In [1]:
# Failed v17

import inspect

def print_variables(*args):
    frame = inspect.currentframe()
    frame = inspect.getouterframes(frame)[1]
    string = inspect.getframeinfo(frame[0]).code_context[0].strip()
    # print('inspect.getframeinfo(frame[0]).code_context',inspect.getframeinfo(frame[0]).code_context)
    # print('string:',string)
    args_base = string[string.find('(') + 1:-1].replace('\n','').split(',')
   
    names = []
    for i in args_base:
        if i.find('=') != -1:
            names.append(i.split('=')[1].strip())
       
        else:
            names.append(i)

    variable_dict = {name:arg for name,arg in zip(names,args)}
    
    ind = names[0]
    print(f'\ntata{ind} start')
    for k,v in variable_dict.items():
        if k==ind:
            continue
        print(f'{k}: {v}')

    print(f'tata{ind} end\n')
    
    
def print_variables(*args):
    pass
    
pp = print_variables
p = lambda x: print(f'pointer {x}')


from collections import deque,Counter

class Node:
    def __init__(self, val):
        self.val = val
        self.well_cost = None
        self.pipes = dict()
        self.is_outer = True
        self.info = {
            'connected_min_well_node':{},
            'connected_min_well_cost':{},
            'connected_pipe_cost':{},
            'connected_pipe_set':{}
        }
        
    def check_outer(self):
        if len(self.pipes)>1:
            self.is_outer = False
        else:
            self.is_outer = True
            
    def add_pipe(self, other_node,cost):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        self.pipes[other_node_val] = cost
        self.check_outer()
    
    def remove_pipe(self, other_node):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        del self.pipes[other_node_val]
        self.check_outer()
        
        

class Solution:
    
    
    def minCostToSupplyWater(self, n, wells, pipes,testing=False):
        
        BASE_NODE = -1#46# 64 # 33 # 25 # 326
        
        
        def packupNode(n, well_cost, pipes):
            # Initial Node
            

            if type(well_cost)==dict:
                node_dict = {i:Node(i) for i in well_cost.keys()}
                
                for k,v in well_cost.items():
                    node_dict[k].well_cost = v
                
            elif type(well_cost) ==list:
                node_dict = {i:Node(i) for i in range(1,n+1)}
                for i in range(len(well_cost)):
                    node_dict[i+1].well_cost = well_cost[i]

            # Add Edges  
            for edge in pipes:
                node_a = node_dict[edge[0]]
                node_b = node_dict[edge[1]]
                cost = edge[2]
                
                if node_a.val in node_b.pipes:
                    if  node_b.pipes[node_a.val]>cost:
                        node_b.add_pipe(node_a.val,cost)
                        node_a.add_pipe(node_b.val,cost)
                else:
                    node_b.add_pipe(node_a.val,cost)
                    node_a.add_pipe(node_b.val,cost)     
                    
            return node_dict


        def checkAround(node, node_dict):
            stack = deque()
            group_set = set()
            stack.append(node)

            while(stack):
                cur = stack.pop()
                group_set.add(cur)
                for pipe, pipe_cost in cur.pipes.items():
                    adj_node = node_dict[pipe]
                    if not adj_node in group_set:
                        if pipe_cost <= cur.well_cost or pipe_cost <= adj_node.well_cost:
                            stack.append(adj_node)    
            return group_set

        
    

        def getGroupCost(group_set, node_dict):

#             well_status = {}
            well_status = {x.val:0 for  x in group_set}
            pipe_status = {}


            connected_node_set = set()
            connected_pipe_set = set()
            
            # Round1 Edge to Fork (or edge)
            def get_pipe_key(node1,node2):
                if (type(node1)==str) or (type(node1)==int):
                    key = tuple(sorted([node1, node2]))
                else:
                    key = tuple(sorted([node1.val, node2.val]))
                return key
            
            
            
            def update_pipe_status(node1,node2,status):
                key = get_pipe_key(node1,node2)
                pipe_status[key] = status 
                
                
                
            def disconnect(node1,node2):
                # pp(' Disconnect',node1.val,node2.val)
                if (type(node1) in (str,int)):
                    node1 = node_dict[node1]
                if (type(node2) in (str,int)):
                    node2 = node_dict[node2]
                node1.remove_pipe(node2)
                node2.remove_pipe(node1)
                
            def clean_edge(node):
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if (cost> neighbor_node.well_cost) and (cost>node.well_cost):
                        disconnect(node,neighbor_node)
                    

            def check_t1(node):# Center Smallest
                if node.val==BASE_NODE:
                    pp(1.01,well_status)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                # pp(' t1',node.val)
                min_node = node.val
                well_cost = node.well_cost
                for neighbor,cost in node.pipes.items():
                    if cost < well_cost:
                        return False,None,None
                return True,min_node,well_cost
                
                
            def check_t2(node): # All neighbor connected, choose the min pipe
                # pp(' t2',node.val)
                if node.val==BASE_NODE:
                    pp(2.01)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                min_node = node.val
                min_cost = node.well_cost
                for neighbor,cost in node.pipes.items():
                    if not neighbor in connected_node_set:
                        return False,None,None
                    if cost<min_cost:
                        min_cost = cost
                        min_node = neighbor
                        
                return True, min_node, min_cost
                    
                
#             def check_t3(node): # Short version
#                 min_node = node.val
#                 min_cost = node.well_cost
#                 for neighbor,cost in node.pipes.items():
#                     if not neighbor in connected_node_set:
#                         neighbor_node = node_dict[neighbor]
#                         for n,_ in neighbor_node.pipes.items():
#                             if n==node.val:
#                                 continue
#                             else:
#                                 if not n in connected_node_set:
#                                     return False,None,None
#                         if (cost+neighbor_node.well_cost)<min_cost:
#                             min_cost = cost+neighbor_node.well_cost
#                             min_node = neighbor_node.val
                            
#                     else:
#                         if cost<min_cost:
#                             min_cost = cost
#                             min_node = neighbor
                
#                 return True,min_node,min_cost
                
                
            def get_connected_group(node, connected_set):
                stack = deque()
                group_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                
                while(stack):
                    cur = stack.pop()
                    group_set.add(cur)
                    for pipe, _ in cur.pipes.items():
                        adj_node = node_dict[pipe]
                        if (not adj_node in group_set) and (not adj_node in connected_set):
                            stack.append(adj_node)   
                            if adj_node.well_cost<min_cost:
                                min_node = adj_node.val
                                min_cost = adj_node.well_cost
                return min_node,min_cost
                
                
            # Unknown
            def check_t4(node): 
                min_node,min_cost = get_connected_group(node, connected_node_set)
                return True, min_node,min_cost
                
            # Not sure
            def check_t5(node):
                pass
                
            # 
            def check_t6(node):  # is outer or connected 
                # pp(6)
                if node.val==BASE_NODE:
                    pp(6.01)
                    p = pp
                    import pdb
                    bp = pdb.set_trace
                else:
                    def p(*args):
                        pass
                    def bp():
                        pass
                
                
                min_node = node.val
                min_node_cost = node.well_cost
                min_pipe_cost = 0
                min_cost = (min_node_cost+min_pipe_cost)
                
                with_additional_node = True
                with_additional_pipe = False
                for neighbor,cost in node.pipes.items():
                    
                    p(6.010,neighbor)
                    
                    neighbor_node = node_dict[neighbor]

                    if (not neighbor in connected_node_set) and neighbor_node.is_outer:
                        if neighbor_node.well_cost<cost:
                            with_additional_node = True
                            with_additional_pipe = False
                            min_node = neighbor
                            return True,min_node,neighbor_node.well_cost, with_additional_node,with_additional_pipe
                    
                    
                    p(6.010,node.val,connected_node_set,neighbor,cost,neighbor_node.val,neighbor_node.well_cost,neighbor_node.is_outer,min_cost,neighbor_node.pipes.keys(),well_status,pipe_status)
                    
                    min_cost = (min_node_cost+min_pipe_cost)
                    
                    
                    if neighbor in connected_node_set:
                        p(6.0101)
                        if min_node_cost==0: # pre is pipe
                            if cost<min_pipe_cost:
                                min_pipe_cost = cost
                                min_node_cost = 0
                                min_node = neighbor
                                with_additional_node = False
                                with_additional_pipe = True
                        else:
                            if cost<min_node_cost:
                                min_pipe_cost = cost
                                min_node_cost = 0
                                min_node = neighbor
                                with_additional_node = False
                                with_additional_pipe = True
                    
                    elif neighbor_node.is_outer:
                        p(6.0102)
                        
                        if min_node_cost==0: # pre is pipe
                            if neighbor_node.well_cost <min_pipe_cost:
                                min_pipe_cost = cost
                                min_node_cost = neighbor_node.well_cost
                                min_node = neighbor
                                with_additional_node = True
                                with_additional_pipe = True
                        else:
#                             bp()
                            if neighbor_node.well_cost <min_node_cost:
                                min_pipe_cost = cost
                                min_node_cost = neighbor_node.well_cost
                                min_node = neighbor
                                with_additional_node = True
                                with_additional_pipe = True
                            
                    elif cost>min_cost:
                        continue
                    else:
                        p(6.0103)
                        return False, None, None, None, None 
                        
                p(6.011,node.val,neighbor,cost,neighbor_node.val,neighbor_node.well_cost,min_node,well_status,pipe_status)
                
                return True, min_node, min_node_cost+min_pipe_cost, with_additional_node,with_additional_pipe
                
                
            def check_t7(node):  # Disconnected Group Min
                # pp(7)
                if node.val==BASE_NODE:
                    pp(7.01)
                    p = pp
                else:
                    def p(*args):
                        pass
                
                stack = deque()
                group_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                
                while(stack):
                    cur = stack.pop()
                    if cur.val in connected_node_set:
                        return False, None, None
                    group_set.add(cur)               
                    if cur.well_cost<min_cost:
                        min_node = cur.val
                        min_cost = cur.well_cost
                    
                    for pipe, _ in cur.pipes.items():
                        adj_node = node_dict[pipe]
                        if adj_node in connected_node_set:
                            return False, None, None
                        elif not adj_node in group_set:
                            stack.append(adj_node)
                p(7.01,'Success')
                return True, min_node,min_cost
                
              
            
#             def check_t8(node):  # is outer min_node in surounding connected edges or surounding not connected edges 
#                                  # but with larger cost 
#                 if node.val==BASE_NODE:
#                     pp(8.01)
#                     p = pp
#                 else:
#                     def p(*args):
#                         pass
                
                
#                 min_node = node.val
#                 min_node_cost = node.well_cost
#                 min_pipe_cost = 0
                
#                 min_cost = min_node_cost+min_pipe_cost
                
#                 with_additional_node = False
#                 not_connected_node_list = []
#                 for neighbor,cost in node.pipes.items():
#                     neighbor_node = node_dict[neighbor]
                    
#                     min_cost = min_node_cost+min_pipe_cost
                    
#                     if neighbor in connected_node_set:
#                         if cost<min_cost:
#                             min_cost = cost
#                             min_node = neighbor
#                             with_additional_node=False
                        
#                     elif neighbor_node.is_outer:
#                         if (neighbor_node.well_cost+cost)<min_cost:
#                             min_cost = neighbor_node.well_cost+cost
#                             min_node = neighbor
#                             with_additional_node=True
                            
#                     else: # not connected not outer
#                         not_connected_node_list.append((neighbor, cost))
                        
#                 for neighbor,cost in not_connected_node_list:
#                     if cost<min_cost:
#                         return False, None, None, None
 
#                 p(8.01,'Success')
#                 return True, min_node,min_cost,with_additional_node


                    
            def check_t9(node):    # Min neighbor node
                if node.val==BASE_NODE:
                    pp(9.01)
                    p = pp
                else:
                    def p(*args):
                        pass

                min_node = node.val
                min_well_cost = node.well_cost
                min_pipe_cost = 0
                
                with_additional_node = False
                for neighbor,cost in node.pipes.items():
                    neighbor_node = node_dict[neighbor]
                    
                    
                    if neighbor in connected_node_set:
                        if min_well_cost==0: 
                            if cost<min_pipe_cost:
                                min_pipe_cost = cost
                                min_node = neighbor
                                min_well_cost = 0
                                with_additional_node = False

                        else:
                            if cost<min_well_cost:
                                min_pipe_cost = cost
                                min_node = neighbor
                                min_well_cost = 0
                                with_additional_node = False
                        # pass
                    
                    else:
                        for nn,cc in neighbor_node.pipes.items():
                            if nn==node.val:
                                continue 
                                
                            nn_node = node_dict[nn]
                            if cc<neighbor_node.well_cost:
                                p(9.011,node.val,neighbor,nn,cc,neighbor_node.well_cost)
                                return False, None, None, None

#                         bp()
                        if min_well_cost==0: 
                            
                            if (neighbor_node.well_cost+cost) < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                            
                            elif neighbor_node.well_cost < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                        else:
                            
                            if cost > (min_pipe_cost+min_well_cost):
                                continue
                            
                            elif (neighbor_node.well_cost+cost) < min_pipe_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                            
                            elif neighbor_node.well_cost < min_well_cost:
                                min_node=neighbor
                                min_pipe_cost = cost
                                min_well_cost = neighbor_node.well_cost
                                with_additional_node = True
                        # pass
                    
                p(9.012,node.val,min_node,min_well_cost,min_pipe_cost,with_additional_node)
                p(9.01,'Success')
                return True,min_node,min_well_cost+min_pipe_cost,with_additional_node
            
            
            
            def check_t10(node):  # Min Node or Already Connected Pipe in a Group
                # pp(10)
                if node.val==BASE_NODE:
                    pp(10.01)
                    p = pp
                else:
                    def p(*args):
                        pass

                stack = deque()
                checked_set = set()
                stack.append(node)

                min_node = node.val
                min_cost = node.well_cost
                pre_node = None
                
                with_additional_node=True
                while(stack):
                    cur = stack.pop()
                    checked_set.add(cur.val)               
                    if cur.well_cost<min_cost:
                        min_node = cur.val
                        min_cost = cur.well_cost
                        with_additional_node=True
                        pre_node = None
                    
                    for adj, cc in cur.pipes.items():
                        adj_node = node_dict[adj]
                        if adj in connected_node_set:
                            checked_set.add(adj)
                            if cc<min_cost:
                                min_node = adj
                                min_cost = cc
                                with_additional_node = False
                                pre_node = cur.val
    
                        elif not adj in checked_set:
                            stack.append(adj_node)   
                
                
                p(10.011,node.val,min_node,min_cost,with_additional_node,pre_node)
                p(10.01,'Success')
                return True,min_node,min_cost,with_additional_node,pre_node
                
            
            def clean_edge_II(node): # Pruning
                min_node = node.val
                min_cost = node.well_cost
                changed_ind=False
                
                with_additional_node=False
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if neighbor in connected_node_set:
                        if cost<min_cost:
                            min_cost = cost
                            min_node = neighbor
                            with_additional_node = False
                    # elif neighbor_node.is_outer:
                    elif (neighbor_node.well_cost+cost)<min_cost:
                            min_cost = neighbor_node.well_cost+cost
                            min_node = neighbor
                            with_additional_node = True
                            
                    else:
                        pass
                    
                for neighbor,cost in list(node.pipes.items()):
                    neighbor_node = node_dict[neighbor]
                    if neighbor in connected_node_set:
                        if cost>min_cost:
                            disconnect(node,neighbor)
                            changed_ind=True
            
                return changed_ind
            
            
            
            for node in group_set:
                clean_edge(node)
                
            # print()
            # for x in group_set:
            #     print('node:',x.val)
            #     # print('info:',x.info)
            #     print('pipes:',x.pipes.keys())
            #     print('is_outer:',x.is_outer)
            #     print()
                
            stack = deque(group_set)
            
            
            import copy
            pre = copy.copy(stack)
            pre_pipe_status = copy.copy(pipe_status)
            pre_well_status = copy.copy(well_status)
            while(stack):
                for i in range(len(stack)):
                    if not stack:
                        break
#                     print('stack:',[x.val for x in stack])
                        
                    cur = stack.popleft()
                    
                    pp('#####################################################',cur.val,well_status)
                    
                    # if cur.val in connected_node_set:
                    #     continue
                    
                    ind, min_node, min_cost = check_t1(cur)
                    if ind:
                        connected_node_set.add(cur.val)
                        well_status[cur.val] = 1
                        continue

                    ind, min_node, min_cost = check_t2(cur)
                    if ind:
                        connected_node_set.add(cur.val)
                        if cur.val!=min_node:
                            pipe_key = get_pipe_key(min_node,cur.val)
                            connected_pipe_set.add(pipe_key)
                            pipe_status[pipe_key] = 1
                        continue  

                    # ind, min_node, min_cost = check_t3(cur)
                    # if ind:
                    #     connected_node_set.add(min_node)
                    #     connected_node_set.add(cur.val)
                    #     if cur.val!=min_node:
                    #         key = get_pipe_key(min_node,cur.val)
                    #         connected_pipe_set.add(key)
                    #     continue

                    ind, min_node, min_cost,with_additional_node,with_additional_pipe = check_t6(cur)
                    if ind:
                        pp(6)
                        if with_additional_node and (not with_additional_pipe):
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            if min_node!=cur.val:
                                stack.append(cur)
                            
                        elif with_additional_node and with_additional_pipe:
                            connected_node_set.add(cur.val)
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            if cur.val!=min_node:
                                pipe_key = get_pipe_key(min_node,cur.val)
                                connected_pipe_set.add(pipe_key)
                                pipe_status[pipe_key] = 1
                        
                        elif (not with_additional_node) and with_additional_pipe:
                            connected_node_set.add(cur.val)
                            if cur.val!=min_node:
                                pipe_key = get_pipe_key(min_node,cur.val)
                                connected_pipe_set.add(pipe_key)
                                pipe_status[pipe_key] = 1
                        else: # (not with_additional_node) and (not with_additional_pipe):
                            pass
                        

                        continue
                              
                    """
                    True, min_node, min_node_cost+min_pipe_cost, with_additional_node,with_additional_pipe
                    
                    """        
                    
                            
                            
                            
                    ind, min_node, min_cost = check_t7(cur)
                    if ind:
                        pp(7,min_node,well_status)
                        connected_node_set.add(min_node)
                        well_status[min_node] = 1
                        
                        if min_node!=cur.val:
                            stack.append(cur)
                        # pp('t7',min_node)
                        continue 
                        

                        
#                     ind, min_node, min_cost,with_additional_node = check_t8(cur)
#                     if ind:
#                         pp(8)
#                         connected_node_set.add(cur.val)
#                         connected_node_set.add(min_node)
#                         if cur.val!=min_node:
#                             pipe_key = get_pipe_key(min_node,cur.val)
#                             connected_pipe_set.add(pipe_key)
#                             pipe_status[pipe_key] = 1
                    
#                         if cur.val==min_node:
#                             well_status[cur.val] = 1
#                         elif with_additional_node:
#                             well_status[min_node] = 1

#                         continue
                          
                            
                    ind, min_node, min_cost,with_additional_node = check_t9(cur) # Inner Net
                    if ind:
                        pp(9)
                        connected_node_set.add(cur.val)
                        connected_node_set.add(min_node)
                        if cur.val!=min_node:
                            pipe_key = get_pipe_key(min_node,cur.val)
                            connected_pipe_set.add(pipe_key)
                            pipe_status[pipe_key] = 1
                    
                        if cur.val==min_node:
                            well_status[cur.val] = 1
                        elif with_additional_node:
                            well_status[min_node] = 1

                        pp(9.1,well_status,pipe_status)
                        continue


                    ind, min_node, min_cost,with_additional_node,pre_node = check_t10(cur)
                    if ind:
                        pp(10,min_node,well_status)
                            
                        if with_additional_node:
                            connected_node_set.add(min_node)
                            well_status[min_node] = 1
                            
                            if min_node!=cur.val:
                                stack.append(cur)
                        else:
                            # if (not min_node) or (not(pre_node))
                            pipe_key = get_pipe_key(min_node,pre_node)
                            connected_pipe_set.add(pipe_key)
                            connected_node_set.add(pre_node)
                            pipe_status[pipe_key] = 1

                        
                            if pre_node!=cur.val:
                                stack.append(cur)

                        # pp('t7',min_node)
                        continue 


                            
                    stack.append(cur)
                    

                    
                # pp(9,[x.val for x in stack],[x.val for x in pre])
                    
                if (stack==pre) and (pre_pipe_status==pipe_status) and (pre_well_status==well_status):
                    
                    changed=False
                    for node in stack:
                        ind = clean_edge_II(node)
                        if ind:
                            changed=True
                    
                    if not changed:
                        break
                else:
                    pre = copy.copy(stack)
                    pre_pipe_status = copy.copy(pipe_status)
                    pre_well_status = copy.copy(well_status)

                    
            print('##########################################################################')
                    
            print("stack:",[x.val for x in stack])
#             print("connected_node_set:", connected_node_set)
#             print("connected_pipe_set:", connected_pipe_set)
                    
                       
            pipe_status = {k:v for k,v in pipe_status.items() if v}
            well_status = {k:v for k,v in well_status.items() if v}
            

#             print()
#             for x in node_dict.values():
#                 print('node:',x.val)
#                 # print('info:',x.info)
#                 print('pipes:',[x for x in x.pipes.keys()])
#                 # print('is_outer:',x.is_outer)
#                 print()


            print("well_status:\n",sorted([x for x in well_status.keys()]))
            print("pipe_status:\n",sorted([x for x in pipe_status.keys()]))
            
            
            
            pipe_cost = sum([pipe_cost_dict[k] for k,v in pipe_status.items() if v])
            node_cost = sum([well_cost_dict[k] for k,v in well_status.items() if v])

            print('node_cost:',node_cost)
            print('pipe_cost:',pipe_cost)
            return pipe_cost, node_cost,well_status,pipe_status


        
        
        
        pipe_cost_dict = {}
        for x in pipes:
            key = tuple(sorted((x[0],x[1])))
            val = x[2]
            
            if key in pipe_cost_dict:
                if pipe_cost_dict[key]>val:
                    pipe_cost_dict[key]=val
            else:
                pipe_cost_dict[key]=val

        if type(wells)==list:
            well_cost_dict = {(i+1):x for i,x in enumerate(wells)}
        elif type(wells)==dict:
            well_cost_dict = {k:v for k,v in wells.items()}
        # Base
        checked_set = set()
        node_dict = packupNode(n, wells, pipes)
        node_list = list(node_dict.values())

        pipe_cost, node_cost,well_status,pipe_status = getGroupCost(node_list, node_dict)
        group_cost = pipe_cost + node_cost
            
            
        if testing: 
            return group_cost,sorted([x for x in well_status.keys()]),sorted([x for x in pipe_status.keys()])
        else:
            return group_cost


    
# n = 5120
# wells = [21128,75528,97977,52262,52378,15538,38502,82313,47321,85603,57516,70456,3340,10685,14501,58724,42337,74534,6019,4958,57135,702,93577,99406,31850,89442,92604,98339,96751,92818,99355,69270,69868,52010,69654,68788,31959,72233,81473,89012,98204,77272,17591,88067,1682,66222,2648,39761,53101,86944,93661,24208,61060,27169,4096,61950,91487,5713,66947,92700,69000,22407,50866,30970,13669,67723,25650,12925,86768,18176,23777,47704,6001,28669,18491,41794,81633,98117,86153,84833,61390,56697,61686,16438,96154,89325,19969,34488,71395,80075,9290,42780,71525,98954,98949,5726,47989,36609,66733,90119,61939,93626,50667,21209,30272,16076,23587,31502,20140,67856,10240,22960,54014,52123,87986,67927,37322,50755,41996,82535,85198,9227,9908,8376,93639,65630,69549,98730,80787,14611,13998,57844,71866,29050,79414,35399,32945,81424,21955,41679,73049,11354,69358,18489,38909,36219,92724,51020,89519,43854,8018,1177,50616,24035,99671,51378,69801,9524,58,94173,21476,60926,95071,97791,80030,19941,13664,40706,47216,14832,11235,1618,25984,52530,54636,98407,18467,39794,71734,32048,43613,68877,1750,36231,85772,10250,82036,61698,4581,8832,61311,75892,50035,22491,94445,97822,57825,83237,26853,58666,92055,23491,5679,34791,6369,37423,20900,97724,5726,10614,38469,29887,78640,84042,26189,58524,51047,68102,33765,3440,20862,31749,61789,37905,90161,54661,58086,43710,74402,1787,8606,1694,84348,62614,69986,61834,24177,37259,66432,28044,54452,43908,57008,24151,74973,15201,34448,68761,76275,34812,82783,25593,65309,54357,34658,50861,40309,99643,49225,62866,56314,54205,7414,6946,42236,75620,19893,79605,292,63612,83196,30451,12402,36348,60488,35269,59872,86581,75990,85045,31249,82809,36114,78399,32493,64972,47424,91529,68309,29235,7094,53707,6607,64663,31229,23737,21931,99922,71809,91458,774,14714,2129,46905,36074,43740,10298,27174,52954,60298,51533,83198,55851,50244,14454,20660,16223,79195,53447,76838,34390,47889,83767,86683,23837,38197,87247,46525,69473,90769,46233,39895,90331,3856,5245,6039,27673,40562,74321,55080,71112,18073,10865,95454,9593,52441,74657,76750,79532,39354,56288,50675,18758,99925,37918,30710,51440,99446,42368,65414,33421,19571,17110,42778,15604,86739,22061,18133,49087,12508,8185,5696,68532,44608,10581,55129,36116,77450,8836,87767,28010,31303,52670,11378,32852,56253,74493,91829,77304,28107,5736,53220,40739,10929,12977,71900,19972,36336,63597,61179,38395,73330,3170,26604,25879,62880,81557,90467,88706,59362,78528,16957,71241,49440,15725,1915,26006,35380,14401,80262,37337,62783,92025,63896,30995,75920,74312,69685,27159,32062,70444,92693,52586,13090,59397,33566,86826,58154,22750,45680,24536,72131,96895,16672,63780,19003,16442,48544,93873,79506,90387,89772,28710,87459,4253,22432,5508,62846,19167,98522,15840,20970,21646,86268,95181,16273,74467,26720,45498,21394,27488,41400,40857,98905,38323,37166,58531,19807,90624,85466,826,66915,89977,67603,88858,54190,56391,53628,29255,44732,90461,87316,72920,10294,94325,46266,32320,84913,27775,46317,25259,62604,98683,1163,50831,20996,67535,28022,31936,9359,57997,53293,16166,90968,5610,98677,87204,32122,64223,49981,66231,94138,63795,34347,44108,16931,12803,52934,90050,10905,38807,56499,69434,75748,75987,28644,8009,9228,37462,36282,74376,24584,18419,36874,84941,10567,28562,24564,4388,89426,70248,97569,83634,78601,90408,24587,876,11564,24023,44134,92407,99304,59507,52676,36490,51811,98936,23078,23451,6912,91728,97804,45705,29015,6501,50648,93394,31486,85833,33121,77695,79898,97513,79456,93290,84536,31622,32336,39514,82426,42270,20375,45038,64750,60304,23190,92543,22771,50340,91463,87263,59310,30115,42401,14075,1839,19666,6201,91372,92347,47312,90591,45247,41791,60831,25649,2183,9785,95802,43927,83036,16446,72458,15378,24760,11584,30490,27446,56556,3947,90231,55869,96391,53320,81259,99427,68218,73419,51813,73002,20792,32167,793,37339,42973,29669,38921,84152,28165,63391,35823,3667,42668,52344,36006,66088,22539,35611,25678,377,10698,37641,4152,25553,19953,79344,76166,46159,94346,81931,29383,40563,1383,33101,24119,67774,75540,54,10297,14159,24397,95229,34503,17591,22643,53823,96961,99632,62902,37168,33152,76881,50996,11969,42557,28398,44837,30287,91066,90878,28572,50129,52533,87683,68314,50450,81159,34636,16141,97481,85268,65125,54871,56141,74785,15739,2596,30257,15052,40850,43972,46468,1478,12625,90174,87667,15451,6492,74535,47633,29540,68661,50153,19973,69822,22596,52483,51971,59536,48421,77723,55879,60123,75458,12363,49026,87090,17887,70196,82654,85048,29062,57160,36882,94244,94262,21539,6242,79710,84186,71852,95820,75439,88436,45074,86037,21724,60272,82634,94848,71507,45058,93257,20090,62973,87047,94777,74805,25555,33799,46180,51568,38778,50209,29587,48009,13611,77011,83699,68421,58897,30217,73538,51615,16195,47134,61167,16458,14992,95421,71095,48104,78506,91875,60334,4114,75527,18786,82351,35532,50241,50520,17270,90251,48971,88583,68346,3848,15025,70858,97811,58037,68107,31215,8917,75557,44214,53772,11833,19310,31803,82175,64860,28944,94860,68953,26949,44917,24732,60193,86879,64017,16658,29788,12836,22687,88036,1727,92302,90009,51181,58144,48586,39924,16946,92452,33424,77306,60740,78015,53771,95852,38535,88149,17595,63739,33295,17344,81835,33448,23512,58607,9968,13270,15219,66043,99079,98340,8320,43418,19105,42643,84442,75356,53399,80425,32583,13078,68747,65624,83734,893,54635,945,79073,87847,40456,74898,90253,45601,23965,9188,40894,3754,79019,99794,57332,15685,91816,83086,53987,43455,70315,33107,31030,3535,24937,10863,8045,84647,82713,5071,15733,42459,14557,64461,91699,16500,99259,52970,37304,4569,23097,13179,29011,88814,16161,3696,93996,93160,88189,69671,79403,62556,33777,11092,13584,6275,61774,7373,93411,71739,58735,50241,73185,20502,25261,86711,42480,39348,42682,11165,90124,39882,11329,60009,82621,81196,22652,17384,71993,8830,86961,23286,56008,21564,42073,97841,81333,36967,89008,64432,89462,31948,59357,50876,92984,31854,75777,54349,75612,2645,83708,73798,84068,18782,67725,51197,20242,78221,15405,88975,51712,53338,56229,70559,93094,42660,91704,31559,37972,79132,63110,24211,87803,59038,15146,80714,72374,73476,17009,52153,42924,35290,46721,63103,9738,13920,69124,11726,43399,80181,93036,60362,94235,27969,45172,23065,6605,16154,26321,68698,56945,10420,90505,69012,67396,89141,47554,69783,82993,4910,66633,78642,60171,97551,24823,8750,57266,74022,91159,57012,25819,35944,79905,10308,46203,47968,95061,15292,81099,74545,72865,27022,98151,11036,34755,58121,66532,41026,3114,54703,84146,15231,15438,9441,74611,63248,31882,92410,83667,89292,64519,76340,70337,31812,11008,76279,20963,31667,75575,38826,59449,64260,24512,27084,33892,94339,65486,7069,17944,87392,98921,78632,58804,4891,20515,45314,61484,68694,49231,94788,35669,52970,67904,36921,6102,58638,41658,73948,97709,50701,70696,58743,94303,62964,87389,73320,87552,16086,97919,92033,52684,65339,37318,90039,69677,81421,978,78297,96989,38235,53587,79889,44260,46184,32704,91898,93954,38755,31545,15054,40644,34988,61926,80204,32829,81642,28595,76613,51572,37339,38967,94174,73311,80507,10750,89006,24961,87684,69276,23445,33301,43699,98773,59384,30661,56264,90151,5850,18180,52408,54646,22614,67348,22613,24805,45947,29500,43287,14864,62135,84902,60446,18211,92923,76670,89875,49353,90483,48189,38758,88789,38625,14797,30270,92055,27904,9988,74765,30139,22782,16514,11091,10572,75398,90492,40731,97848,97439,4262,90919,63479,11307,69550,67871,99525,45190,99847,43972,84558,40801,70605,11570,15760,46120,69357,16621,13938,5398,23311,5720,83340,61227,93356,99224,80281,63292,72418,96202,63819,15608,6783,71776,85763,18057,91241,19993,63631,93025,83683,7753,32760,24773,82670,49424,25092,98290,13525,41696,62152,54031,36463,60279,20657,44645,60436,37415,23642,94797,85267,18304,99226,73993,98763,6658,87030,68355,31437,43022,29304,23320,75433,61626,42079,67876,58293,16608,51941,41930,40880,86964,49379,1077,15350,72960,13884,29503,39425,86562,70778,59841,34248,52260,74490,90093,53189,99288,80278,83298,99575,72594,4562,34793,95116,31373,51355,25006,17448,3263,44201,212,65868,34255,48071,77893,67461,72771,99698,77863,70132,61650,65378,26666,84315,85665,95930,54391,31646,2817,40276,6652,59497,25345,15928,46708,51400,58897,97320,55420,18055,6616,64022,86074,68372,33385,27765,99561,36675,36492,48263,50987,56466,33460,12238,55671,45851,96091,48186,41076,56396,15412,36161,42485,56796,56296,88870,46899,8233,2344,2885,41342,31776,35569,18716,95261,29438,73718,70650,6089,1471,68298,17162,68760,66087,37528,40637,26635,99070,31718,15189,55957,76064,22738,60905,42989,85938,97466,50427,27183,8730,96235,13517,84361,25325,79309,15929,77996,70506,79021,76536,23765,82921,39363,603,40314,44394,53286,29204,71468,57635,34317,63272,36569,95972,943,32101,86247,30776,61664,87536,56005,36357,91667,75500,67288,79466,95007,48684,30231,18924,70970,32506,76005,10590,29909,64396,23674,89669,18915,18373,14115,49806,71403,75369,64577,75182,6167,44909,93849,4498,83941,33918,43641,82119,63457,61137,26533,70646,73593,7797,68141,79481,22767,9325,10241,17247,19839,51908,38091,7748,48673,84165,20139,60560,64666,90829,57871,43645,98423,27581,71138,47389,59847,93497,17181,40712,3395,22902,78394,78270,98706,52705,89097,2807,74834,44882,65030,77593,22725,62296,6737,35532,30852,9400,77320,86042,81493,58511,29323,18102,85251,67220,89253,31261,52417,72485,51266,64426,62836,69716,29557,4970,75907,70234,11188,43237,25894,38253,56197,13576,9403,40668,47581,9439,91457,98782,20608,72358,4031,55695,43430,67698,97238,73269,36596,41483,51941,92234,55751,53951,67526,91253,87054,22002,25081,83501,26084,94748,83638,94522,1796,73818,70702,42083,92016,96918,78500,17090,53413,67636,55676,96824,83066,82011,23003,70910,81928,64516,25695,92082,28501,1118,11722,80628,87657,69458,55520,7348,87634,19571,37504,35679,69084,50096,1347,89816,99799,37729,79758,2035,23594,74602,83917,50023,8962,3168,12504,7853,6379,81057,86219,33646,54178,63859,65010,34130,70307,94265,87551,79211,34376,12890,54603,34827,67616,34647,89431,36513,50999,75722,34306,56047,55858,7534,89107,8928,94114,88974,8692,93547,39919,7773,81787,73825,47923,35681,36029,25393,94352,48007,47078,68736,29625,70072,71398,28919,31743,9530,94858,19626,81906,78556,76955,94531,81944,72022,40212,24387,29111,72259,10573,97955,37106,87612,18276,64241,31172,88174,9686,31806,792,67672,43420,94922,43945,24676,79613,99315,32058,36830,95527,71541,93854,87934,76967,94523,48348,11352,49499,22146,88900,32017,50777,28841,64092,77829,25565,56050,39276,35692,32393,14183,26235,32838,64666,3338,2629,27823,76624,87858,63386,66919,57822,93350,51011,1742,9730,91361,11598,67320,46985,78953,73631,30742,69237,5435,55425,82558,95691,77875,80975,23059,92475,73473,98447,70940,32906,59190,92867,68348,5556,57533,5148,93146,99021,46642,8157,59752,24720,97493,78327,58469,4701,22749,61142,61444,15324,44766,80137,74351,75926,6247,57736,93148,45593,46209,58416,55245,56892,18536,87139,90297,65547,11931,19977,54649,71025,10933,38213,6357,90482,94253,35511,52616,43500,28277,37740,96151,2685,37659,76487,39806,50793,10036,71092,39660,90267,17428,83146,20686,14443,78661,68681,66123,18875,82484,70883,5379,17918,66891,19381,68217,54588,53006,88269,66011,26968,57962,47844,35261,24554,43140,4818,98339,45157,89034,12249,47427,14749,36123,13053,58955,18342,65582,47486,20177,41972,66933,65616,83409,7520,70666,41162,14711,50553,53608,27560,87432,44220,19385,64854,85848,51834,37139,54625,48132,98780,4655,72750,45522,23574,49063,88600,85963,28537,43234,20117,12524,86640,98162,87622,12848,51964,45065,2120,67388,71460,63350,86999,91132,93082,95604,49405,51633,8271,85487,89843,16603,42176,59270,41587,52940,79603,97406,98598,39718,4251,63950,64811,73550,20437,44042,29838,3560,44145,39657,84037,4209,27956,48873,97320,14472,41480,61335,28242,75839,27906,31192,80590,26464,19645,62303,43108,6603,3271,83417,79809,79697,86997,87711,14463,63207,90294,68756,22156,54262,37616,74081,46293,52772,11267,58551,74985,30785,4630,68565,90050,91024,38200,25704,52810,17906,98031,54381,85656,53943,92632,87993,48477,88049,11941,43800,11045,89282,81739,67607,11679,53608,75578,63129,40714,99341,107,71926,31410,29286,77130,21499,42814,26904,22430,5573,72087,64072,72856,12668,64005,67179,15146,37712,21322,82204,28505,39434,60242,66862,18034,57636,33432,85578,73059,34883,27960,55433,82561,17058,63846,10002,4748,61897,29107,92117,19716,32815,82725,36591,26108,95863,86442,24730,51740,46974,13210,51092,63307,18524,98492,82518,32636,53624,96070,75927,94324,97038,37728,51123,33693,96446,83502,28173,43153,52575,39100,98317,36326,70750,39430,44577,99970,24960,90983,34050,61476,60377,24205,92270,89175,21395,86309,45709,49367,39804,25348,20130,86690,90056,88481,68610,7588,95258,15043,22077,35701,32950,59296,26148,96232,61941,14533,49683,14462,34807,47884,4539,87971,54225,91864,45356,91481,19425,77604,47517,71776,4620,30934,45565,90781,37093,76943,13574,98729,77965,59131,89289,90560,32389,9672,46518,41753,29209,2495,47792,42968,91282,49067,37914,59085,74008,84292,67233,74183,23725,56254,92658,99788,34515,66217,71925,40893,51299,81484,60402,9901,69941,62615,79540,67010,44048,40285,61933,33131,61030,26847,38267,38427,68019,83502,9732,52662,70270,21543,37753,7252,33018,5749,53760,71683,13714,49260,21760,51785,66894,52450,14935,18477,91117,30,76977,19710,66006,83182,92762,57229,88468,72434,27652,87346,22963,88011,5623,4713,79992,50880,60056,3975,68946,61503,75702,45244,4232,8881,49090,78240,16620,71194,8722,63768,81328,67869,94755,58265,37032,31670,18900,18995,24721,99862,44229,92752,34348,70048,53240,72430,92202,23,39098,2296,54344,50705,62238,22891,64614,51416,17044,34322,46537,46347,69272,59164,39471,6751,14592,94381,80003,36032,68365,66010,79635,1807,44614,11912,64073,41726,24672,14534,1574,25125,40782,45502,67397,83307,21575,39320,35386,91056,77997,98801,39821,73186,54941,11349,62029,74196,19504,60240,51942,47773,34788,69325,53380,42650,52871,82481,52491,68674,78422,36432,10157,94419,41688,87639,3077,96721,59910,45456,95875,81095,70111,69307,44742,72594,12507,81902,79885,77407,89444,99409,57156,95938,33157,75193,80765,76643,2134,92626,96024,91929,45526,2481,48015,73769,36175,34060,41579,40862,30987,57966,57108,31040,32493,73768,22647,77611,81269,3727,31319,2788,85671,1550,91767,2060,91174,88800,36027,278,63106,61366,30825,372,58844,37840,74682,69582,8124,89595,25539,69754,58881,97901,65104,37638,42371,80762,78709,67005,2639,93094,7934,48969,24945,17455,54564,94063,31628,79576,62928,12150,44378,5761,83162,58070,83952,76246,89737,44145,35155,31442,98690,4559,78946,37369,18335,99094,70935,30451,91512,52094,56661,99509,32490,72150,98741,60800,61749,37544,836,46036,98983,35019,70565,83783,50794,74373,76206,69330,30868,23631,76206,25542,69841,8844,55755,29633,16328,65545,68985,14548,37910,99093,32966,74018,1496,724,9972,97043,73574,19831,22218,29644,59,64508,63119,7295,90883,902,14472,75683,33424,61851,22662,10033,40416,46165,78138,10410,17872,79971,17276,99185,92762,45786,84684,43650,53479,72079,5388,47521,50266,35462,46298,46748,75288,277,10399,41142,25991,7659,14395,50662,53096,99787,85113,89566,76998,63888,52547,70722,35917,47566,12493,94859,29722,75458,87735,25570,29977,39146,82507,64562,67560,8309,17455,64868,1028,72160,84062,47017,45221,25866,64962,6166,48983,2166,20396,97398,25129,20483,68305,7220,62933,92988,98093,97713,99462,8880,81503,79519,56986,31881,24836,71154,90968,8925,10155,99664,17661,47209,19141,84285,96435,55927,45206,65029,50140,98098,7751,33350,38010,45222,75128,51682,98162,62081,67166,42538,19287,92037,44474,33631,57479,17408,14811,89646,11304,9015,83119,9634,856,50989,85187,72807,50417,13607,40718,90950,33187,20470,42955,93058,88566,9267,37255,94204,11603,62841,1341,45421,27349,60869,56422,52117,46688,56547,97762,82184,80347,86153,14670,71562,38909,19882,25599,51008,554,92186,60684,75790,69902,5906,87784,54386,24828,68036,32485,4641,21748,48737,32362,4673,64196,53621,55092,53621,41687,79798,35481,70316,8771,42738,84152,40746,85060,921,15094,87178,59400,21312,74387,23142,66761,77467,29193,63887,22089,79143,57833,36849,18925,37113,11375,32311,67069,48409,76371,70844,18594,24975,30610,73192,15594,7279,40164,63534,83854,47889,23665,67277,15602,46000,31993,14124,7077,1295,52849,96087,55169,14966,1937,6924,82161,55482,62459,77542,11373,83571,68837,82138,41658,48654,52470,57401,60478,38162,81217,13422,91103,20326,84553,35536,13638,53079,61523,85495,91430,20907,4641,88302,15108,17678,22933,288,68523,46200,56806,38298,53443,70613,62562,32996,87451,90425,82910,76968,42735,45889,5564,79522,53787,52951,52868,61298,24467,33897,19426,95719,75753,24456,38083,8540,73329,85622,91612,68821,95612,63791,85748,23294,39552,88423,35241,14057,3803,22588,45023,72259,95016,5355,5576,2381,95050,30646,84734,25421,17750,77872,48873,5430,26093,27439,23696,35630,73749,90260,52953,4989,28523,50330,58968,33821,2873,52426,60092,74590,66956,70998,35820,31882,26054,26563,27709,5697,94421,33199,52004,45997,7390,78934,39495,95019,67882,70302,31972,33059,38188,92355,4310,82834,64121,99908,55510,38983,75328,59823,38474,35409,69678,94048,89858,23507,8962,66673,61917,37641,21241,64437,67867,57187,29146,83277,14917,80060,71933,62529,42437,66735,38978,77265,54602,15810,27158,13003,47161,5905,97220,75586,13371,66901,78146,85225,49487,29131,26311,44404,50095,75831,17716,80676,71769,62170,3060,44213,83884,2271,28941,83628,67297,35473,9810,41926,91269,94833,91527,14541,3056,54069,22129,23060,18354,70078,47906,95235,33930,62060,39971,81442,13572,30083,37232,52203,14277,68192,86450,96615,88266,17705,40882,8317,17799,42953,78266,68182,91105,307,65762,86308,4501,49946,63575,79013,17145,35705,66913,47836,28979,53816,59823,90208,40034,26375,25131,38148,20604,91033,38677,9427,88544,52195,11291,71449,61757,20198,86680,76855,13902,23841,85046,83161,69578,55935,35367,95540,37388,40601,49481,29562,71865,79124,3467,1374,39768,66346,41237,96376,34719,55147,12156,87346,36489,95540,74797,37542,69489,56788,90254,14856,24395,91439,25953,3030,75424,38736,87710,3721,9944,97187,63589,86659,84201,37363,73009,11055,1279,75794,21122,14557,37020,84352,52697,92986,55206,88807,68447,81550,51892,77535,37285,84591,53494,76655,12243,77960,41655,87936,14095,38370,53638,30388,8438,17631,37950,9973,57453,30387,44642,29163,52334,10004,72158,29199,41541,60221,71064,54534,74471,9314,52249,49077,20818,61399,14838,53537,53617,26242,16163,68726,70052,30511,14824,42981,49817,86533,58579,22946,4592,5424,82076,38727,22929,60457,23953,8640,71032,91248,72338,34507,13618,98256,65892,40759,67481,17027,41414,56238,5704,18903,30758,39670,65266,46412,22397,55411,16550,42192,15645,50637,96232,93465,33712,35829,5753,20259,91147,67798,30498,24047,95231,49812,33776,98921,66478,77036,61962,12944,58802,36798,87314,42400,35436,59065,74320,529,94954,56911,98352,66407,71811,61428,90216,61927,9629,78414,87533,2130,13844,2739,68144,59749,68782,82357,22263,59333,72011,84676,38617,99730,98271,11583,5449,24466,89532,71396,70,92693,47326,89119,31097,7516,10951,78232,31229,69627,25953,27923,39917,93665,36088,91566,65819,35045,81989,15525,42815,45446,57392,23651,76214,20471,50028,91889,55723,59182,87938,75573,88730,80424,80905,32160,75726,84790,90064,38436,65043,8899,13568,20759,40620,55390,42919,65040,39531,74169,72514,62058,7239,65214,9870,91044,89263,54000,66248,60112,50703,60109,27628,24071,4413,56252,9631,93167,39080,13937,21509,61919,52693,95432,91423,69858,90216,1751,41848,40512,16004,41812,33395,6984,61922,92481,6523,90063,85178,78092,84525,54555,20106,79362,41937,40680,58008,28539,31414,85396,46811,14339,19476,65131,11410,12725,69293,95961,70908,69982,29716,76319,1280,41564,59056,19274,34312,22856,17685,52300,39328,90096,76208,42706,79285,31443,64749,46330,70489,28147,13432,61560,45818,50399,85065,69796,99013,4211,50930,6575,44928,19382,73888,23945,60222,55596,38833,38516,35713,43468,1315,37091,52416,89688,27658,49344,3277,6924,24083,4024,45094,46511,95910,41865,1276,31258,98099,54524,55655,80741,77967,87767,86067,11357,63372,48613,32980,96141,48195,10632,78580,14005,6596,56109,958,64485,92986,97413,5373,58154,63043,79631,25745,77215,98929,77738,20535,54438,78645,9919,62495,12414,83591,9693,69570,87240,62730,12750,90921,19100,15691,12141,65027,16469,16275,29422,29579,91147,16486,44133,24889,15823,73537,59632,38869,67303,32253,87318,7449,59751,92000,57284,59879,57547,85419,12956,17892,72398,52079,7696,68938,72632,46570,37455,4847,90911,15298,69960,74862,80849,44691,86966,51584,7027,39784,61434,32931,72636,12106,91949,25868,16181,15711,1398,36397,32480,63799,95284,11559,24211,60099,34886,88922,26551,46385,39425,57722,50215,33376,44508,47020,88448,78103,5111,271,9960,14240,47131,61434,47215,55257,6417,30439,59464,68583,43773,7894,81783,19161,41076,62157,25615,19706,4594,46319,37257,39213,86849,94477,36802,40846,71856,61206,82575,94911,16566,34953,49861,80548,88561,94718,98330,66056,23865,48949,84409,67127,51217,34208,77806,82734,89247,18289,65479,60489,33523,65897,93971,48013,88725,93437,35070,85302,75788,84270,75712,99343,35849,80954,67543,64979,78724,94960,62146,46754,78790,64210,1426,3539,39852,6782,98449,31139,84217,24245,26652,61990,89290,4646,29023,61764,84115,39211,6641,65885,88724,1532,32465,2076,31387,63334,98997,3737,82569,58311,56461,8666,2160,88393,23915,63479,84364,45463,64733,66762,65776,69803,82684,67648,69721,82981,88118,7315,91024,22481,10811,46392,4335,33746,70292,34233,7372,72673,65877,63502,92207,48611,4825,4005,60406,28520,15672,20981,98579,43859,21454,46138,49424,5096,77314,21855,52241,85245,41438,5910,17767,11056,1658,74383,95020,33170,61827,24351,28630,78857,73565,24519,39613,39850,49098,78222,62004,12384,70823,50736,91128,78818,82819,76602,56808,45062,5949,16140,83728,90690,37126,17748,25034,17198,140,64986,37277,84684,38500,85144,70229,67561,85312,22664,97716,79676,56735,71555,44389,40859,99313,93215,215,53499,4981,77081,85505,94974,37356,50899,14165,52473,20842,18063,24077,42600,31831,74702,69822,47774,57847,75284,4221,51211,15898,5846,6761,7236,44461,57589,35286,75191,65596,20583,19437,71837,4555,7100,63527,32833,44053,57011,40634,58428,29431,30463,1193,55281,39153,86994,30185,22081,66485,34032,22519,21799,74942,86685,72624,73936,12792,29561,47571,65188,42324,12950,45723,46150,6526,73437,29723,27386,62676,84701,89394,39305,78134,43147,85629,53964,78979,81195,65863,60695,61927,26684,58651,34614,14697,58233,20356,50110,88109,54215,97414,88645,62213,92478,92034,53193,98721,16421,37804,98510,58299,57558,13455,97737,14057,99604,65994,86153,67272,55621,82583,18364,36714,6979,48761,69118,6811,60381,97943,67346,9201,15301,47501,30010,79740,30878,40696,57019,87286,56184,76333,65022,41991,96417,24238,94972,43582,79752,64115,79161,16824,10262,12206,5864,78301,94350,60832,77655,67572,65833,99506,17100,87807,37696,81933,20085,63746,49165,1962,49837,32257,36210,89085,34859,54635,22291,34452,95021,55980,60352,99429,34016,12184,24039,65951,32879,39819,44390,15488,60109,2250,65366,96790,28477,3101,65864,95909,84900,43296,19649,61896,23238,21883,83856,41951,80073,70217,55637,27776,84955,97233,31659,79663,62480,54910,42329,40308,5110,53744,59013,15541,28405,92828,21449,67874,15321,33049,39504,37615,50391,37307,65416,16304,38357,86734,79546,93282,67193,90754,86931,31588,13000,6884,71565,67520,21148,71447,19868,62259,40917,26731,80988,69143,24415,34276,91080,43835,83296,21658,36551,9081,21128,11569,8783,11178,35165,41674,69369,26315,94647,84601,87098,40285,84766,88971,50583,52459,77660,31150,64481,43667,81166,67013,25659,38168,3906,6001,77015,46570,9818,99108,65659,91776,22567,16943,40380,35876,97739,45779,34494,1175,33688,97673,82939,4797,17673,68318,63741,15092,67403,62171,93647,37611,38823,66052,16070,31630,97142,34754,45263,43574,75819,11586,8377,55469,60822,42990,3121,17208,87130,76657,11331,10329,39243,49047,92234,20450,47257,6783,13608,33385,18497,96710,88915,27737,92357,65528,49701,8700,63148,85518,54204,52750,66585,63378,90728,54041,62584,56387,77629,34688,28587,55506,58607,37020,14849,35642,66760,27094,85536,81623,82269,79952,57518,75732,98821,11282,21708,50659,69105,82248,72216,35096,53729,40435,57711,34978,22487,88482,99431,33159,55036,27791,87780,58914,34225,59246,77153,20312,67532,16414,66885,31514,20352,77226,7184,97663,19982,86141,69841,43923,12305,344,95181,97916,631,48324,86704,20333,68493,62557,6229,40285,33575,60122,35958,44132,75580,18273,32511,7332,40778,61307,38826,15430,5272,58748,2765,84425,37602,71905,50871,19840,15291,20505,56193,75538,86994,96945,89850,93935,47791,34148,83936,56757,50698,72638,51296,63868,71512,64289,16861,54018,17201,47812,35626,72992,11308,87302,71411,27793,77758,1885,98490,42434,52292,20551,81253,36600,26879,71292,46751,98476,47684,20384,60749,93371,55041,31423,48033,15030,69255,78059,15085,97734,20409,99042,9226,39539,65063,15928,61686,52652,25451,38708,25678,76740,59837,2527,99947,49495,10804,14228,98739,1935,80138,78277,25252,65263,22754,94645,64649,2988,589,16807,87639,72962,4202,85100,2565,52295,51005,86616,6615,63571,73961,15299,61909,39942,33179,79553,42456,90042,52345,3623,9190,6751,87268,80056,17307,65339,70152,30028,90663,27947,94233,36203,20312,22578,60781,50402,64770,22741,58316,24762,40033,37858,65451,55924,37534,99646,35116,19876,6332,73889,48003,21625,81732,9129,30648,13203,20011,17916,80115,12108,78876,8516,39464,41078,28200,38436,90383,25873,97190,18158,57244,21031,26680,3992,50990,76496,34162,38304,5096,55735,26267,80526,74898,37240,32162,65571,69621,49903,83551,85054,9842,53489,1422,43963,77659,26865,54481,67693,55902,42595,49745,42001,7877,47271,60199,72747,46409,78426,96231,51161,22179,43747,86724,69346,75197,15754,78972,94887,25979,74197,22549,44259,6630,1264,57163,56186,27236,65341,80380,83245,68321,87749,99314,67032,87421,36086,63435,59477,71354,22197,46526,22336,84842,35152,51577,19450,21407,34209,31621,1970,51608,8186,39645,94610,80118,1181,63198,66244,21551,67514,98781,81496,33265,12263,19389,85896,40260,56074,39478,60499,8466,15322,35168,22061,49372,79995,68183,30470,35873,66253,92112,55239,28734,63869,74422,10327,58745,99513,1851,89919,96368,66541,51372,7425,14975,30662,44248,79217,7575,15461,62990,45577,76424,40555,41462,20780,90780,46334,81230,12844,64025,62602,7801,69675,69425,6809,99807,41450,64339,9553,61207,22305,62112,91728,70450,62487,6882,82249,82352,44667,80189,67688,21590,16017,46361,46685,12810,51417,8725,80639,39093,83757,72362,65684,78483,19829,72481,5195,71221,94648,39825,52115,36722,44492,78173,5636,83113,46509,66445,74201,13467,51882,27952,19941,2290,58966,58705,27279,6701,63675,69584,32431,32362,24999,64996,5941,38935,46882,69950,93729,84902,93202,79468,96861,92789,83527,14532,36853,50228,30072,1406,27019,95524,89877,9354,60251,60112,83493,60016,49557,78042,35210,60794,14965,56891,78142,46680,65392,12355,25844,53129,28127,83808,62718,430,72997,45391,70729,44967,93516,46554,34551,63939,11622,35100,43476,31195,88034,44604,56542,28924,9553,86181,55787,44092,75674,8530,51856,30789,63315,67796,10190,54537,81428,62298,2826,80967,54956,13040,74375,37974,22827,80755,6767,24337,62940,35470,60734,98327,550,82739,66199,96684,35874,14648,24666,71733,73243,66182,98242,16020,33092,6553,64541,41616,62969,90440,23874,18308,63988,61921,71731,96330,81274,24542,21093,45101,97560,63704,71507,2632,44914,85538,56867,61886,9895,59770,91508,31713,1648,79980,38503,54297,53649,49342,57272,96315,82835,21519,34409,70242,83629,2125,24545,29118,50842,91458,48664,65802,81514,55461,79547,76147,93929,93834,87581,81279,2369,73991,64939,80337,16143,93021,52943,34972,64429,46756,69125,91955,21872,57040,8277,82102,15986,33022,30067,28810,39087,20150,91277,26679,91667,77151,97365,71976,6151,25083,94425,25161,6358,19464,36560,60205,46734,80530,90413,71582,10605,4573,50793,75696,4142,94,47430,97037,12452,50719,93703,92317,98179,56545,13635,54749,55772,41390,84185,65341,90568,8566,40283,11947,4181,22598,3789,57401,7874,68754,64245,13834,81532,45248,44716,43018,7677,98844,8325,66173,90398,11773,20168,62552,35592,17722,90476,80442,16750,60557,78518,51390,80931,64500,23618,78859,67890,24760,71865,26988,3336,28811,20466,1458,86352,11701,73112,22574,48044,10514,11157,50814,12299,76122,8159,84949,11641,65478,76282,4254,32553,8659,40415,40736,70092,6880,57519,30762,23437,93521,95582,34410,87433,40356,36619,27172,8729,53434,94080,91305,34359,9910,57695,19429,69541,92358,54737,107,52505,20220,41853,59256,15869,12962,98921,53892,92398,8873,89592,40109,77779,24441,24548,38231,68151,84184,63291,18312,84807,27431,43834,29498,96210,26227,43342,68259,2223,59505,36815,87077,42863,35869,19938,29866,90696,85565,4391,42027,1767,86753,59549,49910,15950,74330,85610,71181,31813,34641,43897,69463,74265,122,50100,87541,2636,19103,10499,83777,26702,6900,50459,69182,71281,76128,3679,45613,91349,80008,26930,62451,96895,82128,26558,84845,11932,60692,49608,98772,65609,33281,19180,73051,68029,97250,33612,29706,66441,88862,46343,35617,68696,31902,49127,46744,98216,66352,75962,98126,687,23130,77455,60698,15602,85475,41110,34790,20172,72425,97478,87163,69627,22512,54505,57425,2628,3435,77765,76876,82620,46507,1768,61103,11723,22407,39724,30084,52989,41098,8072,91150,40577,12416,59169,57222,71173,48755,85597,9720,32363,83050,22514,42876,10692,15866,22845,73653,5473,81825,89923,34049,89427,97336,15802,50059,74560,92333,21117,88035,23304,99272,72767,55202,18104,834,4967,43767,22128,79102,24115,87726,52628,75631,87913,31640,50689,99790,21459,26636,39516,820,60504,57633,49557,5486,13090,56078,15882,1801,82471,29914,79397,30762,24029,78630,13711,52041,99675,14764,54417,80684,24699,45287,57680,14773,48437,69743,79483,81854,65548,47088,56030,18712,40025,12189,11977,36866,9402,54647,57579,5240,71557,21134,71560,33030,69395,51564,2841,95428,41459,42797,2729,97963,14516,54867,11646,28343,75107,77785,67319,94933,48069,79944,63892,31982,20253,56685,98010,37426,22251,81046,27472,34048,42116,28147,70139,83482,60935,13322,44296,18351,66464,1989,65029,18875,71808,6522,21280,95759,73978,99212,78241,97990,85089,94966,12160,54424,8410,75102,82116,86704,9406,64458,33661,77597,67972,55012,9965,62058,45513,34931,95936,64923,40570,59206,26266,94026,97975,96992,83193,27368,87254,83971,23368,25015,7945,14194,88599,95306,99719,96550,42961,22015,90071,90435,78000,35605,44032,48207,18629,80421,31648,17061,3805,15114,65016,24681,92855,86274,30927,46413,93204,41627,90116,82316,79212,4821,67308,96416,24863,46486,28084,91464,90111,51754,88853,93820,27547,58952,11521,11136,63803,60684,31816,78000,71484,77553,32201,21700,88097,13442,84322,85018,39427,73720,88886,64031,40618]
# pipes = [[2,1,29701],[3,2,97954],[4,3,14600],[5,2,22110],[7,3,14412],[8,6,35742],[9,4,2560],[10,4,30822],[11,5,24812],[12,1,30583],[13,6,70159],[14,10,51992],[15,8,1765],[16,11,68435],[17,13,26992],[18,15,46846],[19,4,73049],[20,4,96887],[21,9,7896],[22,21,87635],[23,15,29266],[24,4,65804],[25,22,38626],[26,1,23842],[28,12,13943],[29,8,95626],[30,23,90303],[31,9,86959],[32,24,56993],[33,22,65478],[34,32,1745],[35,20,87906],[36,26,95020],[37,1,32630],[38,19,92967],[39,38,82313],[40,27,73805],[41,15,48963],[42,1,99891],[43,19,34162],[44,14,38087],[45,22,57011],[46,7,18719],[47,37,77119],[48,8,19496],[49,46,21348],[51,2,47063],[52,45,49034],[53,23,75239],[54,48,41897],[56,35,66432],[58,3,84395],[59,29,4102],[60,27,35845],[62,9,47710],[63,61,11283],[64,26,91789],[65,59,43149],[66,65,68682],[67,39,30519],[69,61,5680],[70,32,43347],[71,66,84590],[72,59,52431],[73,48,19756],[74,19,67496],[75,57,87276],[76,68,40757],[77,28,74982],[78,71,91211],[79,62,29080],[80,46,88674],[81,74,11198],[82,31,25532],[83,79,43847],[84,7,50451],[85,64,65665],[86,56,39159],[87,82,52212],[88,64,83462],[89,29,97270],[91,83,33747],[92,44,89431],[93,68,84122],[95,28,79585],[96,36,28242],[97,45,62811],[98,16,48256],[99,51,74903],[100,80,55040],[101,83,38146],[102,70,77445],[103,43,93110],[105,31,9519],[106,34,34595],[108,101,87776],[110,36,37237],[111,28,90199],[112,62,74313],[113,82,33433],[114,32,61179],[115,80,84718],[117,86,13810],[118,54,28086],[119,59,85376],[120,115,86910],[121,53,51790],[122,28,29483],[123,101,91981],[125,124,9644],[126,7,73538],[128,1,23474],[129,25,60080],[130,81,75982],[131,56,81311],[132,115,74974],[133,109,69225],[134,128,25868],[135,111,19432],[136,118,92403],[137,132,86544],[138,38,22154],[139,88,31854],[140,98,10505],[142,64,60703],[143,66,80450],[144,17,71173],[145,83,91572],[146,22,58132],[147,87,77145],[148,82,48422],[149,43,95248],[150,77,61978],[151,80,76738],[152,56,73090],[153,46,25304],[154,47,79013],[156,66,77969],[157,147,19431],[158,97,38563],[160,60,12524],[161,64,31287],[162,80,76222],[163,143,32137],[164,66,43935],[165,97,24477],[166,159,94280],[167,9,76610],[168,167,89761],[169,27,67260],[170,34,25753],[171,72,92307],[172,170,67935],[173,9,7909],[174,46,67848],[175,93,78956],[176,72,75403],[177,89,21917],[178,100,82491],[179,40,67362],[180,62,5625],[181,84,69468],[182,100,9665],[185,109,95626],[186,174,62761],[188,70,44662],[190,170,40719],[191,2,35724],[193,81,35502],[194,106,40461],[195,76,55697],[196,97,48251],[197,186,31468],[198,38,28268],[199,131,92398],[200,98,93630],[201,74,32983],[202,129,24667],[203,160,98338],[204,157,83564],[205,76,34448],[206,144,18206],[207,54,21036],[208,154,5146],[209,17,88955],[211,110,81634],[212,169,49900],[213,137,49365],[214,2,2743],[216,102,70265],[217,6,7473],[218,158,93597],[219,218,82025],[220,57,93583],[221,115,17511],[222,149,43398],[223,20,39531],[224,128,98386],[225,158,2478],[226,129,54275],[227,84,57178],[228,76,67722],[229,181,7814],[230,144,31563],[231,176,80503],[232,121,7717],[233,174,77445],[235,197,58732],[236,208,40261],[237,102,36161],[239,113,98426],[240,43,5950],[241,234,44069],[242,215,20773],[244,123,93483],[245,21,24114],[246,16,56866],[247,102,83242],[248,207,42104],[249,228,33878],[250,33,70301],[252,43,12339],[253,56,66095],[254,181,63542],[255,62,80572],[256,254,49480],[257,202,28620],[258,175,21567],[259,202,15336],[260,145,288],[261,198,40005],[262,189,68227],[263,232,50808],[264,72,26817],[265,98,22347],[267,112,30440],[268,18,40357],[269,116,96319],[270,167,82039],[271,207,37300],[272,7,84935],[273,137,26433],[274,195,20485],[275,155,3191],[276,213,1272],[277,207,59565],[280,144,99450],[281,59,91363],[282,201,46568],[283,263,40687],[284,274,43775],[285,159,71086],[286,85,17273],[287,87,16543],[289,10,22913],[290,6,75939],[291,36,94387],[292,30,65390],[293,6,90943],[294,201,39029],[295,49,38436],[296,223,55232],[297,24,21950],[298,273,29748],[299,11,79111],[300,239,1199],[301,157,58695],[302,112,95981],[303,14,27236],[304,40,33067],[305,132,9606],[306,167,91230],[307,176,73139],[308,103,41108],[309,260,57700],[310,34,29689],[311,99,85869],[312,247,27571],[313,91,41434],[314,114,39683],[315,24,81282],[316,205,46011],[317,24,18774],[318,289,49499],[319,35,35476],[320,283,37597],[322,257,74891],[325,322,47060],[326,123,7734],[327,273,48920],[328,239,97516],[329,249,15805],[330,260,52246],[331,138,43065],[332,234,95662],[333,301,89573],[334,148,58966],[335,175,22325],[336,288,90484],[337,211,52119],[338,49,62983],[340,59,75987],[341,180,78837],[342,13,24516],[343,60,20307],[344,263,58061],[345,20,9200],[346,205,71813],[347,258,77860],[348,92,46561],[349,155,95175],[350,14,12250],[351,280,92298],[352,117,33641],[353,329,68156],[354,263,36526],[355,270,15815],[356,23,42824],[357,131,24866],[358,237,64697],[359,350,55239],[360,341,9797],[361,136,89544],[362,73,43099],[363,207,24181],[364,218,39615],[365,273,73084],[366,6,864],[367,84,95143],[368,161,43237],[369,200,60149],[370,207,10091],[371,248,66612],[372,213,87257],[373,318,91407],[374,207,19219],[375,155,27372],[376,223,74371],[378,18,10894],[379,118,87212],[380,110,7549],[381,18,27123],[382,71,61836],[384,211,31408],[385,88,10431],[386,329,86168],[387,88,62655],[388,199,73434],[389,321,31213],[390,285,46509],[392,157,90573],[393,186,86501],[394,251,52085],[396,213,65834],[397,14,10629],[398,196,58288],[399,189,401],[400,29,42458],[401,24,39544],[402,247,33084],[403,293,90173],[404,391,70184],[405,359,19688],[406,230,49814],[407,233,58833],[408,294,40730],[409,302,26525],[410,10,91271],[412,16,92204],[413,89,19524],[414,289,39725],[415,66,20872],[416,352,10054],[417,277,31609],[418,310,93549],[419,138,79149],[420,48,84584],[421,257,49358],[422,391,46325],[423,359,68552],[424,232,17270],[425,384,91335],[426,144,83940],[427,90,98409],[428,396,16607],[429,370,78462],[430,33,8117],[431,152,9942],[432,424,69632],[433,30,85543],[434,11,5518],[436,25,81096],[437,292,82378],[438,51,77858],[439,123,85794],[440,248,42626],[441,387,63758],[442,214,10604],[443,215,41259],[444,116,54035],[445,114,32353],[446,14,49598],[447,315,63931],[448,291,54049],[449,111,30348],[450,158,17723],[451,82,14597],[452,83,69424],[453,327,86389],[454,151,81918],[456,189,96242],[457,70,19960],[458,16,28629],[459,237,93187],[460,159,73685],[461,58,68123],[462,235,56918],[463,398,76694],[464,150,63931],[465,74,442],[466,56,52931],[468,425,4396],[469,194,11191],[470,149,43474],[471,363,54711],[472,172,6269],[473,83,33667],[474,318,6513],[475,405,22053],[476,122,40227],[477,402,19227],[478,95,81886],[479,434,67585],[480,325,57839],[482,471,37584],[483,4,30096],[484,111,47372],[485,203,38591],[487,366,6141],[488,377,54203],[489,59,94963],[490,418,23678],[491,307,17408],[492,14,670],[493,437,77484],[494,280,82940],[495,464,13894],[496,450,64933],[497,305,93614],[498,25,53580],[499,132,39323],[500,371,98644],[501,135,41593],[502,231,47066],[503,87,79360],[504,159,9643],[505,275,38103],[507,248,54503],[508,84,49864],[509,454,71371],[510,380,50407],[512,205,67971],[513,467,97712],[514,374,33787],[515,15,28496],[516,60,76410],[517,348,99842],[518,260,89522],[520,113,47359],[521,198,10440],[522,296,69916],[523,63,83891],[524,60,89749],[525,391,54598],[526,175,5846],[527,153,82127],[528,383,38116],[529,117,16063],[530,399,79407],[531,161,22127],[532,230,76530],[533,85,80682],[534,273,89040],[535,495,80592],[536,109,39842],[537,57,15843],[538,250,96631],[539,493,83378],[540,263,69270],[541,35,61883],[542,159,57255],[543,95,27086],[544,215,6833],[545,73,35151],[546,243,9761],[547,452,24856],[548,84,31190],[549,12,6332],[550,405,37086],[551,301,66342],[552,401,63657],[553,276,66852],[554,22,3203],[555,228,37338],[556,176,22010],[557,381,66493],[558,75,66208],[559,364,23276],[560,97,24924],[561,233,4120],[562,107,67561],[563,509,1127],[564,311,86892],[565,147,13695],[566,440,92139],[567,200,90507],[568,252,74287],[570,161,92535],[571,131,27284],[572,155,30566],[573,462,48670],[574,100,90645],[576,276,85273],[578,174,49536],[579,111,21260],[580,398,2781],[581,525,26866],[582,544,32893],[583,444,84344],[584,387,269],[585,468,60842],[586,6,37878],[587,575,473],[588,106,22681],[589,185,27433],[590,84,85785],[594,227,89197],[595,194,69847],[596,33,39904],[597,105,19893],[598,45,56127],[599,435,44844],[600,415,50742],[601,556,36073],[602,558,1862],[603,448,48189],[604,373,89934],[605,530,56263],[606,106,66369],[607,462,29590],[609,343,81848],[610,495,3754],[611,403,68366],[612,356,4017],[613,400,25750],[614,465,9509],[615,435,72301],[617,541,79715],[618,424,63813],[619,81,47434],[620,152,47336],[621,251,98166],[622,299,69146],[623,201,40688],[624,542,15112],[625,504,14571],[626,276,28761],[627,84,40084],[628,538,82437],[630,104,83542],[631,62,42658],[632,195,89255],[633,197,80995],[634,96,34338],[635,511,49439],[636,200,40008],[637,321,82504],[638,9,63149],[640,191,31739],[641,205,86562],[642,315,89392],[643,199,63529],[644,275,27480],[645,616,68136],[647,227,36936],[648,268,35143],[649,36,63126],[650,370,84467],[651,149,31083],[653,26,53705],[654,12,56687],[656,163,51549],[657,167,88308],[658,78,60839],[659,324,13943],[661,584,22839],[662,317,89481],[663,163,94885],[664,288,85440],[666,607,6691],[667,598,31223],[669,217,86682],[670,414,23396],[671,418,69334],[672,670,91478],[673,53,4716],[674,559,16589],[675,207,79062],[676,111,8928],[677,543,143],[678,229,16663],[679,49,81576],[680,661,70081],[681,667,24489],[682,553,2672],[683,140,69484],[684,102,59446],[685,363,25059],[686,6,62764],[687,283,1382],[688,397,52696],[689,230,58658],[690,176,90750],[691,608,1352],[692,159,36270],[693,450,84846],[694,344,81318],[695,116,90234],[696,404,9678],[697,519,29049],[698,571,94453],[699,644,96083],[700,86,46202],[701,613,99800],[703,277,38585],[704,73,86680],[705,197,43707],[706,25,80290],[707,117,46972],[708,600,85249],[709,90,76060],[710,216,49175],[711,254,96266],[712,266,90950],[713,225,95130],[714,295,35117],[715,16,94893],[716,331,13503],[717,659,11049],[719,590,90696],[720,491,72175],[721,316,86679],[722,12,39992],[724,177,11080],[725,568,26948],[726,322,27388],[727,76,27145],[728,266,75008],[729,527,12535],[730,214,12371],[731,373,39834],[732,454,75407],[734,42,70914],[736,101,54875],[737,4,20308],[738,565,85175],[739,123,60506],[740,631,27167],[741,468,61452],[743,442,35717],[744,693,29093],[746,662,31228],[747,617,19151],[748,246,97610],[749,674,21279],[750,387,46233],[751,546,39260],[752,310,69736],[753,566,9630],[754,257,31508],[755,668,84160],[756,35,44004],[757,658,83541],[759,634,3962],[760,347,75632],[761,49,79612],[762,192,24690],[763,148,4250],[764,244,97829],[765,421,1066],[766,580,7403],[767,216,37692],[769,711,22687],[770,404,88094],[771,671,56506],[772,755,58255],[773,145,52921],[774,609,83960],[775,680,96586],[776,703,99713],[777,599,52375],[779,144,44736],[780,632,89757],[781,36,5731],[782,351,30366],[783,771,57131],[784,310,96517],[785,535,99809],[786,619,68589],[787,396,71272],[788,431,57985],[789,129,27813],[790,369,70912],[791,281,92982],[792,98,86659],[793,546,48369],[795,97,67407],[796,477,37718],[797,66,87249],[799,783,12823],[801,437,3536],[802,11,26811],[804,564,91098],[805,482,27517],[806,168,52001],[807,265,21530],[808,53,76266],[809,361,80924],[810,182,37370],[811,154,37112],[812,556,41290],[814,789,74920],[815,800,56751],[816,729,91805],[817,107,94232],[818,154,80731],[819,268,14737],[820,375,4887],[821,350,44481],[822,72,89776],[823,20,7873],[824,132,21003],[825,528,54949],[826,315,47200],[827,111,74453],[828,512,94245],[829,577,59313],[830,603,14129],[831,690,26452],[832,413,54620],[833,52,58054],[834,711,49276],[835,469,56280],[836,296,9619],[837,371,27141],[838,707,78638],[839,762,2177],[840,136,98498],[841,175,16939],[842,529,10712],[843,320,905],[844,585,66020],[845,450,80295],[846,381,90691],[847,552,28690],[848,619,83992],[849,824,55730],[850,363,43803],[852,52,7169],[853,444,56336],[854,476,56690],[855,622,85799],[857,774,96678],[858,424,49449],[859,402,3931],[861,59,68039],[862,283,69225],[863,668,97126],[865,837,35760],[866,562,10999],[867,185,55837],[868,404,30814],[869,287,24810],[870,208,77452],[871,478,32911],[872,249,33444],[873,245,42964],[874,426,98778],[875,697,70505],[876,128,31069],[877,23,62263],[878,589,55131],[879,355,41219],[880,458,2034],[881,714,40486],[882,854,62129],[883,558,53708],[884,847,7965],[885,816,91508],[886,95,47778],[887,267,10275],[889,206,26758],[890,593,30540],[891,346,26922],[893,568,94898],[894,695,17579],[895,274,37331],[896,454,60062],[897,258,78827],[898,203,1875],[899,665,47366],[900,442,93689],[901,200,13272],[902,792,78544],[903,266,40584],[904,792,47387],[905,659,3977],[906,298,4860],[907,543,6421],[909,40,98118],[910,286,76139],[911,471,29719],[912,873,2364],[913,844,78385],[914,473,29835],[915,89,15642],[916,589,32405],[917,3,41852],[918,517,63462],[920,377,28193],[921,6,27665],[922,32,93519],[923,422,14364],[925,229,37296],[926,804,10557],[927,83,65851],[928,108,99995],[929,680,12483],[930,191,37164],[931,279,80905],[932,409,14026],[933,61,48110],[934,540,10928],[937,446,29444],[938,360,7819],[939,471,40358],[940,357,82004],[941,819,38581],[942,392,47984],[943,652,30315],[944,605,38241],[945,397,29448],[946,915,19271],[947,654,31385],[948,715,63095],[949,44,98741],[950,108,54784],[951,179,54911],[953,849,10447],[954,339,60110],[955,475,72751],[956,511,27716],[957,510,45488],[958,501,75688],[959,167,10869],[960,786,81730],[961,925,29497],[962,882,37332],[963,756,42589],[964,695,69758],[965,376,53733],[967,277,40479],[968,637,55910],[969,676,52573],[970,500,35584],[971,866,76667],[972,335,78272],[973,864,7167],[975,656,95465],[976,763,7587],[977,946,9897],[978,572,15324],[979,791,68963],[980,716,19826],[981,756,41327],[982,462,70389],[983,694,19291],[984,759,26277],[985,269,96491],[986,171,72450],[987,335,33769],[988,28,39546],[989,598,66509],[990,815,58010],[994,246,98920],[995,952,92564],[996,47,19089],[997,815,52660],[998,385,35303],[999,136,34329],[1000,648,73110],[1001,243,253],[1002,729,4232],[1003,918,54115],[1004,636,45857],[1005,254,49621],[1006,915,61092],[1007,664,36241],[1008,114,349],[1009,712,73965],[1010,387,63796],[1011,236,65490],[1012,299,40970],[1014,1007,61676],[1015,597,52376],[1017,549,34623],[1019,1016,25540],[1020,136,52380],[1021,791,24927],[1022,950,98601],[1023,226,1562],[1024,614,9795],[1025,335,30621],[1026,593,56947],[1027,241,25080],[1028,809,15470],[1029,650,35961],[1030,893,69249],[1031,763,46677],[1032,372,78156],[1033,302,99773],[1034,883,85578],[1035,174,10140],[1036,243,41563],[1039,672,15448],[1041,477,40427],[1042,440,94716],[1044,583,53646],[1045,859,43109],[1046,901,19523],[1047,60,71221],[1048,187,43065],[1049,564,10769],[1050,956,54814],[1051,65,81441],[1052,206,18660],[1053,862,27312],[1054,515,6942],[1055,99,37047],[1056,616,41243],[1057,284,33037],[1058,131,17012],[1060,132,60120],[1061,424,65010],[1062,710,6555],[1063,45,30469],[1064,1039,6630],[1065,852,76908],[1067,7,24572],[1068,558,80311],[1069,598,71619],[1070,387,76514],[1072,1,32379],[1073,590,16104],[1074,107,75199],[1075,712,19319],[1076,907,30853],[1078,913,31911],[1079,218,59986],[1080,1051,55941],[1081,54,61929],[1082,367,91902],[1083,924,43746],[1084,906,32858],[1085,183,34173],[1086,375,76226],[1087,145,49924],[1088,714,79879],[1089,170,54271],[1090,726,74548],[1091,798,77665],[1092,60,20506],[1093,330,92122],[1094,1017,5796],[1095,128,27394],[1098,878,52275],[1100,1048,47647],[1101,196,64291],[1102,29,57258],[1103,771,92395],[1104,292,70900],[1105,168,54614],[1106,340,13278],[1107,208,8970],[1108,727,15460],[1109,186,88394],[1110,872,17516],[1112,368,77577],[1113,633,97694],[1114,794,38711],[1115,524,69183],[1116,963,84105],[1117,564,47890],[1118,635,33266],[1119,164,77356],[1120,228,13586],[1121,32,8476],[1122,23,97217],[1123,431,34665],[1124,170,93412],[1125,703,87334],[1126,694,81732],[1127,1068,44270],[1129,538,29571],[1130,227,81957],[1131,198,52023],[1132,1072,16762],[1134,354,99400],[1135,699,46641],[1136,860,68542],[1137,903,34620],[1138,953,21122],[1139,339,99965],[1140,103,63619],[1141,1044,12550],[1142,470,65787],[1143,9,31441],[1144,897,43870],[1145,14,986],[1146,31,54773],[1147,958,70162],[1148,32,57184],[1149,1067,90143],[1150,11,91283],[1151,1131,13805],[1153,1005,7617],[1154,565,87674],[1155,298,23108],[1156,108,98725],[1157,45,50697],[1158,345,48048],[1159,1089,60324],[1160,912,93321],[1161,374,64541],[1162,959,38489],[1163,1007,41410],[1164,623,5394],[1166,989,12787],[1167,723,63135],[1168,201,23866],[1169,883,18205],[1170,198,90217],[1171,197,82553],[1172,1157,7973],[1173,620,76527],[1174,636,56597],[1175,971,93165],[1176,1058,47951],[1177,925,83025],[1178,1150,41159],[1179,41,1078],[1180,799,18057],[1181,600,23302],[1182,247,30510],[1183,474,57167],[1184,676,7280],[1185,254,95340],[1186,167,42062],[1187,422,44527],[1188,780,53033],[1189,353,53881],[1190,155,96905],[1191,132,20734],[1192,192,35140],[1194,184,61905],[1195,973,58618],[1196,32,83850],[1197,30,50460],[1198,345,64698],[1199,18,94838],[1200,684,68193],[1201,390,8651],[1202,258,16873],[1203,1139,70156],[1204,349,36248],[1205,290,66944],[1206,151,75048],[1207,1165,76526],[1208,747,39739],[1209,28,22937],[1210,1005,27970],[1212,1142,75522],[1213,297,43061],[1214,240,14018],[1215,602,61861],[1216,918,96330],[1217,588,44728],[1218,334,23323],[1219,754,5042],[1220,112,54984],[1221,903,90527],[1224,718,9033],[1225,536,51629],[1226,570,34668],[1227,956,39257],[1228,884,52639],[1229,782,89207],[1230,35,5296],[1231,438,80879],[1233,440,68052],[1234,883,63213],[1235,179,96141],[1236,250,73533],[1237,358,83354],[1238,370,68408],[1239,878,58233],[1240,911,75054],[1241,812,64691],[1242,914,50680],[1243,770,50604],[1244,382,93458],[1245,575,81450],[1246,266,68534],[1247,527,48772],[1248,887,73895],[1249,617,24064],[1251,1245,66677],[1252,352,14790],[1253,647,85873],[1254,572,48089],[1256,186,28579],[1257,23,28457],[1259,612,11103],[1260,962,32647],[1262,195,36388],[1263,829,6800],[1264,891,63034],[1265,870,12964],[1266,1047,76341],[1267,587,90963],[1268,1175,58228],[1269,185,32283],[1270,687,10575],[1271,495,2904],[1272,492,50018],[1274,365,82885],[1275,153,99206],[1276,1103,63921],[1277,442,66754],[1278,1130,66462],[1279,150,96234],[1280,1150,26666],[1281,265,57581],[1282,1027,19629],[1283,260,30112],[1284,1186,10459],[1285,508,44090],[1286,831,16935],[1287,490,60197],[1288,187,34489],[1289,1211,49901],[1290,562,21963],[1292,842,71198],[1293,151,54976],[1294,369,55682],[1295,1031,59409],[1296,1285,89738],[1297,175,84503],[1298,741,73013],[1299,709,71213],[1300,163,69017],[1301,377,97307],[1302,644,47722],[1303,48,28534],[1304,802,93100],[1305,183,2428],[1306,143,81543],[1307,744,79377],[1308,1214,69952],[1309,526,98676],[1310,928,23610],[1311,485,32011],[1312,108,16640],[1313,345,59724],[1314,1180,84764],[1315,429,91811],[1316,687,93029],[1317,949,45024],[1318,1021,27980],[1319,555,66426],[1320,1032,62081],[1321,1229,10487],[1322,1159,79489],[1323,748,39178],[1324,461,74206],[1325,726,63863],[1326,909,35424],[1328,635,7373],[1330,583,42792],[1331,856,56747],[1332,1150,66011],[1333,36,82499],[1334,346,69398],[1336,858,65762],[1337,561,60177],[1338,59,45990],[1339,282,48320],[1340,599,97713],[1341,618,88764],[1342,1107,58341],[1343,262,29797],[1345,92,56620],[1346,1312,82414],[1347,70,10017],[1349,1290,64396],[1351,1276,23275],[1352,48,33954],[1353,476,34075],[1354,1125,78417],[1355,1173,62803],[1356,1088,43520],[1357,146,52854],[1358,107,34794],[1359,918,51614],[1360,1353,89488],[1361,908,38986],[1362,38,66757],[1363,268,68867],[1364,1148,44073],[1365,532,72633],[1366,1136,89196],[1367,476,85687],[1368,602,46477],[1369,559,29730],[1370,916,76393],[1371,19,90839],[1372,309,41194],[1373,1121,24041],[1374,681,43968],[1375,613,97413],[1376,1362,65842],[1377,1022,43786],[1378,1174,18594],[1379,58,74456],[1380,1226,41188],[1381,501,84327],[1382,1097,4589],[1383,833,36410],[1384,598,35702],[1385,767,34017],[1386,444,89645],[1388,542,80738],[1389,90,2826],[1390,213,85347],[1391,734,29166],[1393,1072,96409],[1394,553,8271],[1395,179,2230],[1396,1261,73418],[1399,246,8017],[1400,1092,65801],[1401,678,23508],[1402,629,82835],[1404,200,32942],[1405,40,8439],[1406,58,33343],[1407,1090,17269],[1408,740,976],[1409,948,83076],[1410,1327,81532],[1412,75,25421],[1414,1155,43856],[1415,899,40062],[1416,367,96287],[1417,683,2662],[1418,313,84453],[1419,389,97720],[1420,752,29772],[1421,832,74634],[1422,1160,33370],[1425,903,2804],[1426,550,75637],[1427,1040,87343],[1428,1366,14820],[1429,415,38785],[1430,1278,13487],[1431,567,48949],[1432,807,47795],[1433,1353,93754],[1434,870,28897],[1435,1155,12717],[1436,1054,93012],[1437,1258,70247],[1438,451,68809],[1439,402,16171],[1440,99,88933],[1441,1178,87030],[1442,559,4278],[1443,352,25616],[1444,42,61937],[1445,1367,56804],[1446,598,80587],[1448,52,38880],[1449,419,57168],[1450,1262,70998],[1451,800,54836],[1453,1206,61736],[1454,597,84984],[1455,1010,52546],[1456,1290,21483],[1457,1088,5940],[1458,169,99165],[1459,434,29613],[1460,957,39580],[1461,1154,27998],[1462,914,61685],[1463,638,74288],[1464,462,20328],[1465,146,2969],[1466,55,24516],[1467,1355,23636],[1469,260,5032],[1472,338,79476],[1474,794,31960],[1475,333,79966],[1476,941,34278],[1478,704,43050],[1479,114,61231],[1480,801,98296],[1481,718,2047],[1482,926,32449],[1483,443,67181],[1484,1413,91672],[1485,1426,32972],[1486,278,15385],[1487,1401,6861],[1488,677,90408],[1489,1096,21566],[1490,1452,13504],[1491,331,29763],[1492,6,76860],[1493,871,30081],[1494,172,19984],[1495,1094,57495],[1496,1184,63029],[1497,1354,12928],[1498,835,10344],[1499,895,81466],[1500,1349,70236],[1501,305,20088],[1502,412,53171],[1503,659,78764],[1504,655,25927],[1505,147,38445],[1506,1015,69892],[1507,1128,65963],[1508,1355,70080],[1509,173,88174],[1510,758,51179],[1511,1008,27401],[1513,878,94695],[1514,406,31807],[1515,666,43755],[1516,939,81974],[1517,1016,8833],[1518,470,30755],[1519,50,41369],[1520,275,37836],[1521,308,42015],[1522,1412,15351],[1523,285,57167],[1524,302,49610],[1525,1,69359],[1526,1374,46559],[1527,1251,61083],[1528,927,76517],[1529,931,1797],[1530,863,2776],[1531,917,16816],[1532,335,78842],[1533,34,63889],[1534,1219,39489],[1535,722,17797],[1536,1491,68888],[1537,682,52521],[1539,445,44975],[1541,278,57900],[1542,599,18322],[1543,651,9503],[1544,258,62891],[1545,928,14523],[1546,94,71006],[1547,731,77685],[1548,236,28309],[1550,1539,17487],[1551,868,17325],[1552,182,60064],[1553,953,76168],[1554,482,26176],[1555,1554,41161],[1556,656,22451],[1557,194,83440],[1558,48,79620],[1559,478,82745],[1560,744,75405],[1561,1542,32021],[1562,193,82402],[1563,1415,10703],[1564,607,66823],[1565,865,15584],[1566,500,22983],[1567,637,41866],[1568,1133,14499],[1569,1374,48085],[1570,1099,29295],[1571,1061,58459],[1572,1427,50462],[1575,148,90079],[1576,521,5616],[1577,12,68623],[1578,984,19385],[1579,875,28175],[1580,1231,46757],[1582,701,82062],[1583,1072,55296],[1584,76,43894],[1585,1254,11593],[1586,978,36258],[1587,1549,33871],[1588,1145,30375],[1590,1458,81330],[1591,405,91717],[1593,1196,67665],[1594,1516,73678],[1595,1376,52548],[1596,511,45447],[1597,648,93984],[1598,36,63235],[1599,1141,70106],[1600,1538,93658],[1601,176,14318],[1602,683,96433],[1603,6,3566],[1604,1420,99651],[1605,356,1541],[1607,200,76994],[1608,414,42643],[1609,209,31702],[1610,470,12815],[1611,123,25495],[1612,1477,25216],[1613,1176,57217],[1614,314,42236],[1615,925,75626],[1616,356,36801],[1617,823,26946],[1619,1341,29413],[1620,595,56242],[1621,328,79397],[1622,1383,53379],[1623,233,23336],[1624,1541,33776],[1625,167,6950],[1626,638,696],[1627,164,48174],[1628,875,59196],[1629,463,65835],[1631,1003,61351],[1632,392,89541],[1633,522,68950],[1634,180,32748],[1635,1012,81843],[1637,719,4490],[1639,1160,72524],[1640,409,70036],[1642,627,52589],[1643,740,90447],[1644,1208,47774],[1645,1320,54053],[1646,1106,69528],[1647,1150,40525],[1648,1597,18],[1649,380,46144],[1650,90,68408],[1651,1529,47294],[1652,1079,62853],[1653,1150,54067],[1654,1164,67504],[1655,351,20105],[1656,1452,78587],[1657,1396,90214],[1658,278,59150],[1659,1414,80567],[1660,1021,32666],[1661,696,96834],[1662,413,38056],[1664,644,1247],[1665,432,9235],[1666,169,43365],[1668,776,98200],[1669,1206,22963],[1670,1130,56242],[1671,1171,6574],[1674,1363,10204],[1675,1195,84699],[1676,234,93506],[1677,235,70722],[1679,1394,45292],[1680,1529,51633],[1681,1135,47195],[1682,542,35535],[1683,324,33859],[1684,233,52359],[1685,1378,39501],[1686,159,61649],[1687,422,17006],[1688,1635,32544],[1689,623,56884],[1691,803,83],[1692,758,71204],[1693,878,76748],[1694,1604,41796],[1695,1471,73553],[1696,1344,29484],[1697,706,49328],[1698,1061,77002],[1700,701,38657],[1701,961,11891],[1702,1403,76762],[1703,1473,18829],[1704,858,34472],[1705,1107,83820],[1707,103,21744],[1708,1373,87699],[1709,1254,53381],[1710,224,78040],[1711,1026,91283],[1712,1510,81904],[1713,1218,71910],[1714,881,74307],[1715,450,17008],[1716,524,20758],[1717,1077,18922],[1718,426,6142],[1719,1673,24708],[1720,1672,22673],[1721,1061,51947],[1722,1387,85400],[1723,739,63190],[1724,226,41854],[1726,210,64512],[1727,347,24203],[1728,1040,94220],[1729,689,9995],[1730,1560,19567],[1731,1320,96123],[1732,703,82424],[1733,755,78220],[1734,1648,26714],[1735,767,36096],[1737,1009,25871],[1738,1572,5145],[1739,738,81426],[1740,70,90421],[1741,961,16556],[1742,1421,3976],[1743,138,81131],[1744,969,31427],[1745,1547,89625],[1746,1666,38553],[1747,757,37813],[1748,301,12035],[1750,317,79609],[1751,978,3946],[1752,378,88630],[1753,1307,5052],[1754,42,65741],[1755,1448,71008],[1756,1674,20456],[1757,1362,7224],[1758,782,85199],[1759,942,78199],[1761,180,29946],[1762,1553,8212],[1763,868,24273],[1764,162,41003],[1765,1,90961],[1766,1405,55774],[1767,762,95592],[1768,1726,10598],[1769,570,80313],[1771,324,36741],[1772,362,25140],[1773,445,89332],[1774,1533,48982],[1775,1385,39129],[1776,1279,86112],[1777,1032,27089],[1779,1454,34766],[1780,724,87661],[1781,19,61973],[1782,444,99469],[1783,826,84557],[1784,1298,16913],[1785,303,59548],[1786,1279,32759],[1787,591,47664],[1788,1089,38234],[1789,1011,39254],[1791,769,43711],[1792,1778,10440],[1793,1156,28940],[1794,974,75378],[1795,1210,33981],[1796,72,52449],[1798,1718,20993],[1799,663,17307],[1800,1565,50282],[1801,1625,21487],[1802,211,84244],[1804,876,57088],[1805,1474,64278],[1806,872,52006],[1807,1485,83288],[1808,1356,56168],[1809,787,17644],[1810,1674,4892],[1811,1424,98678],[1812,1181,6610],[1813,1094,42801],[1814,303,57027],[1815,945,96016],[1816,1540,90279],[1817,87,26996],[1818,566,89527],[1820,1268,61447],[1821,388,75056],[1822,1454,13665],[1823,96,88854],[1824,93,26858],[1826,1674,7684],[1827,641,26221],[1828,179,97599],[1829,535,23074],[1830,521,2301],[1832,938,12625],[1834,103,84195],[1835,1775,16484],[1836,1089,31998],[1837,488,43211],[1838,996,18504],[1839,1261,92238],[1840,1580,99762],[1841,216,39921],[1842,1614,87827],[1843,312,27048],[1844,458,94118],[1845,238,32505],[1846,417,67817],[1847,196,87086],[1848,481,87862],[1849,957,72420],[1850,924,21972],[1851,945,44397],[1852,26,15100],[1854,112,80135],[1856,877,98938],[1857,1826,12135],[1858,762,78788],[1859,1071,14583],[1861,1576,58950],[1862,148,81901],[1863,920,34735],[1864,500,14383],[1865,486,53463],[1866,1813,4709],[1867,1580,39726],[1868,534,7736],[1869,1416,54710],[1870,1595,62001],[1871,171,52530],[1872,988,29773],[1873,1099,46458],[1874,1744,65858],[1875,569,55721],[1876,570,95922],[1877,990,98942],[1878,1846,22826],[1879,983,74666],[1880,1413,17988],[1881,1046,2412],[1882,1508,68224],[1883,971,36791],[1884,1249,83889],[1885,873,79657],[1886,759,93207],[1888,63,4417],[1889,75,90303],[1892,672,40377],[1893,1043,80449],[1894,699,84171],[1896,730,58237],[1898,780,17909],[1900,1691,47016],[1901,364,29188],[1902,1134,1388],[1903,1379,25838],[1904,1746,44877],[1905,794,7078],[1906,594,56007],[1907,1723,60140],[1908,1650,47677],[1909,137,79481],[1912,289,27911],[1913,1312,3582],[1914,1342,49154],[1915,1139,25802],[1916,1271,87652],[1918,1361,20539],[1920,1247,73281],[1921,562,27815],[1922,1536,36274],[1923,1510,3428],[1924,1356,61660],[1925,1638,26168],[1926,1284,32181],[1927,617,29200],[1928,163,73573],[1929,1563,4238],[1930,1459,86665],[1931,549,30738],[1932,1107,70415],[1933,1211,48121],[1934,1178,3994],[1936,1716,55675],[1937,1140,80126],[1938,627,71618],[1939,1458,93084],[1940,1344,82402],[1941,481,1261],[1942,620,45588],[1943,936,85286],[1944,1916,55626],[1945,1743,31588],[1946,866,88852],[1947,1935,95125],[1948,897,32841],[1949,1340,50949],[1950,934,95611],[1951,1669,31789],[1952,288,71059],[1953,330,17302],[1954,849,12746],[1955,1422,37508],[1956,1017,4480],[1957,823,58719],[1958,1916,65512],[1959,1337,28926],[1960,1697,45187],[1961,645,79442],[1962,1409,78056],[1963,1590,96112],[1964,161,97799],[1965,1774,76020],[1966,737,75005],[1967,1516,68113],[1968,731,1316],[1969,1951,81828],[1970,32,52498],[1971,829,6600],[1972,587,3639],[1973,1211,34628],[1974,454,33525],[1975,1944,31623],[1976,1446,43117],[1977,1654,5354],[1978,1139,28086],[1979,653,7771],[1980,1309,71940],[1982,282,88131],[1983,1479,35519],[1984,1076,64864],[1985,1212,25739],[1986,1913,13813],[1987,1960,42627],[1989,1165,7472],[1990,863,33477],[1991,1956,40231],[1992,889,20056],[1993,18,29013],[1994,1562,87466],[1995,583,55010],[1997,856,77515],[1998,382,87168],[1999,1578,13624],[2001,1108,48441],[2002,1132,79724],[2003,1284,67808],[2004,641,8894],[2005,1284,52411],[2006,1278,43981],[2008,1752,29244],[2009,65,81523],[2010,1602,68647],[2011,1209,65029],[2012,1452,71790],[2013,1628,37263],[2014,633,9544],[2015,1076,38297],[2016,813,49450],[2017,239,63983],[2018,1671,34481],[2019,443,11346],[2021,1282,18896],[2022,836,55087],[2023,1627,78152],[2024,1580,11747],[2025,1797,87033],[2026,882,79129],[2027,201,32580],[2028,560,20877],[2029,1103,71548],[2030,1863,77127],[2031,1830,46879],[2032,751,62894],[2033,777,51242],[2034,1858,73690],[2035,1697,5783],[2036,1812,13180],[2037,846,21033],[2039,575,39011],[2041,1899,84186],[2042,1706,75012],[2043,165,3442],[2045,1961,10443],[2046,596,71950],[2047,1003,94219],[2048,601,31053],[2049,63,89948],[2050,944,7438],[2051,639,56694],[2052,685,20260],[2053,1856,68744],[2054,657,98335],[2055,2009,84753],[2056,1367,77875],[2057,1643,89787],[2058,792,5089],[2059,1569,1950],[2060,169,56565],[2061,82,16928],[2062,1573,470],[2063,542,6916],[2064,1943,66462],[2067,730,55387],[2069,1250,64247],[2070,1488,938],[2071,94,90514],[2072,605,29025],[2073,1082,26394],[2074,1202,94264],[2075,1417,60967],[2076,1045,27009],[2077,1067,82089],[2078,1145,46433],[2079,796,24708],[2080,135,3945],[2081,1464,56941],[2082,1176,64853],[2083,1200,94888],[2084,32,70744],[2086,343,65223],[2087,27,47010],[2088,870,21963],[2089,1659,45436],[2090,44,64359],[2091,2052,32531],[2092,1206,19091],[2093,1849,87745],[2094,1422,79986],[2095,1813,71054],[2096,1447,78772],[2097,1457,1784],[2099,607,305],[2101,1590,47210],[2103,1111,38168],[2104,2022,35470],[2105,1629,19927],[2106,1536,34071],[2108,566,92823],[2109,677,25875],[2110,1817,33904],[2111,571,35240],[2112,741,96757],[2113,1486,78613],[2114,2053,38392],[2115,1724,99260],[2116,1845,66895],[2117,1635,21105],[2118,891,34050],[2119,708,24250],[2120,1331,82794],[2121,956,84026],[2122,1486,21995],[2123,769,56914],[2124,694,80412],[2125,1065,645],[2126,79,96687],[2127,1504,59408],[2128,458,43050],[2130,1594,13307],[2131,1535,99373],[2132,1321,97868],[2133,1759,83226],[2134,1561,37970],[2135,1453,56519],[2136,1275,47035],[2137,1480,44677],[2138,510,75986],[2139,1430,81904],[2141,395,98604],[2143,1812,64733],[2144,46,99363],[2146,1207,17310],[2147,1927,72480],[2148,675,20605],[2149,88,45907],[2150,670,84406],[2151,484,70180],[2152,675,36255],[2153,310,19553],[2154,1068,31005],[2155,703,10580],[2156,1449,8386],[2157,1349,20511],[2158,79,252],[2159,1755,97365],[2160,1920,37848],[2161,601,31738],[2163,1953,37118],[2165,850,54036],[2166,1327,61832],[2167,919,87251],[2168,505,73976],[2169,1618,17476],[2170,544,72201],[2171,617,39041],[2172,1507,56703],[2173,1425,29231],[2174,5,99946],[2175,1234,15449],[2176,1746,84835],[2177,215,12768],[2178,1079,41996],[2179,1483,99892],[2181,1888,42817],[2182,1142,27106],[2183,576,97036],[2184,1318,25675],[2185,935,65725],[2186,1111,52746],[2187,773,77664],[2188,671,4037],[2189,731,32401],[2190,343,28439],[2191,1629,85459],[2192,1663,22616],[2194,723,64190],[2196,1131,27207],[2198,1401,57134],[2199,1992,51555],[2200,2055,54755],[2201,1453,28041],[2203,97,77491],[2204,692,4818],[2205,1767,91743],[2206,1834,65580],[2207,750,87864],[2208,1599,96553],[2209,1338,41079],[2210,1906,15442],[2211,1036,91071],[2212,2179,31891],[2213,116,38651],[2214,634,5407],[2215,790,73219],[2216,1627,46632],[2217,1233,92692],[2218,1389,11892],[2219,1091,310],[2220,2042,4138],[2221,1896,57141],[2222,1943,48854],[2223,948,52713],[2224,326,43624],[2225,1604,32118],[2226,306,47822],[2227,416,29408],[2229,1591,78049],[2230,1331,54044],[2231,302,50714],[2232,394,27780],[2233,2167,35092],[2234,1760,18359],[2235,593,16597],[2236,1405,21589],[2239,777,17972],[2240,2213,50321],[2241,471,33198],[2242,1286,88861],[2243,300,86472],[2245,2132,35193],[2246,2076,68476],[2247,1643,38085],[2248,887,81822],[2250,419,14775],[2251,1034,13565],[2252,791,20932],[2253,1836,67989],[2254,1009,51834],[2255,1331,83849],[2256,2150,52152],[2257,1518,92443],[2258,993,59409],[2259,1332,37819],[2260,133,95590],[2261,1134,7218],[2262,215,2803],[2263,714,85861],[2264,904,1976],[2265,929,59940],[2267,724,19013],[2268,2127,58898],[2269,881,43078],[2270,969,9528],[2271,1779,5966],[2272,789,28826],[2273,1838,77664],[2274,1887,20241],[2275,1793,24149],[2277,444,39736],[2279,91,72484],[2280,908,25158],[2281,729,37842],[2282,55,79670],[2283,873,91619],[2284,2017,67223],[2285,931,27948],[2286,805,44287],[2287,918,78596],[2288,287,74695],[2289,588,33923],[2290,1924,43447],[2291,1871,19670],[2292,1742,23092],[2293,830,36399],[2294,1021,90140],[2295,20,49664],[2296,2093,69054],[2297,1551,34537],[2298,2248,61936],[2300,709,31920],[2301,485,32202],[2302,1828,83649],[2303,452,49966],[2304,1569,42409],[2305,898,66026],[2306,2053,65774],[2307,1640,50699],[2308,72,18222],[2309,1298,13183],[2310,874,52600],[2311,611,78138],[2312,1080,47769],[2313,1554,32481],[2314,1796,45496],[2315,317,50778],[2316,415,72322],[2317,406,83473],[2319,925,92462],[2320,557,59251],[2321,1801,10492],[2322,143,36709],[2324,263,87646],[2325,1076,35982],[2326,11,15974],[2327,1799,27301],[2328,303,67054],[2329,1933,42549],[2330,1737,31216],[2331,1744,86577],[2332,368,7191],[2333,1154,98932],[2335,4,28736],[2336,1782,50871],[2337,1319,84767],[2338,490,13772],[2339,220,15341],[2340,1209,7096],[2341,1234,9301],[2342,1914,54452],[2343,953,19943],[2344,1945,35404],[2345,692,93408],[2346,256,96687],[2347,2272,26920],[2348,819,4325],[2349,2229,458],[2350,2294,71856],[2351,1994,39277],[2352,628,57936],[2353,1743,3756],[2354,322,52759],[2355,1995,1101],[2357,1125,81963],[2358,595,38159],[2359,901,69176],[2360,426,80204],[2361,1110,3807],[2362,741,86148],[2363,175,33573],[2364,909,40522],[2365,1771,914],[2366,2297,55208],[2367,1890,56235],[2368,1804,19519],[2369,850,68637],[2370,1070,29015],[2371,855,63642],[2372,79,4870],[2373,2133,43005],[2374,204,57183],[2376,739,16655],[2377,672,1151],[2378,533,98617],[2379,911,68820],[2380,152,89028],[2381,1556,64362],[2382,439,79083],[2383,2140,49268],[2384,1166,13599],[2385,22,94755],[2386,1692,34636],[2387,1613,1253],[2388,133,64777],[2389,2217,48738],[2390,185,19364],[2391,1078,42950],[2392,597,99864],[2393,1506,60237],[2394,1021,71844],[2395,1206,44646],[2396,379,60834],[2397,199,27747],[2398,1478,36858],[2399,1679,89232],[2400,88,48528],[2401,2050,46105],[2403,1600,69274],[2404,971,21419],[2405,1000,97446],[2406,1624,26158],[2407,82,98831],[2408,1512,25186],[2409,2224,37847],[2411,381,14252],[2412,1061,9908],[2413,41,63348],[2414,1130,74462],[2415,2257,97605],[2416,1011,45431],[2417,526,61727],[2419,630,2855],[2420,460,18714],[2422,2276,5996],[2423,1724,38924],[2425,1447,90106],[2426,1041,73172],[2427,1695,73883],[2428,1812,60478],[2429,1107,55698],[2430,1340,69150],[2431,1725,1998],[2432,1452,44897],[2433,965,86420],[2434,1594,49807],[2435,1228,33124],[2436,189,36504],[2437,2231,27023],[2438,1265,41818],[2439,227,22051],[2440,491,25706],[2441,1458,24269],[2442,1394,91198],[2443,515,60214],[2444,2285,21829],[2445,1469,55308],[2446,2118,47692],[2447,332,91416],[2448,2420,3001],[2450,1655,12678],[2451,2279,8405],[2452,779,41602],[2453,1069,65093],[2454,403,78828],[2455,560,54963],[2457,1472,95585],[2458,1200,27071],[2460,790,49396],[2461,205,26025],[2462,876,29232],[2463,1000,2969],[2466,170,90311],[2467,819,25840],[2468,175,20096],[2469,926,14562],[2470,659,52428],[2471,1436,41523],[2472,2316,41998],[2473,458,77705],[2474,606,57376],[2476,1041,58445],[2477,1560,97281],[2478,2128,21227],[2479,1203,77451],[2480,462,25219],[2481,646,8349],[2482,2274,44492],[2483,375,60246],[2484,406,26080],[2485,1205,67669],[2486,503,26123],[2487,1353,19269],[2488,556,14243],[2489,1425,38485],[2490,1802,36245],[2491,2464,73756],[2492,113,55653],[2493,1303,82183],[2494,1974,72465],[2495,288,70579],[2496,814,46922],[2497,2277,97279],[2498,1964,9719],[2499,1200,29885],[2500,600,36273],[2501,1824,76022],[2502,1705,47172],[2503,1162,39632],[2504,1963,12157],[2505,800,34134],[2506,2459,41601],[2507,1569,69567],[2508,397,17907],[2509,1873,96135],[2510,1772,18118],[2511,1686,39480],[2512,55,56888],[2513,2226,21517],[2514,1847,32961],[2515,2200,63196],[2516,139,50629],[2517,592,73465],[2518,1090,88768],[2519,245,61],[2520,889,46999],[2521,681,3293],[2522,266,40421],[2523,99,17187],[2524,1416,81262],[2525,2339,47652],[2526,2411,59280],[2527,2176,45584],[2528,768,90991],[2529,277,92706],[2530,596,79260],[2531,671,29515],[2532,1126,96099],[2533,223,47097],[2534,525,95300],[2535,1306,70047],[2536,2507,33223],[2537,548,86813],[2538,673,86717],[2539,763,42502],[2541,2029,75460],[2543,777,32627],[2544,1106,68201],[2545,1813,74267],[2546,2044,7021],[2547,2113,36265],[2548,1275,56370],[2549,2135,41896],[2550,454,92434],[2551,378,37484],[2552,412,8121],[2553,2349,28006],[2554,138,19345],[2555,1870,44573],[2556,892,89085],[2557,1848,33033],[2558,2055,75789],[2559,1961,69333],[2560,151,34942],[2561,539,12572],[2562,1548,5950],[2563,1181,48440],[2564,957,2090],[2565,2103,99594],[2566,630,23679],[2567,2399,88550],[2568,1847,48699],[2569,951,38980],[2570,134,4514],[2571,2454,59093],[2572,2285,48149],[2573,522,3184],[2574,2552,77394],[2575,2201,7405],[2576,2472,57539],[2577,2294,19400],[2578,1508,29788],[2579,621,98430],[2580,1351,36518],[2581,453,88711],[2582,2240,12986],[2583,2171,37324],[2584,368,73937],[2585,2184,30738],[2586,1191,19147],[2587,213,57215],[2588,1581,58394],[2589,2411,46165],[2590,1175,12592],[2591,2301,46319],[2593,1985,24217],[2594,946,9783],[2595,335,90881],[2596,2090,26681],[2597,2322,41002],[2598,2334,49024],[2599,1103,37182],[2600,2345,55152],[2601,2334,97269],[2602,2400,82393],[2603,343,99315],[2604,2472,73789],[2605,2055,39070],[2606,1968,32192],[2608,810,39154],[2609,185,54290],[2610,1626,70901],[2611,189,75295],[2613,625,64436],[2614,1660,9165],[2616,2595,39171],[2617,823,77526],[2619,1738,8080],[2620,958,12651],[2622,2354,82223],[2623,1247,79693],[2625,2266,27173],[2626,1111,96604],[2628,1210,42181],[2629,542,12964],[2630,1540,2210],[2631,698,52425],[2632,1715,66822],[2634,607,1686],[2635,2411,36748],[2636,544,19209],[2637,1357,65306],[2638,2317,80823],[2639,412,37170],[2640,836,77594],[2641,1993,80238],[2642,2439,89385],[2643,1824,35166],[2644,1076,15455],[2645,717,99577],[2646,591,73160],[2647,1290,82080],[2648,2139,99751],[2649,489,79278],[2650,1884,62729],[2651,267,92741],[2652,1968,98670],[2655,1219,73786],[2656,1700,63321],[2657,1360,6316],[2658,1731,52349],[2659,2411,44954],[2661,545,93459],[2662,1032,97209],[2663,38,67989],[2664,1567,51330],[2666,125,16130],[2667,2155,95280],[2668,1523,14665],[2669,1379,35613],[2670,749,83583],[2671,1975,13897],[2673,836,64767],[2674,539,77392],[2675,1312,42027],[2676,2262,97176],[2677,1942,74106],[2678,282,8917],[2679,2552,97126],[2680,961,21924],[2681,363,47092],[2682,2154,90139],[2683,1206,78273],[2684,1317,56058],[2685,1221,51641],[2686,1198,44013],[2688,824,38845],[2690,749,27218],[2691,1983,91012],[2693,2241,16990],[2695,1012,65079],[2696,1968,13174],[2697,562,85512],[2698,384,91914],[2699,2310,76270],[2700,722,70268],[2701,2531,43547],[2702,1706,49714],[2703,350,95441],[2704,2629,54206],[2705,1440,18636],[2706,2638,90986],[2707,413,73248],[2708,1459,90762],[2709,337,58776],[2710,2646,3684],[2711,640,22743],[2712,1741,64933],[2714,592,96687],[2715,765,83847],[2716,963,11468],[2718,1681,48558],[2719,2528,1117],[2720,2035,39535],[2721,1525,25968],[2722,813,2185],[2723,93,68941],[2724,1493,66997],[2725,2433,98726],[2726,2689,63219],[2727,1266,68093],[2728,1493,79867],[2729,1102,31808],[2730,1491,81058],[2731,2606,12658],[2733,1343,85065],[2734,1043,77699],[2735,1463,71667],[2736,405,94781],[2737,314,37981],[2738,216,18786],[2739,2549,88759],[2740,1736,86297],[2741,413,87614],[2742,2403,47522],[2743,2391,78111],[2744,1686,52105],[2746,2460,13819],[2747,1844,43111],[2748,674,20474],[2749,298,3246],[2750,465,45958],[2751,610,53648],[2752,1745,33187],[2753,755,91346],[2754,1234,4138],[2755,2437,83810],[2756,714,87228],[2757,1770,45270],[2758,1083,85423],[2759,829,29033],[2760,909,87960],[2761,1821,45312],[2762,1974,63390],[2763,1123,92916],[2764,1537,47883],[2765,542,94799],[2766,1119,20726],[2767,2636,1863],[2768,2244,49905],[2770,332,9915],[2771,420,20266],[2772,1846,16998],[2773,393,53860],[2774,1473,20372],[2775,748,8672],[2776,1901,86785],[2777,1880,60247],[2778,1617,65901],[2779,1614,51262],[2780,1890,1496],[2781,656,37311],[2782,1174,80207],[2784,2714,52303],[2786,1663,17983],[2787,558,94367],[2788,33,75222],[2789,1496,27666],[2790,2175,62718],[2791,1012,57802],[2792,283,39345],[2793,2394,66716],[2794,727,9993],[2795,465,33633],[2797,768,75734],[2798,674,50388],[2799,1030,23704],[2800,98,74370],[2801,776,12792],[2802,940,77667],[2803,572,38708],[2804,2247,5221],[2805,1295,43471],[2806,2758,51458],[2807,1027,90532],[2808,2303,61996],[2809,432,33438],[2810,302,76535],[2811,1755,26831],[2813,2677,8445],[2814,109,74368],[2816,1462,29489],[2817,607,20237],[2818,672,44931],[2819,713,92900],[2820,2175,51526],[2822,545,76339],[2823,2218,13347],[2824,1887,57825],[2825,1017,24770],[2826,1781,77359],[2827,210,94704],[2828,2720,74314],[2829,1941,23895],[2830,659,37003],[2831,388,17531],[2833,1916,77686],[2834,961,16228],[2835,2601,15387],[2837,2614,94323],[2838,1701,19946],[2839,1529,76664],[2840,516,87873],[2841,2699,30643],[2842,2286,33140],[2843,2670,94200],[2844,2226,3481],[2845,2661,69372],[2846,1323,17914],[2847,1366,58503],[2848,1626,1114],[2849,2468,23375],[2850,1266,86616],[2851,1385,61869],[2852,847,98166],[2853,168,80127],[2854,2402,7142],[2855,2206,47055],[2856,285,19093],[2857,250,89446],[2858,760,40701],[2859,332,68552],[2860,1995,86362],[2861,827,88789],[2862,2823,99570],[2863,1755,83878],[2864,236,67068],[2865,562,44904],[2866,2118,18041],[2867,1187,14428],[2868,414,64810],[2871,27,69285],[2872,1797,95509],[2873,1838,80527],[2874,379,10532],[2875,1559,88069],[2876,1033,52479],[2877,100,50371],[2878,626,23137],[2879,1047,39045],[2881,1904,62987],[2882,1343,51764],[2883,591,42800],[2884,2532,86560],[2885,1770,87623],[2886,2670,32526],[2887,46,34004],[2888,2698,64978],[2890,837,15182],[2891,896,93286],[2892,2817,34363],[2894,110,37193],[2895,611,2370],[2896,460,43976],[2897,1555,49623],[2898,1715,9181],[2900,2369,15110],[2901,960,8918],[2902,607,32415],[2903,774,72117],[2904,1628,40657],[2906,521,44129],[2907,492,45003],[2908,1212,6625],[2909,1578,59018],[2910,1091,48434],[2911,64,57721],[2912,1642,85166],[2913,2669,65677],[2914,2167,55693],[2915,2524,41424],[2916,889,80133],[2917,324,1216],[2919,353,86028],[2920,1496,86664],[2921,1129,57113],[2922,314,85420],[2923,2334,60876],[2924,1552,21810],[2925,68,42398],[2926,2911,52942],[2927,2564,51677],[2928,488,56831],[2930,492,77734],[2931,2511,15570],[2932,167,61294],[2933,2692,20385],[2934,886,56154],[2935,1884,29865],[2936,1874,907],[2937,1227,50054],[2938,2361,17921],[2939,62,7889],[2940,1119,34604],[2942,2589,26756],[2943,1435,58732],[2944,1652,79242],[2945,329,32984],[2946,581,49522],[2947,212,66951],[2948,367,75829],[2949,2774,44360],[2950,457,67049],[2951,1995,43559],[2952,1660,21471],[2953,1999,1592],[2954,373,95786],[2955,17,85899],[2956,2263,24490],[2957,2074,55499],[2958,2193,19217],[2959,1172,85484],[2960,707,77635],[2961,497,37718],[2962,1025,89242],[2963,384,4761],[2964,610,12374],[2965,1589,1662],[2966,1989,14142],[2967,584,11572],[2968,2838,1816],[2969,1300,53073],[2970,1272,82955],[2971,1209,43306],[2972,2393,72473],[2973,2964,87446],[2974,182,27840],[2975,1852,37426],[2976,696,20566],[2977,927,23912],[2978,719,82520],[2979,2484,6463],[2981,1671,96068],[2982,817,67210],[2983,2018,55136],[2984,325,86300],[2985,367,69768],[2986,1794,8343],[2987,28,28291],[2989,2893,78264],[2990,421,53655],[2991,226,32168],[2992,2978,38422],[2993,1811,55839],[2994,1411,25383],[2995,1959,96425],[2997,1896,76632],[2998,622,96901],[2999,2648,48295],[3000,748,11112],[3001,1500,68925],[3002,2791,277],[3003,2065,66688],[3004,2581,91888],[3005,2932,2254],[3006,2053,11673],[3007,290,32993],[3008,1897,98847],[3009,802,52498],[3010,744,50834],[3011,2255,91561],[3012,2208,42429],[3013,807,54532],[3014,1208,84466],[3015,938,14627],[3016,1126,92723],[3017,2157,61461],[3018,1437,80824],[3019,851,1174],[3020,2131,14495],[3021,2440,56194],[3022,231,49020],[3023,2510,1812],[3024,235,27442],[3025,350,7948],[3026,653,49316],[3027,2720,79626],[3028,2789,60102],[3029,2321,42394],[3030,2654,22147],[3031,262,68260],[3033,2742,14922],[3035,1346,27468],[3036,1696,44688],[3037,993,34914],[3038,2480,42013],[3040,1798,6908],[3043,1840,64794],[3044,2707,33629],[3046,1713,28827],[3047,1256,6090],[3048,1502,66047],[3049,1884,54962],[3050,1371,96608],[3051,2002,46048],[3052,885,37266],[3053,2492,30408],[3054,22,26769],[3055,897,97225],[3056,752,74818],[3058,1503,31249],[3059,2540,76375],[3060,2145,92508],[3061,2835,48064],[3062,2338,68974],[3063,1097,34123],[3064,648,63193],[3065,295,51877],[3067,1990,20799],[3068,1350,96691],[3069,3013,17931],[3070,2082,74916],[3071,68,623],[3072,686,19849],[3073,1386,2408],[3074,2451,16680],[3076,1779,35729],[3077,983,24351],[3078,3023,13177],[3079,45,98928],[3080,2525,92410],[3082,2396,21590],[3083,325,85916],[3084,1351,38150],[3086,1510,11356],[3087,2258,39831],[3088,2360,82080],[3089,1513,96803],[3090,844,29668],[3091,2636,51743],[3092,73,22754],[3093,1472,56368],[3094,1152,19168],[3095,1705,58075],[3096,62,69253],[3097,694,34465],[3098,1710,6161],[3099,487,47053],[3100,567,51819],[3101,1088,46093],[3102,2479,5185],[3103,1139,5158],[3104,619,54299],[3105,2122,15826],[3107,2193,96891],[3108,1998,38971],[3109,1084,14478],[3110,3011,99360],[3111,1999,5109],[3112,799,71181],[3113,1955,15817],[3114,1057,93834],[3116,2561,98403],[3117,2650,80072],[3118,1037,93759],[3119,1109,74711],[3120,182,64622],[3121,1012,93265],[3122,2813,60543],[3124,2816,72127],[3125,897,80640],[3126,483,27712],[3127,2143,42858],[3128,1092,78056],[3129,2144,1410],[3130,3010,16310],[3131,894,17500],[3133,886,4071],[3134,2156,3099],[3135,1881,73169],[3136,1101,78551],[3137,1608,62568],[3138,322,21116],[3139,2589,34685],[3140,1014,35459],[3141,33,58317],[3142,2341,57668],[3143,1591,57855],[3144,34,65994],[3145,2250,91291],[3146,2233,77397],[3147,2586,47920],[3148,1867,3364],[3149,1457,39836],[3150,2351,24790],[3151,2501,39785],[3152,1206,61865],[3153,1991,6234],[3154,2772,8447],[3155,1599,36331],[3156,14,33000],[3157,1708,43408],[3158,2986,88977],[3159,1223,25324],[3160,723,33845],[3161,2240,58316],[3162,555,47559],[3164,102,32464],[3165,1101,86602],[3166,690,47446],[3167,312,61679],[3168,112,10910],[3169,3099,41671],[3170,2905,74939],[3171,2129,48434],[3172,2544,84068],[3173,1964,7689],[3174,2260,16939],[3175,3128,21086],[3177,2793,50695],[3178,1709,90979],[3179,2097,32927],[3180,2179,80388],[3181,1959,77445],[3182,2584,64029],[3183,1941,39436],[3184,1468,16634],[3186,912,7387],[3187,1082,63811],[3188,2409,29992],[3189,1329,97068],[3190,1524,54374],[3191,1215,11757],[3192,1693,52017],[3193,518,78783],[3195,2178,77763],[3196,2722,19141],[3198,1881,64587],[3199,716,36166],[3200,1830,41809],[3201,2333,20745],[3202,696,45232],[3203,112,23467],[3204,1960,68460],[3205,330,81702],[3207,906,21373],[3208,2271,47377],[3209,22,63391],[3210,49,70491],[3211,438,85392],[3212,2074,61377],[3213,1457,23659],[3214,1108,86807],[3217,2713,76033],[3218,1503,16533],[3219,1970,70857],[3220,412,12881],[3221,1496,13195],[3222,2279,73572],[3223,318,4191],[3224,241,16034],[3225,2433,88798],[3226,1111,45760],[3227,3133,22906],[3228,1538,94500],[3229,988,60074],[3230,128,53898],[3231,3200,8944],[3232,2046,18707],[3233,1554,72060],[3235,1691,18011],[3236,1929,33635],[3237,1659,27129],[3238,2715,95229],[3239,2850,71640],[3240,2122,85451],[3241,2567,89509],[3243,263,17190],[3244,1077,98533],[3246,2445,32195],[3247,1685,27214],[3248,2889,40874],[3249,2847,82418],[3250,379,72502],[3253,2844,81072],[3254,1475,73489],[3255,2517,62456],[3256,2828,25397],[3257,3000,82341],[3258,2865,44401],[3259,998,80009],[3260,1656,87029],[3261,288,22462],[3262,278,16984],[3263,2515,3584],[3264,57,7056],[3265,1011,19999],[3266,2365,75802],[3267,2537,41243],[3268,1831,98833],[3269,979,48232],[3270,1015,53675],[3271,3049,84818],[3272,703,49001],[3273,1570,27072],[3274,997,67429],[3275,34,84877],[3276,210,98405],[3277,2284,42945],[3278,1541,65035],[3279,1129,38094],[3280,2392,74968],[3281,2066,36828],[3282,2434,74952],[3283,1591,76348],[3284,467,20332],[3285,40,89247],[3286,2014,41404],[3287,1308,59921],[3288,378,85141],[3289,476,92885],[3290,217,36789],[3291,1454,44622],[3292,160,55387],[3293,475,92351],[3294,462,28089],[3295,876,99775],[3296,248,69274],[3297,277,56546],[3298,2833,67022],[3299,2100,9270],[3300,3068,86791],[3301,2506,95256],[3302,636,33894],[3304,2508,83421],[3305,2061,37277],[3306,2396,78245],[3307,1103,89040],[3308,306,56072],[3309,1480,89322],[3310,2647,83904],[3312,2153,19675],[3314,2865,8579],[3315,1694,15443],[3316,3246,6588],[3317,1321,82767],[3318,2849,44655],[3319,2808,13194],[3320,2406,45072],[3321,287,37876],[3322,1162,71611],[3323,1533,98562],[3324,1805,11089],[3325,2997,25398],[3326,17,73766],[3327,3231,54325],[3328,2499,93158],[3329,2303,81026],[3330,853,72611],[3331,930,53355],[3332,879,1611],[3333,2038,65225],[3334,2057,99311],[3335,2460,89435],[3336,2523,69074],[3337,9,267],[3338,499,95869],[3339,1702,40196],[3340,588,79487],[3341,583,5735],[3342,834,73326],[3343,877,12359],[3344,2721,86118],[3346,1814,95800],[3347,1270,11129],[3348,258,25518],[3349,2295,33715],[3350,551,51936],[3351,1481,13969],[3352,1390,53442],[3353,2982,97667],[3354,254,54306],[3355,2265,70621],[3356,789,32138],[3357,2909,9399],[3358,519,40628],[3359,2228,88214],[3360,1737,88378],[3361,2748,8402],[3362,1706,52937],[3363,3174,62587],[3364,1248,32204],[3365,538,71799],[3366,678,26041],[3367,1776,94055],[3368,279,62427],[3370,3071,94713],[3371,2641,44716],[3372,649,74525],[3373,2927,76596],[3374,1648,8584],[3375,2159,82944],[3376,2526,52645],[3377,1989,38689],[3378,597,32981],[3379,3023,74310],[3380,1901,6457],[3381,3124,83851],[3382,3314,7860],[3383,1203,26163],[3384,1995,49543],[3385,115,8550],[3386,828,84986],[3387,942,6232],[3388,2004,47858],[3389,3170,5380],[3390,629,18989],[3391,159,14677],[3392,865,41940],[3393,936,92207],[3394,1800,52392],[3395,2327,7842],[3396,2030,55326],[3397,19,9121],[3398,2224,20122],[3399,1408,45595],[3400,668,72503],[3401,1484,78181],[3402,99,6340],[3403,124,90328],[3404,2,26164],[3405,3345,49029],[3406,116,47524],[3407,414,94845],[3408,1655,92235],[3409,2341,58404],[3410,1046,68855],[3411,1766,52738],[3412,1438,169],[3413,1837,71120],[3414,1163,98348],[3415,2772,12479],[3416,177,66832],[3417,2515,72020],[3418,2102,75559],[3419,983,93372],[3420,2403,45295],[3421,274,43962],[3422,1679,81617],[3423,1716,82909],[3424,2036,38372],[3425,2613,73514],[3426,890,99797],[3427,575,14595],[3428,3176,51865],[3431,3181,29846],[3432,516,14819],[3433,1661,95490],[3434,2479,4972],[3435,700,77306],[3436,2302,58188],[3437,3039,35109],[3438,1389,62043],[3439,2693,36121],[3440,2952,60792],[3441,1006,72116],[3442,2801,17466],[3443,1926,65562],[3444,310,27441],[3445,2587,2756],[3446,609,34200],[3447,1586,79325],[3448,962,31125],[3449,420,39582],[3450,836,64202],[3451,2705,78990],[3452,969,55293],[3454,1466,96623],[3455,1326,81524],[3456,2900,1952],[3457,3334,20818],[3458,1689,56323],[3459,218,20705],[3460,3053,95923],[3461,1854,24855],[3462,2862,18628],[3463,1329,61135],[3464,1667,43002],[3465,1886,52017],[3466,425,58914],[3467,842,11630],[3468,2059,39960],[3469,253,82356],[3470,907,11807],[3471,1453,41374],[3472,1750,96646],[3473,86,45674],[3475,96,80440],[3476,396,32028],[3477,285,53326],[3478,313,83916],[3479,878,89387],[3480,1950,84245],[3481,1570,98142],[3482,203,83760],[3483,1450,30727],[3484,1015,79739],[3485,1061,90217],[3486,2359,1547],[3487,1493,94539],[3489,1413,39733],[3490,2733,89229],[3491,2519,82962],[3492,2568,89919],[3493,161,41101],[3495,689,2388],[3496,1630,12929],[3497,1760,62591],[3499,2071,31301],[3500,346,97849],[3501,1728,89646],[3502,3308,88221],[3503,1461,64053],[3505,2795,14696],[3506,1028,14722],[3507,1735,27753],[3508,1539,99994],[3509,3004,11211],[3510,3045,41732],[3512,152,47657],[3514,2409,88755],[3515,1167,47841],[3516,96,4378],[3517,990,20928],[3518,882,77262],[3519,2752,67334],[3520,2748,91988],[3521,305,3505],[3522,735,14163],[3523,749,80341],[3524,2434,71849],[3525,3118,49254],[3526,2892,76686],[3527,1528,35797],[3528,2011,60716],[3529,666,76699],[3530,922,53458],[3531,2049,75457],[3532,644,37514],[3533,3052,61650],[3534,3133,10714],[3535,662,42608],[3536,3290,999],[3538,1671,14340],[3539,591,31619],[3540,2746,94547],[3541,657,26238],[3542,2512,41534],[3543,2358,16938],[3544,2836,28853],[3545,1627,79591],[3546,371,43672],[3547,49,26568],[3548,631,27499],[3550,2143,35416],[3552,3370,79907],[3553,1820,56181],[3554,581,13365],[3555,1529,96351],[3556,1556,67226],[3557,371,9974],[3559,3318,42182],[3560,94,20988],[3561,1503,1735],[3562,3170,24666],[3564,868,68256],[3565,1720,18565],[3568,990,43599],[3569,3212,69884],[3570,2357,93764],[3571,2727,6985],[3572,3515,1464],[3573,2980,41211],[3574,725,47533],[3575,1121,46300],[3576,1810,72554],[3577,1721,52870],[3578,1388,75108],[3580,836,41583],[3581,1514,95608],[3582,3282,15799],[3583,3279,35417],[3584,1879,82910],[3585,1926,67201],[3587,1189,7064],[3588,1143,51183],[3589,880,62572],[3590,1453,29894],[3591,3314,47586],[3593,1264,20453],[3594,1610,74715],[3595,3047,93557],[3596,2846,61101],[3597,2516,52269],[3598,1403,33592],[3599,2196,1983],[3601,1324,26434],[3602,2710,64814],[3603,270,2485],[3604,2659,56686],[3605,2342,65894],[3606,3523,25760],[3607,1221,35324],[3608,447,53921],[3609,1914,96371],[3610,2913,3169],[3611,780,65565],[3612,475,23248],[3613,829,90868],[3614,2213,70845],[3615,2771,9535],[3616,1808,24471],[3617,1351,31078],[3618,1072,97707],[3619,1429,58390],[3620,2211,67124],[3622,640,48482],[3623,3446,40853],[3624,3163,40710],[3625,2082,11967],[3626,514,86179],[3627,2180,73705],[3628,3349,30939],[3629,1819,10352],[3630,2878,11228],[3631,700,9713],[3633,3298,12169],[3634,506,35798],[3635,2515,33257],[3636,2474,67465],[3637,1920,60172],[3638,1429,11259],[3639,3068,87076],[3640,2106,36092],[3641,345,59082],[3642,2155,12204],[3643,1685,84495],[3644,327,63137],[3645,1445,47601],[3646,475,81389],[3647,1129,44549],[3648,1642,75267],[3649,608,33350],[3650,3353,93343],[3651,1442,30874],[3652,2445,78303],[3653,398,9274],[3654,1233,4518],[3655,3603,97570],[3656,3212,81557],[3657,687,20763],[3658,845,20953],[3659,3622,41873],[3660,3137,645],[3662,64,12841],[3663,1828,87369],[3664,2189,77592],[3665,2639,25201],[3666,141,24112],[3667,1299,10887],[3668,3007,95544],[3669,2565,69989],[3670,2810,34839],[3671,2141,22538],[3672,144,49421],[3673,2866,64737],[3674,3599,71954],[3675,2516,57695],[3676,2427,2911],[3677,1515,10125],[3678,1599,56858],[3679,2262,76511],[3680,1112,98285],[3681,218,27341],[3682,1271,20797],[3683,3413,87948],[3684,2210,24027],[3685,705,88213],[3686,3614,50068],[3687,2202,48147],[3688,910,27202],[3689,1566,32526],[3690,1295,86582],[3691,696,27065],[3692,3167,28903],[3693,1508,99950],[3694,2681,53203],[3695,1902,30928],[3696,2420,41523],[3697,1239,49730],[3698,351,72710],[3699,2167,75430],[3700,1373,65086],[3702,2867,67406],[3703,535,90723],[3704,1558,38269],[3705,3532,43022],[3706,1696,41240],[3707,2900,95390],[3708,3419,58762],[3709,716,36026],[3710,1365,92565],[3711,361,31917],[3712,3693,27630],[3713,3031,25821],[3714,1539,6199],[3715,3392,24181],[3716,2149,14751],[3717,2056,10932],[3718,320,62638],[3719,1119,12849],[3720,1119,13447],[3721,2311,89288],[3722,136,45782],[3723,544,66854],[3724,1383,82359],[3725,2813,73194],[3726,1326,37482],[3727,784,65840],[3728,2862,32972],[3729,2500,68780],[3730,3342,93516],[3731,3356,14558],[3732,257,6309],[3733,2966,99196],[3734,2650,72131],[3735,1359,78534],[3736,2251,88411],[3737,2237,16409],[3738,3637,84288],[3739,3625,23200],[3740,2757,19210],[3741,665,40431],[3742,3541,27198],[3743,2753,71878],[3744,21,32754],[3746,2794,34502],[3747,2905,85352],[3749,2452,65963],[3750,871,62701],[3751,898,48817],[3752,1899,26565],[3753,142,63473],[3754,696,78641],[3755,110,55910],[3756,788,40240],[3757,787,88703],[3759,1300,92839],[3761,2816,51063],[3762,239,28130],[3764,3065,32079],[3766,551,58368],[3767,2918,44300],[3768,2633,56023],[3769,1173,74768],[3770,3297,34093],[3771,3571,55865],[3772,3473,42853],[3773,2609,94216],[3774,524,80240],[3775,1512,61218],[3776,774,114],[3777,2574,29772],[3778,3337,59751],[3779,7,54263],[3780,2410,7680],[3781,2712,24962],[3783,2424,63002],[3784,1748,75858],[3787,2278,59059],[3788,1790,82509],[3789,2487,20840],[3790,474,82318],[3791,2916,39379],[3792,1545,64043],[3793,554,47710],[3794,360,56882],[3795,393,52948],[3796,319,17413],[3797,3430,69387],[3798,3344,64597],[3799,359,51171],[3801,2900,58739],[3803,3375,50687],[3804,1035,19710],[3805,1563,33619],[3806,1043,1031],[3807,3619,19650],[3808,3354,27955],[3809,2623,72158],[3810,1713,1948],[3811,3055,86933],[3812,1072,39565],[3813,1239,45532],[3814,525,34163],[3815,2389,83184],[3816,3760,20841],[3817,826,41128],[3818,772,24196],[3819,2069,40134],[3820,579,57496],[3823,949,30078],[3824,1304,717],[3825,1534,90742],[3826,2989,31303],[3827,3730,49075],[3828,3043,15119],[3829,699,41988],[3830,2053,21362],[3831,593,74588],[3832,300,34513],[3833,3233,374],[3834,1933,43248],[3835,3830,28815],[3836,1611,85313],[3838,75,32775],[3839,3018,35167],[3840,735,92534],[3841,2178,54983],[3842,1540,96942],[3843,908,59838],[3844,2518,27552],[3845,2822,8689],[3846,3723,57227],[3847,533,31867],[3850,2546,36201],[3851,312,76117],[3852,18,71258],[3853,2127,89548],[3854,2843,91546],[3855,1226,39352],[3856,1248,49458],[3858,2388,20377],[3859,2946,54039],[3860,471,75029],[3861,3225,59254],[3862,1344,68785],[3863,1981,26504],[3864,1291,17177],[3865,3121,83935],[3866,1402,70223],[3867,3299,22924],[3869,2491,27971],[3870,2960,33895],[3872,3719,36781],[3873,2608,79133],[3874,2445,6384],[3875,10,59020],[3876,391,30514],[3878,3063,19424],[3879,63,7278],[3880,1110,89746],[3881,193,59876],[3882,3124,27732],[3883,2417,27398],[3884,848,90021],[3886,2403,21505],[3887,346,89082],[3888,2893,82639],[3889,3299,56212],[3890,2169,4415],[3891,1324,49483],[3892,2164,85609],[3893,3471,92981],[3894,2110,25787],[3895,3810,84024],[3897,566,93824],[3898,2615,12776],[3899,2014,64581],[3900,380,30676],[3901,2615,85978],[3902,2490,61083],[3903,3226,78879],[3904,1097,59711],[3905,3865,89809],[3906,1387,76021],[3907,874,45741],[3908,3563,72567],[3910,3066,97191],[3911,2967,18571],[3912,3556,64380],[3913,2312,39457],[3914,3913,36919],[3915,1914,18000],[3916,1181,11882],[3917,1190,75508],[3919,3625,3788],[3920,3109,64471],[3921,2298,99976],[3922,3324,19602],[3923,2666,49424],[3924,261,47288],[3925,741,55572],[3926,553,28661],[3928,538,46241],[3929,904,6963],[3930,447,26209],[3931,727,94064],[3932,2151,78891],[3933,1446,83454],[3934,717,91966],[3935,270,89172],[3936,877,69728],[3937,451,31510],[3938,1506,11533],[3940,881,43805],[3941,3551,6291],[3942,3167,7418],[3943,2614,59385],[3944,2102,60670],[3945,3532,78316],[3946,1255,22836],[3947,106,87236],[3949,75,94036],[3950,96,72612],[3951,3392,93142],[3952,1825,64795],[3953,3044,56774],[3954,2923,24705],[3955,2650,14805],[3956,3612,72674],[3958,892,52212],[3959,2530,56694],[3960,2535,94193],[3961,1613,84258],[3962,2942,85257],[3963,1516,30383],[3964,3481,91700],[3965,3096,63828],[3967,2056,74218],[3968,3733,4179],[3969,3438,78593],[3970,2032,73621],[3971,2945,66094],[3972,2108,88385],[3973,1696,10162],[3974,3104,71140],[3975,2931,20720],[3976,1061,14360],[3977,690,86015],[3978,2851,22537],[3979,1299,2278],[3980,481,34728],[3981,982,81241],[3982,557,65388],[3983,2465,35500],[3984,1963,33812],[3985,1985,2722],[3986,2000,58706],[3987,2448,16751],[3988,2634,95254],[3989,1085,83083],[3990,3542,88404],[3992,783,13924],[3994,2341,87602],[3995,1497,89661],[3996,747,86848],[3997,3479,64499],[3999,3923,36045],[4000,16,29485],[4001,659,306],[4002,718,73872],[4003,1827,51597],[4004,2063,26186],[4005,87,28498],[4006,145,57668],[4008,1995,61378],[4009,165,72312],[4010,558,35815],[4011,3289,8353],[4012,2604,79500],[4014,211,80094],[4015,1966,19010],[4016,543,23261],[4017,1541,91786],[4018,2791,91806],[4019,2265,88329],[4020,613,18832],[4021,2293,67849],[4022,1755,79265],[4023,2762,1719],[4024,2944,40343],[4025,2984,54007],[4026,2081,80786],[4028,1988,75366],[4029,644,16138],[4030,1243,86244],[4031,234,98332],[4032,3818,51150],[4033,229,86004],[4034,272,80257],[4035,686,2904],[4036,340,8840],[4037,3495,30444],[4038,1084,42615],[4039,532,30289],[4040,3726,48824],[4041,2445,99852],[4042,1462,35892],[4043,1577,90744],[4044,2206,13005],[4046,3999,53891],[4047,2772,40110],[4048,250,94129],[4049,3110,81060],[4050,2605,17227],[4051,1646,13376],[4052,1608,9129],[4053,1395,71482],[4054,2546,21830],[4055,2494,9557],[4056,2111,11722],[4057,1920,59533],[4058,3514,95802],[4059,2505,50711],[4063,3073,4670],[4064,418,47303],[4065,1145,46280],[4066,2500,12016],[4067,1012,55986],[4068,1196,71946],[4069,1991,92055],[4070,1424,72881],[4071,2462,40894],[4072,637,26855],[4073,3010,59117],[4074,827,96183],[4075,1879,17630],[4076,3751,42159],[4077,1621,8145],[4078,1414,66958],[4079,3027,18473],[4080,733,92935],[4081,1469,98455],[4082,3929,5149],[4083,1470,85525],[4084,1093,28644],[4085,3884,24248],[4086,247,91417],[4087,1297,91128],[4088,733,85544],[4089,680,90468],[4090,2940,17665],[4091,4045,28725],[4092,308,76866],[4093,1412,56868],[4094,2250,32997],[4095,411,59911],[4096,3997,55692],[4097,1636,53059],[4098,2446,50259],[4099,1054,90347],[4100,1103,55792],[4101,162,74325],[4102,1538,46310],[4103,1498,79094],[4106,1499,41651],[4107,1637,74281],[4108,3228,49223],[4109,43,91715],[4110,1787,55420],[4111,50,73730],[4112,875,56896],[4114,3530,46284],[4115,2659,88521],[4116,442,50351],[4117,653,73798],[4118,2403,72979],[4119,289,31886],[4120,2466,10462],[4122,2204,53914],[4123,2580,36172],[4124,3031,66424],[4125,2416,19125],[4126,4064,75147],[4127,52,69626],[4128,2813,69490],[4129,604,65538],[4130,3000,20816],[4131,340,17382],[4132,3227,91627],[4134,2113,7951],[4135,1280,91731],[4136,2665,234],[4137,3890,30778],[4138,3125,17363],[4139,2957,32354],[4140,3172,28562],[4141,3584,91108],[4142,1170,10884],[4143,2767,72653],[4144,3556,96627],[4145,3756,35818],[4146,901,98429],[4147,1385,63750],[4148,1710,64894],[4149,424,7569],[4150,2118,89731],[4151,1889,39566],[4152,717,87304],[4153,3207,62455],[4154,3119,47468],[4155,3484,7799],[4156,1916,80504],[4157,3921,70371],[4158,3933,11071],[4159,2584,37746],[4160,1043,94835],[4161,1857,80927],[4162,306,63913],[4163,3316,40383],[4164,3915,23962],[4166,2608,70578],[4167,326,77323],[4168,2675,76244],[4169,1879,87354],[4170,3816,28456],[4171,1985,58208],[4172,1881,51251],[4173,3611,82132],[4174,1961,93841],[4175,3875,49326],[4176,3627,39118],[4177,1482,48923],[4178,3757,9396],[4179,1389,79894],[4180,2541,63084],[4181,2260,94731],[4182,2676,6477],[4183,4101,73369],[4184,520,5356],[4186,2740,92133],[4187,3604,41353],[4188,1538,83882],[4189,3829,78354],[4190,805,29224],[4191,2322,708],[4192,262,64725],[4193,3863,68592],[4194,2556,46697],[4195,331,98463],[4196,849,26962],[4197,3976,54483],[4198,834,88233],[4199,1933,73565],[4201,342,8181],[4202,1739,34702],[4203,2193,70660],[4204,2151,8586],[4206,262,35380],[4207,1910,54231],[4208,1805,9399],[4209,874,70099],[4210,2992,50488],[4211,3684,33924],[4213,2691,24917],[4214,1919,81617],[4215,3268,69217],[4216,2772,73350],[4217,416,69141],[4218,2039,23229],[4219,718,63601],[4220,2702,73662],[4221,3970,99696],[4222,3751,9655],[4223,2750,11384],[4224,654,73630],[4225,2305,85096],[4226,3922,14641],[4228,1013,20792],[4229,971,64032],[4230,457,12843],[4231,1152,96364],[4232,526,65961],[4233,2483,52463],[4234,1023,48977],[4235,902,92103],[4236,3450,92134],[4237,1743,43147],[4238,3798,80709],[4240,1539,13879],[4242,1838,84656],[4243,1051,77112],[4244,3366,86421],[4245,1257,11768],[4246,2440,72303],[4247,2377,84806],[4248,2044,16029],[4249,2044,12429],[4250,714,14770],[4251,614,75906],[4252,3247,1610],[4253,3580,65871],[4255,3183,16705],[4256,1229,61933],[4258,1742,43294],[4259,3659,48053],[4260,757,69397],[4261,3538,23148],[4262,2463,45079],[4263,2231,965],[4264,60,15313],[4265,1477,5625],[4266,3295,94178],[4267,4094,42139],[4268,994,13692],[4269,151,5828],[4270,2794,28866],[4271,2262,4444],[4272,2506,42169],[4273,3451,98644],[4274,737,54255],[4276,2340,56078],[4277,2415,27111],[4278,400,76394],[4279,369,76528],[4280,2400,31703],[4281,743,65212],[4282,450,76874],[4283,2907,48893],[4284,4126,29691],[4285,1930,45404],[4286,1678,72814],[4288,1187,28122],[4289,712,95321],[4290,818,30317],[4291,4241,13830],[4292,3023,16875],[4293,716,97696],[4295,2173,36721],[4296,794,1163],[4298,189,12758],[4299,1168,72489],[4300,2894,98625],[4301,3025,57956],[4302,2154,40060],[4303,1116,38444],[4304,1892,86128],[4305,2543,69066],[4307,3668,8750],[4308,1868,60886],[4310,2908,45791],[4311,3013,33909],[4312,167,46457],[4313,2118,29308],[4314,695,44187],[4315,37,10420],[4316,1884,25051],[4317,3748,37996],[4318,4208,71695],[4319,3150,46145],[4320,373,37194],[4321,1503,56806],[4322,704,61158],[4323,3608,81452],[4324,4279,82490],[4325,3939,33970],[4326,628,75564],[4327,1167,37299],[4328,507,23108],[4329,143,25098],[4330,1850,16717],[4331,3895,78807],[4332,1158,462],[4333,1296,7849],[4334,710,42025],[4335,3556,32405],[4336,3360,10547],[4337,3492,67515],[4338,236,59048],[4340,245,36689],[4341,25,45305],[4342,3569,27916],[4343,2566,40436],[4344,2656,95786],[4346,3,30762],[4347,934,68687],[4348,1391,55454],[4349,3967,70198],[4350,3661,41386],[4351,662,85324],[4352,2671,11811],[4353,209,65061],[4354,2746,34715],[4355,476,22683],[4356,3724,99073],[4357,3605,47495],[4358,1506,16395],[4359,1765,98293],[4360,2717,22339],[4361,1449,18170],[4362,2111,44600],[4363,3311,10469],[4364,2648,50151],[4365,2906,60744],[4366,1709,6902],[4367,1954,62076],[4368,4354,74],[4369,3703,9137],[4370,3992,29697],[4371,2328,17369],[4372,3346,33481],[4374,1113,36569],[4375,2439,17384],[4377,1509,19927],[4379,187,4014],[4380,2242,29377],[4381,2292,85635],[4382,2983,75941],[4383,3424,92623],[4384,3146,21370],[4385,3333,65780],[4386,1055,77574],[4387,1097,10963],[4388,2341,48211],[4389,4072,35220],[4390,649,97439],[4391,2565,72473],[4392,592,54171],[4393,1406,53697],[4394,958,11910],[4395,4092,88202],[4396,922,19617],[4397,3021,88737],[4398,1419,68698],[4399,1498,4610],[4400,3494,95688],[4401,815,50937],[4402,3543,22563],[4403,1276,10155],[4404,4187,39475],[4405,1347,79133],[4406,3244,57132],[4408,2489,56226],[4409,2220,25301],[4411,1482,8935],[4412,4124,24797],[4413,3119,13737],[4414,3400,9730],[4415,913,48976],[4416,2250,35040],[4417,1398,42595],[4418,442,38542],[4419,3225,4062],[4420,4201,42624],[4421,2994,42213],[4422,4384,55947],[4423,2843,98129],[4424,1390,45685],[4425,1027,5103],[4426,2655,34488],[4428,852,19412],[4429,4263,11512],[4430,765,59319],[4431,3888,95852],[4432,2395,29391],[4434,3286,42870],[4435,1615,57712],[4436,507,48157],[4437,2034,52642],[4438,3893,32478],[4440,3872,26149],[4441,2693,10991],[4442,79,90521],[4443,2537,10122],[4444,1437,6607],[4445,2089,15752],[4446,3823,6372],[4447,1757,14700],[4448,826,4921],[4449,4341,65553],[4451,3154,97845],[4452,1715,65415],[4453,3490,11734],[4455,2556,39117],[4456,1764,858],[4457,2066,9273],[4458,1245,7333],[4459,1729,41324],[4460,4069,54238],[4461,3239,63038],[4462,2836,91595],[4463,2605,69978],[4464,2043,67282],[4465,2953,1348],[4466,1905,60577],[4467,1694,97941],[4468,4117,95062],[4469,1063,41201],[4470,500,67824],[4471,831,74093],[4472,3568,29757],[4473,1133,38190],[4474,2210,955],[4475,710,74274],[4476,1668,56981],[4477,2780,90085],[4478,3383,72840],[4479,162,53493],[4480,920,40285],[4481,4262,89921],[4482,886,14260],[4485,3151,65582],[4486,4287,37157],[4487,3217,35690],[4488,4334,58308],[4489,411,30585],[4490,955,64378],[4491,389,76702],[4492,2936,86495],[4493,2960,78630],[4494,3336,68705],[4495,3760,50907],[4496,244,21107],[4497,3746,43100],[4498,2302,86858],[4499,331,1146],[4501,492,10622],[4502,3168,78832],[4503,1048,82409],[4504,2451,21823],[4505,147,26654],[4507,2644,19818],[4508,3512,37214],[4509,4148,23850],[4510,2057,29667],[4511,2629,5349],[4513,2338,56643],[4514,287,262],[4515,1863,84310],[4516,1343,50904],[4517,3075,24898],[4519,849,6909],[4520,1218,12657],[4521,1803,73859],[4522,3519,66584],[4523,2888,61987],[4525,3598,52937],[4526,599,53104],[4527,827,24585],[4528,3134,24894],[4529,1429,81999],[4530,2162,53958],[4531,272,69014],[4532,1580,24352],[4533,2524,42459],[4534,137,88046],[4536,1946,58427],[4537,2792,10417],[4538,1773,24880],[4540,2862,74689],[4541,1924,36163],[4542,2624,61649],[4543,1097,2350],[4544,3010,78577],[4545,734,10204],[4546,1137,93033],[4547,1619,56947],[4548,2257,75487],[4549,3708,56245],[4551,1633,4979],[4552,1834,10694],[4553,4279,10740],[4554,1642,63955],[4555,1078,40295],[4556,2727,36377],[4559,221,70913],[4561,4057,80725],[4562,3133,70490],[4564,2586,55600],[4565,2127,2914],[4567,2096,61631],[4568,330,15106],[4569,3505,85057],[4570,625,47390],[4571,42,74390],[4573,339,70910],[4574,531,56403],[4575,2494,62586],[4576,4443,44459],[4577,4069,994],[4578,582,26287],[4579,181,6535],[4580,896,2264],[4581,207,26175],[4582,3930,82741],[4583,554,62338],[4584,2965,43359],[4587,4563,19578],[4588,511,41921],[4589,1317,92930],[4590,3692,32923],[4592,1925,70897],[4593,1341,70465],[4594,1682,62172],[4595,2617,52330],[4596,3620,41755],[4599,3474,74026],[4600,1525,45963],[4601,3301,27392],[4602,2774,80792],[4603,3547,54814],[4604,1733,36031],[4605,2648,23890],[4606,4044,24145],[4607,3935,69],[4608,512,32042],[4609,3843,19351],[4610,219,85944],[4612,236,22376],[4613,3495,55273],[4614,2412,57773],[4615,3029,76077],[4616,4227,92066],[4617,2572,48678],[4618,3961,75607],[4619,4403,53632],[4620,1457,44180],[4621,767,82577],[4622,3610,73591],[4623,3856,96113],[4624,4380,74693],[4625,1741,89687],[4626,3011,72936],[4627,1551,28008],[4629,3863,2790],[4631,3100,88376],[4632,3300,36887],[4633,2078,16920],[4634,880,89834],[4635,3317,10360],[4636,2312,44358],[4637,3941,36694],[4638,3566,50530],[4639,3418,73645],[4640,4076,98068],[4642,2337,76888],[4643,462,7785],[4644,3151,91670],[4645,3556,37850],[4646,1301,54336],[4647,4573,5840],[4648,3498,56437],[4649,351,492],[4650,1941,90555],[4651,712,71979],[4652,1168,59707],[4653,3246,67881],[4654,2803,68071],[4655,1994,19134],[4656,280,36336],[4657,3430,46341],[4658,2464,91836],[4659,767,31303],[4660,3506,99471],[4661,2812,19525],[4662,3614,28252],[4663,1196,20726],[4664,3579,76276],[4665,460,62055],[4666,2523,53987],[4667,2483,51240],[4668,4400,86515],[4669,3875,71449],[4670,275,20068],[4671,2034,70255],[4672,1152,16872],[4673,2066,41395],[4675,4235,48122],[4676,2347,57119],[4677,4639,55483],[4678,348,42413],[4679,1189,54809],[4680,1754,20605],[4681,4153,53813],[4682,2940,67182],[4683,2248,18798],[4684,265,10562],[4685,3445,26879],[4686,3930,40546],[4687,3009,71918],[4689,3468,64514],[4690,3077,7934],[4691,446,54744],[4692,2339,87990],[4694,619,90505],[4695,1027,33571],[4696,1141,17127],[4697,1983,60800],[4698,4251,6593],[4699,2504,8054],[4700,1244,48651],[4701,606,88097],[4702,646,17027],[4703,3056,5260],[4704,1555,81720],[4705,1058,29955],[4706,2266,91377],[4707,1002,420],[4708,4611,92630],[4709,2330,19631],[4710,490,11745],[4711,58,1527],[4712,2375,56187],[4713,697,89398],[4714,2159,51468],[4715,351,84174],[4716,636,85617],[4717,2921,5106],[4718,1912,1617],[4719,1707,87332],[4720,31,92812],[4721,1621,69959],[4722,1227,39705],[4723,3029,28961],[4724,1004,29637],[4725,3228,27106],[4726,304,15665],[4727,4191,5959],[4728,4722,84178],[4729,210,88319],[4730,2663,68302],[4732,1073,98515],[4733,3048,43250],[4734,3137,22302],[4735,3098,65452],[4736,4384,22158],[4737,3939,68858],[4738,1739,39670],[4739,1750,12257],[4740,2660,21235],[4741,189,83097],[4742,2086,960],[4743,4393,9888],[4744,1355,69990],[4745,2385,45075],[4746,408,50863],[4748,4086,63782],[4749,4375,93868],[4750,2895,20883],[4751,4460,89526],[4752,4630,27321],[4753,2033,28438],[4754,984,28540],[4755,3826,66233],[4756,2094,55854],[4757,1039,86123],[4758,2162,20459],[4759,351,72337],[4761,2100,7813],[4762,4573,44134],[4763,2105,89859],[4765,822,38035],[4766,3073,55804],[4767,4599,87743],[4768,4062,38863],[4769,98,54933],[4770,2116,90750],[4771,3201,28480],[4772,315,78300],[4773,4599,26223],[4775,3619,55164],[4776,148,86898],[4777,4556,19967],[4778,146,10762],[4780,1073,49926],[4781,4467,22168],[4782,500,80698],[4783,283,48366],[4784,3464,27727],[4785,1744,56400],[4787,372,71228],[4788,3289,90011],[4789,4480,32684],[4790,2664,41808],[4791,3072,91167],[4792,4016,51614],[4793,3445,9524],[4794,4453,99332],[4795,4559,18133],[4796,3952,51345],[4797,4788,52396],[4799,3346,45613],[4800,2792,77832],[4801,2072,70024],[4802,617,89525],[4803,245,72605],[4804,4086,62971],[4807,2594,12957],[4808,3714,48598],[4809,2616,18693],[4810,74,62882],[4811,2973,85141],[4813,1573,13286],[4814,944,24285],[4815,393,48508],[4816,92,66773],[4817,1613,46200],[4818,1913,85145],[4819,2181,52386],[4820,2187,57368],[4821,2615,35193],[4822,3356,16635],[4823,3936,31775],[4824,345,48639],[4825,3576,51506],[4826,3542,79028],[4827,874,3710],[4829,1028,11124],[4831,4442,75460],[4832,3378,14580],[4833,3632,2249],[4834,3419,47740],[4835,2068,1344],[4836,425,72083],[4837,4825,24877],[4838,909,19974],[4839,2311,25057],[4840,649,22635],[4841,3753,38523],[4842,951,42616],[4843,1665,35938],[4844,3279,86232],[4845,4249,6468],[4846,1085,59835],[4847,2058,71088],[4848,1542,25647],[4849,647,6423],[4850,2809,58914],[4851,3049,59313],[4852,2243,3898],[4853,2961,23283],[4854,3657,66324],[4856,3769,47780],[4857,3439,9107],[4858,4289,30038],[4859,1405,86108],[4860,3061,77134],[4861,3647,9451],[4862,713,88792],[4863,4653,55572],[4864,690,51356],[4865,1796,65553],[4866,1439,98863],[4867,3477,75925],[4868,4573,56861],[4869,2989,1267],[4870,3130,1877],[4872,3504,86658],[4873,3957,9637],[4874,3968,64806],[4875,2497,6132],[4876,1462,22218],[4877,4469,50945],[4878,4148,48824],[4880,61,92026],[4881,4260,12448],[4882,2837,10496],[4883,4176,93769],[4884,1879,78062],[4885,152,64406],[4886,3935,12204],[4887,3086,85979],[4888,86,82447],[4889,545,7204],[4890,2504,34936],[4892,2488,84929],[4893,2558,43869],[4894,4697,16978],[4895,1815,20708],[4896,3725,73424],[4897,2814,74613],[4898,3980,87183],[4900,4264,9058],[4901,741,38291],[4902,2584,63022],[4903,4220,25044],[4904,2247,27226],[4905,2539,15981],[4906,3461,80179],[4908,1385,38442],[4910,1748,2983],[4911,943,95061],[4912,29,77590],[4913,798,81584],[4914,4352,71546],[4915,528,59703],[4916,2635,59531],[4917,515,35616],[4918,2753,96032],[4919,4574,26653],[4920,4195,30293],[4921,2108,86492],[4922,730,73331],[4923,3138,89370],[4924,1032,46090],[4925,1604,2204],[4926,2301,18726],[4927,4593,94106],[4928,2775,60368],[4929,3338,31890],[4930,3697,62844],[4931,1977,68316],[4932,134,71315],[4933,1104,45573],[4935,3020,89794],[4936,2113,77051],[4937,3956,92873],[4938,3814,95395],[4940,525,67922],[4941,1369,76376],[4942,2820,7764],[4944,3583,47405],[4945,678,98918],[4946,3641,74773],[4947,1153,20622],[4948,3818,81255],[4949,4542,95558],[4950,4177,89776],[4951,374,22579],[4953,4468,63616],[4954,3236,32701],[4955,3035,10149],[4956,2197,62741],[4957,3049,13547],[4958,1435,97768],[4959,1322,99894],[4960,2884,83152],[4961,2090,53156],[4962,538,82698],[4963,2588,81769],[4965,2275,21276],[4966,3118,87679],[4967,1177,59162],[4968,4121,65210],[4969,3239,10432],[4970,3068,95365],[4971,2626,90152],[4972,183,60398],[4973,552,32485],[4974,4844,24576],[4975,1795,90547],[4976,678,44605],[4977,2612,78816],[4978,2495,68568],[4979,12,98743],[4980,214,22124],[4981,4755,1098],[4982,174,40306],[4983,4420,79277],[4984,949,1863],[4985,2445,39103],[4986,2451,74467],[4987,4513,49190],[4988,1398,51750],[4989,4347,83126],[4990,754,1916],[4991,1745,44967],[4992,186,15327],[4993,2379,25658],[4994,2315,7855],[4995,2498,68184],[4996,1108,83621],[4997,2430,52934],[4998,4116,61384],[4999,4646,97322],[5000,4815,2714],[5001,3627,58553],[5002,1711,20167],[5003,3474,55766],[5005,2124,13224],[5006,373,24479],[5007,89,94483],[5008,1237,56902],[5009,4012,8034],[5010,833,86052],[5011,4339,87828],[5012,732,44549],[5013,2032,88365],[5014,390,96121],[5015,3054,43712],[5016,1254,11188],[5017,1885,91180],[5018,249,50844],[5019,4340,56613],[5020,396,30292],[5021,289,78828],[5022,3261,51253],[5023,573,2970],[5024,3393,48925],[5026,2952,46418],[5028,2895,83452],[5029,4494,44285],[5030,509,57283],[5031,2342,98032],[5032,511,69230],[5033,4919,48916],[5034,1308,1113],[5035,3827,48818],[5036,2598,41938],[5037,921,5822],[5038,2247,39021],[5039,293,74176],[5040,183,71763],[5041,4333,6059],[5042,4787,11116],[5043,4268,41184],[5044,3453,47232],[5045,885,83205],[5046,2261,55624],[5047,2646,88047],[5048,2027,41878],[5049,3562,33251],[5050,2682,21808],[5052,1810,21656],[5053,1994,6962],[5054,3236,84129],[5055,778,89003],[5058,3076,79013],[5059,3828,64991],[5060,4031,37419],[5061,1087,92433],[5062,191,68136],[5063,2730,46290],[5064,1065,9715],[5065,462,92878],[5066,1766,92559],[5067,2749,95270],[5068,3682,34381],[5069,4977,14521],[5070,4152,9447],[5071,4286,25801],[5072,647,12664],[5074,4526,89566],[5075,2476,16395],[5076,3345,70782],[5077,2681,33611],[5078,3210,93618],[5079,1756,38443],[5080,4425,76266],[5081,3280,13123],[5082,2383,4185],[5083,2799,57486],[5084,4183,28623],[5085,2682,5750],[5086,2453,71868],[5087,1513,18449],[5088,1558,1777],[5089,1970,89319],[5090,696,94673],[5091,4104,59945],[5092,1719,90679],[5093,759,36574],[5094,5016,75070],[5095,1032,79605],[5096,2266,5669],[5097,2407,63608],[5098,4004,61859],[5100,4269,93517],[5101,996,5688],[5102,4371,69525],[5103,1766,45087],[5104,3304,79888],[5105,301,63307],[5106,233,72376],[5107,4104,82064],[5108,3458,56577],[5109,2588,29428],[5110,3446,39224],[5111,5049,88607],[5112,2112,49804],[5113,383,37855],[5114,2169,74609],[5115,2950,20200],[5116,4243,61173],[5117,2460,88193],[5119,183,86090],[5120,3401,49682]]
    

# s = Solution()    
# res = s.minCostToSupplyWater(n, wells, pipes)

In [2]:
# v8 with connected node and edges

import pdb
bp = pdb.set_trace
import sys
import matplotlib.pyplot as plt
import networkx as nx
import pylab

def plot_graph(n, nodes,edges,node_connected=[],edge_connected=[],special_list=[],start_i=0, size=10):
    fig, ax = plt.subplots(figsize=(size, size)) # Size figure size

    G = nx.Graph()
   
    if type(nodes)==list:
        new_nodes = {}
        for i,v in enumerate(nodes):
            new_nodes[i+1] = v
        nodes = new_nodes

    if start_i:
        for k,v in nodes.items():
            G.add_node(k, weight=v, color='pink')
           
    else:
        for k,v in nodes.items():
            G.add_node(k, weight=v, color='pink')

    labels = {n: f"{n}\nx\n{G.nodes[n]['weight']}" for n in G.nodes}
   
#     print('G.nodes:',G.nodes)


    # Modify Node
    for i in node_connected:
        G.nodes[i]['color'] = 'red'    
        
    for i in special_list:
        G.nodes[i]['color'] = 'blue'
             
    node_colors = nx.get_node_attributes(G,'color').values()
             
        
    # Plot Edges
    for edge in edges:
        cost = edge[2]
        edge = sorted(edge[:2])
        G.add_edge(edge[0],edge[1],color='black', weight=cost)

                 
    # Modify edge
    for edge in edge_connected:
        G[edge[0]][edge[1]]['color'] = 'red'
                       
             
    edge_colors = nx.get_edge_attributes(G,'color').values()
#     pos = nx.spring_layout(G, k=20) # Distance between nodes
#     pos = nx.random_layout(G, center=None, dim=2, seed=None)
#     pos = nx.shell_layout(G)
#     pos = nx.spectral_layout(G)
#     pos = nx.kamada_kawai_layout(G)
    pos = nx.planar_layout(G, scale=1)  
             
    # Draw Edge Labels
    nx.draw_networkx_edge_labels(
        G, pos,
            edge_labels=dict([((u,v,),d['weight'])
                     for u,v,d in G.edges(data=True)]),
        font_color='red',font_size=10
    )
             
             
    # Drow Nodes and Edges
    nx.draw(
        G, pos, width=5, linewidths=1,edge_color=edge_colors,
        node_size=2000, node_color=node_colors, alpha=0.7,
        labels=labels,font_size=10
    )

    """
    edge_color='b'
    width=5
    node_size=2000
    """
    plt.savefig('tmp.png')
    plt.show()

In [3]:
from collections import deque,Counter

class Node:
    def __init__(self, val):
        self.val = val
        self.well_cost = None
        self.pipes = dict()
        self.is_outer = True
    def check_outer(self):
        if len(self.pipes)>1:
            self.is_outer = False
        else:
            self.is_outer = True
            
    def add_pipe(self, other_node,cost):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        self.pipes[other_node_val] = cost
        self.check_outer()
    
    def remove_pipe(self, other_node):
        if type(other_node) in (str,int):
            other_node_val = other_node
        else:
            other_node_val = other_node.val
        
        del self.pipes[other_node_val]
        self.check_outer()

def packupNode(n, well_cost, pipes):
    # Initial Node
    node_dict = {i:Node(i) for i in range(1,n+1)}

    for i in range(len(well_cost)):
        node_dict[i+1].well_cost = well_cost[i]

    # Add Edges  
    for edge in pipes:
        node_a = node_dict[edge[0]]
        node_b = node_dict[edge[1]]
        cost = edge[2]

        if node_a.val in node_b.pipes:
            if  node_b.pipes[node_a.val]>cost:
                node_b.add_pipe(node_a.val,cost)
                node_a.add_pipe(node_b.val,cost)
        else:
            node_b.add_pipe(node_a.val,cost)
            node_a.add_pipe(node_b.val,cost)     

    return node_dict


def get_connected_group(node,checked_node_set):
    stack = deque()
    group_set = set()
    stack.append(node)

    while(stack):
        cur = stack.pop()
        group_set.add(cur)
        checked_node_set.add(cur)
        for pipe, _ in cur.pipes.items():
            adj_node = node_dict[pipe]
            if (not adj_node in group_set):
                stack.append(adj_node)   
    return group_set


def get_groups(node_list):
    group_list = []
    checked_node_set = set()
    for node in node_list:
        if node in checked_node_set:
            continue
        else:
            group = get_connected_group(node,checked_node_set)
            group_list.append(group)
    return group_list


n = 5120
wells = [21128,75528,97977,52262,52378,15538,38502,82313,47321,85603,57516,70456,3340,10685,14501,58724,42337,74534,6019,4958,57135,702,93577,99406,31850,89442,92604,98339,96751,92818,99355,69270,69868,52010,69654,68788,31959,72233,81473,89012,98204,77272,17591,88067,1682,66222,2648,39761,53101,86944,93661,24208,61060,27169,4096,61950,91487,5713,66947,92700,69000,22407,50866,30970,13669,67723,25650,12925,86768,18176,23777,47704,6001,28669,18491,41794,81633,98117,86153,84833,61390,56697,61686,16438,96154,89325,19969,34488,71395,80075,9290,42780,71525,98954,98949,5726,47989,36609,66733,90119,61939,93626,50667,21209,30272,16076,23587,31502,20140,67856,10240,22960,54014,52123,87986,67927,37322,50755,41996,82535,85198,9227,9908,8376,93639,65630,69549,98730,80787,14611,13998,57844,71866,29050,79414,35399,32945,81424,21955,41679,73049,11354,69358,18489,38909,36219,92724,51020,89519,43854,8018,1177,50616,24035,99671,51378,69801,9524,58,94173,21476,60926,95071,97791,80030,19941,13664,40706,47216,14832,11235,1618,25984,52530,54636,98407,18467,39794,71734,32048,43613,68877,1750,36231,85772,10250,82036,61698,4581,8832,61311,75892,50035,22491,94445,97822,57825,83237,26853,58666,92055,23491,5679,34791,6369,37423,20900,97724,5726,10614,38469,29887,78640,84042,26189,58524,51047,68102,33765,3440,20862,31749,61789,37905,90161,54661,58086,43710,74402,1787,8606,1694,84348,62614,69986,61834,24177,37259,66432,28044,54452,43908,57008,24151,74973,15201,34448,68761,76275,34812,82783,25593,65309,54357,34658,50861,40309,99643,49225,62866,56314,54205,7414,6946,42236,75620,19893,79605,292,63612,83196,30451,12402,36348,60488,35269,59872,86581,75990,85045,31249,82809,36114,78399,32493,64972,47424,91529,68309,29235,7094,53707,6607,64663,31229,23737,21931,99922,71809,91458,774,14714,2129,46905,36074,43740,10298,27174,52954,60298,51533,83198,55851,50244,14454,20660,16223,79195,53447,76838,34390,47889,83767,86683,23837,38197,87247,46525,69473,90769,46233,39895,90331,3856,5245,6039,27673,40562,74321,55080,71112,18073,10865,95454,9593,52441,74657,76750,79532,39354,56288,50675,18758,99925,37918,30710,51440,99446,42368,65414,33421,19571,17110,42778,15604,86739,22061,18133,49087,12508,8185,5696,68532,44608,10581,55129,36116,77450,8836,87767,28010,31303,52670,11378,32852,56253,74493,91829,77304,28107,5736,53220,40739,10929,12977,71900,19972,36336,63597,61179,38395,73330,3170,26604,25879,62880,81557,90467,88706,59362,78528,16957,71241,49440,15725,1915,26006,35380,14401,80262,37337,62783,92025,63896,30995,75920,74312,69685,27159,32062,70444,92693,52586,13090,59397,33566,86826,58154,22750,45680,24536,72131,96895,16672,63780,19003,16442,48544,93873,79506,90387,89772,28710,87459,4253,22432,5508,62846,19167,98522,15840,20970,21646,86268,95181,16273,74467,26720,45498,21394,27488,41400,40857,98905,38323,37166,58531,19807,90624,85466,826,66915,89977,67603,88858,54190,56391,53628,29255,44732,90461,87316,72920,10294,94325,46266,32320,84913,27775,46317,25259,62604,98683,1163,50831,20996,67535,28022,31936,9359,57997,53293,16166,90968,5610,98677,87204,32122,64223,49981,66231,94138,63795,34347,44108,16931,12803,52934,90050,10905,38807,56499,69434,75748,75987,28644,8009,9228,37462,36282,74376,24584,18419,36874,84941,10567,28562,24564,4388,89426,70248,97569,83634,78601,90408,24587,876,11564,24023,44134,92407,99304,59507,52676,36490,51811,98936,23078,23451,6912,91728,97804,45705,29015,6501,50648,93394,31486,85833,33121,77695,79898,97513,79456,93290,84536,31622,32336,39514,82426,42270,20375,45038,64750,60304,23190,92543,22771,50340,91463,87263,59310,30115,42401,14075,1839,19666,6201,91372,92347,47312,90591,45247,41791,60831,25649,2183,9785,95802,43927,83036,16446,72458,15378,24760,11584,30490,27446,56556,3947,90231,55869,96391,53320,81259,99427,68218,73419,51813,73002,20792,32167,793,37339,42973,29669,38921,84152,28165,63391,35823,3667,42668,52344,36006,66088,22539,35611,25678,377,10698,37641,4152,25553,19953,79344,76166,46159,94346,81931,29383,40563,1383,33101,24119,67774,75540,54,10297,14159,24397,95229,34503,17591,22643,53823,96961,99632,62902,37168,33152,76881,50996,11969,42557,28398,44837,30287,91066,90878,28572,50129,52533,87683,68314,50450,81159,34636,16141,97481,85268,65125,54871,56141,74785,15739,2596,30257,15052,40850,43972,46468,1478,12625,90174,87667,15451,6492,74535,47633,29540,68661,50153,19973,69822,22596,52483,51971,59536,48421,77723,55879,60123,75458,12363,49026,87090,17887,70196,82654,85048,29062,57160,36882,94244,94262,21539,6242,79710,84186,71852,95820,75439,88436,45074,86037,21724,60272,82634,94848,71507,45058,93257,20090,62973,87047,94777,74805,25555,33799,46180,51568,38778,50209,29587,48009,13611,77011,83699,68421,58897,30217,73538,51615,16195,47134,61167,16458,14992,95421,71095,48104,78506,91875,60334,4114,75527,18786,82351,35532,50241,50520,17270,90251,48971,88583,68346,3848,15025,70858,97811,58037,68107,31215,8917,75557,44214,53772,11833,19310,31803,82175,64860,28944,94860,68953,26949,44917,24732,60193,86879,64017,16658,29788,12836,22687,88036,1727,92302,90009,51181,58144,48586,39924,16946,92452,33424,77306,60740,78015,53771,95852,38535,88149,17595,63739,33295,17344,81835,33448,23512,58607,9968,13270,15219,66043,99079,98340,8320,43418,19105,42643,84442,75356,53399,80425,32583,13078,68747,65624,83734,893,54635,945,79073,87847,40456,74898,90253,45601,23965,9188,40894,3754,79019,99794,57332,15685,91816,83086,53987,43455,70315,33107,31030,3535,24937,10863,8045,84647,82713,5071,15733,42459,14557,64461,91699,16500,99259,52970,37304,4569,23097,13179,29011,88814,16161,3696,93996,93160,88189,69671,79403,62556,33777,11092,13584,6275,61774,7373,93411,71739,58735,50241,73185,20502,25261,86711,42480,39348,42682,11165,90124,39882,11329,60009,82621,81196,22652,17384,71993,8830,86961,23286,56008,21564,42073,97841,81333,36967,89008,64432,89462,31948,59357,50876,92984,31854,75777,54349,75612,2645,83708,73798,84068,18782,67725,51197,20242,78221,15405,88975,51712,53338,56229,70559,93094,42660,91704,31559,37972,79132,63110,24211,87803,59038,15146,80714,72374,73476,17009,52153,42924,35290,46721,63103,9738,13920,69124,11726,43399,80181,93036,60362,94235,27969,45172,23065,6605,16154,26321,68698,56945,10420,90505,69012,67396,89141,47554,69783,82993,4910,66633,78642,60171,97551,24823,8750,57266,74022,91159,57012,25819,35944,79905,10308,46203,47968,95061,15292,81099,74545,72865,27022,98151,11036,34755,58121,66532,41026,3114,54703,84146,15231,15438,9441,74611,63248,31882,92410,83667,89292,64519,76340,70337,31812,11008,76279,20963,31667,75575,38826,59449,64260,24512,27084,33892,94339,65486,7069,17944,87392,98921,78632,58804,4891,20515,45314,61484,68694,49231,94788,35669,52970,67904,36921,6102,58638,41658,73948,97709,50701,70696,58743,94303,62964,87389,73320,87552,16086,97919,92033,52684,65339,37318,90039,69677,81421,978,78297,96989,38235,53587,79889,44260,46184,32704,91898,93954,38755,31545,15054,40644,34988,61926,80204,32829,81642,28595,76613,51572,37339,38967,94174,73311,80507,10750,89006,24961,87684,69276,23445,33301,43699,98773,59384,30661,56264,90151,5850,18180,52408,54646,22614,67348,22613,24805,45947,29500,43287,14864,62135,84902,60446,18211,92923,76670,89875,49353,90483,48189,38758,88789,38625,14797,30270,92055,27904,9988,74765,30139,22782,16514,11091,10572,75398,90492,40731,97848,97439,4262,90919,63479,11307,69550,67871,99525,45190,99847,43972,84558,40801,70605,11570,15760,46120,69357,16621,13938,5398,23311,5720,83340,61227,93356,99224,80281,63292,72418,96202,63819,15608,6783,71776,85763,18057,91241,19993,63631,93025,83683,7753,32760,24773,82670,49424,25092,98290,13525,41696,62152,54031,36463,60279,20657,44645,60436,37415,23642,94797,85267,18304,99226,73993,98763,6658,87030,68355,31437,43022,29304,23320,75433,61626,42079,67876,58293,16608,51941,41930,40880,86964,49379,1077,15350,72960,13884,29503,39425,86562,70778,59841,34248,52260,74490,90093,53189,99288,80278,83298,99575,72594,4562,34793,95116,31373,51355,25006,17448,3263,44201,212,65868,34255,48071,77893,67461,72771,99698,77863,70132,61650,65378,26666,84315,85665,95930,54391,31646,2817,40276,6652,59497,25345,15928,46708,51400,58897,97320,55420,18055,6616,64022,86074,68372,33385,27765,99561,36675,36492,48263,50987,56466,33460,12238,55671,45851,96091,48186,41076,56396,15412,36161,42485,56796,56296,88870,46899,8233,2344,2885,41342,31776,35569,18716,95261,29438,73718,70650,6089,1471,68298,17162,68760,66087,37528,40637,26635,99070,31718,15189,55957,76064,22738,60905,42989,85938,97466,50427,27183,8730,96235,13517,84361,25325,79309,15929,77996,70506,79021,76536,23765,82921,39363,603,40314,44394,53286,29204,71468,57635,34317,63272,36569,95972,943,32101,86247,30776,61664,87536,56005,36357,91667,75500,67288,79466,95007,48684,30231,18924,70970,32506,76005,10590,29909,64396,23674,89669,18915,18373,14115,49806,71403,75369,64577,75182,6167,44909,93849,4498,83941,33918,43641,82119,63457,61137,26533,70646,73593,7797,68141,79481,22767,9325,10241,17247,19839,51908,38091,7748,48673,84165,20139,60560,64666,90829,57871,43645,98423,27581,71138,47389,59847,93497,17181,40712,3395,22902,78394,78270,98706,52705,89097,2807,74834,44882,65030,77593,22725,62296,6737,35532,30852,9400,77320,86042,81493,58511,29323,18102,85251,67220,89253,31261,52417,72485,51266,64426,62836,69716,29557,4970,75907,70234,11188,43237,25894,38253,56197,13576,9403,40668,47581,9439,91457,98782,20608,72358,4031,55695,43430,67698,97238,73269,36596,41483,51941,92234,55751,53951,67526,91253,87054,22002,25081,83501,26084,94748,83638,94522,1796,73818,70702,42083,92016,96918,78500,17090,53413,67636,55676,96824,83066,82011,23003,70910,81928,64516,25695,92082,28501,1118,11722,80628,87657,69458,55520,7348,87634,19571,37504,35679,69084,50096,1347,89816,99799,37729,79758,2035,23594,74602,83917,50023,8962,3168,12504,7853,6379,81057,86219,33646,54178,63859,65010,34130,70307,94265,87551,79211,34376,12890,54603,34827,67616,34647,89431,36513,50999,75722,34306,56047,55858,7534,89107,8928,94114,88974,8692,93547,39919,7773,81787,73825,47923,35681,36029,25393,94352,48007,47078,68736,29625,70072,71398,28919,31743,9530,94858,19626,81906,78556,76955,94531,81944,72022,40212,24387,29111,72259,10573,97955,37106,87612,18276,64241,31172,88174,9686,31806,792,67672,43420,94922,43945,24676,79613,99315,32058,36830,95527,71541,93854,87934,76967,94523,48348,11352,49499,22146,88900,32017,50777,28841,64092,77829,25565,56050,39276,35692,32393,14183,26235,32838,64666,3338,2629,27823,76624,87858,63386,66919,57822,93350,51011,1742,9730,91361,11598,67320,46985,78953,73631,30742,69237,5435,55425,82558,95691,77875,80975,23059,92475,73473,98447,70940,32906,59190,92867,68348,5556,57533,5148,93146,99021,46642,8157,59752,24720,97493,78327,58469,4701,22749,61142,61444,15324,44766,80137,74351,75926,6247,57736,93148,45593,46209,58416,55245,56892,18536,87139,90297,65547,11931,19977,54649,71025,10933,38213,6357,90482,94253,35511,52616,43500,28277,37740,96151,2685,37659,76487,39806,50793,10036,71092,39660,90267,17428,83146,20686,14443,78661,68681,66123,18875,82484,70883,5379,17918,66891,19381,68217,54588,53006,88269,66011,26968,57962,47844,35261,24554,43140,4818,98339,45157,89034,12249,47427,14749,36123,13053,58955,18342,65582,47486,20177,41972,66933,65616,83409,7520,70666,41162,14711,50553,53608,27560,87432,44220,19385,64854,85848,51834,37139,54625,48132,98780,4655,72750,45522,23574,49063,88600,85963,28537,43234,20117,12524,86640,98162,87622,12848,51964,45065,2120,67388,71460,63350,86999,91132,93082,95604,49405,51633,8271,85487,89843,16603,42176,59270,41587,52940,79603,97406,98598,39718,4251,63950,64811,73550,20437,44042,29838,3560,44145,39657,84037,4209,27956,48873,97320,14472,41480,61335,28242,75839,27906,31192,80590,26464,19645,62303,43108,6603,3271,83417,79809,79697,86997,87711,14463,63207,90294,68756,22156,54262,37616,74081,46293,52772,11267,58551,74985,30785,4630,68565,90050,91024,38200,25704,52810,17906,98031,54381,85656,53943,92632,87993,48477,88049,11941,43800,11045,89282,81739,67607,11679,53608,75578,63129,40714,99341,107,71926,31410,29286,77130,21499,42814,26904,22430,5573,72087,64072,72856,12668,64005,67179,15146,37712,21322,82204,28505,39434,60242,66862,18034,57636,33432,85578,73059,34883,27960,55433,82561,17058,63846,10002,4748,61897,29107,92117,19716,32815,82725,36591,26108,95863,86442,24730,51740,46974,13210,51092,63307,18524,98492,82518,32636,53624,96070,75927,94324,97038,37728,51123,33693,96446,83502,28173,43153,52575,39100,98317,36326,70750,39430,44577,99970,24960,90983,34050,61476,60377,24205,92270,89175,21395,86309,45709,49367,39804,25348,20130,86690,90056,88481,68610,7588,95258,15043,22077,35701,32950,59296,26148,96232,61941,14533,49683,14462,34807,47884,4539,87971,54225,91864,45356,91481,19425,77604,47517,71776,4620,30934,45565,90781,37093,76943,13574,98729,77965,59131,89289,90560,32389,9672,46518,41753,29209,2495,47792,42968,91282,49067,37914,59085,74008,84292,67233,74183,23725,56254,92658,99788,34515,66217,71925,40893,51299,81484,60402,9901,69941,62615,79540,67010,44048,40285,61933,33131,61030,26847,38267,38427,68019,83502,9732,52662,70270,21543,37753,7252,33018,5749,53760,71683,13714,49260,21760,51785,66894,52450,14935,18477,91117,30,76977,19710,66006,83182,92762,57229,88468,72434,27652,87346,22963,88011,5623,4713,79992,50880,60056,3975,68946,61503,75702,45244,4232,8881,49090,78240,16620,71194,8722,63768,81328,67869,94755,58265,37032,31670,18900,18995,24721,99862,44229,92752,34348,70048,53240,72430,92202,23,39098,2296,54344,50705,62238,22891,64614,51416,17044,34322,46537,46347,69272,59164,39471,6751,14592,94381,80003,36032,68365,66010,79635,1807,44614,11912,64073,41726,24672,14534,1574,25125,40782,45502,67397,83307,21575,39320,35386,91056,77997,98801,39821,73186,54941,11349,62029,74196,19504,60240,51942,47773,34788,69325,53380,42650,52871,82481,52491,68674,78422,36432,10157,94419,41688,87639,3077,96721,59910,45456,95875,81095,70111,69307,44742,72594,12507,81902,79885,77407,89444,99409,57156,95938,33157,75193,80765,76643,2134,92626,96024,91929,45526,2481,48015,73769,36175,34060,41579,40862,30987,57966,57108,31040,32493,73768,22647,77611,81269,3727,31319,2788,85671,1550,91767,2060,91174,88800,36027,278,63106,61366,30825,372,58844,37840,74682,69582,8124,89595,25539,69754,58881,97901,65104,37638,42371,80762,78709,67005,2639,93094,7934,48969,24945,17455,54564,94063,31628,79576,62928,12150,44378,5761,83162,58070,83952,76246,89737,44145,35155,31442,98690,4559,78946,37369,18335,99094,70935,30451,91512,52094,56661,99509,32490,72150,98741,60800,61749,37544,836,46036,98983,35019,70565,83783,50794,74373,76206,69330,30868,23631,76206,25542,69841,8844,55755,29633,16328,65545,68985,14548,37910,99093,32966,74018,1496,724,9972,97043,73574,19831,22218,29644,59,64508,63119,7295,90883,902,14472,75683,33424,61851,22662,10033,40416,46165,78138,10410,17872,79971,17276,99185,92762,45786,84684,43650,53479,72079,5388,47521,50266,35462,46298,46748,75288,277,10399,41142,25991,7659,14395,50662,53096,99787,85113,89566,76998,63888,52547,70722,35917,47566,12493,94859,29722,75458,87735,25570,29977,39146,82507,64562,67560,8309,17455,64868,1028,72160,84062,47017,45221,25866,64962,6166,48983,2166,20396,97398,25129,20483,68305,7220,62933,92988,98093,97713,99462,8880,81503,79519,56986,31881,24836,71154,90968,8925,10155,99664,17661,47209,19141,84285,96435,55927,45206,65029,50140,98098,7751,33350,38010,45222,75128,51682,98162,62081,67166,42538,19287,92037,44474,33631,57479,17408,14811,89646,11304,9015,83119,9634,856,50989,85187,72807,50417,13607,40718,90950,33187,20470,42955,93058,88566,9267,37255,94204,11603,62841,1341,45421,27349,60869,56422,52117,46688,56547,97762,82184,80347,86153,14670,71562,38909,19882,25599,51008,554,92186,60684,75790,69902,5906,87784,54386,24828,68036,32485,4641,21748,48737,32362,4673,64196,53621,55092,53621,41687,79798,35481,70316,8771,42738,84152,40746,85060,921,15094,87178,59400,21312,74387,23142,66761,77467,29193,63887,22089,79143,57833,36849,18925,37113,11375,32311,67069,48409,76371,70844,18594,24975,30610,73192,15594,7279,40164,63534,83854,47889,23665,67277,15602,46000,31993,14124,7077,1295,52849,96087,55169,14966,1937,6924,82161,55482,62459,77542,11373,83571,68837,82138,41658,48654,52470,57401,60478,38162,81217,13422,91103,20326,84553,35536,13638,53079,61523,85495,91430,20907,4641,88302,15108,17678,22933,288,68523,46200,56806,38298,53443,70613,62562,32996,87451,90425,82910,76968,42735,45889,5564,79522,53787,52951,52868,61298,24467,33897,19426,95719,75753,24456,38083,8540,73329,85622,91612,68821,95612,63791,85748,23294,39552,88423,35241,14057,3803,22588,45023,72259,95016,5355,5576,2381,95050,30646,84734,25421,17750,77872,48873,5430,26093,27439,23696,35630,73749,90260,52953,4989,28523,50330,58968,33821,2873,52426,60092,74590,66956,70998,35820,31882,26054,26563,27709,5697,94421,33199,52004,45997,7390,78934,39495,95019,67882,70302,31972,33059,38188,92355,4310,82834,64121,99908,55510,38983,75328,59823,38474,35409,69678,94048,89858,23507,8962,66673,61917,37641,21241,64437,67867,57187,29146,83277,14917,80060,71933,62529,42437,66735,38978,77265,54602,15810,27158,13003,47161,5905,97220,75586,13371,66901,78146,85225,49487,29131,26311,44404,50095,75831,17716,80676,71769,62170,3060,44213,83884,2271,28941,83628,67297,35473,9810,41926,91269,94833,91527,14541,3056,54069,22129,23060,18354,70078,47906,95235,33930,62060,39971,81442,13572,30083,37232,52203,14277,68192,86450,96615,88266,17705,40882,8317,17799,42953,78266,68182,91105,307,65762,86308,4501,49946,63575,79013,17145,35705,66913,47836,28979,53816,59823,90208,40034,26375,25131,38148,20604,91033,38677,9427,88544,52195,11291,71449,61757,20198,86680,76855,13902,23841,85046,83161,69578,55935,35367,95540,37388,40601,49481,29562,71865,79124,3467,1374,39768,66346,41237,96376,34719,55147,12156,87346,36489,95540,74797,37542,69489,56788,90254,14856,24395,91439,25953,3030,75424,38736,87710,3721,9944,97187,63589,86659,84201,37363,73009,11055,1279,75794,21122,14557,37020,84352,52697,92986,55206,88807,68447,81550,51892,77535,37285,84591,53494,76655,12243,77960,41655,87936,14095,38370,53638,30388,8438,17631,37950,9973,57453,30387,44642,29163,52334,10004,72158,29199,41541,60221,71064,54534,74471,9314,52249,49077,20818,61399,14838,53537,53617,26242,16163,68726,70052,30511,14824,42981,49817,86533,58579,22946,4592,5424,82076,38727,22929,60457,23953,8640,71032,91248,72338,34507,13618,98256,65892,40759,67481,17027,41414,56238,5704,18903,30758,39670,65266,46412,22397,55411,16550,42192,15645,50637,96232,93465,33712,35829,5753,20259,91147,67798,30498,24047,95231,49812,33776,98921,66478,77036,61962,12944,58802,36798,87314,42400,35436,59065,74320,529,94954,56911,98352,66407,71811,61428,90216,61927,9629,78414,87533,2130,13844,2739,68144,59749,68782,82357,22263,59333,72011,84676,38617,99730,98271,11583,5449,24466,89532,71396,70,92693,47326,89119,31097,7516,10951,78232,31229,69627,25953,27923,39917,93665,36088,91566,65819,35045,81989,15525,42815,45446,57392,23651,76214,20471,50028,91889,55723,59182,87938,75573,88730,80424,80905,32160,75726,84790,90064,38436,65043,8899,13568,20759,40620,55390,42919,65040,39531,74169,72514,62058,7239,65214,9870,91044,89263,54000,66248,60112,50703,60109,27628,24071,4413,56252,9631,93167,39080,13937,21509,61919,52693,95432,91423,69858,90216,1751,41848,40512,16004,41812,33395,6984,61922,92481,6523,90063,85178,78092,84525,54555,20106,79362,41937,40680,58008,28539,31414,85396,46811,14339,19476,65131,11410,12725,69293,95961,70908,69982,29716,76319,1280,41564,59056,19274,34312,22856,17685,52300,39328,90096,76208,42706,79285,31443,64749,46330,70489,28147,13432,61560,45818,50399,85065,69796,99013,4211,50930,6575,44928,19382,73888,23945,60222,55596,38833,38516,35713,43468,1315,37091,52416,89688,27658,49344,3277,6924,24083,4024,45094,46511,95910,41865,1276,31258,98099,54524,55655,80741,77967,87767,86067,11357,63372,48613,32980,96141,48195,10632,78580,14005,6596,56109,958,64485,92986,97413,5373,58154,63043,79631,25745,77215,98929,77738,20535,54438,78645,9919,62495,12414,83591,9693,69570,87240,62730,12750,90921,19100,15691,12141,65027,16469,16275,29422,29579,91147,16486,44133,24889,15823,73537,59632,38869,67303,32253,87318,7449,59751,92000,57284,59879,57547,85419,12956,17892,72398,52079,7696,68938,72632,46570,37455,4847,90911,15298,69960,74862,80849,44691,86966,51584,7027,39784,61434,32931,72636,12106,91949,25868,16181,15711,1398,36397,32480,63799,95284,11559,24211,60099,34886,88922,26551,46385,39425,57722,50215,33376,44508,47020,88448,78103,5111,271,9960,14240,47131,61434,47215,55257,6417,30439,59464,68583,43773,7894,81783,19161,41076,62157,25615,19706,4594,46319,37257,39213,86849,94477,36802,40846,71856,61206,82575,94911,16566,34953,49861,80548,88561,94718,98330,66056,23865,48949,84409,67127,51217,34208,77806,82734,89247,18289,65479,60489,33523,65897,93971,48013,88725,93437,35070,85302,75788,84270,75712,99343,35849,80954,67543,64979,78724,94960,62146,46754,78790,64210,1426,3539,39852,6782,98449,31139,84217,24245,26652,61990,89290,4646,29023,61764,84115,39211,6641,65885,88724,1532,32465,2076,31387,63334,98997,3737,82569,58311,56461,8666,2160,88393,23915,63479,84364,45463,64733,66762,65776,69803,82684,67648,69721,82981,88118,7315,91024,22481,10811,46392,4335,33746,70292,34233,7372,72673,65877,63502,92207,48611,4825,4005,60406,28520,15672,20981,98579,43859,21454,46138,49424,5096,77314,21855,52241,85245,41438,5910,17767,11056,1658,74383,95020,33170,61827,24351,28630,78857,73565,24519,39613,39850,49098,78222,62004,12384,70823,50736,91128,78818,82819,76602,56808,45062,5949,16140,83728,90690,37126,17748,25034,17198,140,64986,37277,84684,38500,85144,70229,67561,85312,22664,97716,79676,56735,71555,44389,40859,99313,93215,215,53499,4981,77081,85505,94974,37356,50899,14165,52473,20842,18063,24077,42600,31831,74702,69822,47774,57847,75284,4221,51211,15898,5846,6761,7236,44461,57589,35286,75191,65596,20583,19437,71837,4555,7100,63527,32833,44053,57011,40634,58428,29431,30463,1193,55281,39153,86994,30185,22081,66485,34032,22519,21799,74942,86685,72624,73936,12792,29561,47571,65188,42324,12950,45723,46150,6526,73437,29723,27386,62676,84701,89394,39305,78134,43147,85629,53964,78979,81195,65863,60695,61927,26684,58651,34614,14697,58233,20356,50110,88109,54215,97414,88645,62213,92478,92034,53193,98721,16421,37804,98510,58299,57558,13455,97737,14057,99604,65994,86153,67272,55621,82583,18364,36714,6979,48761,69118,6811,60381,97943,67346,9201,15301,47501,30010,79740,30878,40696,57019,87286,56184,76333,65022,41991,96417,24238,94972,43582,79752,64115,79161,16824,10262,12206,5864,78301,94350,60832,77655,67572,65833,99506,17100,87807,37696,81933,20085,63746,49165,1962,49837,32257,36210,89085,34859,54635,22291,34452,95021,55980,60352,99429,34016,12184,24039,65951,32879,39819,44390,15488,60109,2250,65366,96790,28477,3101,65864,95909,84900,43296,19649,61896,23238,21883,83856,41951,80073,70217,55637,27776,84955,97233,31659,79663,62480,54910,42329,40308,5110,53744,59013,15541,28405,92828,21449,67874,15321,33049,39504,37615,50391,37307,65416,16304,38357,86734,79546,93282,67193,90754,86931,31588,13000,6884,71565,67520,21148,71447,19868,62259,40917,26731,80988,69143,24415,34276,91080,43835,83296,21658,36551,9081,21128,11569,8783,11178,35165,41674,69369,26315,94647,84601,87098,40285,84766,88971,50583,52459,77660,31150,64481,43667,81166,67013,25659,38168,3906,6001,77015,46570,9818,99108,65659,91776,22567,16943,40380,35876,97739,45779,34494,1175,33688,97673,82939,4797,17673,68318,63741,15092,67403,62171,93647,37611,38823,66052,16070,31630,97142,34754,45263,43574,75819,11586,8377,55469,60822,42990,3121,17208,87130,76657,11331,10329,39243,49047,92234,20450,47257,6783,13608,33385,18497,96710,88915,27737,92357,65528,49701,8700,63148,85518,54204,52750,66585,63378,90728,54041,62584,56387,77629,34688,28587,55506,58607,37020,14849,35642,66760,27094,85536,81623,82269,79952,57518,75732,98821,11282,21708,50659,69105,82248,72216,35096,53729,40435,57711,34978,22487,88482,99431,33159,55036,27791,87780,58914,34225,59246,77153,20312,67532,16414,66885,31514,20352,77226,7184,97663,19982,86141,69841,43923,12305,344,95181,97916,631,48324,86704,20333,68493,62557,6229,40285,33575,60122,35958,44132,75580,18273,32511,7332,40778,61307,38826,15430,5272,58748,2765,84425,37602,71905,50871,19840,15291,20505,56193,75538,86994,96945,89850,93935,47791,34148,83936,56757,50698,72638,51296,63868,71512,64289,16861,54018,17201,47812,35626,72992,11308,87302,71411,27793,77758,1885,98490,42434,52292,20551,81253,36600,26879,71292,46751,98476,47684,20384,60749,93371,55041,31423,48033,15030,69255,78059,15085,97734,20409,99042,9226,39539,65063,15928,61686,52652,25451,38708,25678,76740,59837,2527,99947,49495,10804,14228,98739,1935,80138,78277,25252,65263,22754,94645,64649,2988,589,16807,87639,72962,4202,85100,2565,52295,51005,86616,6615,63571,73961,15299,61909,39942,33179,79553,42456,90042,52345,3623,9190,6751,87268,80056,17307,65339,70152,30028,90663,27947,94233,36203,20312,22578,60781,50402,64770,22741,58316,24762,40033,37858,65451,55924,37534,99646,35116,19876,6332,73889,48003,21625,81732,9129,30648,13203,20011,17916,80115,12108,78876,8516,39464,41078,28200,38436,90383,25873,97190,18158,57244,21031,26680,3992,50990,76496,34162,38304,5096,55735,26267,80526,74898,37240,32162,65571,69621,49903,83551,85054,9842,53489,1422,43963,77659,26865,54481,67693,55902,42595,49745,42001,7877,47271,60199,72747,46409,78426,96231,51161,22179,43747,86724,69346,75197,15754,78972,94887,25979,74197,22549,44259,6630,1264,57163,56186,27236,65341,80380,83245,68321,87749,99314,67032,87421,36086,63435,59477,71354,22197,46526,22336,84842,35152,51577,19450,21407,34209,31621,1970,51608,8186,39645,94610,80118,1181,63198,66244,21551,67514,98781,81496,33265,12263,19389,85896,40260,56074,39478,60499,8466,15322,35168,22061,49372,79995,68183,30470,35873,66253,92112,55239,28734,63869,74422,10327,58745,99513,1851,89919,96368,66541,51372,7425,14975,30662,44248,79217,7575,15461,62990,45577,76424,40555,41462,20780,90780,46334,81230,12844,64025,62602,7801,69675,69425,6809,99807,41450,64339,9553,61207,22305,62112,91728,70450,62487,6882,82249,82352,44667,80189,67688,21590,16017,46361,46685,12810,51417,8725,80639,39093,83757,72362,65684,78483,19829,72481,5195,71221,94648,39825,52115,36722,44492,78173,5636,83113,46509,66445,74201,13467,51882,27952,19941,2290,58966,58705,27279,6701,63675,69584,32431,32362,24999,64996,5941,38935,46882,69950,93729,84902,93202,79468,96861,92789,83527,14532,36853,50228,30072,1406,27019,95524,89877,9354,60251,60112,83493,60016,49557,78042,35210,60794,14965,56891,78142,46680,65392,12355,25844,53129,28127,83808,62718,430,72997,45391,70729,44967,93516,46554,34551,63939,11622,35100,43476,31195,88034,44604,56542,28924,9553,86181,55787,44092,75674,8530,51856,30789,63315,67796,10190,54537,81428,62298,2826,80967,54956,13040,74375,37974,22827,80755,6767,24337,62940,35470,60734,98327,550,82739,66199,96684,35874,14648,24666,71733,73243,66182,98242,16020,33092,6553,64541,41616,62969,90440,23874,18308,63988,61921,71731,96330,81274,24542,21093,45101,97560,63704,71507,2632,44914,85538,56867,61886,9895,59770,91508,31713,1648,79980,38503,54297,53649,49342,57272,96315,82835,21519,34409,70242,83629,2125,24545,29118,50842,91458,48664,65802,81514,55461,79547,76147,93929,93834,87581,81279,2369,73991,64939,80337,16143,93021,52943,34972,64429,46756,69125,91955,21872,57040,8277,82102,15986,33022,30067,28810,39087,20150,91277,26679,91667,77151,97365,71976,6151,25083,94425,25161,6358,19464,36560,60205,46734,80530,90413,71582,10605,4573,50793,75696,4142,94,47430,97037,12452,50719,93703,92317,98179,56545,13635,54749,55772,41390,84185,65341,90568,8566,40283,11947,4181,22598,3789,57401,7874,68754,64245,13834,81532,45248,44716,43018,7677,98844,8325,66173,90398,11773,20168,62552,35592,17722,90476,80442,16750,60557,78518,51390,80931,64500,23618,78859,67890,24760,71865,26988,3336,28811,20466,1458,86352,11701,73112,22574,48044,10514,11157,50814,12299,76122,8159,84949,11641,65478,76282,4254,32553,8659,40415,40736,70092,6880,57519,30762,23437,93521,95582,34410,87433,40356,36619,27172,8729,53434,94080,91305,34359,9910,57695,19429,69541,92358,54737,107,52505,20220,41853,59256,15869,12962,98921,53892,92398,8873,89592,40109,77779,24441,24548,38231,68151,84184,63291,18312,84807,27431,43834,29498,96210,26227,43342,68259,2223,59505,36815,87077,42863,35869,19938,29866,90696,85565,4391,42027,1767,86753,59549,49910,15950,74330,85610,71181,31813,34641,43897,69463,74265,122,50100,87541,2636,19103,10499,83777,26702,6900,50459,69182,71281,76128,3679,45613,91349,80008,26930,62451,96895,82128,26558,84845,11932,60692,49608,98772,65609,33281,19180,73051,68029,97250,33612,29706,66441,88862,46343,35617,68696,31902,49127,46744,98216,66352,75962,98126,687,23130,77455,60698,15602,85475,41110,34790,20172,72425,97478,87163,69627,22512,54505,57425,2628,3435,77765,76876,82620,46507,1768,61103,11723,22407,39724,30084,52989,41098,8072,91150,40577,12416,59169,57222,71173,48755,85597,9720,32363,83050,22514,42876,10692,15866,22845,73653,5473,81825,89923,34049,89427,97336,15802,50059,74560,92333,21117,88035,23304,99272,72767,55202,18104,834,4967,43767,22128,79102,24115,87726,52628,75631,87913,31640,50689,99790,21459,26636,39516,820,60504,57633,49557,5486,13090,56078,15882,1801,82471,29914,79397,30762,24029,78630,13711,52041,99675,14764,54417,80684,24699,45287,57680,14773,48437,69743,79483,81854,65548,47088,56030,18712,40025,12189,11977,36866,9402,54647,57579,5240,71557,21134,71560,33030,69395,51564,2841,95428,41459,42797,2729,97963,14516,54867,11646,28343,75107,77785,67319,94933,48069,79944,63892,31982,20253,56685,98010,37426,22251,81046,27472,34048,42116,28147,70139,83482,60935,13322,44296,18351,66464,1989,65029,18875,71808,6522,21280,95759,73978,99212,78241,97990,85089,94966,12160,54424,8410,75102,82116,86704,9406,64458,33661,77597,67972,55012,9965,62058,45513,34931,95936,64923,40570,59206,26266,94026,97975,96992,83193,27368,87254,83971,23368,25015,7945,14194,88599,95306,99719,96550,42961,22015,90071,90435,78000,35605,44032,48207,18629,80421,31648,17061,3805,15114,65016,24681,92855,86274,30927,46413,93204,41627,90116,82316,79212,4821,67308,96416,24863,46486,28084,91464,90111,51754,88853,93820,27547,58952,11521,11136,63803,60684,31816,78000,71484,77553,32201,21700,88097,13442,84322,85018,39427,73720,88886,64031,40618]
pipes = [[2,1,29701],[3,2,97954],[4,3,14600],[5,2,22110],[7,3,14412],[8,6,35742],[9,4,2560],[10,4,30822],[11,5,24812],[12,1,30583],[13,6,70159],[14,10,51992],[15,8,1765],[16,11,68435],[17,13,26992],[18,15,46846],[19,4,73049],[20,4,96887],[21,9,7896],[22,21,87635],[23,15,29266],[24,4,65804],[25,22,38626],[26,1,23842],[28,12,13943],[29,8,95626],[30,23,90303],[31,9,86959],[32,24,56993],[33,22,65478],[34,32,1745],[35,20,87906],[36,26,95020],[37,1,32630],[38,19,92967],[39,38,82313],[40,27,73805],[41,15,48963],[42,1,99891],[43,19,34162],[44,14,38087],[45,22,57011],[46,7,18719],[47,37,77119],[48,8,19496],[49,46,21348],[51,2,47063],[52,45,49034],[53,23,75239],[54,48,41897],[56,35,66432],[58,3,84395],[59,29,4102],[60,27,35845],[62,9,47710],[63,61,11283],[64,26,91789],[65,59,43149],[66,65,68682],[67,39,30519],[69,61,5680],[70,32,43347],[71,66,84590],[72,59,52431],[73,48,19756],[74,19,67496],[75,57,87276],[76,68,40757],[77,28,74982],[78,71,91211],[79,62,29080],[80,46,88674],[81,74,11198],[82,31,25532],[83,79,43847],[84,7,50451],[85,64,65665],[86,56,39159],[87,82,52212],[88,64,83462],[89,29,97270],[91,83,33747],[92,44,89431],[93,68,84122],[95,28,79585],[96,36,28242],[97,45,62811],[98,16,48256],[99,51,74903],[100,80,55040],[101,83,38146],[102,70,77445],[103,43,93110],[105,31,9519],[106,34,34595],[108,101,87776],[110,36,37237],[111,28,90199],[112,62,74313],[113,82,33433],[114,32,61179],[115,80,84718],[117,86,13810],[118,54,28086],[119,59,85376],[120,115,86910],[121,53,51790],[122,28,29483],[123,101,91981],[125,124,9644],[126,7,73538],[128,1,23474],[129,25,60080],[130,81,75982],[131,56,81311],[132,115,74974],[133,109,69225],[134,128,25868],[135,111,19432],[136,118,92403],[137,132,86544],[138,38,22154],[139,88,31854],[140,98,10505],[142,64,60703],[143,66,80450],[144,17,71173],[145,83,91572],[146,22,58132],[147,87,77145],[148,82,48422],[149,43,95248],[150,77,61978],[151,80,76738],[152,56,73090],[153,46,25304],[154,47,79013],[156,66,77969],[157,147,19431],[158,97,38563],[160,60,12524],[161,64,31287],[162,80,76222],[163,143,32137],[164,66,43935],[165,97,24477],[166,159,94280],[167,9,76610],[168,167,89761],[169,27,67260],[170,34,25753],[171,72,92307],[172,170,67935],[173,9,7909],[174,46,67848],[175,93,78956],[176,72,75403],[177,89,21917],[178,100,82491],[179,40,67362],[180,62,5625],[181,84,69468],[182,100,9665],[185,109,95626],[186,174,62761],[188,70,44662],[190,170,40719],[191,2,35724],[193,81,35502],[194,106,40461],[195,76,55697],[196,97,48251],[197,186,31468],[198,38,28268],[199,131,92398],[200,98,93630],[201,74,32983],[202,129,24667],[203,160,98338],[204,157,83564],[205,76,34448],[206,144,18206],[207,54,21036],[208,154,5146],[209,17,88955],[211,110,81634],[212,169,49900],[213,137,49365],[214,2,2743],[216,102,70265],[217,6,7473],[218,158,93597],[219,218,82025],[220,57,93583],[221,115,17511],[222,149,43398],[223,20,39531],[224,128,98386],[225,158,2478],[226,129,54275],[227,84,57178],[228,76,67722],[229,181,7814],[230,144,31563],[231,176,80503],[232,121,7717],[233,174,77445],[235,197,58732],[236,208,40261],[237,102,36161],[239,113,98426],[240,43,5950],[241,234,44069],[242,215,20773],[244,123,93483],[245,21,24114],[246,16,56866],[247,102,83242],[248,207,42104],[249,228,33878],[250,33,70301],[252,43,12339],[253,56,66095],[254,181,63542],[255,62,80572],[256,254,49480],[257,202,28620],[258,175,21567],[259,202,15336],[260,145,288],[261,198,40005],[262,189,68227],[263,232,50808],[264,72,26817],[265,98,22347],[267,112,30440],[268,18,40357],[269,116,96319],[270,167,82039],[271,207,37300],[272,7,84935],[273,137,26433],[274,195,20485],[275,155,3191],[276,213,1272],[277,207,59565],[280,144,99450],[281,59,91363],[282,201,46568],[283,263,40687],[284,274,43775],[285,159,71086],[286,85,17273],[287,87,16543],[289,10,22913],[290,6,75939],[291,36,94387],[292,30,65390],[293,6,90943],[294,201,39029],[295,49,38436],[296,223,55232],[297,24,21950],[298,273,29748],[299,11,79111],[300,239,1199],[301,157,58695],[302,112,95981],[303,14,27236],[304,40,33067],[305,132,9606],[306,167,91230],[307,176,73139],[308,103,41108],[309,260,57700],[310,34,29689],[311,99,85869],[312,247,27571],[313,91,41434],[314,114,39683],[315,24,81282],[316,205,46011],[317,24,18774],[318,289,49499],[319,35,35476],[320,283,37597],[322,257,74891],[325,322,47060],[326,123,7734],[327,273,48920],[328,239,97516],[329,249,15805],[330,260,52246],[331,138,43065],[332,234,95662],[333,301,89573],[334,148,58966],[335,175,22325],[336,288,90484],[337,211,52119],[338,49,62983],[340,59,75987],[341,180,78837],[342,13,24516],[343,60,20307],[344,263,58061],[345,20,9200],[346,205,71813],[347,258,77860],[348,92,46561],[349,155,95175],[350,14,12250],[351,280,92298],[352,117,33641],[353,329,68156],[354,263,36526],[355,270,15815],[356,23,42824],[357,131,24866],[358,237,64697],[359,350,55239],[360,341,9797],[361,136,89544],[362,73,43099],[363,207,24181],[364,218,39615],[365,273,73084],[366,6,864],[367,84,95143],[368,161,43237],[369,200,60149],[370,207,10091],[371,248,66612],[372,213,87257],[373,318,91407],[374,207,19219],[375,155,27372],[376,223,74371],[378,18,10894],[379,118,87212],[380,110,7549],[381,18,27123],[382,71,61836],[384,211,31408],[385,88,10431],[386,329,86168],[387,88,62655],[388,199,73434],[389,321,31213],[390,285,46509],[392,157,90573],[393,186,86501],[394,251,52085],[396,213,65834],[397,14,10629],[398,196,58288],[399,189,401],[400,29,42458],[401,24,39544],[402,247,33084],[403,293,90173],[404,391,70184],[405,359,19688],[406,230,49814],[407,233,58833],[408,294,40730],[409,302,26525],[410,10,91271],[412,16,92204],[413,89,19524],[414,289,39725],[415,66,20872],[416,352,10054],[417,277,31609],[418,310,93549],[419,138,79149],[420,48,84584],[421,257,49358],[422,391,46325],[423,359,68552],[424,232,17270],[425,384,91335],[426,144,83940],[427,90,98409],[428,396,16607],[429,370,78462],[430,33,8117],[431,152,9942],[432,424,69632],[433,30,85543],[434,11,5518],[436,25,81096],[437,292,82378],[438,51,77858],[439,123,85794],[440,248,42626],[441,387,63758],[442,214,10604],[443,215,41259],[444,116,54035],[445,114,32353],[446,14,49598],[447,315,63931],[448,291,54049],[449,111,30348],[450,158,17723],[451,82,14597],[452,83,69424],[453,327,86389],[454,151,81918],[456,189,96242],[457,70,19960],[458,16,28629],[459,237,93187],[460,159,73685],[461,58,68123],[462,235,56918],[463,398,76694],[464,150,63931],[465,74,442],[466,56,52931],[468,425,4396],[469,194,11191],[470,149,43474],[471,363,54711],[472,172,6269],[473,83,33667],[474,318,6513],[475,405,22053],[476,122,40227],[477,402,19227],[478,95,81886],[479,434,67585],[480,325,57839],[482,471,37584],[483,4,30096],[484,111,47372],[485,203,38591],[487,366,6141],[488,377,54203],[489,59,94963],[490,418,23678],[491,307,17408],[492,14,670],[493,437,77484],[494,280,82940],[495,464,13894],[496,450,64933],[497,305,93614],[498,25,53580],[499,132,39323],[500,371,98644],[501,135,41593],[502,231,47066],[503,87,79360],[504,159,9643],[505,275,38103],[507,248,54503],[508,84,49864],[509,454,71371],[510,380,50407],[512,205,67971],[513,467,97712],[514,374,33787],[515,15,28496],[516,60,76410],[517,348,99842],[518,260,89522],[520,113,47359],[521,198,10440],[522,296,69916],[523,63,83891],[524,60,89749],[525,391,54598],[526,175,5846],[527,153,82127],[528,383,38116],[529,117,16063],[530,399,79407],[531,161,22127],[532,230,76530],[533,85,80682],[534,273,89040],[535,495,80592],[536,109,39842],[537,57,15843],[538,250,96631],[539,493,83378],[540,263,69270],[541,35,61883],[542,159,57255],[543,95,27086],[544,215,6833],[545,73,35151],[546,243,9761],[547,452,24856],[548,84,31190],[549,12,6332],[550,405,37086],[551,301,66342],[552,401,63657],[553,276,66852],[554,22,3203],[555,228,37338],[556,176,22010],[557,381,66493],[558,75,66208],[559,364,23276],[560,97,24924],[561,233,4120],[562,107,67561],[563,509,1127],[564,311,86892],[565,147,13695],[566,440,92139],[567,200,90507],[568,252,74287],[570,161,92535],[571,131,27284],[572,155,30566],[573,462,48670],[574,100,90645],[576,276,85273],[578,174,49536],[579,111,21260],[580,398,2781],[581,525,26866],[582,544,32893],[583,444,84344],[584,387,269],[585,468,60842],[586,6,37878],[587,575,473],[588,106,22681],[589,185,27433],[590,84,85785],[594,227,89197],[595,194,69847],[596,33,39904],[597,105,19893],[598,45,56127],[599,435,44844],[600,415,50742],[601,556,36073],[602,558,1862],[603,448,48189],[604,373,89934],[605,530,56263],[606,106,66369],[607,462,29590],[609,343,81848],[610,495,3754],[611,403,68366],[612,356,4017],[613,400,25750],[614,465,9509],[615,435,72301],[617,541,79715],[618,424,63813],[619,81,47434],[620,152,47336],[621,251,98166],[622,299,69146],[623,201,40688],[624,542,15112],[625,504,14571],[626,276,28761],[627,84,40084],[628,538,82437],[630,104,83542],[631,62,42658],[632,195,89255],[633,197,80995],[634,96,34338],[635,511,49439],[636,200,40008],[637,321,82504],[638,9,63149],[640,191,31739],[641,205,86562],[642,315,89392],[643,199,63529],[644,275,27480],[645,616,68136],[647,227,36936],[648,268,35143],[649,36,63126],[650,370,84467],[651,149,31083],[653,26,53705],[654,12,56687],[656,163,51549],[657,167,88308],[658,78,60839],[659,324,13943],[661,584,22839],[662,317,89481],[663,163,94885],[664,288,85440],[666,607,6691],[667,598,31223],[669,217,86682],[670,414,23396],[671,418,69334],[672,670,91478],[673,53,4716],[674,559,16589],[675,207,79062],[676,111,8928],[677,543,143],[678,229,16663],[679,49,81576],[680,661,70081],[681,667,24489],[682,553,2672],[683,140,69484],[684,102,59446],[685,363,25059],[686,6,62764],[687,283,1382],[688,397,52696],[689,230,58658],[690,176,90750],[691,608,1352],[692,159,36270],[693,450,84846],[694,344,81318],[695,116,90234],[696,404,9678],[697,519,29049],[698,571,94453],[699,644,96083],[700,86,46202],[701,613,99800],[703,277,38585],[704,73,86680],[705,197,43707],[706,25,80290],[707,117,46972],[708,600,85249],[709,90,76060],[710,216,49175],[711,254,96266],[712,266,90950],[713,225,95130],[714,295,35117],[715,16,94893],[716,331,13503],[717,659,11049],[719,590,90696],[720,491,72175],[721,316,86679],[722,12,39992],[724,177,11080],[725,568,26948],[726,322,27388],[727,76,27145],[728,266,75008],[729,527,12535],[730,214,12371],[731,373,39834],[732,454,75407],[734,42,70914],[736,101,54875],[737,4,20308],[738,565,85175],[739,123,60506],[740,631,27167],[741,468,61452],[743,442,35717],[744,693,29093],[746,662,31228],[747,617,19151],[748,246,97610],[749,674,21279],[750,387,46233],[751,546,39260],[752,310,69736],[753,566,9630],[754,257,31508],[755,668,84160],[756,35,44004],[757,658,83541],[759,634,3962],[760,347,75632],[761,49,79612],[762,192,24690],[763,148,4250],[764,244,97829],[765,421,1066],[766,580,7403],[767,216,37692],[769,711,22687],[770,404,88094],[771,671,56506],[772,755,58255],[773,145,52921],[774,609,83960],[775,680,96586],[776,703,99713],[777,599,52375],[779,144,44736],[780,632,89757],[781,36,5731],[782,351,30366],[783,771,57131],[784,310,96517],[785,535,99809],[786,619,68589],[787,396,71272],[788,431,57985],[789,129,27813],[790,369,70912],[791,281,92982],[792,98,86659],[793,546,48369],[795,97,67407],[796,477,37718],[797,66,87249],[799,783,12823],[801,437,3536],[802,11,26811],[804,564,91098],[805,482,27517],[806,168,52001],[807,265,21530],[808,53,76266],[809,361,80924],[810,182,37370],[811,154,37112],[812,556,41290],[814,789,74920],[815,800,56751],[816,729,91805],[817,107,94232],[818,154,80731],[819,268,14737],[820,375,4887],[821,350,44481],[822,72,89776],[823,20,7873],[824,132,21003],[825,528,54949],[826,315,47200],[827,111,74453],[828,512,94245],[829,577,59313],[830,603,14129],[831,690,26452],[832,413,54620],[833,52,58054],[834,711,49276],[835,469,56280],[836,296,9619],[837,371,27141],[838,707,78638],[839,762,2177],[840,136,98498],[841,175,16939],[842,529,10712],[843,320,905],[844,585,66020],[845,450,80295],[846,381,90691],[847,552,28690],[848,619,83992],[849,824,55730],[850,363,43803],[852,52,7169],[853,444,56336],[854,476,56690],[855,622,85799],[857,774,96678],[858,424,49449],[859,402,3931],[861,59,68039],[862,283,69225],[863,668,97126],[865,837,35760],[866,562,10999],[867,185,55837],[868,404,30814],[869,287,24810],[870,208,77452],[871,478,32911],[872,249,33444],[873,245,42964],[874,426,98778],[875,697,70505],[876,128,31069],[877,23,62263],[878,589,55131],[879,355,41219],[880,458,2034],[881,714,40486],[882,854,62129],[883,558,53708],[884,847,7965],[885,816,91508],[886,95,47778],[887,267,10275],[889,206,26758],[890,593,30540],[891,346,26922],[893,568,94898],[894,695,17579],[895,274,37331],[896,454,60062],[897,258,78827],[898,203,1875],[899,665,47366],[900,442,93689],[901,200,13272],[902,792,78544],[903,266,40584],[904,792,47387],[905,659,3977],[906,298,4860],[907,543,6421],[909,40,98118],[910,286,76139],[911,471,29719],[912,873,2364],[913,844,78385],[914,473,29835],[915,89,15642],[916,589,32405],[917,3,41852],[918,517,63462],[920,377,28193],[921,6,27665],[922,32,93519],[923,422,14364],[925,229,37296],[926,804,10557],[927,83,65851],[928,108,99995],[929,680,12483],[930,191,37164],[931,279,80905],[932,409,14026],[933,61,48110],[934,540,10928],[937,446,29444],[938,360,7819],[939,471,40358],[940,357,82004],[941,819,38581],[942,392,47984],[943,652,30315],[944,605,38241],[945,397,29448],[946,915,19271],[947,654,31385],[948,715,63095],[949,44,98741],[950,108,54784],[951,179,54911],[953,849,10447],[954,339,60110],[955,475,72751],[956,511,27716],[957,510,45488],[958,501,75688],[959,167,10869],[960,786,81730],[961,925,29497],[962,882,37332],[963,756,42589],[964,695,69758],[965,376,53733],[967,277,40479],[968,637,55910],[969,676,52573],[970,500,35584],[971,866,76667],[972,335,78272],[973,864,7167],[975,656,95465],[976,763,7587],[977,946,9897],[978,572,15324],[979,791,68963],[980,716,19826],[981,756,41327],[982,462,70389],[983,694,19291],[984,759,26277],[985,269,96491],[986,171,72450],[987,335,33769],[988,28,39546],[989,598,66509],[990,815,58010],[994,246,98920],[995,952,92564],[996,47,19089],[997,815,52660],[998,385,35303],[999,136,34329],[1000,648,73110],[1001,243,253],[1002,729,4232],[1003,918,54115],[1004,636,45857],[1005,254,49621],[1006,915,61092],[1007,664,36241],[1008,114,349],[1009,712,73965],[1010,387,63796],[1011,236,65490],[1012,299,40970],[1014,1007,61676],[1015,597,52376],[1017,549,34623],[1019,1016,25540],[1020,136,52380],[1021,791,24927],[1022,950,98601],[1023,226,1562],[1024,614,9795],[1025,335,30621],[1026,593,56947],[1027,241,25080],[1028,809,15470],[1029,650,35961],[1030,893,69249],[1031,763,46677],[1032,372,78156],[1033,302,99773],[1034,883,85578],[1035,174,10140],[1036,243,41563],[1039,672,15448],[1041,477,40427],[1042,440,94716],[1044,583,53646],[1045,859,43109],[1046,901,19523],[1047,60,71221],[1048,187,43065],[1049,564,10769],[1050,956,54814],[1051,65,81441],[1052,206,18660],[1053,862,27312],[1054,515,6942],[1055,99,37047],[1056,616,41243],[1057,284,33037],[1058,131,17012],[1060,132,60120],[1061,424,65010],[1062,710,6555],[1063,45,30469],[1064,1039,6630],[1065,852,76908],[1067,7,24572],[1068,558,80311],[1069,598,71619],[1070,387,76514],[1072,1,32379],[1073,590,16104],[1074,107,75199],[1075,712,19319],[1076,907,30853],[1078,913,31911],[1079,218,59986],[1080,1051,55941],[1081,54,61929],[1082,367,91902],[1083,924,43746],[1084,906,32858],[1085,183,34173],[1086,375,76226],[1087,145,49924],[1088,714,79879],[1089,170,54271],[1090,726,74548],[1091,798,77665],[1092,60,20506],[1093,330,92122],[1094,1017,5796],[1095,128,27394],[1098,878,52275],[1100,1048,47647],[1101,196,64291],[1102,29,57258],[1103,771,92395],[1104,292,70900],[1105,168,54614],[1106,340,13278],[1107,208,8970],[1108,727,15460],[1109,186,88394],[1110,872,17516],[1112,368,77577],[1113,633,97694],[1114,794,38711],[1115,524,69183],[1116,963,84105],[1117,564,47890],[1118,635,33266],[1119,164,77356],[1120,228,13586],[1121,32,8476],[1122,23,97217],[1123,431,34665],[1124,170,93412],[1125,703,87334],[1126,694,81732],[1127,1068,44270],[1129,538,29571],[1130,227,81957],[1131,198,52023],[1132,1072,16762],[1134,354,99400],[1135,699,46641],[1136,860,68542],[1137,903,34620],[1138,953,21122],[1139,339,99965],[1140,103,63619],[1141,1044,12550],[1142,470,65787],[1143,9,31441],[1144,897,43870],[1145,14,986],[1146,31,54773],[1147,958,70162],[1148,32,57184],[1149,1067,90143],[1150,11,91283],[1151,1131,13805],[1153,1005,7617],[1154,565,87674],[1155,298,23108],[1156,108,98725],[1157,45,50697],[1158,345,48048],[1159,1089,60324],[1160,912,93321],[1161,374,64541],[1162,959,38489],[1163,1007,41410],[1164,623,5394],[1166,989,12787],[1167,723,63135],[1168,201,23866],[1169,883,18205],[1170,198,90217],[1171,197,82553],[1172,1157,7973],[1173,620,76527],[1174,636,56597],[1175,971,93165],[1176,1058,47951],[1177,925,83025],[1178,1150,41159],[1179,41,1078],[1180,799,18057],[1181,600,23302],[1182,247,30510],[1183,474,57167],[1184,676,7280],[1185,254,95340],[1186,167,42062],[1187,422,44527],[1188,780,53033],[1189,353,53881],[1190,155,96905],[1191,132,20734],[1192,192,35140],[1194,184,61905],[1195,973,58618],[1196,32,83850],[1197,30,50460],[1198,345,64698],[1199,18,94838],[1200,684,68193],[1201,390,8651],[1202,258,16873],[1203,1139,70156],[1204,349,36248],[1205,290,66944],[1206,151,75048],[1207,1165,76526],[1208,747,39739],[1209,28,22937],[1210,1005,27970],[1212,1142,75522],[1213,297,43061],[1214,240,14018],[1215,602,61861],[1216,918,96330],[1217,588,44728],[1218,334,23323],[1219,754,5042],[1220,112,54984],[1221,903,90527],[1224,718,9033],[1225,536,51629],[1226,570,34668],[1227,956,39257],[1228,884,52639],[1229,782,89207],[1230,35,5296],[1231,438,80879],[1233,440,68052],[1234,883,63213],[1235,179,96141],[1236,250,73533],[1237,358,83354],[1238,370,68408],[1239,878,58233],[1240,911,75054],[1241,812,64691],[1242,914,50680],[1243,770,50604],[1244,382,93458],[1245,575,81450],[1246,266,68534],[1247,527,48772],[1248,887,73895],[1249,617,24064],[1251,1245,66677],[1252,352,14790],[1253,647,85873],[1254,572,48089],[1256,186,28579],[1257,23,28457],[1259,612,11103],[1260,962,32647],[1262,195,36388],[1263,829,6800],[1264,891,63034],[1265,870,12964],[1266,1047,76341],[1267,587,90963],[1268,1175,58228],[1269,185,32283],[1270,687,10575],[1271,495,2904],[1272,492,50018],[1274,365,82885],[1275,153,99206],[1276,1103,63921],[1277,442,66754],[1278,1130,66462],[1279,150,96234],[1280,1150,26666],[1281,265,57581],[1282,1027,19629],[1283,260,30112],[1284,1186,10459],[1285,508,44090],[1286,831,16935],[1287,490,60197],[1288,187,34489],[1289,1211,49901],[1290,562,21963],[1292,842,71198],[1293,151,54976],[1294,369,55682],[1295,1031,59409],[1296,1285,89738],[1297,175,84503],[1298,741,73013],[1299,709,71213],[1300,163,69017],[1301,377,97307],[1302,644,47722],[1303,48,28534],[1304,802,93100],[1305,183,2428],[1306,143,81543],[1307,744,79377],[1308,1214,69952],[1309,526,98676],[1310,928,23610],[1311,485,32011],[1312,108,16640],[1313,345,59724],[1314,1180,84764],[1315,429,91811],[1316,687,93029],[1317,949,45024],[1318,1021,27980],[1319,555,66426],[1320,1032,62081],[1321,1229,10487],[1322,1159,79489],[1323,748,39178],[1324,461,74206],[1325,726,63863],[1326,909,35424],[1328,635,7373],[1330,583,42792],[1331,856,56747],[1332,1150,66011],[1333,36,82499],[1334,346,69398],[1336,858,65762],[1337,561,60177],[1338,59,45990],[1339,282,48320],[1340,599,97713],[1341,618,88764],[1342,1107,58341],[1343,262,29797],[1345,92,56620],[1346,1312,82414],[1347,70,10017],[1349,1290,64396],[1351,1276,23275],[1352,48,33954],[1353,476,34075],[1354,1125,78417],[1355,1173,62803],[1356,1088,43520],[1357,146,52854],[1358,107,34794],[1359,918,51614],[1360,1353,89488],[1361,908,38986],[1362,38,66757],[1363,268,68867],[1364,1148,44073],[1365,532,72633],[1366,1136,89196],[1367,476,85687],[1368,602,46477],[1369,559,29730],[1370,916,76393],[1371,19,90839],[1372,309,41194],[1373,1121,24041],[1374,681,43968],[1375,613,97413],[1376,1362,65842],[1377,1022,43786],[1378,1174,18594],[1379,58,74456],[1380,1226,41188],[1381,501,84327],[1382,1097,4589],[1383,833,36410],[1384,598,35702],[1385,767,34017],[1386,444,89645],[1388,542,80738],[1389,90,2826],[1390,213,85347],[1391,734,29166],[1393,1072,96409],[1394,553,8271],[1395,179,2230],[1396,1261,73418],[1399,246,8017],[1400,1092,65801],[1401,678,23508],[1402,629,82835],[1404,200,32942],[1405,40,8439],[1406,58,33343],[1407,1090,17269],[1408,740,976],[1409,948,83076],[1410,1327,81532],[1412,75,25421],[1414,1155,43856],[1415,899,40062],[1416,367,96287],[1417,683,2662],[1418,313,84453],[1419,389,97720],[1420,752,29772],[1421,832,74634],[1422,1160,33370],[1425,903,2804],[1426,550,75637],[1427,1040,87343],[1428,1366,14820],[1429,415,38785],[1430,1278,13487],[1431,567,48949],[1432,807,47795],[1433,1353,93754],[1434,870,28897],[1435,1155,12717],[1436,1054,93012],[1437,1258,70247],[1438,451,68809],[1439,402,16171],[1440,99,88933],[1441,1178,87030],[1442,559,4278],[1443,352,25616],[1444,42,61937],[1445,1367,56804],[1446,598,80587],[1448,52,38880],[1449,419,57168],[1450,1262,70998],[1451,800,54836],[1453,1206,61736],[1454,597,84984],[1455,1010,52546],[1456,1290,21483],[1457,1088,5940],[1458,169,99165],[1459,434,29613],[1460,957,39580],[1461,1154,27998],[1462,914,61685],[1463,638,74288],[1464,462,20328],[1465,146,2969],[1466,55,24516],[1467,1355,23636],[1469,260,5032],[1472,338,79476],[1474,794,31960],[1475,333,79966],[1476,941,34278],[1478,704,43050],[1479,114,61231],[1480,801,98296],[1481,718,2047],[1482,926,32449],[1483,443,67181],[1484,1413,91672],[1485,1426,32972],[1486,278,15385],[1487,1401,6861],[1488,677,90408],[1489,1096,21566],[1490,1452,13504],[1491,331,29763],[1492,6,76860],[1493,871,30081],[1494,172,19984],[1495,1094,57495],[1496,1184,63029],[1497,1354,12928],[1498,835,10344],[1499,895,81466],[1500,1349,70236],[1501,305,20088],[1502,412,53171],[1503,659,78764],[1504,655,25927],[1505,147,38445],[1506,1015,69892],[1507,1128,65963],[1508,1355,70080],[1509,173,88174],[1510,758,51179],[1511,1008,27401],[1513,878,94695],[1514,406,31807],[1515,666,43755],[1516,939,81974],[1517,1016,8833],[1518,470,30755],[1519,50,41369],[1520,275,37836],[1521,308,42015],[1522,1412,15351],[1523,285,57167],[1524,302,49610],[1525,1,69359],[1526,1374,46559],[1527,1251,61083],[1528,927,76517],[1529,931,1797],[1530,863,2776],[1531,917,16816],[1532,335,78842],[1533,34,63889],[1534,1219,39489],[1535,722,17797],[1536,1491,68888],[1537,682,52521],[1539,445,44975],[1541,278,57900],[1542,599,18322],[1543,651,9503],[1544,258,62891],[1545,928,14523],[1546,94,71006],[1547,731,77685],[1548,236,28309],[1550,1539,17487],[1551,868,17325],[1552,182,60064],[1553,953,76168],[1554,482,26176],[1555,1554,41161],[1556,656,22451],[1557,194,83440],[1558,48,79620],[1559,478,82745],[1560,744,75405],[1561,1542,32021],[1562,193,82402],[1563,1415,10703],[1564,607,66823],[1565,865,15584],[1566,500,22983],[1567,637,41866],[1568,1133,14499],[1569,1374,48085],[1570,1099,29295],[1571,1061,58459],[1572,1427,50462],[1575,148,90079],[1576,521,5616],[1577,12,68623],[1578,984,19385],[1579,875,28175],[1580,1231,46757],[1582,701,82062],[1583,1072,55296],[1584,76,43894],[1585,1254,11593],[1586,978,36258],[1587,1549,33871],[1588,1145,30375],[1590,1458,81330],[1591,405,91717],[1593,1196,67665],[1594,1516,73678],[1595,1376,52548],[1596,511,45447],[1597,648,93984],[1598,36,63235],[1599,1141,70106],[1600,1538,93658],[1601,176,14318],[1602,683,96433],[1603,6,3566],[1604,1420,99651],[1605,356,1541],[1607,200,76994],[1608,414,42643],[1609,209,31702],[1610,470,12815],[1611,123,25495],[1612,1477,25216],[1613,1176,57217],[1614,314,42236],[1615,925,75626],[1616,356,36801],[1617,823,26946],[1619,1341,29413],[1620,595,56242],[1621,328,79397],[1622,1383,53379],[1623,233,23336],[1624,1541,33776],[1625,167,6950],[1626,638,696],[1627,164,48174],[1628,875,59196],[1629,463,65835],[1631,1003,61351],[1632,392,89541],[1633,522,68950],[1634,180,32748],[1635,1012,81843],[1637,719,4490],[1639,1160,72524],[1640,409,70036],[1642,627,52589],[1643,740,90447],[1644,1208,47774],[1645,1320,54053],[1646,1106,69528],[1647,1150,40525],[1648,1597,18],[1649,380,46144],[1650,90,68408],[1651,1529,47294],[1652,1079,62853],[1653,1150,54067],[1654,1164,67504],[1655,351,20105],[1656,1452,78587],[1657,1396,90214],[1658,278,59150],[1659,1414,80567],[1660,1021,32666],[1661,696,96834],[1662,413,38056],[1664,644,1247],[1665,432,9235],[1666,169,43365],[1668,776,98200],[1669,1206,22963],[1670,1130,56242],[1671,1171,6574],[1674,1363,10204],[1675,1195,84699],[1676,234,93506],[1677,235,70722],[1679,1394,45292],[1680,1529,51633],[1681,1135,47195],[1682,542,35535],[1683,324,33859],[1684,233,52359],[1685,1378,39501],[1686,159,61649],[1687,422,17006],[1688,1635,32544],[1689,623,56884],[1691,803,83],[1692,758,71204],[1693,878,76748],[1694,1604,41796],[1695,1471,73553],[1696,1344,29484],[1697,706,49328],[1698,1061,77002],[1700,701,38657],[1701,961,11891],[1702,1403,76762],[1703,1473,18829],[1704,858,34472],[1705,1107,83820],[1707,103,21744],[1708,1373,87699],[1709,1254,53381],[1710,224,78040],[1711,1026,91283],[1712,1510,81904],[1713,1218,71910],[1714,881,74307],[1715,450,17008],[1716,524,20758],[1717,1077,18922],[1718,426,6142],[1719,1673,24708],[1720,1672,22673],[1721,1061,51947],[1722,1387,85400],[1723,739,63190],[1724,226,41854],[1726,210,64512],[1727,347,24203],[1728,1040,94220],[1729,689,9995],[1730,1560,19567],[1731,1320,96123],[1732,703,82424],[1733,755,78220],[1734,1648,26714],[1735,767,36096],[1737,1009,25871],[1738,1572,5145],[1739,738,81426],[1740,70,90421],[1741,961,16556],[1742,1421,3976],[1743,138,81131],[1744,969,31427],[1745,1547,89625],[1746,1666,38553],[1747,757,37813],[1748,301,12035],[1750,317,79609],[1751,978,3946],[1752,378,88630],[1753,1307,5052],[1754,42,65741],[1755,1448,71008],[1756,1674,20456],[1757,1362,7224],[1758,782,85199],[1759,942,78199],[1761,180,29946],[1762,1553,8212],[1763,868,24273],[1764,162,41003],[1765,1,90961],[1766,1405,55774],[1767,762,95592],[1768,1726,10598],[1769,570,80313],[1771,324,36741],[1772,362,25140],[1773,445,89332],[1774,1533,48982],[1775,1385,39129],[1776,1279,86112],[1777,1032,27089],[1779,1454,34766],[1780,724,87661],[1781,19,61973],[1782,444,99469],[1783,826,84557],[1784,1298,16913],[1785,303,59548],[1786,1279,32759],[1787,591,47664],[1788,1089,38234],[1789,1011,39254],[1791,769,43711],[1792,1778,10440],[1793,1156,28940],[1794,974,75378],[1795,1210,33981],[1796,72,52449],[1798,1718,20993],[1799,663,17307],[1800,1565,50282],[1801,1625,21487],[1802,211,84244],[1804,876,57088],[1805,1474,64278],[1806,872,52006],[1807,1485,83288],[1808,1356,56168],[1809,787,17644],[1810,1674,4892],[1811,1424,98678],[1812,1181,6610],[1813,1094,42801],[1814,303,57027],[1815,945,96016],[1816,1540,90279],[1817,87,26996],[1818,566,89527],[1820,1268,61447],[1821,388,75056],[1822,1454,13665],[1823,96,88854],[1824,93,26858],[1826,1674,7684],[1827,641,26221],[1828,179,97599],[1829,535,23074],[1830,521,2301],[1832,938,12625],[1834,103,84195],[1835,1775,16484],[1836,1089,31998],[1837,488,43211],[1838,996,18504],[1839,1261,92238],[1840,1580,99762],[1841,216,39921],[1842,1614,87827],[1843,312,27048],[1844,458,94118],[1845,238,32505],[1846,417,67817],[1847,196,87086],[1848,481,87862],[1849,957,72420],[1850,924,21972],[1851,945,44397],[1852,26,15100],[1854,112,80135],[1856,877,98938],[1857,1826,12135],[1858,762,78788],[1859,1071,14583],[1861,1576,58950],[1862,148,81901],[1863,920,34735],[1864,500,14383],[1865,486,53463],[1866,1813,4709],[1867,1580,39726],[1868,534,7736],[1869,1416,54710],[1870,1595,62001],[1871,171,52530],[1872,988,29773],[1873,1099,46458],[1874,1744,65858],[1875,569,55721],[1876,570,95922],[1877,990,98942],[1878,1846,22826],[1879,983,74666],[1880,1413,17988],[1881,1046,2412],[1882,1508,68224],[1883,971,36791],[1884,1249,83889],[1885,873,79657],[1886,759,93207],[1888,63,4417],[1889,75,90303],[1892,672,40377],[1893,1043,80449],[1894,699,84171],[1896,730,58237],[1898,780,17909],[1900,1691,47016],[1901,364,29188],[1902,1134,1388],[1903,1379,25838],[1904,1746,44877],[1905,794,7078],[1906,594,56007],[1907,1723,60140],[1908,1650,47677],[1909,137,79481],[1912,289,27911],[1913,1312,3582],[1914,1342,49154],[1915,1139,25802],[1916,1271,87652],[1918,1361,20539],[1920,1247,73281],[1921,562,27815],[1922,1536,36274],[1923,1510,3428],[1924,1356,61660],[1925,1638,26168],[1926,1284,32181],[1927,617,29200],[1928,163,73573],[1929,1563,4238],[1930,1459,86665],[1931,549,30738],[1932,1107,70415],[1933,1211,48121],[1934,1178,3994],[1936,1716,55675],[1937,1140,80126],[1938,627,71618],[1939,1458,93084],[1940,1344,82402],[1941,481,1261],[1942,620,45588],[1943,936,85286],[1944,1916,55626],[1945,1743,31588],[1946,866,88852],[1947,1935,95125],[1948,897,32841],[1949,1340,50949],[1950,934,95611],[1951,1669,31789],[1952,288,71059],[1953,330,17302],[1954,849,12746],[1955,1422,37508],[1956,1017,4480],[1957,823,58719],[1958,1916,65512],[1959,1337,28926],[1960,1697,45187],[1961,645,79442],[1962,1409,78056],[1963,1590,96112],[1964,161,97799],[1965,1774,76020],[1966,737,75005],[1967,1516,68113],[1968,731,1316],[1969,1951,81828],[1970,32,52498],[1971,829,6600],[1972,587,3639],[1973,1211,34628],[1974,454,33525],[1975,1944,31623],[1976,1446,43117],[1977,1654,5354],[1978,1139,28086],[1979,653,7771],[1980,1309,71940],[1982,282,88131],[1983,1479,35519],[1984,1076,64864],[1985,1212,25739],[1986,1913,13813],[1987,1960,42627],[1989,1165,7472],[1990,863,33477],[1991,1956,40231],[1992,889,20056],[1993,18,29013],[1994,1562,87466],[1995,583,55010],[1997,856,77515],[1998,382,87168],[1999,1578,13624],[2001,1108,48441],[2002,1132,79724],[2003,1284,67808],[2004,641,8894],[2005,1284,52411],[2006,1278,43981],[2008,1752,29244],[2009,65,81523],[2010,1602,68647],[2011,1209,65029],[2012,1452,71790],[2013,1628,37263],[2014,633,9544],[2015,1076,38297],[2016,813,49450],[2017,239,63983],[2018,1671,34481],[2019,443,11346],[2021,1282,18896],[2022,836,55087],[2023,1627,78152],[2024,1580,11747],[2025,1797,87033],[2026,882,79129],[2027,201,32580],[2028,560,20877],[2029,1103,71548],[2030,1863,77127],[2031,1830,46879],[2032,751,62894],[2033,777,51242],[2034,1858,73690],[2035,1697,5783],[2036,1812,13180],[2037,846,21033],[2039,575,39011],[2041,1899,84186],[2042,1706,75012],[2043,165,3442],[2045,1961,10443],[2046,596,71950],[2047,1003,94219],[2048,601,31053],[2049,63,89948],[2050,944,7438],[2051,639,56694],[2052,685,20260],[2053,1856,68744],[2054,657,98335],[2055,2009,84753],[2056,1367,77875],[2057,1643,89787],[2058,792,5089],[2059,1569,1950],[2060,169,56565],[2061,82,16928],[2062,1573,470],[2063,542,6916],[2064,1943,66462],[2067,730,55387],[2069,1250,64247],[2070,1488,938],[2071,94,90514],[2072,605,29025],[2073,1082,26394],[2074,1202,94264],[2075,1417,60967],[2076,1045,27009],[2077,1067,82089],[2078,1145,46433],[2079,796,24708],[2080,135,3945],[2081,1464,56941],[2082,1176,64853],[2083,1200,94888],[2084,32,70744],[2086,343,65223],[2087,27,47010],[2088,870,21963],[2089,1659,45436],[2090,44,64359],[2091,2052,32531],[2092,1206,19091],[2093,1849,87745],[2094,1422,79986],[2095,1813,71054],[2096,1447,78772],[2097,1457,1784],[2099,607,305],[2101,1590,47210],[2103,1111,38168],[2104,2022,35470],[2105,1629,19927],[2106,1536,34071],[2108,566,92823],[2109,677,25875],[2110,1817,33904],[2111,571,35240],[2112,741,96757],[2113,1486,78613],[2114,2053,38392],[2115,1724,99260],[2116,1845,66895],[2117,1635,21105],[2118,891,34050],[2119,708,24250],[2120,1331,82794],[2121,956,84026],[2122,1486,21995],[2123,769,56914],[2124,694,80412],[2125,1065,645],[2126,79,96687],[2127,1504,59408],[2128,458,43050],[2130,1594,13307],[2131,1535,99373],[2132,1321,97868],[2133,1759,83226],[2134,1561,37970],[2135,1453,56519],[2136,1275,47035],[2137,1480,44677],[2138,510,75986],[2139,1430,81904],[2141,395,98604],[2143,1812,64733],[2144,46,99363],[2146,1207,17310],[2147,1927,72480],[2148,675,20605],[2149,88,45907],[2150,670,84406],[2151,484,70180],[2152,675,36255],[2153,310,19553],[2154,1068,31005],[2155,703,10580],[2156,1449,8386],[2157,1349,20511],[2158,79,252],[2159,1755,97365],[2160,1920,37848],[2161,601,31738],[2163,1953,37118],[2165,850,54036],[2166,1327,61832],[2167,919,87251],[2168,505,73976],[2169,1618,17476],[2170,544,72201],[2171,617,39041],[2172,1507,56703],[2173,1425,29231],[2174,5,99946],[2175,1234,15449],[2176,1746,84835],[2177,215,12768],[2178,1079,41996],[2179,1483,99892],[2181,1888,42817],[2182,1142,27106],[2183,576,97036],[2184,1318,25675],[2185,935,65725],[2186,1111,52746],[2187,773,77664],[2188,671,4037],[2189,731,32401],[2190,343,28439],[2191,1629,85459],[2192,1663,22616],[2194,723,64190],[2196,1131,27207],[2198,1401,57134],[2199,1992,51555],[2200,2055,54755],[2201,1453,28041],[2203,97,77491],[2204,692,4818],[2205,1767,91743],[2206,1834,65580],[2207,750,87864],[2208,1599,96553],[2209,1338,41079],[2210,1906,15442],[2211,1036,91071],[2212,2179,31891],[2213,116,38651],[2214,634,5407],[2215,790,73219],[2216,1627,46632],[2217,1233,92692],[2218,1389,11892],[2219,1091,310],[2220,2042,4138],[2221,1896,57141],[2222,1943,48854],[2223,948,52713],[2224,326,43624],[2225,1604,32118],[2226,306,47822],[2227,416,29408],[2229,1591,78049],[2230,1331,54044],[2231,302,50714],[2232,394,27780],[2233,2167,35092],[2234,1760,18359],[2235,593,16597],[2236,1405,21589],[2239,777,17972],[2240,2213,50321],[2241,471,33198],[2242,1286,88861],[2243,300,86472],[2245,2132,35193],[2246,2076,68476],[2247,1643,38085],[2248,887,81822],[2250,419,14775],[2251,1034,13565],[2252,791,20932],[2253,1836,67989],[2254,1009,51834],[2255,1331,83849],[2256,2150,52152],[2257,1518,92443],[2258,993,59409],[2259,1332,37819],[2260,133,95590],[2261,1134,7218],[2262,215,2803],[2263,714,85861],[2264,904,1976],[2265,929,59940],[2267,724,19013],[2268,2127,58898],[2269,881,43078],[2270,969,9528],[2271,1779,5966],[2272,789,28826],[2273,1838,77664],[2274,1887,20241],[2275,1793,24149],[2277,444,39736],[2279,91,72484],[2280,908,25158],[2281,729,37842],[2282,55,79670],[2283,873,91619],[2284,2017,67223],[2285,931,27948],[2286,805,44287],[2287,918,78596],[2288,287,74695],[2289,588,33923],[2290,1924,43447],[2291,1871,19670],[2292,1742,23092],[2293,830,36399],[2294,1021,90140],[2295,20,49664],[2296,2093,69054],[2297,1551,34537],[2298,2248,61936],[2300,709,31920],[2301,485,32202],[2302,1828,83649],[2303,452,49966],[2304,1569,42409],[2305,898,66026],[2306,2053,65774],[2307,1640,50699],[2308,72,18222],[2309,1298,13183],[2310,874,52600],[2311,611,78138],[2312,1080,47769],[2313,1554,32481],[2314,1796,45496],[2315,317,50778],[2316,415,72322],[2317,406,83473],[2319,925,92462],[2320,557,59251],[2321,1801,10492],[2322,143,36709],[2324,263,87646],[2325,1076,35982],[2326,11,15974],[2327,1799,27301],[2328,303,67054],[2329,1933,42549],[2330,1737,31216],[2331,1744,86577],[2332,368,7191],[2333,1154,98932],[2335,4,28736],[2336,1782,50871],[2337,1319,84767],[2338,490,13772],[2339,220,15341],[2340,1209,7096],[2341,1234,9301],[2342,1914,54452],[2343,953,19943],[2344,1945,35404],[2345,692,93408],[2346,256,96687],[2347,2272,26920],[2348,819,4325],[2349,2229,458],[2350,2294,71856],[2351,1994,39277],[2352,628,57936],[2353,1743,3756],[2354,322,52759],[2355,1995,1101],[2357,1125,81963],[2358,595,38159],[2359,901,69176],[2360,426,80204],[2361,1110,3807],[2362,741,86148],[2363,175,33573],[2364,909,40522],[2365,1771,914],[2366,2297,55208],[2367,1890,56235],[2368,1804,19519],[2369,850,68637],[2370,1070,29015],[2371,855,63642],[2372,79,4870],[2373,2133,43005],[2374,204,57183],[2376,739,16655],[2377,672,1151],[2378,533,98617],[2379,911,68820],[2380,152,89028],[2381,1556,64362],[2382,439,79083],[2383,2140,49268],[2384,1166,13599],[2385,22,94755],[2386,1692,34636],[2387,1613,1253],[2388,133,64777],[2389,2217,48738],[2390,185,19364],[2391,1078,42950],[2392,597,99864],[2393,1506,60237],[2394,1021,71844],[2395,1206,44646],[2396,379,60834],[2397,199,27747],[2398,1478,36858],[2399,1679,89232],[2400,88,48528],[2401,2050,46105],[2403,1600,69274],[2404,971,21419],[2405,1000,97446],[2406,1624,26158],[2407,82,98831],[2408,1512,25186],[2409,2224,37847],[2411,381,14252],[2412,1061,9908],[2413,41,63348],[2414,1130,74462],[2415,2257,97605],[2416,1011,45431],[2417,526,61727],[2419,630,2855],[2420,460,18714],[2422,2276,5996],[2423,1724,38924],[2425,1447,90106],[2426,1041,73172],[2427,1695,73883],[2428,1812,60478],[2429,1107,55698],[2430,1340,69150],[2431,1725,1998],[2432,1452,44897],[2433,965,86420],[2434,1594,49807],[2435,1228,33124],[2436,189,36504],[2437,2231,27023],[2438,1265,41818],[2439,227,22051],[2440,491,25706],[2441,1458,24269],[2442,1394,91198],[2443,515,60214],[2444,2285,21829],[2445,1469,55308],[2446,2118,47692],[2447,332,91416],[2448,2420,3001],[2450,1655,12678],[2451,2279,8405],[2452,779,41602],[2453,1069,65093],[2454,403,78828],[2455,560,54963],[2457,1472,95585],[2458,1200,27071],[2460,790,49396],[2461,205,26025],[2462,876,29232],[2463,1000,2969],[2466,170,90311],[2467,819,25840],[2468,175,20096],[2469,926,14562],[2470,659,52428],[2471,1436,41523],[2472,2316,41998],[2473,458,77705],[2474,606,57376],[2476,1041,58445],[2477,1560,97281],[2478,2128,21227],[2479,1203,77451],[2480,462,25219],[2481,646,8349],[2482,2274,44492],[2483,375,60246],[2484,406,26080],[2485,1205,67669],[2486,503,26123],[2487,1353,19269],[2488,556,14243],[2489,1425,38485],[2490,1802,36245],[2491,2464,73756],[2492,113,55653],[2493,1303,82183],[2494,1974,72465],[2495,288,70579],[2496,814,46922],[2497,2277,97279],[2498,1964,9719],[2499,1200,29885],[2500,600,36273],[2501,1824,76022],[2502,1705,47172],[2503,1162,39632],[2504,1963,12157],[2505,800,34134],[2506,2459,41601],[2507,1569,69567],[2508,397,17907],[2509,1873,96135],[2510,1772,18118],[2511,1686,39480],[2512,55,56888],[2513,2226,21517],[2514,1847,32961],[2515,2200,63196],[2516,139,50629],[2517,592,73465],[2518,1090,88768],[2519,245,61],[2520,889,46999],[2521,681,3293],[2522,266,40421],[2523,99,17187],[2524,1416,81262],[2525,2339,47652],[2526,2411,59280],[2527,2176,45584],[2528,768,90991],[2529,277,92706],[2530,596,79260],[2531,671,29515],[2532,1126,96099],[2533,223,47097],[2534,525,95300],[2535,1306,70047],[2536,2507,33223],[2537,548,86813],[2538,673,86717],[2539,763,42502],[2541,2029,75460],[2543,777,32627],[2544,1106,68201],[2545,1813,74267],[2546,2044,7021],[2547,2113,36265],[2548,1275,56370],[2549,2135,41896],[2550,454,92434],[2551,378,37484],[2552,412,8121],[2553,2349,28006],[2554,138,19345],[2555,1870,44573],[2556,892,89085],[2557,1848,33033],[2558,2055,75789],[2559,1961,69333],[2560,151,34942],[2561,539,12572],[2562,1548,5950],[2563,1181,48440],[2564,957,2090],[2565,2103,99594],[2566,630,23679],[2567,2399,88550],[2568,1847,48699],[2569,951,38980],[2570,134,4514],[2571,2454,59093],[2572,2285,48149],[2573,522,3184],[2574,2552,77394],[2575,2201,7405],[2576,2472,57539],[2577,2294,19400],[2578,1508,29788],[2579,621,98430],[2580,1351,36518],[2581,453,88711],[2582,2240,12986],[2583,2171,37324],[2584,368,73937],[2585,2184,30738],[2586,1191,19147],[2587,213,57215],[2588,1581,58394],[2589,2411,46165],[2590,1175,12592],[2591,2301,46319],[2593,1985,24217],[2594,946,9783],[2595,335,90881],[2596,2090,26681],[2597,2322,41002],[2598,2334,49024],[2599,1103,37182],[2600,2345,55152],[2601,2334,97269],[2602,2400,82393],[2603,343,99315],[2604,2472,73789],[2605,2055,39070],[2606,1968,32192],[2608,810,39154],[2609,185,54290],[2610,1626,70901],[2611,189,75295],[2613,625,64436],[2614,1660,9165],[2616,2595,39171],[2617,823,77526],[2619,1738,8080],[2620,958,12651],[2622,2354,82223],[2623,1247,79693],[2625,2266,27173],[2626,1111,96604],[2628,1210,42181],[2629,542,12964],[2630,1540,2210],[2631,698,52425],[2632,1715,66822],[2634,607,1686],[2635,2411,36748],[2636,544,19209],[2637,1357,65306],[2638,2317,80823],[2639,412,37170],[2640,836,77594],[2641,1993,80238],[2642,2439,89385],[2643,1824,35166],[2644,1076,15455],[2645,717,99577],[2646,591,73160],[2647,1290,82080],[2648,2139,99751],[2649,489,79278],[2650,1884,62729],[2651,267,92741],[2652,1968,98670],[2655,1219,73786],[2656,1700,63321],[2657,1360,6316],[2658,1731,52349],[2659,2411,44954],[2661,545,93459],[2662,1032,97209],[2663,38,67989],[2664,1567,51330],[2666,125,16130],[2667,2155,95280],[2668,1523,14665],[2669,1379,35613],[2670,749,83583],[2671,1975,13897],[2673,836,64767],[2674,539,77392],[2675,1312,42027],[2676,2262,97176],[2677,1942,74106],[2678,282,8917],[2679,2552,97126],[2680,961,21924],[2681,363,47092],[2682,2154,90139],[2683,1206,78273],[2684,1317,56058],[2685,1221,51641],[2686,1198,44013],[2688,824,38845],[2690,749,27218],[2691,1983,91012],[2693,2241,16990],[2695,1012,65079],[2696,1968,13174],[2697,562,85512],[2698,384,91914],[2699,2310,76270],[2700,722,70268],[2701,2531,43547],[2702,1706,49714],[2703,350,95441],[2704,2629,54206],[2705,1440,18636],[2706,2638,90986],[2707,413,73248],[2708,1459,90762],[2709,337,58776],[2710,2646,3684],[2711,640,22743],[2712,1741,64933],[2714,592,96687],[2715,765,83847],[2716,963,11468],[2718,1681,48558],[2719,2528,1117],[2720,2035,39535],[2721,1525,25968],[2722,813,2185],[2723,93,68941],[2724,1493,66997],[2725,2433,98726],[2726,2689,63219],[2727,1266,68093],[2728,1493,79867],[2729,1102,31808],[2730,1491,81058],[2731,2606,12658],[2733,1343,85065],[2734,1043,77699],[2735,1463,71667],[2736,405,94781],[2737,314,37981],[2738,216,18786],[2739,2549,88759],[2740,1736,86297],[2741,413,87614],[2742,2403,47522],[2743,2391,78111],[2744,1686,52105],[2746,2460,13819],[2747,1844,43111],[2748,674,20474],[2749,298,3246],[2750,465,45958],[2751,610,53648],[2752,1745,33187],[2753,755,91346],[2754,1234,4138],[2755,2437,83810],[2756,714,87228],[2757,1770,45270],[2758,1083,85423],[2759,829,29033],[2760,909,87960],[2761,1821,45312],[2762,1974,63390],[2763,1123,92916],[2764,1537,47883],[2765,542,94799],[2766,1119,20726],[2767,2636,1863],[2768,2244,49905],[2770,332,9915],[2771,420,20266],[2772,1846,16998],[2773,393,53860],[2774,1473,20372],[2775,748,8672],[2776,1901,86785],[2777,1880,60247],[2778,1617,65901],[2779,1614,51262],[2780,1890,1496],[2781,656,37311],[2782,1174,80207],[2784,2714,52303],[2786,1663,17983],[2787,558,94367],[2788,33,75222],[2789,1496,27666],[2790,2175,62718],[2791,1012,57802],[2792,283,39345],[2793,2394,66716],[2794,727,9993],[2795,465,33633],[2797,768,75734],[2798,674,50388],[2799,1030,23704],[2800,98,74370],[2801,776,12792],[2802,940,77667],[2803,572,38708],[2804,2247,5221],[2805,1295,43471],[2806,2758,51458],[2807,1027,90532],[2808,2303,61996],[2809,432,33438],[2810,302,76535],[2811,1755,26831],[2813,2677,8445],[2814,109,74368],[2816,1462,29489],[2817,607,20237],[2818,672,44931],[2819,713,92900],[2820,2175,51526],[2822,545,76339],[2823,2218,13347],[2824,1887,57825],[2825,1017,24770],[2826,1781,77359],[2827,210,94704],[2828,2720,74314],[2829,1941,23895],[2830,659,37003],[2831,388,17531],[2833,1916,77686],[2834,961,16228],[2835,2601,15387],[2837,2614,94323],[2838,1701,19946],[2839,1529,76664],[2840,516,87873],[2841,2699,30643],[2842,2286,33140],[2843,2670,94200],[2844,2226,3481],[2845,2661,69372],[2846,1323,17914],[2847,1366,58503],[2848,1626,1114],[2849,2468,23375],[2850,1266,86616],[2851,1385,61869],[2852,847,98166],[2853,168,80127],[2854,2402,7142],[2855,2206,47055],[2856,285,19093],[2857,250,89446],[2858,760,40701],[2859,332,68552],[2860,1995,86362],[2861,827,88789],[2862,2823,99570],[2863,1755,83878],[2864,236,67068],[2865,562,44904],[2866,2118,18041],[2867,1187,14428],[2868,414,64810],[2871,27,69285],[2872,1797,95509],[2873,1838,80527],[2874,379,10532],[2875,1559,88069],[2876,1033,52479],[2877,100,50371],[2878,626,23137],[2879,1047,39045],[2881,1904,62987],[2882,1343,51764],[2883,591,42800],[2884,2532,86560],[2885,1770,87623],[2886,2670,32526],[2887,46,34004],[2888,2698,64978],[2890,837,15182],[2891,896,93286],[2892,2817,34363],[2894,110,37193],[2895,611,2370],[2896,460,43976],[2897,1555,49623],[2898,1715,9181],[2900,2369,15110],[2901,960,8918],[2902,607,32415],[2903,774,72117],[2904,1628,40657],[2906,521,44129],[2907,492,45003],[2908,1212,6625],[2909,1578,59018],[2910,1091,48434],[2911,64,57721],[2912,1642,85166],[2913,2669,65677],[2914,2167,55693],[2915,2524,41424],[2916,889,80133],[2917,324,1216],[2919,353,86028],[2920,1496,86664],[2921,1129,57113],[2922,314,85420],[2923,2334,60876],[2924,1552,21810],[2925,68,42398],[2926,2911,52942],[2927,2564,51677],[2928,488,56831],[2930,492,77734],[2931,2511,15570],[2932,167,61294],[2933,2692,20385],[2934,886,56154],[2935,1884,29865],[2936,1874,907],[2937,1227,50054],[2938,2361,17921],[2939,62,7889],[2940,1119,34604],[2942,2589,26756],[2943,1435,58732],[2944,1652,79242],[2945,329,32984],[2946,581,49522],[2947,212,66951],[2948,367,75829],[2949,2774,44360],[2950,457,67049],[2951,1995,43559],[2952,1660,21471],[2953,1999,1592],[2954,373,95786],[2955,17,85899],[2956,2263,24490],[2957,2074,55499],[2958,2193,19217],[2959,1172,85484],[2960,707,77635],[2961,497,37718],[2962,1025,89242],[2963,384,4761],[2964,610,12374],[2965,1589,1662],[2966,1989,14142],[2967,584,11572],[2968,2838,1816],[2969,1300,53073],[2970,1272,82955],[2971,1209,43306],[2972,2393,72473],[2973,2964,87446],[2974,182,27840],[2975,1852,37426],[2976,696,20566],[2977,927,23912],[2978,719,82520],[2979,2484,6463],[2981,1671,96068],[2982,817,67210],[2983,2018,55136],[2984,325,86300],[2985,367,69768],[2986,1794,8343],[2987,28,28291],[2989,2893,78264],[2990,421,53655],[2991,226,32168],[2992,2978,38422],[2993,1811,55839],[2994,1411,25383],[2995,1959,96425],[2997,1896,76632],[2998,622,96901],[2999,2648,48295],[3000,748,11112],[3001,1500,68925],[3002,2791,277],[3003,2065,66688],[3004,2581,91888],[3005,2932,2254],[3006,2053,11673],[3007,290,32993],[3008,1897,98847],[3009,802,52498],[3010,744,50834],[3011,2255,91561],[3012,2208,42429],[3013,807,54532],[3014,1208,84466],[3015,938,14627],[3016,1126,92723],[3017,2157,61461],[3018,1437,80824],[3019,851,1174],[3020,2131,14495],[3021,2440,56194],[3022,231,49020],[3023,2510,1812],[3024,235,27442],[3025,350,7948],[3026,653,49316],[3027,2720,79626],[3028,2789,60102],[3029,2321,42394],[3030,2654,22147],[3031,262,68260],[3033,2742,14922],[3035,1346,27468],[3036,1696,44688],[3037,993,34914],[3038,2480,42013],[3040,1798,6908],[3043,1840,64794],[3044,2707,33629],[3046,1713,28827],[3047,1256,6090],[3048,1502,66047],[3049,1884,54962],[3050,1371,96608],[3051,2002,46048],[3052,885,37266],[3053,2492,30408],[3054,22,26769],[3055,897,97225],[3056,752,74818],[3058,1503,31249],[3059,2540,76375],[3060,2145,92508],[3061,2835,48064],[3062,2338,68974],[3063,1097,34123],[3064,648,63193],[3065,295,51877],[3067,1990,20799],[3068,1350,96691],[3069,3013,17931],[3070,2082,74916],[3071,68,623],[3072,686,19849],[3073,1386,2408],[3074,2451,16680],[3076,1779,35729],[3077,983,24351],[3078,3023,13177],[3079,45,98928],[3080,2525,92410],[3082,2396,21590],[3083,325,85916],[3084,1351,38150],[3086,1510,11356],[3087,2258,39831],[3088,2360,82080],[3089,1513,96803],[3090,844,29668],[3091,2636,51743],[3092,73,22754],[3093,1472,56368],[3094,1152,19168],[3095,1705,58075],[3096,62,69253],[3097,694,34465],[3098,1710,6161],[3099,487,47053],[3100,567,51819],[3101,1088,46093],[3102,2479,5185],[3103,1139,5158],[3104,619,54299],[3105,2122,15826],[3107,2193,96891],[3108,1998,38971],[3109,1084,14478],[3110,3011,99360],[3111,1999,5109],[3112,799,71181],[3113,1955,15817],[3114,1057,93834],[3116,2561,98403],[3117,2650,80072],[3118,1037,93759],[3119,1109,74711],[3120,182,64622],[3121,1012,93265],[3122,2813,60543],[3124,2816,72127],[3125,897,80640],[3126,483,27712],[3127,2143,42858],[3128,1092,78056],[3129,2144,1410],[3130,3010,16310],[3131,894,17500],[3133,886,4071],[3134,2156,3099],[3135,1881,73169],[3136,1101,78551],[3137,1608,62568],[3138,322,21116],[3139,2589,34685],[3140,1014,35459],[3141,33,58317],[3142,2341,57668],[3143,1591,57855],[3144,34,65994],[3145,2250,91291],[3146,2233,77397],[3147,2586,47920],[3148,1867,3364],[3149,1457,39836],[3150,2351,24790],[3151,2501,39785],[3152,1206,61865],[3153,1991,6234],[3154,2772,8447],[3155,1599,36331],[3156,14,33000],[3157,1708,43408],[3158,2986,88977],[3159,1223,25324],[3160,723,33845],[3161,2240,58316],[3162,555,47559],[3164,102,32464],[3165,1101,86602],[3166,690,47446],[3167,312,61679],[3168,112,10910],[3169,3099,41671],[3170,2905,74939],[3171,2129,48434],[3172,2544,84068],[3173,1964,7689],[3174,2260,16939],[3175,3128,21086],[3177,2793,50695],[3178,1709,90979],[3179,2097,32927],[3180,2179,80388],[3181,1959,77445],[3182,2584,64029],[3183,1941,39436],[3184,1468,16634],[3186,912,7387],[3187,1082,63811],[3188,2409,29992],[3189,1329,97068],[3190,1524,54374],[3191,1215,11757],[3192,1693,52017],[3193,518,78783],[3195,2178,77763],[3196,2722,19141],[3198,1881,64587],[3199,716,36166],[3200,1830,41809],[3201,2333,20745],[3202,696,45232],[3203,112,23467],[3204,1960,68460],[3205,330,81702],[3207,906,21373],[3208,2271,47377],[3209,22,63391],[3210,49,70491],[3211,438,85392],[3212,2074,61377],[3213,1457,23659],[3214,1108,86807],[3217,2713,76033],[3218,1503,16533],[3219,1970,70857],[3220,412,12881],[3221,1496,13195],[3222,2279,73572],[3223,318,4191],[3224,241,16034],[3225,2433,88798],[3226,1111,45760],[3227,3133,22906],[3228,1538,94500],[3229,988,60074],[3230,128,53898],[3231,3200,8944],[3232,2046,18707],[3233,1554,72060],[3235,1691,18011],[3236,1929,33635],[3237,1659,27129],[3238,2715,95229],[3239,2850,71640],[3240,2122,85451],[3241,2567,89509],[3243,263,17190],[3244,1077,98533],[3246,2445,32195],[3247,1685,27214],[3248,2889,40874],[3249,2847,82418],[3250,379,72502],[3253,2844,81072],[3254,1475,73489],[3255,2517,62456],[3256,2828,25397],[3257,3000,82341],[3258,2865,44401],[3259,998,80009],[3260,1656,87029],[3261,288,22462],[3262,278,16984],[3263,2515,3584],[3264,57,7056],[3265,1011,19999],[3266,2365,75802],[3267,2537,41243],[3268,1831,98833],[3269,979,48232],[3270,1015,53675],[3271,3049,84818],[3272,703,49001],[3273,1570,27072],[3274,997,67429],[3275,34,84877],[3276,210,98405],[3277,2284,42945],[3278,1541,65035],[3279,1129,38094],[3280,2392,74968],[3281,2066,36828],[3282,2434,74952],[3283,1591,76348],[3284,467,20332],[3285,40,89247],[3286,2014,41404],[3287,1308,59921],[3288,378,85141],[3289,476,92885],[3290,217,36789],[3291,1454,44622],[3292,160,55387],[3293,475,92351],[3294,462,28089],[3295,876,99775],[3296,248,69274],[3297,277,56546],[3298,2833,67022],[3299,2100,9270],[3300,3068,86791],[3301,2506,95256],[3302,636,33894],[3304,2508,83421],[3305,2061,37277],[3306,2396,78245],[3307,1103,89040],[3308,306,56072],[3309,1480,89322],[3310,2647,83904],[3312,2153,19675],[3314,2865,8579],[3315,1694,15443],[3316,3246,6588],[3317,1321,82767],[3318,2849,44655],[3319,2808,13194],[3320,2406,45072],[3321,287,37876],[3322,1162,71611],[3323,1533,98562],[3324,1805,11089],[3325,2997,25398],[3326,17,73766],[3327,3231,54325],[3328,2499,93158],[3329,2303,81026],[3330,853,72611],[3331,930,53355],[3332,879,1611],[3333,2038,65225],[3334,2057,99311],[3335,2460,89435],[3336,2523,69074],[3337,9,267],[3338,499,95869],[3339,1702,40196],[3340,588,79487],[3341,583,5735],[3342,834,73326],[3343,877,12359],[3344,2721,86118],[3346,1814,95800],[3347,1270,11129],[3348,258,25518],[3349,2295,33715],[3350,551,51936],[3351,1481,13969],[3352,1390,53442],[3353,2982,97667],[3354,254,54306],[3355,2265,70621],[3356,789,32138],[3357,2909,9399],[3358,519,40628],[3359,2228,88214],[3360,1737,88378],[3361,2748,8402],[3362,1706,52937],[3363,3174,62587],[3364,1248,32204],[3365,538,71799],[3366,678,26041],[3367,1776,94055],[3368,279,62427],[3370,3071,94713],[3371,2641,44716],[3372,649,74525],[3373,2927,76596],[3374,1648,8584],[3375,2159,82944],[3376,2526,52645],[3377,1989,38689],[3378,597,32981],[3379,3023,74310],[3380,1901,6457],[3381,3124,83851],[3382,3314,7860],[3383,1203,26163],[3384,1995,49543],[3385,115,8550],[3386,828,84986],[3387,942,6232],[3388,2004,47858],[3389,3170,5380],[3390,629,18989],[3391,159,14677],[3392,865,41940],[3393,936,92207],[3394,1800,52392],[3395,2327,7842],[3396,2030,55326],[3397,19,9121],[3398,2224,20122],[3399,1408,45595],[3400,668,72503],[3401,1484,78181],[3402,99,6340],[3403,124,90328],[3404,2,26164],[3405,3345,49029],[3406,116,47524],[3407,414,94845],[3408,1655,92235],[3409,2341,58404],[3410,1046,68855],[3411,1766,52738],[3412,1438,169],[3413,1837,71120],[3414,1163,98348],[3415,2772,12479],[3416,177,66832],[3417,2515,72020],[3418,2102,75559],[3419,983,93372],[3420,2403,45295],[3421,274,43962],[3422,1679,81617],[3423,1716,82909],[3424,2036,38372],[3425,2613,73514],[3426,890,99797],[3427,575,14595],[3428,3176,51865],[3431,3181,29846],[3432,516,14819],[3433,1661,95490],[3434,2479,4972],[3435,700,77306],[3436,2302,58188],[3437,3039,35109],[3438,1389,62043],[3439,2693,36121],[3440,2952,60792],[3441,1006,72116],[3442,2801,17466],[3443,1926,65562],[3444,310,27441],[3445,2587,2756],[3446,609,34200],[3447,1586,79325],[3448,962,31125],[3449,420,39582],[3450,836,64202],[3451,2705,78990],[3452,969,55293],[3454,1466,96623],[3455,1326,81524],[3456,2900,1952],[3457,3334,20818],[3458,1689,56323],[3459,218,20705],[3460,3053,95923],[3461,1854,24855],[3462,2862,18628],[3463,1329,61135],[3464,1667,43002],[3465,1886,52017],[3466,425,58914],[3467,842,11630],[3468,2059,39960],[3469,253,82356],[3470,907,11807],[3471,1453,41374],[3472,1750,96646],[3473,86,45674],[3475,96,80440],[3476,396,32028],[3477,285,53326],[3478,313,83916],[3479,878,89387],[3480,1950,84245],[3481,1570,98142],[3482,203,83760],[3483,1450,30727],[3484,1015,79739],[3485,1061,90217],[3486,2359,1547],[3487,1493,94539],[3489,1413,39733],[3490,2733,89229],[3491,2519,82962],[3492,2568,89919],[3493,161,41101],[3495,689,2388],[3496,1630,12929],[3497,1760,62591],[3499,2071,31301],[3500,346,97849],[3501,1728,89646],[3502,3308,88221],[3503,1461,64053],[3505,2795,14696],[3506,1028,14722],[3507,1735,27753],[3508,1539,99994],[3509,3004,11211],[3510,3045,41732],[3512,152,47657],[3514,2409,88755],[3515,1167,47841],[3516,96,4378],[3517,990,20928],[3518,882,77262],[3519,2752,67334],[3520,2748,91988],[3521,305,3505],[3522,735,14163],[3523,749,80341],[3524,2434,71849],[3525,3118,49254],[3526,2892,76686],[3527,1528,35797],[3528,2011,60716],[3529,666,76699],[3530,922,53458],[3531,2049,75457],[3532,644,37514],[3533,3052,61650],[3534,3133,10714],[3535,662,42608],[3536,3290,999],[3538,1671,14340],[3539,591,31619],[3540,2746,94547],[3541,657,26238],[3542,2512,41534],[3543,2358,16938],[3544,2836,28853],[3545,1627,79591],[3546,371,43672],[3547,49,26568],[3548,631,27499],[3550,2143,35416],[3552,3370,79907],[3553,1820,56181],[3554,581,13365],[3555,1529,96351],[3556,1556,67226],[3557,371,9974],[3559,3318,42182],[3560,94,20988],[3561,1503,1735],[3562,3170,24666],[3564,868,68256],[3565,1720,18565],[3568,990,43599],[3569,3212,69884],[3570,2357,93764],[3571,2727,6985],[3572,3515,1464],[3573,2980,41211],[3574,725,47533],[3575,1121,46300],[3576,1810,72554],[3577,1721,52870],[3578,1388,75108],[3580,836,41583],[3581,1514,95608],[3582,3282,15799],[3583,3279,35417],[3584,1879,82910],[3585,1926,67201],[3587,1189,7064],[3588,1143,51183],[3589,880,62572],[3590,1453,29894],[3591,3314,47586],[3593,1264,20453],[3594,1610,74715],[3595,3047,93557],[3596,2846,61101],[3597,2516,52269],[3598,1403,33592],[3599,2196,1983],[3601,1324,26434],[3602,2710,64814],[3603,270,2485],[3604,2659,56686],[3605,2342,65894],[3606,3523,25760],[3607,1221,35324],[3608,447,53921],[3609,1914,96371],[3610,2913,3169],[3611,780,65565],[3612,475,23248],[3613,829,90868],[3614,2213,70845],[3615,2771,9535],[3616,1808,24471],[3617,1351,31078],[3618,1072,97707],[3619,1429,58390],[3620,2211,67124],[3622,640,48482],[3623,3446,40853],[3624,3163,40710],[3625,2082,11967],[3626,514,86179],[3627,2180,73705],[3628,3349,30939],[3629,1819,10352],[3630,2878,11228],[3631,700,9713],[3633,3298,12169],[3634,506,35798],[3635,2515,33257],[3636,2474,67465],[3637,1920,60172],[3638,1429,11259],[3639,3068,87076],[3640,2106,36092],[3641,345,59082],[3642,2155,12204],[3643,1685,84495],[3644,327,63137],[3645,1445,47601],[3646,475,81389],[3647,1129,44549],[3648,1642,75267],[3649,608,33350],[3650,3353,93343],[3651,1442,30874],[3652,2445,78303],[3653,398,9274],[3654,1233,4518],[3655,3603,97570],[3656,3212,81557],[3657,687,20763],[3658,845,20953],[3659,3622,41873],[3660,3137,645],[3662,64,12841],[3663,1828,87369],[3664,2189,77592],[3665,2639,25201],[3666,141,24112],[3667,1299,10887],[3668,3007,95544],[3669,2565,69989],[3670,2810,34839],[3671,2141,22538],[3672,144,49421],[3673,2866,64737],[3674,3599,71954],[3675,2516,57695],[3676,2427,2911],[3677,1515,10125],[3678,1599,56858],[3679,2262,76511],[3680,1112,98285],[3681,218,27341],[3682,1271,20797],[3683,3413,87948],[3684,2210,24027],[3685,705,88213],[3686,3614,50068],[3687,2202,48147],[3688,910,27202],[3689,1566,32526],[3690,1295,86582],[3691,696,27065],[3692,3167,28903],[3693,1508,99950],[3694,2681,53203],[3695,1902,30928],[3696,2420,41523],[3697,1239,49730],[3698,351,72710],[3699,2167,75430],[3700,1373,65086],[3702,2867,67406],[3703,535,90723],[3704,1558,38269],[3705,3532,43022],[3706,1696,41240],[3707,2900,95390],[3708,3419,58762],[3709,716,36026],[3710,1365,92565],[3711,361,31917],[3712,3693,27630],[3713,3031,25821],[3714,1539,6199],[3715,3392,24181],[3716,2149,14751],[3717,2056,10932],[3718,320,62638],[3719,1119,12849],[3720,1119,13447],[3721,2311,89288],[3722,136,45782],[3723,544,66854],[3724,1383,82359],[3725,2813,73194],[3726,1326,37482],[3727,784,65840],[3728,2862,32972],[3729,2500,68780],[3730,3342,93516],[3731,3356,14558],[3732,257,6309],[3733,2966,99196],[3734,2650,72131],[3735,1359,78534],[3736,2251,88411],[3737,2237,16409],[3738,3637,84288],[3739,3625,23200],[3740,2757,19210],[3741,665,40431],[3742,3541,27198],[3743,2753,71878],[3744,21,32754],[3746,2794,34502],[3747,2905,85352],[3749,2452,65963],[3750,871,62701],[3751,898,48817],[3752,1899,26565],[3753,142,63473],[3754,696,78641],[3755,110,55910],[3756,788,40240],[3757,787,88703],[3759,1300,92839],[3761,2816,51063],[3762,239,28130],[3764,3065,32079],[3766,551,58368],[3767,2918,44300],[3768,2633,56023],[3769,1173,74768],[3770,3297,34093],[3771,3571,55865],[3772,3473,42853],[3773,2609,94216],[3774,524,80240],[3775,1512,61218],[3776,774,114],[3777,2574,29772],[3778,3337,59751],[3779,7,54263],[3780,2410,7680],[3781,2712,24962],[3783,2424,63002],[3784,1748,75858],[3787,2278,59059],[3788,1790,82509],[3789,2487,20840],[3790,474,82318],[3791,2916,39379],[3792,1545,64043],[3793,554,47710],[3794,360,56882],[3795,393,52948],[3796,319,17413],[3797,3430,69387],[3798,3344,64597],[3799,359,51171],[3801,2900,58739],[3803,3375,50687],[3804,1035,19710],[3805,1563,33619],[3806,1043,1031],[3807,3619,19650],[3808,3354,27955],[3809,2623,72158],[3810,1713,1948],[3811,3055,86933],[3812,1072,39565],[3813,1239,45532],[3814,525,34163],[3815,2389,83184],[3816,3760,20841],[3817,826,41128],[3818,772,24196],[3819,2069,40134],[3820,579,57496],[3823,949,30078],[3824,1304,717],[3825,1534,90742],[3826,2989,31303],[3827,3730,49075],[3828,3043,15119],[3829,699,41988],[3830,2053,21362],[3831,593,74588],[3832,300,34513],[3833,3233,374],[3834,1933,43248],[3835,3830,28815],[3836,1611,85313],[3838,75,32775],[3839,3018,35167],[3840,735,92534],[3841,2178,54983],[3842,1540,96942],[3843,908,59838],[3844,2518,27552],[3845,2822,8689],[3846,3723,57227],[3847,533,31867],[3850,2546,36201],[3851,312,76117],[3852,18,71258],[3853,2127,89548],[3854,2843,91546],[3855,1226,39352],[3856,1248,49458],[3858,2388,20377],[3859,2946,54039],[3860,471,75029],[3861,3225,59254],[3862,1344,68785],[3863,1981,26504],[3864,1291,17177],[3865,3121,83935],[3866,1402,70223],[3867,3299,22924],[3869,2491,27971],[3870,2960,33895],[3872,3719,36781],[3873,2608,79133],[3874,2445,6384],[3875,10,59020],[3876,391,30514],[3878,3063,19424],[3879,63,7278],[3880,1110,89746],[3881,193,59876],[3882,3124,27732],[3883,2417,27398],[3884,848,90021],[3886,2403,21505],[3887,346,89082],[3888,2893,82639],[3889,3299,56212],[3890,2169,4415],[3891,1324,49483],[3892,2164,85609],[3893,3471,92981],[3894,2110,25787],[3895,3810,84024],[3897,566,93824],[3898,2615,12776],[3899,2014,64581],[3900,380,30676],[3901,2615,85978],[3902,2490,61083],[3903,3226,78879],[3904,1097,59711],[3905,3865,89809],[3906,1387,76021],[3907,874,45741],[3908,3563,72567],[3910,3066,97191],[3911,2967,18571],[3912,3556,64380],[3913,2312,39457],[3914,3913,36919],[3915,1914,18000],[3916,1181,11882],[3917,1190,75508],[3919,3625,3788],[3920,3109,64471],[3921,2298,99976],[3922,3324,19602],[3923,2666,49424],[3924,261,47288],[3925,741,55572],[3926,553,28661],[3928,538,46241],[3929,904,6963],[3930,447,26209],[3931,727,94064],[3932,2151,78891],[3933,1446,83454],[3934,717,91966],[3935,270,89172],[3936,877,69728],[3937,451,31510],[3938,1506,11533],[3940,881,43805],[3941,3551,6291],[3942,3167,7418],[3943,2614,59385],[3944,2102,60670],[3945,3532,78316],[3946,1255,22836],[3947,106,87236],[3949,75,94036],[3950,96,72612],[3951,3392,93142],[3952,1825,64795],[3953,3044,56774],[3954,2923,24705],[3955,2650,14805],[3956,3612,72674],[3958,892,52212],[3959,2530,56694],[3960,2535,94193],[3961,1613,84258],[3962,2942,85257],[3963,1516,30383],[3964,3481,91700],[3965,3096,63828],[3967,2056,74218],[3968,3733,4179],[3969,3438,78593],[3970,2032,73621],[3971,2945,66094],[3972,2108,88385],[3973,1696,10162],[3974,3104,71140],[3975,2931,20720],[3976,1061,14360],[3977,690,86015],[3978,2851,22537],[3979,1299,2278],[3980,481,34728],[3981,982,81241],[3982,557,65388],[3983,2465,35500],[3984,1963,33812],[3985,1985,2722],[3986,2000,58706],[3987,2448,16751],[3988,2634,95254],[3989,1085,83083],[3990,3542,88404],[3992,783,13924],[3994,2341,87602],[3995,1497,89661],[3996,747,86848],[3997,3479,64499],[3999,3923,36045],[4000,16,29485],[4001,659,306],[4002,718,73872],[4003,1827,51597],[4004,2063,26186],[4005,87,28498],[4006,145,57668],[4008,1995,61378],[4009,165,72312],[4010,558,35815],[4011,3289,8353],[4012,2604,79500],[4014,211,80094],[4015,1966,19010],[4016,543,23261],[4017,1541,91786],[4018,2791,91806],[4019,2265,88329],[4020,613,18832],[4021,2293,67849],[4022,1755,79265],[4023,2762,1719],[4024,2944,40343],[4025,2984,54007],[4026,2081,80786],[4028,1988,75366],[4029,644,16138],[4030,1243,86244],[4031,234,98332],[4032,3818,51150],[4033,229,86004],[4034,272,80257],[4035,686,2904],[4036,340,8840],[4037,3495,30444],[4038,1084,42615],[4039,532,30289],[4040,3726,48824],[4041,2445,99852],[4042,1462,35892],[4043,1577,90744],[4044,2206,13005],[4046,3999,53891],[4047,2772,40110],[4048,250,94129],[4049,3110,81060],[4050,2605,17227],[4051,1646,13376],[4052,1608,9129],[4053,1395,71482],[4054,2546,21830],[4055,2494,9557],[4056,2111,11722],[4057,1920,59533],[4058,3514,95802],[4059,2505,50711],[4063,3073,4670],[4064,418,47303],[4065,1145,46280],[4066,2500,12016],[4067,1012,55986],[4068,1196,71946],[4069,1991,92055],[4070,1424,72881],[4071,2462,40894],[4072,637,26855],[4073,3010,59117],[4074,827,96183],[4075,1879,17630],[4076,3751,42159],[4077,1621,8145],[4078,1414,66958],[4079,3027,18473],[4080,733,92935],[4081,1469,98455],[4082,3929,5149],[4083,1470,85525],[4084,1093,28644],[4085,3884,24248],[4086,247,91417],[4087,1297,91128],[4088,733,85544],[4089,680,90468],[4090,2940,17665],[4091,4045,28725],[4092,308,76866],[4093,1412,56868],[4094,2250,32997],[4095,411,59911],[4096,3997,55692],[4097,1636,53059],[4098,2446,50259],[4099,1054,90347],[4100,1103,55792],[4101,162,74325],[4102,1538,46310],[4103,1498,79094],[4106,1499,41651],[4107,1637,74281],[4108,3228,49223],[4109,43,91715],[4110,1787,55420],[4111,50,73730],[4112,875,56896],[4114,3530,46284],[4115,2659,88521],[4116,442,50351],[4117,653,73798],[4118,2403,72979],[4119,289,31886],[4120,2466,10462],[4122,2204,53914],[4123,2580,36172],[4124,3031,66424],[4125,2416,19125],[4126,4064,75147],[4127,52,69626],[4128,2813,69490],[4129,604,65538],[4130,3000,20816],[4131,340,17382],[4132,3227,91627],[4134,2113,7951],[4135,1280,91731],[4136,2665,234],[4137,3890,30778],[4138,3125,17363],[4139,2957,32354],[4140,3172,28562],[4141,3584,91108],[4142,1170,10884],[4143,2767,72653],[4144,3556,96627],[4145,3756,35818],[4146,901,98429],[4147,1385,63750],[4148,1710,64894],[4149,424,7569],[4150,2118,89731],[4151,1889,39566],[4152,717,87304],[4153,3207,62455],[4154,3119,47468],[4155,3484,7799],[4156,1916,80504],[4157,3921,70371],[4158,3933,11071],[4159,2584,37746],[4160,1043,94835],[4161,1857,80927],[4162,306,63913],[4163,3316,40383],[4164,3915,23962],[4166,2608,70578],[4167,326,77323],[4168,2675,76244],[4169,1879,87354],[4170,3816,28456],[4171,1985,58208],[4172,1881,51251],[4173,3611,82132],[4174,1961,93841],[4175,3875,49326],[4176,3627,39118],[4177,1482,48923],[4178,3757,9396],[4179,1389,79894],[4180,2541,63084],[4181,2260,94731],[4182,2676,6477],[4183,4101,73369],[4184,520,5356],[4186,2740,92133],[4187,3604,41353],[4188,1538,83882],[4189,3829,78354],[4190,805,29224],[4191,2322,708],[4192,262,64725],[4193,3863,68592],[4194,2556,46697],[4195,331,98463],[4196,849,26962],[4197,3976,54483],[4198,834,88233],[4199,1933,73565],[4201,342,8181],[4202,1739,34702],[4203,2193,70660],[4204,2151,8586],[4206,262,35380],[4207,1910,54231],[4208,1805,9399],[4209,874,70099],[4210,2992,50488],[4211,3684,33924],[4213,2691,24917],[4214,1919,81617],[4215,3268,69217],[4216,2772,73350],[4217,416,69141],[4218,2039,23229],[4219,718,63601],[4220,2702,73662],[4221,3970,99696],[4222,3751,9655],[4223,2750,11384],[4224,654,73630],[4225,2305,85096],[4226,3922,14641],[4228,1013,20792],[4229,971,64032],[4230,457,12843],[4231,1152,96364],[4232,526,65961],[4233,2483,52463],[4234,1023,48977],[4235,902,92103],[4236,3450,92134],[4237,1743,43147],[4238,3798,80709],[4240,1539,13879],[4242,1838,84656],[4243,1051,77112],[4244,3366,86421],[4245,1257,11768],[4246,2440,72303],[4247,2377,84806],[4248,2044,16029],[4249,2044,12429],[4250,714,14770],[4251,614,75906],[4252,3247,1610],[4253,3580,65871],[4255,3183,16705],[4256,1229,61933],[4258,1742,43294],[4259,3659,48053],[4260,757,69397],[4261,3538,23148],[4262,2463,45079],[4263,2231,965],[4264,60,15313],[4265,1477,5625],[4266,3295,94178],[4267,4094,42139],[4268,994,13692],[4269,151,5828],[4270,2794,28866],[4271,2262,4444],[4272,2506,42169],[4273,3451,98644],[4274,737,54255],[4276,2340,56078],[4277,2415,27111],[4278,400,76394],[4279,369,76528],[4280,2400,31703],[4281,743,65212],[4282,450,76874],[4283,2907,48893],[4284,4126,29691],[4285,1930,45404],[4286,1678,72814],[4288,1187,28122],[4289,712,95321],[4290,818,30317],[4291,4241,13830],[4292,3023,16875],[4293,716,97696],[4295,2173,36721],[4296,794,1163],[4298,189,12758],[4299,1168,72489],[4300,2894,98625],[4301,3025,57956],[4302,2154,40060],[4303,1116,38444],[4304,1892,86128],[4305,2543,69066],[4307,3668,8750],[4308,1868,60886],[4310,2908,45791],[4311,3013,33909],[4312,167,46457],[4313,2118,29308],[4314,695,44187],[4315,37,10420],[4316,1884,25051],[4317,3748,37996],[4318,4208,71695],[4319,3150,46145],[4320,373,37194],[4321,1503,56806],[4322,704,61158],[4323,3608,81452],[4324,4279,82490],[4325,3939,33970],[4326,628,75564],[4327,1167,37299],[4328,507,23108],[4329,143,25098],[4330,1850,16717],[4331,3895,78807],[4332,1158,462],[4333,1296,7849],[4334,710,42025],[4335,3556,32405],[4336,3360,10547],[4337,3492,67515],[4338,236,59048],[4340,245,36689],[4341,25,45305],[4342,3569,27916],[4343,2566,40436],[4344,2656,95786],[4346,3,30762],[4347,934,68687],[4348,1391,55454],[4349,3967,70198],[4350,3661,41386],[4351,662,85324],[4352,2671,11811],[4353,209,65061],[4354,2746,34715],[4355,476,22683],[4356,3724,99073],[4357,3605,47495],[4358,1506,16395],[4359,1765,98293],[4360,2717,22339],[4361,1449,18170],[4362,2111,44600],[4363,3311,10469],[4364,2648,50151],[4365,2906,60744],[4366,1709,6902],[4367,1954,62076],[4368,4354,74],[4369,3703,9137],[4370,3992,29697],[4371,2328,17369],[4372,3346,33481],[4374,1113,36569],[4375,2439,17384],[4377,1509,19927],[4379,187,4014],[4380,2242,29377],[4381,2292,85635],[4382,2983,75941],[4383,3424,92623],[4384,3146,21370],[4385,3333,65780],[4386,1055,77574],[4387,1097,10963],[4388,2341,48211],[4389,4072,35220],[4390,649,97439],[4391,2565,72473],[4392,592,54171],[4393,1406,53697],[4394,958,11910],[4395,4092,88202],[4396,922,19617],[4397,3021,88737],[4398,1419,68698],[4399,1498,4610],[4400,3494,95688],[4401,815,50937],[4402,3543,22563],[4403,1276,10155],[4404,4187,39475],[4405,1347,79133],[4406,3244,57132],[4408,2489,56226],[4409,2220,25301],[4411,1482,8935],[4412,4124,24797],[4413,3119,13737],[4414,3400,9730],[4415,913,48976],[4416,2250,35040],[4417,1398,42595],[4418,442,38542],[4419,3225,4062],[4420,4201,42624],[4421,2994,42213],[4422,4384,55947],[4423,2843,98129],[4424,1390,45685],[4425,1027,5103],[4426,2655,34488],[4428,852,19412],[4429,4263,11512],[4430,765,59319],[4431,3888,95852],[4432,2395,29391],[4434,3286,42870],[4435,1615,57712],[4436,507,48157],[4437,2034,52642],[4438,3893,32478],[4440,3872,26149],[4441,2693,10991],[4442,79,90521],[4443,2537,10122],[4444,1437,6607],[4445,2089,15752],[4446,3823,6372],[4447,1757,14700],[4448,826,4921],[4449,4341,65553],[4451,3154,97845],[4452,1715,65415],[4453,3490,11734],[4455,2556,39117],[4456,1764,858],[4457,2066,9273],[4458,1245,7333],[4459,1729,41324],[4460,4069,54238],[4461,3239,63038],[4462,2836,91595],[4463,2605,69978],[4464,2043,67282],[4465,2953,1348],[4466,1905,60577],[4467,1694,97941],[4468,4117,95062],[4469,1063,41201],[4470,500,67824],[4471,831,74093],[4472,3568,29757],[4473,1133,38190],[4474,2210,955],[4475,710,74274],[4476,1668,56981],[4477,2780,90085],[4478,3383,72840],[4479,162,53493],[4480,920,40285],[4481,4262,89921],[4482,886,14260],[4485,3151,65582],[4486,4287,37157],[4487,3217,35690],[4488,4334,58308],[4489,411,30585],[4490,955,64378],[4491,389,76702],[4492,2936,86495],[4493,2960,78630],[4494,3336,68705],[4495,3760,50907],[4496,244,21107],[4497,3746,43100],[4498,2302,86858],[4499,331,1146],[4501,492,10622],[4502,3168,78832],[4503,1048,82409],[4504,2451,21823],[4505,147,26654],[4507,2644,19818],[4508,3512,37214],[4509,4148,23850],[4510,2057,29667],[4511,2629,5349],[4513,2338,56643],[4514,287,262],[4515,1863,84310],[4516,1343,50904],[4517,3075,24898],[4519,849,6909],[4520,1218,12657],[4521,1803,73859],[4522,3519,66584],[4523,2888,61987],[4525,3598,52937],[4526,599,53104],[4527,827,24585],[4528,3134,24894],[4529,1429,81999],[4530,2162,53958],[4531,272,69014],[4532,1580,24352],[4533,2524,42459],[4534,137,88046],[4536,1946,58427],[4537,2792,10417],[4538,1773,24880],[4540,2862,74689],[4541,1924,36163],[4542,2624,61649],[4543,1097,2350],[4544,3010,78577],[4545,734,10204],[4546,1137,93033],[4547,1619,56947],[4548,2257,75487],[4549,3708,56245],[4551,1633,4979],[4552,1834,10694],[4553,4279,10740],[4554,1642,63955],[4555,1078,40295],[4556,2727,36377],[4559,221,70913],[4561,4057,80725],[4562,3133,70490],[4564,2586,55600],[4565,2127,2914],[4567,2096,61631],[4568,330,15106],[4569,3505,85057],[4570,625,47390],[4571,42,74390],[4573,339,70910],[4574,531,56403],[4575,2494,62586],[4576,4443,44459],[4577,4069,994],[4578,582,26287],[4579,181,6535],[4580,896,2264],[4581,207,26175],[4582,3930,82741],[4583,554,62338],[4584,2965,43359],[4587,4563,19578],[4588,511,41921],[4589,1317,92930],[4590,3692,32923],[4592,1925,70897],[4593,1341,70465],[4594,1682,62172],[4595,2617,52330],[4596,3620,41755],[4599,3474,74026],[4600,1525,45963],[4601,3301,27392],[4602,2774,80792],[4603,3547,54814],[4604,1733,36031],[4605,2648,23890],[4606,4044,24145],[4607,3935,69],[4608,512,32042],[4609,3843,19351],[4610,219,85944],[4612,236,22376],[4613,3495,55273],[4614,2412,57773],[4615,3029,76077],[4616,4227,92066],[4617,2572,48678],[4618,3961,75607],[4619,4403,53632],[4620,1457,44180],[4621,767,82577],[4622,3610,73591],[4623,3856,96113],[4624,4380,74693],[4625,1741,89687],[4626,3011,72936],[4627,1551,28008],[4629,3863,2790],[4631,3100,88376],[4632,3300,36887],[4633,2078,16920],[4634,880,89834],[4635,3317,10360],[4636,2312,44358],[4637,3941,36694],[4638,3566,50530],[4639,3418,73645],[4640,4076,98068],[4642,2337,76888],[4643,462,7785],[4644,3151,91670],[4645,3556,37850],[4646,1301,54336],[4647,4573,5840],[4648,3498,56437],[4649,351,492],[4650,1941,90555],[4651,712,71979],[4652,1168,59707],[4653,3246,67881],[4654,2803,68071],[4655,1994,19134],[4656,280,36336],[4657,3430,46341],[4658,2464,91836],[4659,767,31303],[4660,3506,99471],[4661,2812,19525],[4662,3614,28252],[4663,1196,20726],[4664,3579,76276],[4665,460,62055],[4666,2523,53987],[4667,2483,51240],[4668,4400,86515],[4669,3875,71449],[4670,275,20068],[4671,2034,70255],[4672,1152,16872],[4673,2066,41395],[4675,4235,48122],[4676,2347,57119],[4677,4639,55483],[4678,348,42413],[4679,1189,54809],[4680,1754,20605],[4681,4153,53813],[4682,2940,67182],[4683,2248,18798],[4684,265,10562],[4685,3445,26879],[4686,3930,40546],[4687,3009,71918],[4689,3468,64514],[4690,3077,7934],[4691,446,54744],[4692,2339,87990],[4694,619,90505],[4695,1027,33571],[4696,1141,17127],[4697,1983,60800],[4698,4251,6593],[4699,2504,8054],[4700,1244,48651],[4701,606,88097],[4702,646,17027],[4703,3056,5260],[4704,1555,81720],[4705,1058,29955],[4706,2266,91377],[4707,1002,420],[4708,4611,92630],[4709,2330,19631],[4710,490,11745],[4711,58,1527],[4712,2375,56187],[4713,697,89398],[4714,2159,51468],[4715,351,84174],[4716,636,85617],[4717,2921,5106],[4718,1912,1617],[4719,1707,87332],[4720,31,92812],[4721,1621,69959],[4722,1227,39705],[4723,3029,28961],[4724,1004,29637],[4725,3228,27106],[4726,304,15665],[4727,4191,5959],[4728,4722,84178],[4729,210,88319],[4730,2663,68302],[4732,1073,98515],[4733,3048,43250],[4734,3137,22302],[4735,3098,65452],[4736,4384,22158],[4737,3939,68858],[4738,1739,39670],[4739,1750,12257],[4740,2660,21235],[4741,189,83097],[4742,2086,960],[4743,4393,9888],[4744,1355,69990],[4745,2385,45075],[4746,408,50863],[4748,4086,63782],[4749,4375,93868],[4750,2895,20883],[4751,4460,89526],[4752,4630,27321],[4753,2033,28438],[4754,984,28540],[4755,3826,66233],[4756,2094,55854],[4757,1039,86123],[4758,2162,20459],[4759,351,72337],[4761,2100,7813],[4762,4573,44134],[4763,2105,89859],[4765,822,38035],[4766,3073,55804],[4767,4599,87743],[4768,4062,38863],[4769,98,54933],[4770,2116,90750],[4771,3201,28480],[4772,315,78300],[4773,4599,26223],[4775,3619,55164],[4776,148,86898],[4777,4556,19967],[4778,146,10762],[4780,1073,49926],[4781,4467,22168],[4782,500,80698],[4783,283,48366],[4784,3464,27727],[4785,1744,56400],[4787,372,71228],[4788,3289,90011],[4789,4480,32684],[4790,2664,41808],[4791,3072,91167],[4792,4016,51614],[4793,3445,9524],[4794,4453,99332],[4795,4559,18133],[4796,3952,51345],[4797,4788,52396],[4799,3346,45613],[4800,2792,77832],[4801,2072,70024],[4802,617,89525],[4803,245,72605],[4804,4086,62971],[4807,2594,12957],[4808,3714,48598],[4809,2616,18693],[4810,74,62882],[4811,2973,85141],[4813,1573,13286],[4814,944,24285],[4815,393,48508],[4816,92,66773],[4817,1613,46200],[4818,1913,85145],[4819,2181,52386],[4820,2187,57368],[4821,2615,35193],[4822,3356,16635],[4823,3936,31775],[4824,345,48639],[4825,3576,51506],[4826,3542,79028],[4827,874,3710],[4829,1028,11124],[4831,4442,75460],[4832,3378,14580],[4833,3632,2249],[4834,3419,47740],[4835,2068,1344],[4836,425,72083],[4837,4825,24877],[4838,909,19974],[4839,2311,25057],[4840,649,22635],[4841,3753,38523],[4842,951,42616],[4843,1665,35938],[4844,3279,86232],[4845,4249,6468],[4846,1085,59835],[4847,2058,71088],[4848,1542,25647],[4849,647,6423],[4850,2809,58914],[4851,3049,59313],[4852,2243,3898],[4853,2961,23283],[4854,3657,66324],[4856,3769,47780],[4857,3439,9107],[4858,4289,30038],[4859,1405,86108],[4860,3061,77134],[4861,3647,9451],[4862,713,88792],[4863,4653,55572],[4864,690,51356],[4865,1796,65553],[4866,1439,98863],[4867,3477,75925],[4868,4573,56861],[4869,2989,1267],[4870,3130,1877],[4872,3504,86658],[4873,3957,9637],[4874,3968,64806],[4875,2497,6132],[4876,1462,22218],[4877,4469,50945],[4878,4148,48824],[4880,61,92026],[4881,4260,12448],[4882,2837,10496],[4883,4176,93769],[4884,1879,78062],[4885,152,64406],[4886,3935,12204],[4887,3086,85979],[4888,86,82447],[4889,545,7204],[4890,2504,34936],[4892,2488,84929],[4893,2558,43869],[4894,4697,16978],[4895,1815,20708],[4896,3725,73424],[4897,2814,74613],[4898,3980,87183],[4900,4264,9058],[4901,741,38291],[4902,2584,63022],[4903,4220,25044],[4904,2247,27226],[4905,2539,15981],[4906,3461,80179],[4908,1385,38442],[4910,1748,2983],[4911,943,95061],[4912,29,77590],[4913,798,81584],[4914,4352,71546],[4915,528,59703],[4916,2635,59531],[4917,515,35616],[4918,2753,96032],[4919,4574,26653],[4920,4195,30293],[4921,2108,86492],[4922,730,73331],[4923,3138,89370],[4924,1032,46090],[4925,1604,2204],[4926,2301,18726],[4927,4593,94106],[4928,2775,60368],[4929,3338,31890],[4930,3697,62844],[4931,1977,68316],[4932,134,71315],[4933,1104,45573],[4935,3020,89794],[4936,2113,77051],[4937,3956,92873],[4938,3814,95395],[4940,525,67922],[4941,1369,76376],[4942,2820,7764],[4944,3583,47405],[4945,678,98918],[4946,3641,74773],[4947,1153,20622],[4948,3818,81255],[4949,4542,95558],[4950,4177,89776],[4951,374,22579],[4953,4468,63616],[4954,3236,32701],[4955,3035,10149],[4956,2197,62741],[4957,3049,13547],[4958,1435,97768],[4959,1322,99894],[4960,2884,83152],[4961,2090,53156],[4962,538,82698],[4963,2588,81769],[4965,2275,21276],[4966,3118,87679],[4967,1177,59162],[4968,4121,65210],[4969,3239,10432],[4970,3068,95365],[4971,2626,90152],[4972,183,60398],[4973,552,32485],[4974,4844,24576],[4975,1795,90547],[4976,678,44605],[4977,2612,78816],[4978,2495,68568],[4979,12,98743],[4980,214,22124],[4981,4755,1098],[4982,174,40306],[4983,4420,79277],[4984,949,1863],[4985,2445,39103],[4986,2451,74467],[4987,4513,49190],[4988,1398,51750],[4989,4347,83126],[4990,754,1916],[4991,1745,44967],[4992,186,15327],[4993,2379,25658],[4994,2315,7855],[4995,2498,68184],[4996,1108,83621],[4997,2430,52934],[4998,4116,61384],[4999,4646,97322],[5000,4815,2714],[5001,3627,58553],[5002,1711,20167],[5003,3474,55766],[5005,2124,13224],[5006,373,24479],[5007,89,94483],[5008,1237,56902],[5009,4012,8034],[5010,833,86052],[5011,4339,87828],[5012,732,44549],[5013,2032,88365],[5014,390,96121],[5015,3054,43712],[5016,1254,11188],[5017,1885,91180],[5018,249,50844],[5019,4340,56613],[5020,396,30292],[5021,289,78828],[5022,3261,51253],[5023,573,2970],[5024,3393,48925],[5026,2952,46418],[5028,2895,83452],[5029,4494,44285],[5030,509,57283],[5031,2342,98032],[5032,511,69230],[5033,4919,48916],[5034,1308,1113],[5035,3827,48818],[5036,2598,41938],[5037,921,5822],[5038,2247,39021],[5039,293,74176],[5040,183,71763],[5041,4333,6059],[5042,4787,11116],[5043,4268,41184],[5044,3453,47232],[5045,885,83205],[5046,2261,55624],[5047,2646,88047],[5048,2027,41878],[5049,3562,33251],[5050,2682,21808],[5052,1810,21656],[5053,1994,6962],[5054,3236,84129],[5055,778,89003],[5058,3076,79013],[5059,3828,64991],[5060,4031,37419],[5061,1087,92433],[5062,191,68136],[5063,2730,46290],[5064,1065,9715],[5065,462,92878],[5066,1766,92559],[5067,2749,95270],[5068,3682,34381],[5069,4977,14521],[5070,4152,9447],[5071,4286,25801],[5072,647,12664],[5074,4526,89566],[5075,2476,16395],[5076,3345,70782],[5077,2681,33611],[5078,3210,93618],[5079,1756,38443],[5080,4425,76266],[5081,3280,13123],[5082,2383,4185],[5083,2799,57486],[5084,4183,28623],[5085,2682,5750],[5086,2453,71868],[5087,1513,18449],[5088,1558,1777],[5089,1970,89319],[5090,696,94673],[5091,4104,59945],[5092,1719,90679],[5093,759,36574],[5094,5016,75070],[5095,1032,79605],[5096,2266,5669],[5097,2407,63608],[5098,4004,61859],[5100,4269,93517],[5101,996,5688],[5102,4371,69525],[5103,1766,45087],[5104,3304,79888],[5105,301,63307],[5106,233,72376],[5107,4104,82064],[5108,3458,56577],[5109,2588,29428],[5110,3446,39224],[5111,5049,88607],[5112,2112,49804],[5113,383,37855],[5114,2169,74609],[5115,2950,20200],[5116,4243,61173],[5117,2460,88193],[5119,183,86090],[5120,3401,49682]]

node_dict = packupNode(n, wells, pipes)
node_list = list(node_dict.values())

group_list = get_groups(node_list)

In [4]:
import json
def dump_json(var,filename='tmp.json'):
    with open(filename, "w") as outfile:
        json.dump(var, outfile)
       
       
def read_json(filename):
    with open(filename,'r') as f:
        data = json.load(f)
        return data


In [5]:
tmp = group_list
tmp = [x for x in tmp if len(x)>100]
len(tmp)
# tmp

base = tmp[-4]

base = [x.val for x in base]
print('base:',len(base))

n = len(base)
wells_all = [21128,75528,97977,52262,52378,15538,38502,82313,47321,85603,57516,70456,3340,10685,14501,58724,42337,74534,6019,4958,57135,702,93577,99406,31850,89442,92604,98339,96751,92818,99355,69270,69868,52010,69654,68788,31959,72233,81473,89012,98204,77272,17591,88067,1682,66222,2648,39761,53101,86944,93661,24208,61060,27169,4096,61950,91487,5713,66947,92700,69000,22407,50866,30970,13669,67723,25650,12925,86768,18176,23777,47704,6001,28669,18491,41794,81633,98117,86153,84833,61390,56697,61686,16438,96154,89325,19969,34488,71395,80075,9290,42780,71525,98954,98949,5726,47989,36609,66733,90119,61939,93626,50667,21209,30272,16076,23587,31502,20140,67856,10240,22960,54014,52123,87986,67927,37322,50755,41996,82535,85198,9227,9908,8376,93639,65630,69549,98730,80787,14611,13998,57844,71866,29050,79414,35399,32945,81424,21955,41679,73049,11354,69358,18489,38909,36219,92724,51020,89519,43854,8018,1177,50616,24035,99671,51378,69801,9524,58,94173,21476,60926,95071,97791,80030,19941,13664,40706,47216,14832,11235,1618,25984,52530,54636,98407,18467,39794,71734,32048,43613,68877,1750,36231,85772,10250,82036,61698,4581,8832,61311,75892,50035,22491,94445,97822,57825,83237,26853,58666,92055,23491,5679,34791,6369,37423,20900,97724,5726,10614,38469,29887,78640,84042,26189,58524,51047,68102,33765,3440,20862,31749,61789,37905,90161,54661,58086,43710,74402,1787,8606,1694,84348,62614,69986,61834,24177,37259,66432,28044,54452,43908,57008,24151,74973,15201,34448,68761,76275,34812,82783,25593,65309,54357,34658,50861,40309,99643,49225,62866,56314,54205,7414,6946,42236,75620,19893,79605,292,63612,83196,30451,12402,36348,60488,35269,59872,86581,75990,85045,31249,82809,36114,78399,32493,64972,47424,91529,68309,29235,7094,53707,6607,64663,31229,23737,21931,99922,71809,91458,774,14714,2129,46905,36074,43740,10298,27174,52954,60298,51533,83198,55851,50244,14454,20660,16223,79195,53447,76838,34390,47889,83767,86683,23837,38197,87247,46525,69473,90769,46233,39895,90331,3856,5245,6039,27673,40562,74321,55080,71112,18073,10865,95454,9593,52441,74657,76750,79532,39354,56288,50675,18758,99925,37918,30710,51440,99446,42368,65414,33421,19571,17110,42778,15604,86739,22061,18133,49087,12508,8185,5696,68532,44608,10581,55129,36116,77450,8836,87767,28010,31303,52670,11378,32852,56253,74493,91829,77304,28107,5736,53220,40739,10929,12977,71900,19972,36336,63597,61179,38395,73330,3170,26604,25879,62880,81557,90467,88706,59362,78528,16957,71241,49440,15725,1915,26006,35380,14401,80262,37337,62783,92025,63896,30995,75920,74312,69685,27159,32062,70444,92693,52586,13090,59397,33566,86826,58154,22750,45680,24536,72131,96895,16672,63780,19003,16442,48544,93873,79506,90387,89772,28710,87459,4253,22432,5508,62846,19167,98522,15840,20970,21646,86268,95181,16273,74467,26720,45498,21394,27488,41400,40857,98905,38323,37166,58531,19807,90624,85466,826,66915,89977,67603,88858,54190,56391,53628,29255,44732,90461,87316,72920,10294,94325,46266,32320,84913,27775,46317,25259,62604,98683,1163,50831,20996,67535,28022,31936,9359,57997,53293,16166,90968,5610,98677,87204,32122,64223,49981,66231,94138,63795,34347,44108,16931,12803,52934,90050,10905,38807,56499,69434,75748,75987,28644,8009,9228,37462,36282,74376,24584,18419,36874,84941,10567,28562,24564,4388,89426,70248,97569,83634,78601,90408,24587,876,11564,24023,44134,92407,99304,59507,52676,36490,51811,98936,23078,23451,6912,91728,97804,45705,29015,6501,50648,93394,31486,85833,33121,77695,79898,97513,79456,93290,84536,31622,32336,39514,82426,42270,20375,45038,64750,60304,23190,92543,22771,50340,91463,87263,59310,30115,42401,14075,1839,19666,6201,91372,92347,47312,90591,45247,41791,60831,25649,2183,9785,95802,43927,83036,16446,72458,15378,24760,11584,30490,27446,56556,3947,90231,55869,96391,53320,81259,99427,68218,73419,51813,73002,20792,32167,793,37339,42973,29669,38921,84152,28165,63391,35823,3667,42668,52344,36006,66088,22539,35611,25678,377,10698,37641,4152,25553,19953,79344,76166,46159,94346,81931,29383,40563,1383,33101,24119,67774,75540,54,10297,14159,24397,95229,34503,17591,22643,53823,96961,99632,62902,37168,33152,76881,50996,11969,42557,28398,44837,30287,91066,90878,28572,50129,52533,87683,68314,50450,81159,34636,16141,97481,85268,65125,54871,56141,74785,15739,2596,30257,15052,40850,43972,46468,1478,12625,90174,87667,15451,6492,74535,47633,29540,68661,50153,19973,69822,22596,52483,51971,59536,48421,77723,55879,60123,75458,12363,49026,87090,17887,70196,82654,85048,29062,57160,36882,94244,94262,21539,6242,79710,84186,71852,95820,75439,88436,45074,86037,21724,60272,82634,94848,71507,45058,93257,20090,62973,87047,94777,74805,25555,33799,46180,51568,38778,50209,29587,48009,13611,77011,83699,68421,58897,30217,73538,51615,16195,47134,61167,16458,14992,95421,71095,48104,78506,91875,60334,4114,75527,18786,82351,35532,50241,50520,17270,90251,48971,88583,68346,3848,15025,70858,97811,58037,68107,31215,8917,75557,44214,53772,11833,19310,31803,82175,64860,28944,94860,68953,26949,44917,24732,60193,86879,64017,16658,29788,12836,22687,88036,1727,92302,90009,51181,58144,48586,39924,16946,92452,33424,77306,60740,78015,53771,95852,38535,88149,17595,63739,33295,17344,81835,33448,23512,58607,9968,13270,15219,66043,99079,98340,8320,43418,19105,42643,84442,75356,53399,80425,32583,13078,68747,65624,83734,893,54635,945,79073,87847,40456,74898,90253,45601,23965,9188,40894,3754,79019,99794,57332,15685,91816,83086,53987,43455,70315,33107,31030,3535,24937,10863,8045,84647,82713,5071,15733,42459,14557,64461,91699,16500,99259,52970,37304,4569,23097,13179,29011,88814,16161,3696,93996,93160,88189,69671,79403,62556,33777,11092,13584,6275,61774,7373,93411,71739,58735,50241,73185,20502,25261,86711,42480,39348,42682,11165,90124,39882,11329,60009,82621,81196,22652,17384,71993,8830,86961,23286,56008,21564,42073,97841,81333,36967,89008,64432,89462,31948,59357,50876,92984,31854,75777,54349,75612,2645,83708,73798,84068,18782,67725,51197,20242,78221,15405,88975,51712,53338,56229,70559,93094,42660,91704,31559,37972,79132,63110,24211,87803,59038,15146,80714,72374,73476,17009,52153,42924,35290,46721,63103,9738,13920,69124,11726,43399,80181,93036,60362,94235,27969,45172,23065,6605,16154,26321,68698,56945,10420,90505,69012,67396,89141,47554,69783,82993,4910,66633,78642,60171,97551,24823,8750,57266,74022,91159,57012,25819,35944,79905,10308,46203,47968,95061,15292,81099,74545,72865,27022,98151,11036,34755,58121,66532,41026,3114,54703,84146,15231,15438,9441,74611,63248,31882,92410,83667,89292,64519,76340,70337,31812,11008,76279,20963,31667,75575,38826,59449,64260,24512,27084,33892,94339,65486,7069,17944,87392,98921,78632,58804,4891,20515,45314,61484,68694,49231,94788,35669,52970,67904,36921,6102,58638,41658,73948,97709,50701,70696,58743,94303,62964,87389,73320,87552,16086,97919,92033,52684,65339,37318,90039,69677,81421,978,78297,96989,38235,53587,79889,44260,46184,32704,91898,93954,38755,31545,15054,40644,34988,61926,80204,32829,81642,28595,76613,51572,37339,38967,94174,73311,80507,10750,89006,24961,87684,69276,23445,33301,43699,98773,59384,30661,56264,90151,5850,18180,52408,54646,22614,67348,22613,24805,45947,29500,43287,14864,62135,84902,60446,18211,92923,76670,89875,49353,90483,48189,38758,88789,38625,14797,30270,92055,27904,9988,74765,30139,22782,16514,11091,10572,75398,90492,40731,97848,97439,4262,90919,63479,11307,69550,67871,99525,45190,99847,43972,84558,40801,70605,11570,15760,46120,69357,16621,13938,5398,23311,5720,83340,61227,93356,99224,80281,63292,72418,96202,63819,15608,6783,71776,85763,18057,91241,19993,63631,93025,83683,7753,32760,24773,82670,49424,25092,98290,13525,41696,62152,54031,36463,60279,20657,44645,60436,37415,23642,94797,85267,18304,99226,73993,98763,6658,87030,68355,31437,43022,29304,23320,75433,61626,42079,67876,58293,16608,51941,41930,40880,86964,49379,1077,15350,72960,13884,29503,39425,86562,70778,59841,34248,52260,74490,90093,53189,99288,80278,83298,99575,72594,4562,34793,95116,31373,51355,25006,17448,3263,44201,212,65868,34255,48071,77893,67461,72771,99698,77863,70132,61650,65378,26666,84315,85665,95930,54391,31646,2817,40276,6652,59497,25345,15928,46708,51400,58897,97320,55420,18055,6616,64022,86074,68372,33385,27765,99561,36675,36492,48263,50987,56466,33460,12238,55671,45851,96091,48186,41076,56396,15412,36161,42485,56796,56296,88870,46899,8233,2344,2885,41342,31776,35569,18716,95261,29438,73718,70650,6089,1471,68298,17162,68760,66087,37528,40637,26635,99070,31718,15189,55957,76064,22738,60905,42989,85938,97466,50427,27183,8730,96235,13517,84361,25325,79309,15929,77996,70506,79021,76536,23765,82921,39363,603,40314,44394,53286,29204,71468,57635,34317,63272,36569,95972,943,32101,86247,30776,61664,87536,56005,36357,91667,75500,67288,79466,95007,48684,30231,18924,70970,32506,76005,10590,29909,64396,23674,89669,18915,18373,14115,49806,71403,75369,64577,75182,6167,44909,93849,4498,83941,33918,43641,82119,63457,61137,26533,70646,73593,7797,68141,79481,22767,9325,10241,17247,19839,51908,38091,7748,48673,84165,20139,60560,64666,90829,57871,43645,98423,27581,71138,47389,59847,93497,17181,40712,3395,22902,78394,78270,98706,52705,89097,2807,74834,44882,65030,77593,22725,62296,6737,35532,30852,9400,77320,86042,81493,58511,29323,18102,85251,67220,89253,31261,52417,72485,51266,64426,62836,69716,29557,4970,75907,70234,11188,43237,25894,38253,56197,13576,9403,40668,47581,9439,91457,98782,20608,72358,4031,55695,43430,67698,97238,73269,36596,41483,51941,92234,55751,53951,67526,91253,87054,22002,25081,83501,26084,94748,83638,94522,1796,73818,70702,42083,92016,96918,78500,17090,53413,67636,55676,96824,83066,82011,23003,70910,81928,64516,25695,92082,28501,1118,11722,80628,87657,69458,55520,7348,87634,19571,37504,35679,69084,50096,1347,89816,99799,37729,79758,2035,23594,74602,83917,50023,8962,3168,12504,7853,6379,81057,86219,33646,54178,63859,65010,34130,70307,94265,87551,79211,34376,12890,54603,34827,67616,34647,89431,36513,50999,75722,34306,56047,55858,7534,89107,8928,94114,88974,8692,93547,39919,7773,81787,73825,47923,35681,36029,25393,94352,48007,47078,68736,29625,70072,71398,28919,31743,9530,94858,19626,81906,78556,76955,94531,81944,72022,40212,24387,29111,72259,10573,97955,37106,87612,18276,64241,31172,88174,9686,31806,792,67672,43420,94922,43945,24676,79613,99315,32058,36830,95527,71541,93854,87934,76967,94523,48348,11352,49499,22146,88900,32017,50777,28841,64092,77829,25565,56050,39276,35692,32393,14183,26235,32838,64666,3338,2629,27823,76624,87858,63386,66919,57822,93350,51011,1742,9730,91361,11598,67320,46985,78953,73631,30742,69237,5435,55425,82558,95691,77875,80975,23059,92475,73473,98447,70940,32906,59190,92867,68348,5556,57533,5148,93146,99021,46642,8157,59752,24720,97493,78327,58469,4701,22749,61142,61444,15324,44766,80137,74351,75926,6247,57736,93148,45593,46209,58416,55245,56892,18536,87139,90297,65547,11931,19977,54649,71025,10933,38213,6357,90482,94253,35511,52616,43500,28277,37740,96151,2685,37659,76487,39806,50793,10036,71092,39660,90267,17428,83146,20686,14443,78661,68681,66123,18875,82484,70883,5379,17918,66891,19381,68217,54588,53006,88269,66011,26968,57962,47844,35261,24554,43140,4818,98339,45157,89034,12249,47427,14749,36123,13053,58955,18342,65582,47486,20177,41972,66933,65616,83409,7520,70666,41162,14711,50553,53608,27560,87432,44220,19385,64854,85848,51834,37139,54625,48132,98780,4655,72750,45522,23574,49063,88600,85963,28537,43234,20117,12524,86640,98162,87622,12848,51964,45065,2120,67388,71460,63350,86999,91132,93082,95604,49405,51633,8271,85487,89843,16603,42176,59270,41587,52940,79603,97406,98598,39718,4251,63950,64811,73550,20437,44042,29838,3560,44145,39657,84037,4209,27956,48873,97320,14472,41480,61335,28242,75839,27906,31192,80590,26464,19645,62303,43108,6603,3271,83417,79809,79697,86997,87711,14463,63207,90294,68756,22156,54262,37616,74081,46293,52772,11267,58551,74985,30785,4630,68565,90050,91024,38200,25704,52810,17906,98031,54381,85656,53943,92632,87993,48477,88049,11941,43800,11045,89282,81739,67607,11679,53608,75578,63129,40714,99341,107,71926,31410,29286,77130,21499,42814,26904,22430,5573,72087,64072,72856,12668,64005,67179,15146,37712,21322,82204,28505,39434,60242,66862,18034,57636,33432,85578,73059,34883,27960,55433,82561,17058,63846,10002,4748,61897,29107,92117,19716,32815,82725,36591,26108,95863,86442,24730,51740,46974,13210,51092,63307,18524,98492,82518,32636,53624,96070,75927,94324,97038,37728,51123,33693,96446,83502,28173,43153,52575,39100,98317,36326,70750,39430,44577,99970,24960,90983,34050,61476,60377,24205,92270,89175,21395,86309,45709,49367,39804,25348,20130,86690,90056,88481,68610,7588,95258,15043,22077,35701,32950,59296,26148,96232,61941,14533,49683,14462,34807,47884,4539,87971,54225,91864,45356,91481,19425,77604,47517,71776,4620,30934,45565,90781,37093,76943,13574,98729,77965,59131,89289,90560,32389,9672,46518,41753,29209,2495,47792,42968,91282,49067,37914,59085,74008,84292,67233,74183,23725,56254,92658,99788,34515,66217,71925,40893,51299,81484,60402,9901,69941,62615,79540,67010,44048,40285,61933,33131,61030,26847,38267,38427,68019,83502,9732,52662,70270,21543,37753,7252,33018,5749,53760,71683,13714,49260,21760,51785,66894,52450,14935,18477,91117,30,76977,19710,66006,83182,92762,57229,88468,72434,27652,87346,22963,88011,5623,4713,79992,50880,60056,3975,68946,61503,75702,45244,4232,8881,49090,78240,16620,71194,8722,63768,81328,67869,94755,58265,37032,31670,18900,18995,24721,99862,44229,92752,34348,70048,53240,72430,92202,23,39098,2296,54344,50705,62238,22891,64614,51416,17044,34322,46537,46347,69272,59164,39471,6751,14592,94381,80003,36032,68365,66010,79635,1807,44614,11912,64073,41726,24672,14534,1574,25125,40782,45502,67397,83307,21575,39320,35386,91056,77997,98801,39821,73186,54941,11349,62029,74196,19504,60240,51942,47773,34788,69325,53380,42650,52871,82481,52491,68674,78422,36432,10157,94419,41688,87639,3077,96721,59910,45456,95875,81095,70111,69307,44742,72594,12507,81902,79885,77407,89444,99409,57156,95938,33157,75193,80765,76643,2134,92626,96024,91929,45526,2481,48015,73769,36175,34060,41579,40862,30987,57966,57108,31040,32493,73768,22647,77611,81269,3727,31319,2788,85671,1550,91767,2060,91174,88800,36027,278,63106,61366,30825,372,58844,37840,74682,69582,8124,89595,25539,69754,58881,97901,65104,37638,42371,80762,78709,67005,2639,93094,7934,48969,24945,17455,54564,94063,31628,79576,62928,12150,44378,5761,83162,58070,83952,76246,89737,44145,35155,31442,98690,4559,78946,37369,18335,99094,70935,30451,91512,52094,56661,99509,32490,72150,98741,60800,61749,37544,836,46036,98983,35019,70565,83783,50794,74373,76206,69330,30868,23631,76206,25542,69841,8844,55755,29633,16328,65545,68985,14548,37910,99093,32966,74018,1496,724,9972,97043,73574,19831,22218,29644,59,64508,63119,7295,90883,902,14472,75683,33424,61851,22662,10033,40416,46165,78138,10410,17872,79971,17276,99185,92762,45786,84684,43650,53479,72079,5388,47521,50266,35462,46298,46748,75288,277,10399,41142,25991,7659,14395,50662,53096,99787,85113,89566,76998,63888,52547,70722,35917,47566,12493,94859,29722,75458,87735,25570,29977,39146,82507,64562,67560,8309,17455,64868,1028,72160,84062,47017,45221,25866,64962,6166,48983,2166,20396,97398,25129,20483,68305,7220,62933,92988,98093,97713,99462,8880,81503,79519,56986,31881,24836,71154,90968,8925,10155,99664,17661,47209,19141,84285,96435,55927,45206,65029,50140,98098,7751,33350,38010,45222,75128,51682,98162,62081,67166,42538,19287,92037,44474,33631,57479,17408,14811,89646,11304,9015,83119,9634,856,50989,85187,72807,50417,13607,40718,90950,33187,20470,42955,93058,88566,9267,37255,94204,11603,62841,1341,45421,27349,60869,56422,52117,46688,56547,97762,82184,80347,86153,14670,71562,38909,19882,25599,51008,554,92186,60684,75790,69902,5906,87784,54386,24828,68036,32485,4641,21748,48737,32362,4673,64196,53621,55092,53621,41687,79798,35481,70316,8771,42738,84152,40746,85060,921,15094,87178,59400,21312,74387,23142,66761,77467,29193,63887,22089,79143,57833,36849,18925,37113,11375,32311,67069,48409,76371,70844,18594,24975,30610,73192,15594,7279,40164,63534,83854,47889,23665,67277,15602,46000,31993,14124,7077,1295,52849,96087,55169,14966,1937,6924,82161,55482,62459,77542,11373,83571,68837,82138,41658,48654,52470,57401,60478,38162,81217,13422,91103,20326,84553,35536,13638,53079,61523,85495,91430,20907,4641,88302,15108,17678,22933,288,68523,46200,56806,38298,53443,70613,62562,32996,87451,90425,82910,76968,42735,45889,5564,79522,53787,52951,52868,61298,24467,33897,19426,95719,75753,24456,38083,8540,73329,85622,91612,68821,95612,63791,85748,23294,39552,88423,35241,14057,3803,22588,45023,72259,95016,5355,5576,2381,95050,30646,84734,25421,17750,77872,48873,5430,26093,27439,23696,35630,73749,90260,52953,4989,28523,50330,58968,33821,2873,52426,60092,74590,66956,70998,35820,31882,26054,26563,27709,5697,94421,33199,52004,45997,7390,78934,39495,95019,67882,70302,31972,33059,38188,92355,4310,82834,64121,99908,55510,38983,75328,59823,38474,35409,69678,94048,89858,23507,8962,66673,61917,37641,21241,64437,67867,57187,29146,83277,14917,80060,71933,62529,42437,66735,38978,77265,54602,15810,27158,13003,47161,5905,97220,75586,13371,66901,78146,85225,49487,29131,26311,44404,50095,75831,17716,80676,71769,62170,3060,44213,83884,2271,28941,83628,67297,35473,9810,41926,91269,94833,91527,14541,3056,54069,22129,23060,18354,70078,47906,95235,33930,62060,39971,81442,13572,30083,37232,52203,14277,68192,86450,96615,88266,17705,40882,8317,17799,42953,78266,68182,91105,307,65762,86308,4501,49946,63575,79013,17145,35705,66913,47836,28979,53816,59823,90208,40034,26375,25131,38148,20604,91033,38677,9427,88544,52195,11291,71449,61757,20198,86680,76855,13902,23841,85046,83161,69578,55935,35367,95540,37388,40601,49481,29562,71865,79124,3467,1374,39768,66346,41237,96376,34719,55147,12156,87346,36489,95540,74797,37542,69489,56788,90254,14856,24395,91439,25953,3030,75424,38736,87710,3721,9944,97187,63589,86659,84201,37363,73009,11055,1279,75794,21122,14557,37020,84352,52697,92986,55206,88807,68447,81550,51892,77535,37285,84591,53494,76655,12243,77960,41655,87936,14095,38370,53638,30388,8438,17631,37950,9973,57453,30387,44642,29163,52334,10004,72158,29199,41541,60221,71064,54534,74471,9314,52249,49077,20818,61399,14838,53537,53617,26242,16163,68726,70052,30511,14824,42981,49817,86533,58579,22946,4592,5424,82076,38727,22929,60457,23953,8640,71032,91248,72338,34507,13618,98256,65892,40759,67481,17027,41414,56238,5704,18903,30758,39670,65266,46412,22397,55411,16550,42192,15645,50637,96232,93465,33712,35829,5753,20259,91147,67798,30498,24047,95231,49812,33776,98921,66478,77036,61962,12944,58802,36798,87314,42400,35436,59065,74320,529,94954,56911,98352,66407,71811,61428,90216,61927,9629,78414,87533,2130,13844,2739,68144,59749,68782,82357,22263,59333,72011,84676,38617,99730,98271,11583,5449,24466,89532,71396,70,92693,47326,89119,31097,7516,10951,78232,31229,69627,25953,27923,39917,93665,36088,91566,65819,35045,81989,15525,42815,45446,57392,23651,76214,20471,50028,91889,55723,59182,87938,75573,88730,80424,80905,32160,75726,84790,90064,38436,65043,8899,13568,20759,40620,55390,42919,65040,39531,74169,72514,62058,7239,65214,9870,91044,89263,54000,66248,60112,50703,60109,27628,24071,4413,56252,9631,93167,39080,13937,21509,61919,52693,95432,91423,69858,90216,1751,41848,40512,16004,41812,33395,6984,61922,92481,6523,90063,85178,78092,84525,54555,20106,79362,41937,40680,58008,28539,31414,85396,46811,14339,19476,65131,11410,12725,69293,95961,70908,69982,29716,76319,1280,41564,59056,19274,34312,22856,17685,52300,39328,90096,76208,42706,79285,31443,64749,46330,70489,28147,13432,61560,45818,50399,85065,69796,99013,4211,50930,6575,44928,19382,73888,23945,60222,55596,38833,38516,35713,43468,1315,37091,52416,89688,27658,49344,3277,6924,24083,4024,45094,46511,95910,41865,1276,31258,98099,54524,55655,80741,77967,87767,86067,11357,63372,48613,32980,96141,48195,10632,78580,14005,6596,56109,958,64485,92986,97413,5373,58154,63043,79631,25745,77215,98929,77738,20535,54438,78645,9919,62495,12414,83591,9693,69570,87240,62730,12750,90921,19100,15691,12141,65027,16469,16275,29422,29579,91147,16486,44133,24889,15823,73537,59632,38869,67303,32253,87318,7449,59751,92000,57284,59879,57547,85419,12956,17892,72398,52079,7696,68938,72632,46570,37455,4847,90911,15298,69960,74862,80849,44691,86966,51584,7027,39784,61434,32931,72636,12106,91949,25868,16181,15711,1398,36397,32480,63799,95284,11559,24211,60099,34886,88922,26551,46385,39425,57722,50215,33376,44508,47020,88448,78103,5111,271,9960,14240,47131,61434,47215,55257,6417,30439,59464,68583,43773,7894,81783,19161,41076,62157,25615,19706,4594,46319,37257,39213,86849,94477,36802,40846,71856,61206,82575,94911,16566,34953,49861,80548,88561,94718,98330,66056,23865,48949,84409,67127,51217,34208,77806,82734,89247,18289,65479,60489,33523,65897,93971,48013,88725,93437,35070,85302,75788,84270,75712,99343,35849,80954,67543,64979,78724,94960,62146,46754,78790,64210,1426,3539,39852,6782,98449,31139,84217,24245,26652,61990,89290,4646,29023,61764,84115,39211,6641,65885,88724,1532,32465,2076,31387,63334,98997,3737,82569,58311,56461,8666,2160,88393,23915,63479,84364,45463,64733,66762,65776,69803,82684,67648,69721,82981,88118,7315,91024,22481,10811,46392,4335,33746,70292,34233,7372,72673,65877,63502,92207,48611,4825,4005,60406,28520,15672,20981,98579,43859,21454,46138,49424,5096,77314,21855,52241,85245,41438,5910,17767,11056,1658,74383,95020,33170,61827,24351,28630,78857,73565,24519,39613,39850,49098,78222,62004,12384,70823,50736,91128,78818,82819,76602,56808,45062,5949,16140,83728,90690,37126,17748,25034,17198,140,64986,37277,84684,38500,85144,70229,67561,85312,22664,97716,79676,56735,71555,44389,40859,99313,93215,215,53499,4981,77081,85505,94974,37356,50899,14165,52473,20842,18063,24077,42600,31831,74702,69822,47774,57847,75284,4221,51211,15898,5846,6761,7236,44461,57589,35286,75191,65596,20583,19437,71837,4555,7100,63527,32833,44053,57011,40634,58428,29431,30463,1193,55281,39153,86994,30185,22081,66485,34032,22519,21799,74942,86685,72624,73936,12792,29561,47571,65188,42324,12950,45723,46150,6526,73437,29723,27386,62676,84701,89394,39305,78134,43147,85629,53964,78979,81195,65863,60695,61927,26684,58651,34614,14697,58233,20356,50110,88109,54215,97414,88645,62213,92478,92034,53193,98721,16421,37804,98510,58299,57558,13455,97737,14057,99604,65994,86153,67272,55621,82583,18364,36714,6979,48761,69118,6811,60381,97943,67346,9201,15301,47501,30010,79740,30878,40696,57019,87286,56184,76333,65022,41991,96417,24238,94972,43582,79752,64115,79161,16824,10262,12206,5864,78301,94350,60832,77655,67572,65833,99506,17100,87807,37696,81933,20085,63746,49165,1962,49837,32257,36210,89085,34859,54635,22291,34452,95021,55980,60352,99429,34016,12184,24039,65951,32879,39819,44390,15488,60109,2250,65366,96790,28477,3101,65864,95909,84900,43296,19649,61896,23238,21883,83856,41951,80073,70217,55637,27776,84955,97233,31659,79663,62480,54910,42329,40308,5110,53744,59013,15541,28405,92828,21449,67874,15321,33049,39504,37615,50391,37307,65416,16304,38357,86734,79546,93282,67193,90754,86931,31588,13000,6884,71565,67520,21148,71447,19868,62259,40917,26731,80988,69143,24415,34276,91080,43835,83296,21658,36551,9081,21128,11569,8783,11178,35165,41674,69369,26315,94647,84601,87098,40285,84766,88971,50583,52459,77660,31150,64481,43667,81166,67013,25659,38168,3906,6001,77015,46570,9818,99108,65659,91776,22567,16943,40380,35876,97739,45779,34494,1175,33688,97673,82939,4797,17673,68318,63741,15092,67403,62171,93647,37611,38823,66052,16070,31630,97142,34754,45263,43574,75819,11586,8377,55469,60822,42990,3121,17208,87130,76657,11331,10329,39243,49047,92234,20450,47257,6783,13608,33385,18497,96710,88915,27737,92357,65528,49701,8700,63148,85518,54204,52750,66585,63378,90728,54041,62584,56387,77629,34688,28587,55506,58607,37020,14849,35642,66760,27094,85536,81623,82269,79952,57518,75732,98821,11282,21708,50659,69105,82248,72216,35096,53729,40435,57711,34978,22487,88482,99431,33159,55036,27791,87780,58914,34225,59246,77153,20312,67532,16414,66885,31514,20352,77226,7184,97663,19982,86141,69841,43923,12305,344,95181,97916,631,48324,86704,20333,68493,62557,6229,40285,33575,60122,35958,44132,75580,18273,32511,7332,40778,61307,38826,15430,5272,58748,2765,84425,37602,71905,50871,19840,15291,20505,56193,75538,86994,96945,89850,93935,47791,34148,83936,56757,50698,72638,51296,63868,71512,64289,16861,54018,17201,47812,35626,72992,11308,87302,71411,27793,77758,1885,98490,42434,52292,20551,81253,36600,26879,71292,46751,98476,47684,20384,60749,93371,55041,31423,48033,15030,69255,78059,15085,97734,20409,99042,9226,39539,65063,15928,61686,52652,25451,38708,25678,76740,59837,2527,99947,49495,10804,14228,98739,1935,80138,78277,25252,65263,22754,94645,64649,2988,589,16807,87639,72962,4202,85100,2565,52295,51005,86616,6615,63571,73961,15299,61909,39942,33179,79553,42456,90042,52345,3623,9190,6751,87268,80056,17307,65339,70152,30028,90663,27947,94233,36203,20312,22578,60781,50402,64770,22741,58316,24762,40033,37858,65451,55924,37534,99646,35116,19876,6332,73889,48003,21625,81732,9129,30648,13203,20011,17916,80115,12108,78876,8516,39464,41078,28200,38436,90383,25873,97190,18158,57244,21031,26680,3992,50990,76496,34162,38304,5096,55735,26267,80526,74898,37240,32162,65571,69621,49903,83551,85054,9842,53489,1422,43963,77659,26865,54481,67693,55902,42595,49745,42001,7877,47271,60199,72747,46409,78426,96231,51161,22179,43747,86724,69346,75197,15754,78972,94887,25979,74197,22549,44259,6630,1264,57163,56186,27236,65341,80380,83245,68321,87749,99314,67032,87421,36086,63435,59477,71354,22197,46526,22336,84842,35152,51577,19450,21407,34209,31621,1970,51608,8186,39645,94610,80118,1181,63198,66244,21551,67514,98781,81496,33265,12263,19389,85896,40260,56074,39478,60499,8466,15322,35168,22061,49372,79995,68183,30470,35873,66253,92112,55239,28734,63869,74422,10327,58745,99513,1851,89919,96368,66541,51372,7425,14975,30662,44248,79217,7575,15461,62990,45577,76424,40555,41462,20780,90780,46334,81230,12844,64025,62602,7801,69675,69425,6809,99807,41450,64339,9553,61207,22305,62112,91728,70450,62487,6882,82249,82352,44667,80189,67688,21590,16017,46361,46685,12810,51417,8725,80639,39093,83757,72362,65684,78483,19829,72481,5195,71221,94648,39825,52115,36722,44492,78173,5636,83113,46509,66445,74201,13467,51882,27952,19941,2290,58966,58705,27279,6701,63675,69584,32431,32362,24999,64996,5941,38935,46882,69950,93729,84902,93202,79468,96861,92789,83527,14532,36853,50228,30072,1406,27019,95524,89877,9354,60251,60112,83493,60016,49557,78042,35210,60794,14965,56891,78142,46680,65392,12355,25844,53129,28127,83808,62718,430,72997,45391,70729,44967,93516,46554,34551,63939,11622,35100,43476,31195,88034,44604,56542,28924,9553,86181,55787,44092,75674,8530,51856,30789,63315,67796,10190,54537,81428,62298,2826,80967,54956,13040,74375,37974,22827,80755,6767,24337,62940,35470,60734,98327,550,82739,66199,96684,35874,14648,24666,71733,73243,66182,98242,16020,33092,6553,64541,41616,62969,90440,23874,18308,63988,61921,71731,96330,81274,24542,21093,45101,97560,63704,71507,2632,44914,85538,56867,61886,9895,59770,91508,31713,1648,79980,38503,54297,53649,49342,57272,96315,82835,21519,34409,70242,83629,2125,24545,29118,50842,91458,48664,65802,81514,55461,79547,76147,93929,93834,87581,81279,2369,73991,64939,80337,16143,93021,52943,34972,64429,46756,69125,91955,21872,57040,8277,82102,15986,33022,30067,28810,39087,20150,91277,26679,91667,77151,97365,71976,6151,25083,94425,25161,6358,19464,36560,60205,46734,80530,90413,71582,10605,4573,50793,75696,4142,94,47430,97037,12452,50719,93703,92317,98179,56545,13635,54749,55772,41390,84185,65341,90568,8566,40283,11947,4181,22598,3789,57401,7874,68754,64245,13834,81532,45248,44716,43018,7677,98844,8325,66173,90398,11773,20168,62552,35592,17722,90476,80442,16750,60557,78518,51390,80931,64500,23618,78859,67890,24760,71865,26988,3336,28811,20466,1458,86352,11701,73112,22574,48044,10514,11157,50814,12299,76122,8159,84949,11641,65478,76282,4254,32553,8659,40415,40736,70092,6880,57519,30762,23437,93521,95582,34410,87433,40356,36619,27172,8729,53434,94080,91305,34359,9910,57695,19429,69541,92358,54737,107,52505,20220,41853,59256,15869,12962,98921,53892,92398,8873,89592,40109,77779,24441,24548,38231,68151,84184,63291,18312,84807,27431,43834,29498,96210,26227,43342,68259,2223,59505,36815,87077,42863,35869,19938,29866,90696,85565,4391,42027,1767,86753,59549,49910,15950,74330,85610,71181,31813,34641,43897,69463,74265,122,50100,87541,2636,19103,10499,83777,26702,6900,50459,69182,71281,76128,3679,45613,91349,80008,26930,62451,96895,82128,26558,84845,11932,60692,49608,98772,65609,33281,19180,73051,68029,97250,33612,29706,66441,88862,46343,35617,68696,31902,49127,46744,98216,66352,75962,98126,687,23130,77455,60698,15602,85475,41110,34790,20172,72425,97478,87163,69627,22512,54505,57425,2628,3435,77765,76876,82620,46507,1768,61103,11723,22407,39724,30084,52989,41098,8072,91150,40577,12416,59169,57222,71173,48755,85597,9720,32363,83050,22514,42876,10692,15866,22845,73653,5473,81825,89923,34049,89427,97336,15802,50059,74560,92333,21117,88035,23304,99272,72767,55202,18104,834,4967,43767,22128,79102,24115,87726,52628,75631,87913,31640,50689,99790,21459,26636,39516,820,60504,57633,49557,5486,13090,56078,15882,1801,82471,29914,79397,30762,24029,78630,13711,52041,99675,14764,54417,80684,24699,45287,57680,14773,48437,69743,79483,81854,65548,47088,56030,18712,40025,12189,11977,36866,9402,54647,57579,5240,71557,21134,71560,33030,69395,51564,2841,95428,41459,42797,2729,97963,14516,54867,11646,28343,75107,77785,67319,94933,48069,79944,63892,31982,20253,56685,98010,37426,22251,81046,27472,34048,42116,28147,70139,83482,60935,13322,44296,18351,66464,1989,65029,18875,71808,6522,21280,95759,73978,99212,78241,97990,85089,94966,12160,54424,8410,75102,82116,86704,9406,64458,33661,77597,67972,55012,9965,62058,45513,34931,95936,64923,40570,59206,26266,94026,97975,96992,83193,27368,87254,83971,23368,25015,7945,14194,88599,95306,99719,96550,42961,22015,90071,90435,78000,35605,44032,48207,18629,80421,31648,17061,3805,15114,65016,24681,92855,86274,30927,46413,93204,41627,90116,82316,79212,4821,67308,96416,24863,46486,28084,91464,90111,51754,88853,93820,27547,58952,11521,11136,63803,60684,31816,78000,71484,77553,32201,21700,88097,13442,84322,85018,39427,73720,88886,64031,40618]
pipes_all = [[2,1,29701],[3,2,97954],[4,3,14600],[5,2,22110],[7,3,14412],[8,6,35742],[9,4,2560],[10,4,30822],[11,5,24812],[12,1,30583],[13,6,70159],[14,10,51992],[15,8,1765],[16,11,68435],[17,13,26992],[18,15,46846],[19,4,73049],[20,4,96887],[21,9,7896],[22,21,87635],[23,15,29266],[24,4,65804],[25,22,38626],[26,1,23842],[28,12,13943],[29,8,95626],[30,23,90303],[31,9,86959],[32,24,56993],[33,22,65478],[34,32,1745],[35,20,87906],[36,26,95020],[37,1,32630],[38,19,92967],[39,38,82313],[40,27,73805],[41,15,48963],[42,1,99891],[43,19,34162],[44,14,38087],[45,22,57011],[46,7,18719],[47,37,77119],[48,8,19496],[49,46,21348],[51,2,47063],[52,45,49034],[53,23,75239],[54,48,41897],[56,35,66432],[58,3,84395],[59,29,4102],[60,27,35845],[62,9,47710],[63,61,11283],[64,26,91789],[65,59,43149],[66,65,68682],[67,39,30519],[69,61,5680],[70,32,43347],[71,66,84590],[72,59,52431],[73,48,19756],[74,19,67496],[75,57,87276],[76,68,40757],[77,28,74982],[78,71,91211],[79,62,29080],[80,46,88674],[81,74,11198],[82,31,25532],[83,79,43847],[84,7,50451],[85,64,65665],[86,56,39159],[87,82,52212],[88,64,83462],[89,29,97270],[91,83,33747],[92,44,89431],[93,68,84122],[95,28,79585],[96,36,28242],[97,45,62811],[98,16,48256],[99,51,74903],[100,80,55040],[101,83,38146],[102,70,77445],[103,43,93110],[105,31,9519],[106,34,34595],[108,101,87776],[110,36,37237],[111,28,90199],[112,62,74313],[113,82,33433],[114,32,61179],[115,80,84718],[117,86,13810],[118,54,28086],[119,59,85376],[120,115,86910],[121,53,51790],[122,28,29483],[123,101,91981],[125,124,9644],[126,7,73538],[128,1,23474],[129,25,60080],[130,81,75982],[131,56,81311],[132,115,74974],[133,109,69225],[134,128,25868],[135,111,19432],[136,118,92403],[137,132,86544],[138,38,22154],[139,88,31854],[140,98,10505],[142,64,60703],[143,66,80450],[144,17,71173],[145,83,91572],[146,22,58132],[147,87,77145],[148,82,48422],[149,43,95248],[150,77,61978],[151,80,76738],[152,56,73090],[153,46,25304],[154,47,79013],[156,66,77969],[157,147,19431],[158,97,38563],[160,60,12524],[161,64,31287],[162,80,76222],[163,143,32137],[164,66,43935],[165,97,24477],[166,159,94280],[167,9,76610],[168,167,89761],[169,27,67260],[170,34,25753],[171,72,92307],[172,170,67935],[173,9,7909],[174,46,67848],[175,93,78956],[176,72,75403],[177,89,21917],[178,100,82491],[179,40,67362],[180,62,5625],[181,84,69468],[182,100,9665],[185,109,95626],[186,174,62761],[188,70,44662],[190,170,40719],[191,2,35724],[193,81,35502],[194,106,40461],[195,76,55697],[196,97,48251],[197,186,31468],[198,38,28268],[199,131,92398],[200,98,93630],[201,74,32983],[202,129,24667],[203,160,98338],[204,157,83564],[205,76,34448],[206,144,18206],[207,54,21036],[208,154,5146],[209,17,88955],[211,110,81634],[212,169,49900],[213,137,49365],[214,2,2743],[216,102,70265],[217,6,7473],[218,158,93597],[219,218,82025],[220,57,93583],[221,115,17511],[222,149,43398],[223,20,39531],[224,128,98386],[225,158,2478],[226,129,54275],[227,84,57178],[228,76,67722],[229,181,7814],[230,144,31563],[231,176,80503],[232,121,7717],[233,174,77445],[235,197,58732],[236,208,40261],[237,102,36161],[239,113,98426],[240,43,5950],[241,234,44069],[242,215,20773],[244,123,93483],[245,21,24114],[246,16,56866],[247,102,83242],[248,207,42104],[249,228,33878],[250,33,70301],[252,43,12339],[253,56,66095],[254,181,63542],[255,62,80572],[256,254,49480],[257,202,28620],[258,175,21567],[259,202,15336],[260,145,288],[261,198,40005],[262,189,68227],[263,232,50808],[264,72,26817],[265,98,22347],[267,112,30440],[268,18,40357],[269,116,96319],[270,167,82039],[271,207,37300],[272,7,84935],[273,137,26433],[274,195,20485],[275,155,3191],[276,213,1272],[277,207,59565],[280,144,99450],[281,59,91363],[282,201,46568],[283,263,40687],[284,274,43775],[285,159,71086],[286,85,17273],[287,87,16543],[289,10,22913],[290,6,75939],[291,36,94387],[292,30,65390],[293,6,90943],[294,201,39029],[295,49,38436],[296,223,55232],[297,24,21950],[298,273,29748],[299,11,79111],[300,239,1199],[301,157,58695],[302,112,95981],[303,14,27236],[304,40,33067],[305,132,9606],[306,167,91230],[307,176,73139],[308,103,41108],[309,260,57700],[310,34,29689],[311,99,85869],[312,247,27571],[313,91,41434],[314,114,39683],[315,24,81282],[316,205,46011],[317,24,18774],[318,289,49499],[319,35,35476],[320,283,37597],[322,257,74891],[325,322,47060],[326,123,7734],[327,273,48920],[328,239,97516],[329,249,15805],[330,260,52246],[331,138,43065],[332,234,95662],[333,301,89573],[334,148,58966],[335,175,22325],[336,288,90484],[337,211,52119],[338,49,62983],[340,59,75987],[341,180,78837],[342,13,24516],[343,60,20307],[344,263,58061],[345,20,9200],[346,205,71813],[347,258,77860],[348,92,46561],[349,155,95175],[350,14,12250],[351,280,92298],[352,117,33641],[353,329,68156],[354,263,36526],[355,270,15815],[356,23,42824],[357,131,24866],[358,237,64697],[359,350,55239],[360,341,9797],[361,136,89544],[362,73,43099],[363,207,24181],[364,218,39615],[365,273,73084],[366,6,864],[367,84,95143],[368,161,43237],[369,200,60149],[370,207,10091],[371,248,66612],[372,213,87257],[373,318,91407],[374,207,19219],[375,155,27372],[376,223,74371],[378,18,10894],[379,118,87212],[380,110,7549],[381,18,27123],[382,71,61836],[384,211,31408],[385,88,10431],[386,329,86168],[387,88,62655],[388,199,73434],[389,321,31213],[390,285,46509],[392,157,90573],[393,186,86501],[394,251,52085],[396,213,65834],[397,14,10629],[398,196,58288],[399,189,401],[400,29,42458],[401,24,39544],[402,247,33084],[403,293,90173],[404,391,70184],[405,359,19688],[406,230,49814],[407,233,58833],[408,294,40730],[409,302,26525],[410,10,91271],[412,16,92204],[413,89,19524],[414,289,39725],[415,66,20872],[416,352,10054],[417,277,31609],[418,310,93549],[419,138,79149],[420,48,84584],[421,257,49358],[422,391,46325],[423,359,68552],[424,232,17270],[425,384,91335],[426,144,83940],[427,90,98409],[428,396,16607],[429,370,78462],[430,33,8117],[431,152,9942],[432,424,69632],[433,30,85543],[434,11,5518],[436,25,81096],[437,292,82378],[438,51,77858],[439,123,85794],[440,248,42626],[441,387,63758],[442,214,10604],[443,215,41259],[444,116,54035],[445,114,32353],[446,14,49598],[447,315,63931],[448,291,54049],[449,111,30348],[450,158,17723],[451,82,14597],[452,83,69424],[453,327,86389],[454,151,81918],[456,189,96242],[457,70,19960],[458,16,28629],[459,237,93187],[460,159,73685],[461,58,68123],[462,235,56918],[463,398,76694],[464,150,63931],[465,74,442],[466,56,52931],[468,425,4396],[469,194,11191],[470,149,43474],[471,363,54711],[472,172,6269],[473,83,33667],[474,318,6513],[475,405,22053],[476,122,40227],[477,402,19227],[478,95,81886],[479,434,67585],[480,325,57839],[482,471,37584],[483,4,30096],[484,111,47372],[485,203,38591],[487,366,6141],[488,377,54203],[489,59,94963],[490,418,23678],[491,307,17408],[492,14,670],[493,437,77484],[494,280,82940],[495,464,13894],[496,450,64933],[497,305,93614],[498,25,53580],[499,132,39323],[500,371,98644],[501,135,41593],[502,231,47066],[503,87,79360],[504,159,9643],[505,275,38103],[507,248,54503],[508,84,49864],[509,454,71371],[510,380,50407],[512,205,67971],[513,467,97712],[514,374,33787],[515,15,28496],[516,60,76410],[517,348,99842],[518,260,89522],[520,113,47359],[521,198,10440],[522,296,69916],[523,63,83891],[524,60,89749],[525,391,54598],[526,175,5846],[527,153,82127],[528,383,38116],[529,117,16063],[530,399,79407],[531,161,22127],[532,230,76530],[533,85,80682],[534,273,89040],[535,495,80592],[536,109,39842],[537,57,15843],[538,250,96631],[539,493,83378],[540,263,69270],[541,35,61883],[542,159,57255],[543,95,27086],[544,215,6833],[545,73,35151],[546,243,9761],[547,452,24856],[548,84,31190],[549,12,6332],[550,405,37086],[551,301,66342],[552,401,63657],[553,276,66852],[554,22,3203],[555,228,37338],[556,176,22010],[557,381,66493],[558,75,66208],[559,364,23276],[560,97,24924],[561,233,4120],[562,107,67561],[563,509,1127],[564,311,86892],[565,147,13695],[566,440,92139],[567,200,90507],[568,252,74287],[570,161,92535],[571,131,27284],[572,155,30566],[573,462,48670],[574,100,90645],[576,276,85273],[578,174,49536],[579,111,21260],[580,398,2781],[581,525,26866],[582,544,32893],[583,444,84344],[584,387,269],[585,468,60842],[586,6,37878],[587,575,473],[588,106,22681],[589,185,27433],[590,84,85785],[594,227,89197],[595,194,69847],[596,33,39904],[597,105,19893],[598,45,56127],[599,435,44844],[600,415,50742],[601,556,36073],[602,558,1862],[603,448,48189],[604,373,89934],[605,530,56263],[606,106,66369],[607,462,29590],[609,343,81848],[610,495,3754],[611,403,68366],[612,356,4017],[613,400,25750],[614,465,9509],[615,435,72301],[617,541,79715],[618,424,63813],[619,81,47434],[620,152,47336],[621,251,98166],[622,299,69146],[623,201,40688],[624,542,15112],[625,504,14571],[626,276,28761],[627,84,40084],[628,538,82437],[630,104,83542],[631,62,42658],[632,195,89255],[633,197,80995],[634,96,34338],[635,511,49439],[636,200,40008],[637,321,82504],[638,9,63149],[640,191,31739],[641,205,86562],[642,315,89392],[643,199,63529],[644,275,27480],[645,616,68136],[647,227,36936],[648,268,35143],[649,36,63126],[650,370,84467],[651,149,31083],[653,26,53705],[654,12,56687],[656,163,51549],[657,167,88308],[658,78,60839],[659,324,13943],[661,584,22839],[662,317,89481],[663,163,94885],[664,288,85440],[666,607,6691],[667,598,31223],[669,217,86682],[670,414,23396],[671,418,69334],[672,670,91478],[673,53,4716],[674,559,16589],[675,207,79062],[676,111,8928],[677,543,143],[678,229,16663],[679,49,81576],[680,661,70081],[681,667,24489],[682,553,2672],[683,140,69484],[684,102,59446],[685,363,25059],[686,6,62764],[687,283,1382],[688,397,52696],[689,230,58658],[690,176,90750],[691,608,1352],[692,159,36270],[693,450,84846],[694,344,81318],[695,116,90234],[696,404,9678],[697,519,29049],[698,571,94453],[699,644,96083],[700,86,46202],[701,613,99800],[703,277,38585],[704,73,86680],[705,197,43707],[706,25,80290],[707,117,46972],[708,600,85249],[709,90,76060],[710,216,49175],[711,254,96266],[712,266,90950],[713,225,95130],[714,295,35117],[715,16,94893],[716,331,13503],[717,659,11049],[719,590,90696],[720,491,72175],[721,316,86679],[722,12,39992],[724,177,11080],[725,568,26948],[726,322,27388],[727,76,27145],[728,266,75008],[729,527,12535],[730,214,12371],[731,373,39834],[732,454,75407],[734,42,70914],[736,101,54875],[737,4,20308],[738,565,85175],[739,123,60506],[740,631,27167],[741,468,61452],[743,442,35717],[744,693,29093],[746,662,31228],[747,617,19151],[748,246,97610],[749,674,21279],[750,387,46233],[751,546,39260],[752,310,69736],[753,566,9630],[754,257,31508],[755,668,84160],[756,35,44004],[757,658,83541],[759,634,3962],[760,347,75632],[761,49,79612],[762,192,24690],[763,148,4250],[764,244,97829],[765,421,1066],[766,580,7403],[767,216,37692],[769,711,22687],[770,404,88094],[771,671,56506],[772,755,58255],[773,145,52921],[774,609,83960],[775,680,96586],[776,703,99713],[777,599,52375],[779,144,44736],[780,632,89757],[781,36,5731],[782,351,30366],[783,771,57131],[784,310,96517],[785,535,99809],[786,619,68589],[787,396,71272],[788,431,57985],[789,129,27813],[790,369,70912],[791,281,92982],[792,98,86659],[793,546,48369],[795,97,67407],[796,477,37718],[797,66,87249],[799,783,12823],[801,437,3536],[802,11,26811],[804,564,91098],[805,482,27517],[806,168,52001],[807,265,21530],[808,53,76266],[809,361,80924],[810,182,37370],[811,154,37112],[812,556,41290],[814,789,74920],[815,800,56751],[816,729,91805],[817,107,94232],[818,154,80731],[819,268,14737],[820,375,4887],[821,350,44481],[822,72,89776],[823,20,7873],[824,132,21003],[825,528,54949],[826,315,47200],[827,111,74453],[828,512,94245],[829,577,59313],[830,603,14129],[831,690,26452],[832,413,54620],[833,52,58054],[834,711,49276],[835,469,56280],[836,296,9619],[837,371,27141],[838,707,78638],[839,762,2177],[840,136,98498],[841,175,16939],[842,529,10712],[843,320,905],[844,585,66020],[845,450,80295],[846,381,90691],[847,552,28690],[848,619,83992],[849,824,55730],[850,363,43803],[852,52,7169],[853,444,56336],[854,476,56690],[855,622,85799],[857,774,96678],[858,424,49449],[859,402,3931],[861,59,68039],[862,283,69225],[863,668,97126],[865,837,35760],[866,562,10999],[867,185,55837],[868,404,30814],[869,287,24810],[870,208,77452],[871,478,32911],[872,249,33444],[873,245,42964],[874,426,98778],[875,697,70505],[876,128,31069],[877,23,62263],[878,589,55131],[879,355,41219],[880,458,2034],[881,714,40486],[882,854,62129],[883,558,53708],[884,847,7965],[885,816,91508],[886,95,47778],[887,267,10275],[889,206,26758],[890,593,30540],[891,346,26922],[893,568,94898],[894,695,17579],[895,274,37331],[896,454,60062],[897,258,78827],[898,203,1875],[899,665,47366],[900,442,93689],[901,200,13272],[902,792,78544],[903,266,40584],[904,792,47387],[905,659,3977],[906,298,4860],[907,543,6421],[909,40,98118],[910,286,76139],[911,471,29719],[912,873,2364],[913,844,78385],[914,473,29835],[915,89,15642],[916,589,32405],[917,3,41852],[918,517,63462],[920,377,28193],[921,6,27665],[922,32,93519],[923,422,14364],[925,229,37296],[926,804,10557],[927,83,65851],[928,108,99995],[929,680,12483],[930,191,37164],[931,279,80905],[932,409,14026],[933,61,48110],[934,540,10928],[937,446,29444],[938,360,7819],[939,471,40358],[940,357,82004],[941,819,38581],[942,392,47984],[943,652,30315],[944,605,38241],[945,397,29448],[946,915,19271],[947,654,31385],[948,715,63095],[949,44,98741],[950,108,54784],[951,179,54911],[953,849,10447],[954,339,60110],[955,475,72751],[956,511,27716],[957,510,45488],[958,501,75688],[959,167,10869],[960,786,81730],[961,925,29497],[962,882,37332],[963,756,42589],[964,695,69758],[965,376,53733],[967,277,40479],[968,637,55910],[969,676,52573],[970,500,35584],[971,866,76667],[972,335,78272],[973,864,7167],[975,656,95465],[976,763,7587],[977,946,9897],[978,572,15324],[979,791,68963],[980,716,19826],[981,756,41327],[982,462,70389],[983,694,19291],[984,759,26277],[985,269,96491],[986,171,72450],[987,335,33769],[988,28,39546],[989,598,66509],[990,815,58010],[994,246,98920],[995,952,92564],[996,47,19089],[997,815,52660],[998,385,35303],[999,136,34329],[1000,648,73110],[1001,243,253],[1002,729,4232],[1003,918,54115],[1004,636,45857],[1005,254,49621],[1006,915,61092],[1007,664,36241],[1008,114,349],[1009,712,73965],[1010,387,63796],[1011,236,65490],[1012,299,40970],[1014,1007,61676],[1015,597,52376],[1017,549,34623],[1019,1016,25540],[1020,136,52380],[1021,791,24927],[1022,950,98601],[1023,226,1562],[1024,614,9795],[1025,335,30621],[1026,593,56947],[1027,241,25080],[1028,809,15470],[1029,650,35961],[1030,893,69249],[1031,763,46677],[1032,372,78156],[1033,302,99773],[1034,883,85578],[1035,174,10140],[1036,243,41563],[1039,672,15448],[1041,477,40427],[1042,440,94716],[1044,583,53646],[1045,859,43109],[1046,901,19523],[1047,60,71221],[1048,187,43065],[1049,564,10769],[1050,956,54814],[1051,65,81441],[1052,206,18660],[1053,862,27312],[1054,515,6942],[1055,99,37047],[1056,616,41243],[1057,284,33037],[1058,131,17012],[1060,132,60120],[1061,424,65010],[1062,710,6555],[1063,45,30469],[1064,1039,6630],[1065,852,76908],[1067,7,24572],[1068,558,80311],[1069,598,71619],[1070,387,76514],[1072,1,32379],[1073,590,16104],[1074,107,75199],[1075,712,19319],[1076,907,30853],[1078,913,31911],[1079,218,59986],[1080,1051,55941],[1081,54,61929],[1082,367,91902],[1083,924,43746],[1084,906,32858],[1085,183,34173],[1086,375,76226],[1087,145,49924],[1088,714,79879],[1089,170,54271],[1090,726,74548],[1091,798,77665],[1092,60,20506],[1093,330,92122],[1094,1017,5796],[1095,128,27394],[1098,878,52275],[1100,1048,47647],[1101,196,64291],[1102,29,57258],[1103,771,92395],[1104,292,70900],[1105,168,54614],[1106,340,13278],[1107,208,8970],[1108,727,15460],[1109,186,88394],[1110,872,17516],[1112,368,77577],[1113,633,97694],[1114,794,38711],[1115,524,69183],[1116,963,84105],[1117,564,47890],[1118,635,33266],[1119,164,77356],[1120,228,13586],[1121,32,8476],[1122,23,97217],[1123,431,34665],[1124,170,93412],[1125,703,87334],[1126,694,81732],[1127,1068,44270],[1129,538,29571],[1130,227,81957],[1131,198,52023],[1132,1072,16762],[1134,354,99400],[1135,699,46641],[1136,860,68542],[1137,903,34620],[1138,953,21122],[1139,339,99965],[1140,103,63619],[1141,1044,12550],[1142,470,65787],[1143,9,31441],[1144,897,43870],[1145,14,986],[1146,31,54773],[1147,958,70162],[1148,32,57184],[1149,1067,90143],[1150,11,91283],[1151,1131,13805],[1153,1005,7617],[1154,565,87674],[1155,298,23108],[1156,108,98725],[1157,45,50697],[1158,345,48048],[1159,1089,60324],[1160,912,93321],[1161,374,64541],[1162,959,38489],[1163,1007,41410],[1164,623,5394],[1166,989,12787],[1167,723,63135],[1168,201,23866],[1169,883,18205],[1170,198,90217],[1171,197,82553],[1172,1157,7973],[1173,620,76527],[1174,636,56597],[1175,971,93165],[1176,1058,47951],[1177,925,83025],[1178,1150,41159],[1179,41,1078],[1180,799,18057],[1181,600,23302],[1182,247,30510],[1183,474,57167],[1184,676,7280],[1185,254,95340],[1186,167,42062],[1187,422,44527],[1188,780,53033],[1189,353,53881],[1190,155,96905],[1191,132,20734],[1192,192,35140],[1194,184,61905],[1195,973,58618],[1196,32,83850],[1197,30,50460],[1198,345,64698],[1199,18,94838],[1200,684,68193],[1201,390,8651],[1202,258,16873],[1203,1139,70156],[1204,349,36248],[1205,290,66944],[1206,151,75048],[1207,1165,76526],[1208,747,39739],[1209,28,22937],[1210,1005,27970],[1212,1142,75522],[1213,297,43061],[1214,240,14018],[1215,602,61861],[1216,918,96330],[1217,588,44728],[1218,334,23323],[1219,754,5042],[1220,112,54984],[1221,903,90527],[1224,718,9033],[1225,536,51629],[1226,570,34668],[1227,956,39257],[1228,884,52639],[1229,782,89207],[1230,35,5296],[1231,438,80879],[1233,440,68052],[1234,883,63213],[1235,179,96141],[1236,250,73533],[1237,358,83354],[1238,370,68408],[1239,878,58233],[1240,911,75054],[1241,812,64691],[1242,914,50680],[1243,770,50604],[1244,382,93458],[1245,575,81450],[1246,266,68534],[1247,527,48772],[1248,887,73895],[1249,617,24064],[1251,1245,66677],[1252,352,14790],[1253,647,85873],[1254,572,48089],[1256,186,28579],[1257,23,28457],[1259,612,11103],[1260,962,32647],[1262,195,36388],[1263,829,6800],[1264,891,63034],[1265,870,12964],[1266,1047,76341],[1267,587,90963],[1268,1175,58228],[1269,185,32283],[1270,687,10575],[1271,495,2904],[1272,492,50018],[1274,365,82885],[1275,153,99206],[1276,1103,63921],[1277,442,66754],[1278,1130,66462],[1279,150,96234],[1280,1150,26666],[1281,265,57581],[1282,1027,19629],[1283,260,30112],[1284,1186,10459],[1285,508,44090],[1286,831,16935],[1287,490,60197],[1288,187,34489],[1289,1211,49901],[1290,562,21963],[1292,842,71198],[1293,151,54976],[1294,369,55682],[1295,1031,59409],[1296,1285,89738],[1297,175,84503],[1298,741,73013],[1299,709,71213],[1300,163,69017],[1301,377,97307],[1302,644,47722],[1303,48,28534],[1304,802,93100],[1305,183,2428],[1306,143,81543],[1307,744,79377],[1308,1214,69952],[1309,526,98676],[1310,928,23610],[1311,485,32011],[1312,108,16640],[1313,345,59724],[1314,1180,84764],[1315,429,91811],[1316,687,93029],[1317,949,45024],[1318,1021,27980],[1319,555,66426],[1320,1032,62081],[1321,1229,10487],[1322,1159,79489],[1323,748,39178],[1324,461,74206],[1325,726,63863],[1326,909,35424],[1328,635,7373],[1330,583,42792],[1331,856,56747],[1332,1150,66011],[1333,36,82499],[1334,346,69398],[1336,858,65762],[1337,561,60177],[1338,59,45990],[1339,282,48320],[1340,599,97713],[1341,618,88764],[1342,1107,58341],[1343,262,29797],[1345,92,56620],[1346,1312,82414],[1347,70,10017],[1349,1290,64396],[1351,1276,23275],[1352,48,33954],[1353,476,34075],[1354,1125,78417],[1355,1173,62803],[1356,1088,43520],[1357,146,52854],[1358,107,34794],[1359,918,51614],[1360,1353,89488],[1361,908,38986],[1362,38,66757],[1363,268,68867],[1364,1148,44073],[1365,532,72633],[1366,1136,89196],[1367,476,85687],[1368,602,46477],[1369,559,29730],[1370,916,76393],[1371,19,90839],[1372,309,41194],[1373,1121,24041],[1374,681,43968],[1375,613,97413],[1376,1362,65842],[1377,1022,43786],[1378,1174,18594],[1379,58,74456],[1380,1226,41188],[1381,501,84327],[1382,1097,4589],[1383,833,36410],[1384,598,35702],[1385,767,34017],[1386,444,89645],[1388,542,80738],[1389,90,2826],[1390,213,85347],[1391,734,29166],[1393,1072,96409],[1394,553,8271],[1395,179,2230],[1396,1261,73418],[1399,246,8017],[1400,1092,65801],[1401,678,23508],[1402,629,82835],[1404,200,32942],[1405,40,8439],[1406,58,33343],[1407,1090,17269],[1408,740,976],[1409,948,83076],[1410,1327,81532],[1412,75,25421],[1414,1155,43856],[1415,899,40062],[1416,367,96287],[1417,683,2662],[1418,313,84453],[1419,389,97720],[1420,752,29772],[1421,832,74634],[1422,1160,33370],[1425,903,2804],[1426,550,75637],[1427,1040,87343],[1428,1366,14820],[1429,415,38785],[1430,1278,13487],[1431,567,48949],[1432,807,47795],[1433,1353,93754],[1434,870,28897],[1435,1155,12717],[1436,1054,93012],[1437,1258,70247],[1438,451,68809],[1439,402,16171],[1440,99,88933],[1441,1178,87030],[1442,559,4278],[1443,352,25616],[1444,42,61937],[1445,1367,56804],[1446,598,80587],[1448,52,38880],[1449,419,57168],[1450,1262,70998],[1451,800,54836],[1453,1206,61736],[1454,597,84984],[1455,1010,52546],[1456,1290,21483],[1457,1088,5940],[1458,169,99165],[1459,434,29613],[1460,957,39580],[1461,1154,27998],[1462,914,61685],[1463,638,74288],[1464,462,20328],[1465,146,2969],[1466,55,24516],[1467,1355,23636],[1469,260,5032],[1472,338,79476],[1474,794,31960],[1475,333,79966],[1476,941,34278],[1478,704,43050],[1479,114,61231],[1480,801,98296],[1481,718,2047],[1482,926,32449],[1483,443,67181],[1484,1413,91672],[1485,1426,32972],[1486,278,15385],[1487,1401,6861],[1488,677,90408],[1489,1096,21566],[1490,1452,13504],[1491,331,29763],[1492,6,76860],[1493,871,30081],[1494,172,19984],[1495,1094,57495],[1496,1184,63029],[1497,1354,12928],[1498,835,10344],[1499,895,81466],[1500,1349,70236],[1501,305,20088],[1502,412,53171],[1503,659,78764],[1504,655,25927],[1505,147,38445],[1506,1015,69892],[1507,1128,65963],[1508,1355,70080],[1509,173,88174],[1510,758,51179],[1511,1008,27401],[1513,878,94695],[1514,406,31807],[1515,666,43755],[1516,939,81974],[1517,1016,8833],[1518,470,30755],[1519,50,41369],[1520,275,37836],[1521,308,42015],[1522,1412,15351],[1523,285,57167],[1524,302,49610],[1525,1,69359],[1526,1374,46559],[1527,1251,61083],[1528,927,76517],[1529,931,1797],[1530,863,2776],[1531,917,16816],[1532,335,78842],[1533,34,63889],[1534,1219,39489],[1535,722,17797],[1536,1491,68888],[1537,682,52521],[1539,445,44975],[1541,278,57900],[1542,599,18322],[1543,651,9503],[1544,258,62891],[1545,928,14523],[1546,94,71006],[1547,731,77685],[1548,236,28309],[1550,1539,17487],[1551,868,17325],[1552,182,60064],[1553,953,76168],[1554,482,26176],[1555,1554,41161],[1556,656,22451],[1557,194,83440],[1558,48,79620],[1559,478,82745],[1560,744,75405],[1561,1542,32021],[1562,193,82402],[1563,1415,10703],[1564,607,66823],[1565,865,15584],[1566,500,22983],[1567,637,41866],[1568,1133,14499],[1569,1374,48085],[1570,1099,29295],[1571,1061,58459],[1572,1427,50462],[1575,148,90079],[1576,521,5616],[1577,12,68623],[1578,984,19385],[1579,875,28175],[1580,1231,46757],[1582,701,82062],[1583,1072,55296],[1584,76,43894],[1585,1254,11593],[1586,978,36258],[1587,1549,33871],[1588,1145,30375],[1590,1458,81330],[1591,405,91717],[1593,1196,67665],[1594,1516,73678],[1595,1376,52548],[1596,511,45447],[1597,648,93984],[1598,36,63235],[1599,1141,70106],[1600,1538,93658],[1601,176,14318],[1602,683,96433],[1603,6,3566],[1604,1420,99651],[1605,356,1541],[1607,200,76994],[1608,414,42643],[1609,209,31702],[1610,470,12815],[1611,123,25495],[1612,1477,25216],[1613,1176,57217],[1614,314,42236],[1615,925,75626],[1616,356,36801],[1617,823,26946],[1619,1341,29413],[1620,595,56242],[1621,328,79397],[1622,1383,53379],[1623,233,23336],[1624,1541,33776],[1625,167,6950],[1626,638,696],[1627,164,48174],[1628,875,59196],[1629,463,65835],[1631,1003,61351],[1632,392,89541],[1633,522,68950],[1634,180,32748],[1635,1012,81843],[1637,719,4490],[1639,1160,72524],[1640,409,70036],[1642,627,52589],[1643,740,90447],[1644,1208,47774],[1645,1320,54053],[1646,1106,69528],[1647,1150,40525],[1648,1597,18],[1649,380,46144],[1650,90,68408],[1651,1529,47294],[1652,1079,62853],[1653,1150,54067],[1654,1164,67504],[1655,351,20105],[1656,1452,78587],[1657,1396,90214],[1658,278,59150],[1659,1414,80567],[1660,1021,32666],[1661,696,96834],[1662,413,38056],[1664,644,1247],[1665,432,9235],[1666,169,43365],[1668,776,98200],[1669,1206,22963],[1670,1130,56242],[1671,1171,6574],[1674,1363,10204],[1675,1195,84699],[1676,234,93506],[1677,235,70722],[1679,1394,45292],[1680,1529,51633],[1681,1135,47195],[1682,542,35535],[1683,324,33859],[1684,233,52359],[1685,1378,39501],[1686,159,61649],[1687,422,17006],[1688,1635,32544],[1689,623,56884],[1691,803,83],[1692,758,71204],[1693,878,76748],[1694,1604,41796],[1695,1471,73553],[1696,1344,29484],[1697,706,49328],[1698,1061,77002],[1700,701,38657],[1701,961,11891],[1702,1403,76762],[1703,1473,18829],[1704,858,34472],[1705,1107,83820],[1707,103,21744],[1708,1373,87699],[1709,1254,53381],[1710,224,78040],[1711,1026,91283],[1712,1510,81904],[1713,1218,71910],[1714,881,74307],[1715,450,17008],[1716,524,20758],[1717,1077,18922],[1718,426,6142],[1719,1673,24708],[1720,1672,22673],[1721,1061,51947],[1722,1387,85400],[1723,739,63190],[1724,226,41854],[1726,210,64512],[1727,347,24203],[1728,1040,94220],[1729,689,9995],[1730,1560,19567],[1731,1320,96123],[1732,703,82424],[1733,755,78220],[1734,1648,26714],[1735,767,36096],[1737,1009,25871],[1738,1572,5145],[1739,738,81426],[1740,70,90421],[1741,961,16556],[1742,1421,3976],[1743,138,81131],[1744,969,31427],[1745,1547,89625],[1746,1666,38553],[1747,757,37813],[1748,301,12035],[1750,317,79609],[1751,978,3946],[1752,378,88630],[1753,1307,5052],[1754,42,65741],[1755,1448,71008],[1756,1674,20456],[1757,1362,7224],[1758,782,85199],[1759,942,78199],[1761,180,29946],[1762,1553,8212],[1763,868,24273],[1764,162,41003],[1765,1,90961],[1766,1405,55774],[1767,762,95592],[1768,1726,10598],[1769,570,80313],[1771,324,36741],[1772,362,25140],[1773,445,89332],[1774,1533,48982],[1775,1385,39129],[1776,1279,86112],[1777,1032,27089],[1779,1454,34766],[1780,724,87661],[1781,19,61973],[1782,444,99469],[1783,826,84557],[1784,1298,16913],[1785,303,59548],[1786,1279,32759],[1787,591,47664],[1788,1089,38234],[1789,1011,39254],[1791,769,43711],[1792,1778,10440],[1793,1156,28940],[1794,974,75378],[1795,1210,33981],[1796,72,52449],[1798,1718,20993],[1799,663,17307],[1800,1565,50282],[1801,1625,21487],[1802,211,84244],[1804,876,57088],[1805,1474,64278],[1806,872,52006],[1807,1485,83288],[1808,1356,56168],[1809,787,17644],[1810,1674,4892],[1811,1424,98678],[1812,1181,6610],[1813,1094,42801],[1814,303,57027],[1815,945,96016],[1816,1540,90279],[1817,87,26996],[1818,566,89527],[1820,1268,61447],[1821,388,75056],[1822,1454,13665],[1823,96,88854],[1824,93,26858],[1826,1674,7684],[1827,641,26221],[1828,179,97599],[1829,535,23074],[1830,521,2301],[1832,938,12625],[1834,103,84195],[1835,1775,16484],[1836,1089,31998],[1837,488,43211],[1838,996,18504],[1839,1261,92238],[1840,1580,99762],[1841,216,39921],[1842,1614,87827],[1843,312,27048],[1844,458,94118],[1845,238,32505],[1846,417,67817],[1847,196,87086],[1848,481,87862],[1849,957,72420],[1850,924,21972],[1851,945,44397],[1852,26,15100],[1854,112,80135],[1856,877,98938],[1857,1826,12135],[1858,762,78788],[1859,1071,14583],[1861,1576,58950],[1862,148,81901],[1863,920,34735],[1864,500,14383],[1865,486,53463],[1866,1813,4709],[1867,1580,39726],[1868,534,7736],[1869,1416,54710],[1870,1595,62001],[1871,171,52530],[1872,988,29773],[1873,1099,46458],[1874,1744,65858],[1875,569,55721],[1876,570,95922],[1877,990,98942],[1878,1846,22826],[1879,983,74666],[1880,1413,17988],[1881,1046,2412],[1882,1508,68224],[1883,971,36791],[1884,1249,83889],[1885,873,79657],[1886,759,93207],[1888,63,4417],[1889,75,90303],[1892,672,40377],[1893,1043,80449],[1894,699,84171],[1896,730,58237],[1898,780,17909],[1900,1691,47016],[1901,364,29188],[1902,1134,1388],[1903,1379,25838],[1904,1746,44877],[1905,794,7078],[1906,594,56007],[1907,1723,60140],[1908,1650,47677],[1909,137,79481],[1912,289,27911],[1913,1312,3582],[1914,1342,49154],[1915,1139,25802],[1916,1271,87652],[1918,1361,20539],[1920,1247,73281],[1921,562,27815],[1922,1536,36274],[1923,1510,3428],[1924,1356,61660],[1925,1638,26168],[1926,1284,32181],[1927,617,29200],[1928,163,73573],[1929,1563,4238],[1930,1459,86665],[1931,549,30738],[1932,1107,70415],[1933,1211,48121],[1934,1178,3994],[1936,1716,55675],[1937,1140,80126],[1938,627,71618],[1939,1458,93084],[1940,1344,82402],[1941,481,1261],[1942,620,45588],[1943,936,85286],[1944,1916,55626],[1945,1743,31588],[1946,866,88852],[1947,1935,95125],[1948,897,32841],[1949,1340,50949],[1950,934,95611],[1951,1669,31789],[1952,288,71059],[1953,330,17302],[1954,849,12746],[1955,1422,37508],[1956,1017,4480],[1957,823,58719],[1958,1916,65512],[1959,1337,28926],[1960,1697,45187],[1961,645,79442],[1962,1409,78056],[1963,1590,96112],[1964,161,97799],[1965,1774,76020],[1966,737,75005],[1967,1516,68113],[1968,731,1316],[1969,1951,81828],[1970,32,52498],[1971,829,6600],[1972,587,3639],[1973,1211,34628],[1974,454,33525],[1975,1944,31623],[1976,1446,43117],[1977,1654,5354],[1978,1139,28086],[1979,653,7771],[1980,1309,71940],[1982,282,88131],[1983,1479,35519],[1984,1076,64864],[1985,1212,25739],[1986,1913,13813],[1987,1960,42627],[1989,1165,7472],[1990,863,33477],[1991,1956,40231],[1992,889,20056],[1993,18,29013],[1994,1562,87466],[1995,583,55010],[1997,856,77515],[1998,382,87168],[1999,1578,13624],[2001,1108,48441],[2002,1132,79724],[2003,1284,67808],[2004,641,8894],[2005,1284,52411],[2006,1278,43981],[2008,1752,29244],[2009,65,81523],[2010,1602,68647],[2011,1209,65029],[2012,1452,71790],[2013,1628,37263],[2014,633,9544],[2015,1076,38297],[2016,813,49450],[2017,239,63983],[2018,1671,34481],[2019,443,11346],[2021,1282,18896],[2022,836,55087],[2023,1627,78152],[2024,1580,11747],[2025,1797,87033],[2026,882,79129],[2027,201,32580],[2028,560,20877],[2029,1103,71548],[2030,1863,77127],[2031,1830,46879],[2032,751,62894],[2033,777,51242],[2034,1858,73690],[2035,1697,5783],[2036,1812,13180],[2037,846,21033],[2039,575,39011],[2041,1899,84186],[2042,1706,75012],[2043,165,3442],[2045,1961,10443],[2046,596,71950],[2047,1003,94219],[2048,601,31053],[2049,63,89948],[2050,944,7438],[2051,639,56694],[2052,685,20260],[2053,1856,68744],[2054,657,98335],[2055,2009,84753],[2056,1367,77875],[2057,1643,89787],[2058,792,5089],[2059,1569,1950],[2060,169,56565],[2061,82,16928],[2062,1573,470],[2063,542,6916],[2064,1943,66462],[2067,730,55387],[2069,1250,64247],[2070,1488,938],[2071,94,90514],[2072,605,29025],[2073,1082,26394],[2074,1202,94264],[2075,1417,60967],[2076,1045,27009],[2077,1067,82089],[2078,1145,46433],[2079,796,24708],[2080,135,3945],[2081,1464,56941],[2082,1176,64853],[2083,1200,94888],[2084,32,70744],[2086,343,65223],[2087,27,47010],[2088,870,21963],[2089,1659,45436],[2090,44,64359],[2091,2052,32531],[2092,1206,19091],[2093,1849,87745],[2094,1422,79986],[2095,1813,71054],[2096,1447,78772],[2097,1457,1784],[2099,607,305],[2101,1590,47210],[2103,1111,38168],[2104,2022,35470],[2105,1629,19927],[2106,1536,34071],[2108,566,92823],[2109,677,25875],[2110,1817,33904],[2111,571,35240],[2112,741,96757],[2113,1486,78613],[2114,2053,38392],[2115,1724,99260],[2116,1845,66895],[2117,1635,21105],[2118,891,34050],[2119,708,24250],[2120,1331,82794],[2121,956,84026],[2122,1486,21995],[2123,769,56914],[2124,694,80412],[2125,1065,645],[2126,79,96687],[2127,1504,59408],[2128,458,43050],[2130,1594,13307],[2131,1535,99373],[2132,1321,97868],[2133,1759,83226],[2134,1561,37970],[2135,1453,56519],[2136,1275,47035],[2137,1480,44677],[2138,510,75986],[2139,1430,81904],[2141,395,98604],[2143,1812,64733],[2144,46,99363],[2146,1207,17310],[2147,1927,72480],[2148,675,20605],[2149,88,45907],[2150,670,84406],[2151,484,70180],[2152,675,36255],[2153,310,19553],[2154,1068,31005],[2155,703,10580],[2156,1449,8386],[2157,1349,20511],[2158,79,252],[2159,1755,97365],[2160,1920,37848],[2161,601,31738],[2163,1953,37118],[2165,850,54036],[2166,1327,61832],[2167,919,87251],[2168,505,73976],[2169,1618,17476],[2170,544,72201],[2171,617,39041],[2172,1507,56703],[2173,1425,29231],[2174,5,99946],[2175,1234,15449],[2176,1746,84835],[2177,215,12768],[2178,1079,41996],[2179,1483,99892],[2181,1888,42817],[2182,1142,27106],[2183,576,97036],[2184,1318,25675],[2185,935,65725],[2186,1111,52746],[2187,773,77664],[2188,671,4037],[2189,731,32401],[2190,343,28439],[2191,1629,85459],[2192,1663,22616],[2194,723,64190],[2196,1131,27207],[2198,1401,57134],[2199,1992,51555],[2200,2055,54755],[2201,1453,28041],[2203,97,77491],[2204,692,4818],[2205,1767,91743],[2206,1834,65580],[2207,750,87864],[2208,1599,96553],[2209,1338,41079],[2210,1906,15442],[2211,1036,91071],[2212,2179,31891],[2213,116,38651],[2214,634,5407],[2215,790,73219],[2216,1627,46632],[2217,1233,92692],[2218,1389,11892],[2219,1091,310],[2220,2042,4138],[2221,1896,57141],[2222,1943,48854],[2223,948,52713],[2224,326,43624],[2225,1604,32118],[2226,306,47822],[2227,416,29408],[2229,1591,78049],[2230,1331,54044],[2231,302,50714],[2232,394,27780],[2233,2167,35092],[2234,1760,18359],[2235,593,16597],[2236,1405,21589],[2239,777,17972],[2240,2213,50321],[2241,471,33198],[2242,1286,88861],[2243,300,86472],[2245,2132,35193],[2246,2076,68476],[2247,1643,38085],[2248,887,81822],[2250,419,14775],[2251,1034,13565],[2252,791,20932],[2253,1836,67989],[2254,1009,51834],[2255,1331,83849],[2256,2150,52152],[2257,1518,92443],[2258,993,59409],[2259,1332,37819],[2260,133,95590],[2261,1134,7218],[2262,215,2803],[2263,714,85861],[2264,904,1976],[2265,929,59940],[2267,724,19013],[2268,2127,58898],[2269,881,43078],[2270,969,9528],[2271,1779,5966],[2272,789,28826],[2273,1838,77664],[2274,1887,20241],[2275,1793,24149],[2277,444,39736],[2279,91,72484],[2280,908,25158],[2281,729,37842],[2282,55,79670],[2283,873,91619],[2284,2017,67223],[2285,931,27948],[2286,805,44287],[2287,918,78596],[2288,287,74695],[2289,588,33923],[2290,1924,43447],[2291,1871,19670],[2292,1742,23092],[2293,830,36399],[2294,1021,90140],[2295,20,49664],[2296,2093,69054],[2297,1551,34537],[2298,2248,61936],[2300,709,31920],[2301,485,32202],[2302,1828,83649],[2303,452,49966],[2304,1569,42409],[2305,898,66026],[2306,2053,65774],[2307,1640,50699],[2308,72,18222],[2309,1298,13183],[2310,874,52600],[2311,611,78138],[2312,1080,47769],[2313,1554,32481],[2314,1796,45496],[2315,317,50778],[2316,415,72322],[2317,406,83473],[2319,925,92462],[2320,557,59251],[2321,1801,10492],[2322,143,36709],[2324,263,87646],[2325,1076,35982],[2326,11,15974],[2327,1799,27301],[2328,303,67054],[2329,1933,42549],[2330,1737,31216],[2331,1744,86577],[2332,368,7191],[2333,1154,98932],[2335,4,28736],[2336,1782,50871],[2337,1319,84767],[2338,490,13772],[2339,220,15341],[2340,1209,7096],[2341,1234,9301],[2342,1914,54452],[2343,953,19943],[2344,1945,35404],[2345,692,93408],[2346,256,96687],[2347,2272,26920],[2348,819,4325],[2349,2229,458],[2350,2294,71856],[2351,1994,39277],[2352,628,57936],[2353,1743,3756],[2354,322,52759],[2355,1995,1101],[2357,1125,81963],[2358,595,38159],[2359,901,69176],[2360,426,80204],[2361,1110,3807],[2362,741,86148],[2363,175,33573],[2364,909,40522],[2365,1771,914],[2366,2297,55208],[2367,1890,56235],[2368,1804,19519],[2369,850,68637],[2370,1070,29015],[2371,855,63642],[2372,79,4870],[2373,2133,43005],[2374,204,57183],[2376,739,16655],[2377,672,1151],[2378,533,98617],[2379,911,68820],[2380,152,89028],[2381,1556,64362],[2382,439,79083],[2383,2140,49268],[2384,1166,13599],[2385,22,94755],[2386,1692,34636],[2387,1613,1253],[2388,133,64777],[2389,2217,48738],[2390,185,19364],[2391,1078,42950],[2392,597,99864],[2393,1506,60237],[2394,1021,71844],[2395,1206,44646],[2396,379,60834],[2397,199,27747],[2398,1478,36858],[2399,1679,89232],[2400,88,48528],[2401,2050,46105],[2403,1600,69274],[2404,971,21419],[2405,1000,97446],[2406,1624,26158],[2407,82,98831],[2408,1512,25186],[2409,2224,37847],[2411,381,14252],[2412,1061,9908],[2413,41,63348],[2414,1130,74462],[2415,2257,97605],[2416,1011,45431],[2417,526,61727],[2419,630,2855],[2420,460,18714],[2422,2276,5996],[2423,1724,38924],[2425,1447,90106],[2426,1041,73172],[2427,1695,73883],[2428,1812,60478],[2429,1107,55698],[2430,1340,69150],[2431,1725,1998],[2432,1452,44897],[2433,965,86420],[2434,1594,49807],[2435,1228,33124],[2436,189,36504],[2437,2231,27023],[2438,1265,41818],[2439,227,22051],[2440,491,25706],[2441,1458,24269],[2442,1394,91198],[2443,515,60214],[2444,2285,21829],[2445,1469,55308],[2446,2118,47692],[2447,332,91416],[2448,2420,3001],[2450,1655,12678],[2451,2279,8405],[2452,779,41602],[2453,1069,65093],[2454,403,78828],[2455,560,54963],[2457,1472,95585],[2458,1200,27071],[2460,790,49396],[2461,205,26025],[2462,876,29232],[2463,1000,2969],[2466,170,90311],[2467,819,25840],[2468,175,20096],[2469,926,14562],[2470,659,52428],[2471,1436,41523],[2472,2316,41998],[2473,458,77705],[2474,606,57376],[2476,1041,58445],[2477,1560,97281],[2478,2128,21227],[2479,1203,77451],[2480,462,25219],[2481,646,8349],[2482,2274,44492],[2483,375,60246],[2484,406,26080],[2485,1205,67669],[2486,503,26123],[2487,1353,19269],[2488,556,14243],[2489,1425,38485],[2490,1802,36245],[2491,2464,73756],[2492,113,55653],[2493,1303,82183],[2494,1974,72465],[2495,288,70579],[2496,814,46922],[2497,2277,97279],[2498,1964,9719],[2499,1200,29885],[2500,600,36273],[2501,1824,76022],[2502,1705,47172],[2503,1162,39632],[2504,1963,12157],[2505,800,34134],[2506,2459,41601],[2507,1569,69567],[2508,397,17907],[2509,1873,96135],[2510,1772,18118],[2511,1686,39480],[2512,55,56888],[2513,2226,21517],[2514,1847,32961],[2515,2200,63196],[2516,139,50629],[2517,592,73465],[2518,1090,88768],[2519,245,61],[2520,889,46999],[2521,681,3293],[2522,266,40421],[2523,99,17187],[2524,1416,81262],[2525,2339,47652],[2526,2411,59280],[2527,2176,45584],[2528,768,90991],[2529,277,92706],[2530,596,79260],[2531,671,29515],[2532,1126,96099],[2533,223,47097],[2534,525,95300],[2535,1306,70047],[2536,2507,33223],[2537,548,86813],[2538,673,86717],[2539,763,42502],[2541,2029,75460],[2543,777,32627],[2544,1106,68201],[2545,1813,74267],[2546,2044,7021],[2547,2113,36265],[2548,1275,56370],[2549,2135,41896],[2550,454,92434],[2551,378,37484],[2552,412,8121],[2553,2349,28006],[2554,138,19345],[2555,1870,44573],[2556,892,89085],[2557,1848,33033],[2558,2055,75789],[2559,1961,69333],[2560,151,34942],[2561,539,12572],[2562,1548,5950],[2563,1181,48440],[2564,957,2090],[2565,2103,99594],[2566,630,23679],[2567,2399,88550],[2568,1847,48699],[2569,951,38980],[2570,134,4514],[2571,2454,59093],[2572,2285,48149],[2573,522,3184],[2574,2552,77394],[2575,2201,7405],[2576,2472,57539],[2577,2294,19400],[2578,1508,29788],[2579,621,98430],[2580,1351,36518],[2581,453,88711],[2582,2240,12986],[2583,2171,37324],[2584,368,73937],[2585,2184,30738],[2586,1191,19147],[2587,213,57215],[2588,1581,58394],[2589,2411,46165],[2590,1175,12592],[2591,2301,46319],[2593,1985,24217],[2594,946,9783],[2595,335,90881],[2596,2090,26681],[2597,2322,41002],[2598,2334,49024],[2599,1103,37182],[2600,2345,55152],[2601,2334,97269],[2602,2400,82393],[2603,343,99315],[2604,2472,73789],[2605,2055,39070],[2606,1968,32192],[2608,810,39154],[2609,185,54290],[2610,1626,70901],[2611,189,75295],[2613,625,64436],[2614,1660,9165],[2616,2595,39171],[2617,823,77526],[2619,1738,8080],[2620,958,12651],[2622,2354,82223],[2623,1247,79693],[2625,2266,27173],[2626,1111,96604],[2628,1210,42181],[2629,542,12964],[2630,1540,2210],[2631,698,52425],[2632,1715,66822],[2634,607,1686],[2635,2411,36748],[2636,544,19209],[2637,1357,65306],[2638,2317,80823],[2639,412,37170],[2640,836,77594],[2641,1993,80238],[2642,2439,89385],[2643,1824,35166],[2644,1076,15455],[2645,717,99577],[2646,591,73160],[2647,1290,82080],[2648,2139,99751],[2649,489,79278],[2650,1884,62729],[2651,267,92741],[2652,1968,98670],[2655,1219,73786],[2656,1700,63321],[2657,1360,6316],[2658,1731,52349],[2659,2411,44954],[2661,545,93459],[2662,1032,97209],[2663,38,67989],[2664,1567,51330],[2666,125,16130],[2667,2155,95280],[2668,1523,14665],[2669,1379,35613],[2670,749,83583],[2671,1975,13897],[2673,836,64767],[2674,539,77392],[2675,1312,42027],[2676,2262,97176],[2677,1942,74106],[2678,282,8917],[2679,2552,97126],[2680,961,21924],[2681,363,47092],[2682,2154,90139],[2683,1206,78273],[2684,1317,56058],[2685,1221,51641],[2686,1198,44013],[2688,824,38845],[2690,749,27218],[2691,1983,91012],[2693,2241,16990],[2695,1012,65079],[2696,1968,13174],[2697,562,85512],[2698,384,91914],[2699,2310,76270],[2700,722,70268],[2701,2531,43547],[2702,1706,49714],[2703,350,95441],[2704,2629,54206],[2705,1440,18636],[2706,2638,90986],[2707,413,73248],[2708,1459,90762],[2709,337,58776],[2710,2646,3684],[2711,640,22743],[2712,1741,64933],[2714,592,96687],[2715,765,83847],[2716,963,11468],[2718,1681,48558],[2719,2528,1117],[2720,2035,39535],[2721,1525,25968],[2722,813,2185],[2723,93,68941],[2724,1493,66997],[2725,2433,98726],[2726,2689,63219],[2727,1266,68093],[2728,1493,79867],[2729,1102,31808],[2730,1491,81058],[2731,2606,12658],[2733,1343,85065],[2734,1043,77699],[2735,1463,71667],[2736,405,94781],[2737,314,37981],[2738,216,18786],[2739,2549,88759],[2740,1736,86297],[2741,413,87614],[2742,2403,47522],[2743,2391,78111],[2744,1686,52105],[2746,2460,13819],[2747,1844,43111],[2748,674,20474],[2749,298,3246],[2750,465,45958],[2751,610,53648],[2752,1745,33187],[2753,755,91346],[2754,1234,4138],[2755,2437,83810],[2756,714,87228],[2757,1770,45270],[2758,1083,85423],[2759,829,29033],[2760,909,87960],[2761,1821,45312],[2762,1974,63390],[2763,1123,92916],[2764,1537,47883],[2765,542,94799],[2766,1119,20726],[2767,2636,1863],[2768,2244,49905],[2770,332,9915],[2771,420,20266],[2772,1846,16998],[2773,393,53860],[2774,1473,20372],[2775,748,8672],[2776,1901,86785],[2777,1880,60247],[2778,1617,65901],[2779,1614,51262],[2780,1890,1496],[2781,656,37311],[2782,1174,80207],[2784,2714,52303],[2786,1663,17983],[2787,558,94367],[2788,33,75222],[2789,1496,27666],[2790,2175,62718],[2791,1012,57802],[2792,283,39345],[2793,2394,66716],[2794,727,9993],[2795,465,33633],[2797,768,75734],[2798,674,50388],[2799,1030,23704],[2800,98,74370],[2801,776,12792],[2802,940,77667],[2803,572,38708],[2804,2247,5221],[2805,1295,43471],[2806,2758,51458],[2807,1027,90532],[2808,2303,61996],[2809,432,33438],[2810,302,76535],[2811,1755,26831],[2813,2677,8445],[2814,109,74368],[2816,1462,29489],[2817,607,20237],[2818,672,44931],[2819,713,92900],[2820,2175,51526],[2822,545,76339],[2823,2218,13347],[2824,1887,57825],[2825,1017,24770],[2826,1781,77359],[2827,210,94704],[2828,2720,74314],[2829,1941,23895],[2830,659,37003],[2831,388,17531],[2833,1916,77686],[2834,961,16228],[2835,2601,15387],[2837,2614,94323],[2838,1701,19946],[2839,1529,76664],[2840,516,87873],[2841,2699,30643],[2842,2286,33140],[2843,2670,94200],[2844,2226,3481],[2845,2661,69372],[2846,1323,17914],[2847,1366,58503],[2848,1626,1114],[2849,2468,23375],[2850,1266,86616],[2851,1385,61869],[2852,847,98166],[2853,168,80127],[2854,2402,7142],[2855,2206,47055],[2856,285,19093],[2857,250,89446],[2858,760,40701],[2859,332,68552],[2860,1995,86362],[2861,827,88789],[2862,2823,99570],[2863,1755,83878],[2864,236,67068],[2865,562,44904],[2866,2118,18041],[2867,1187,14428],[2868,414,64810],[2871,27,69285],[2872,1797,95509],[2873,1838,80527],[2874,379,10532],[2875,1559,88069],[2876,1033,52479],[2877,100,50371],[2878,626,23137],[2879,1047,39045],[2881,1904,62987],[2882,1343,51764],[2883,591,42800],[2884,2532,86560],[2885,1770,87623],[2886,2670,32526],[2887,46,34004],[2888,2698,64978],[2890,837,15182],[2891,896,93286],[2892,2817,34363],[2894,110,37193],[2895,611,2370],[2896,460,43976],[2897,1555,49623],[2898,1715,9181],[2900,2369,15110],[2901,960,8918],[2902,607,32415],[2903,774,72117],[2904,1628,40657],[2906,521,44129],[2907,492,45003],[2908,1212,6625],[2909,1578,59018],[2910,1091,48434],[2911,64,57721],[2912,1642,85166],[2913,2669,65677],[2914,2167,55693],[2915,2524,41424],[2916,889,80133],[2917,324,1216],[2919,353,86028],[2920,1496,86664],[2921,1129,57113],[2922,314,85420],[2923,2334,60876],[2924,1552,21810],[2925,68,42398],[2926,2911,52942],[2927,2564,51677],[2928,488,56831],[2930,492,77734],[2931,2511,15570],[2932,167,61294],[2933,2692,20385],[2934,886,56154],[2935,1884,29865],[2936,1874,907],[2937,1227,50054],[2938,2361,17921],[2939,62,7889],[2940,1119,34604],[2942,2589,26756],[2943,1435,58732],[2944,1652,79242],[2945,329,32984],[2946,581,49522],[2947,212,66951],[2948,367,75829],[2949,2774,44360],[2950,457,67049],[2951,1995,43559],[2952,1660,21471],[2953,1999,1592],[2954,373,95786],[2955,17,85899],[2956,2263,24490],[2957,2074,55499],[2958,2193,19217],[2959,1172,85484],[2960,707,77635],[2961,497,37718],[2962,1025,89242],[2963,384,4761],[2964,610,12374],[2965,1589,1662],[2966,1989,14142],[2967,584,11572],[2968,2838,1816],[2969,1300,53073],[2970,1272,82955],[2971,1209,43306],[2972,2393,72473],[2973,2964,87446],[2974,182,27840],[2975,1852,37426],[2976,696,20566],[2977,927,23912],[2978,719,82520],[2979,2484,6463],[2981,1671,96068],[2982,817,67210],[2983,2018,55136],[2984,325,86300],[2985,367,69768],[2986,1794,8343],[2987,28,28291],[2989,2893,78264],[2990,421,53655],[2991,226,32168],[2992,2978,38422],[2993,1811,55839],[2994,1411,25383],[2995,1959,96425],[2997,1896,76632],[2998,622,96901],[2999,2648,48295],[3000,748,11112],[3001,1500,68925],[3002,2791,277],[3003,2065,66688],[3004,2581,91888],[3005,2932,2254],[3006,2053,11673],[3007,290,32993],[3008,1897,98847],[3009,802,52498],[3010,744,50834],[3011,2255,91561],[3012,2208,42429],[3013,807,54532],[3014,1208,84466],[3015,938,14627],[3016,1126,92723],[3017,2157,61461],[3018,1437,80824],[3019,851,1174],[3020,2131,14495],[3021,2440,56194],[3022,231,49020],[3023,2510,1812],[3024,235,27442],[3025,350,7948],[3026,653,49316],[3027,2720,79626],[3028,2789,60102],[3029,2321,42394],[3030,2654,22147],[3031,262,68260],[3033,2742,14922],[3035,1346,27468],[3036,1696,44688],[3037,993,34914],[3038,2480,42013],[3040,1798,6908],[3043,1840,64794],[3044,2707,33629],[3046,1713,28827],[3047,1256,6090],[3048,1502,66047],[3049,1884,54962],[3050,1371,96608],[3051,2002,46048],[3052,885,37266],[3053,2492,30408],[3054,22,26769],[3055,897,97225],[3056,752,74818],[3058,1503,31249],[3059,2540,76375],[3060,2145,92508],[3061,2835,48064],[3062,2338,68974],[3063,1097,34123],[3064,648,63193],[3065,295,51877],[3067,1990,20799],[3068,1350,96691],[3069,3013,17931],[3070,2082,74916],[3071,68,623],[3072,686,19849],[3073,1386,2408],[3074,2451,16680],[3076,1779,35729],[3077,983,24351],[3078,3023,13177],[3079,45,98928],[3080,2525,92410],[3082,2396,21590],[3083,325,85916],[3084,1351,38150],[3086,1510,11356],[3087,2258,39831],[3088,2360,82080],[3089,1513,96803],[3090,844,29668],[3091,2636,51743],[3092,73,22754],[3093,1472,56368],[3094,1152,19168],[3095,1705,58075],[3096,62,69253],[3097,694,34465],[3098,1710,6161],[3099,487,47053],[3100,567,51819],[3101,1088,46093],[3102,2479,5185],[3103,1139,5158],[3104,619,54299],[3105,2122,15826],[3107,2193,96891],[3108,1998,38971],[3109,1084,14478],[3110,3011,99360],[3111,1999,5109],[3112,799,71181],[3113,1955,15817],[3114,1057,93834],[3116,2561,98403],[3117,2650,80072],[3118,1037,93759],[3119,1109,74711],[3120,182,64622],[3121,1012,93265],[3122,2813,60543],[3124,2816,72127],[3125,897,80640],[3126,483,27712],[3127,2143,42858],[3128,1092,78056],[3129,2144,1410],[3130,3010,16310],[3131,894,17500],[3133,886,4071],[3134,2156,3099],[3135,1881,73169],[3136,1101,78551],[3137,1608,62568],[3138,322,21116],[3139,2589,34685],[3140,1014,35459],[3141,33,58317],[3142,2341,57668],[3143,1591,57855],[3144,34,65994],[3145,2250,91291],[3146,2233,77397],[3147,2586,47920],[3148,1867,3364],[3149,1457,39836],[3150,2351,24790],[3151,2501,39785],[3152,1206,61865],[3153,1991,6234],[3154,2772,8447],[3155,1599,36331],[3156,14,33000],[3157,1708,43408],[3158,2986,88977],[3159,1223,25324],[3160,723,33845],[3161,2240,58316],[3162,555,47559],[3164,102,32464],[3165,1101,86602],[3166,690,47446],[3167,312,61679],[3168,112,10910],[3169,3099,41671],[3170,2905,74939],[3171,2129,48434],[3172,2544,84068],[3173,1964,7689],[3174,2260,16939],[3175,3128,21086],[3177,2793,50695],[3178,1709,90979],[3179,2097,32927],[3180,2179,80388],[3181,1959,77445],[3182,2584,64029],[3183,1941,39436],[3184,1468,16634],[3186,912,7387],[3187,1082,63811],[3188,2409,29992],[3189,1329,97068],[3190,1524,54374],[3191,1215,11757],[3192,1693,52017],[3193,518,78783],[3195,2178,77763],[3196,2722,19141],[3198,1881,64587],[3199,716,36166],[3200,1830,41809],[3201,2333,20745],[3202,696,45232],[3203,112,23467],[3204,1960,68460],[3205,330,81702],[3207,906,21373],[3208,2271,47377],[3209,22,63391],[3210,49,70491],[3211,438,85392],[3212,2074,61377],[3213,1457,23659],[3214,1108,86807],[3217,2713,76033],[3218,1503,16533],[3219,1970,70857],[3220,412,12881],[3221,1496,13195],[3222,2279,73572],[3223,318,4191],[3224,241,16034],[3225,2433,88798],[3226,1111,45760],[3227,3133,22906],[3228,1538,94500],[3229,988,60074],[3230,128,53898],[3231,3200,8944],[3232,2046,18707],[3233,1554,72060],[3235,1691,18011],[3236,1929,33635],[3237,1659,27129],[3238,2715,95229],[3239,2850,71640],[3240,2122,85451],[3241,2567,89509],[3243,263,17190],[3244,1077,98533],[3246,2445,32195],[3247,1685,27214],[3248,2889,40874],[3249,2847,82418],[3250,379,72502],[3253,2844,81072],[3254,1475,73489],[3255,2517,62456],[3256,2828,25397],[3257,3000,82341],[3258,2865,44401],[3259,998,80009],[3260,1656,87029],[3261,288,22462],[3262,278,16984],[3263,2515,3584],[3264,57,7056],[3265,1011,19999],[3266,2365,75802],[3267,2537,41243],[3268,1831,98833],[3269,979,48232],[3270,1015,53675],[3271,3049,84818],[3272,703,49001],[3273,1570,27072],[3274,997,67429],[3275,34,84877],[3276,210,98405],[3277,2284,42945],[3278,1541,65035],[3279,1129,38094],[3280,2392,74968],[3281,2066,36828],[3282,2434,74952],[3283,1591,76348],[3284,467,20332],[3285,40,89247],[3286,2014,41404],[3287,1308,59921],[3288,378,85141],[3289,476,92885],[3290,217,36789],[3291,1454,44622],[3292,160,55387],[3293,475,92351],[3294,462,28089],[3295,876,99775],[3296,248,69274],[3297,277,56546],[3298,2833,67022],[3299,2100,9270],[3300,3068,86791],[3301,2506,95256],[3302,636,33894],[3304,2508,83421],[3305,2061,37277],[3306,2396,78245],[3307,1103,89040],[3308,306,56072],[3309,1480,89322],[3310,2647,83904],[3312,2153,19675],[3314,2865,8579],[3315,1694,15443],[3316,3246,6588],[3317,1321,82767],[3318,2849,44655],[3319,2808,13194],[3320,2406,45072],[3321,287,37876],[3322,1162,71611],[3323,1533,98562],[3324,1805,11089],[3325,2997,25398],[3326,17,73766],[3327,3231,54325],[3328,2499,93158],[3329,2303,81026],[3330,853,72611],[3331,930,53355],[3332,879,1611],[3333,2038,65225],[3334,2057,99311],[3335,2460,89435],[3336,2523,69074],[3337,9,267],[3338,499,95869],[3339,1702,40196],[3340,588,79487],[3341,583,5735],[3342,834,73326],[3343,877,12359],[3344,2721,86118],[3346,1814,95800],[3347,1270,11129],[3348,258,25518],[3349,2295,33715],[3350,551,51936],[3351,1481,13969],[3352,1390,53442],[3353,2982,97667],[3354,254,54306],[3355,2265,70621],[3356,789,32138],[3357,2909,9399],[3358,519,40628],[3359,2228,88214],[3360,1737,88378],[3361,2748,8402],[3362,1706,52937],[3363,3174,62587],[3364,1248,32204],[3365,538,71799],[3366,678,26041],[3367,1776,94055],[3368,279,62427],[3370,3071,94713],[3371,2641,44716],[3372,649,74525],[3373,2927,76596],[3374,1648,8584],[3375,2159,82944],[3376,2526,52645],[3377,1989,38689],[3378,597,32981],[3379,3023,74310],[3380,1901,6457],[3381,3124,83851],[3382,3314,7860],[3383,1203,26163],[3384,1995,49543],[3385,115,8550],[3386,828,84986],[3387,942,6232],[3388,2004,47858],[3389,3170,5380],[3390,629,18989],[3391,159,14677],[3392,865,41940],[3393,936,92207],[3394,1800,52392],[3395,2327,7842],[3396,2030,55326],[3397,19,9121],[3398,2224,20122],[3399,1408,45595],[3400,668,72503],[3401,1484,78181],[3402,99,6340],[3403,124,90328],[3404,2,26164],[3405,3345,49029],[3406,116,47524],[3407,414,94845],[3408,1655,92235],[3409,2341,58404],[3410,1046,68855],[3411,1766,52738],[3412,1438,169],[3413,1837,71120],[3414,1163,98348],[3415,2772,12479],[3416,177,66832],[3417,2515,72020],[3418,2102,75559],[3419,983,93372],[3420,2403,45295],[3421,274,43962],[3422,1679,81617],[3423,1716,82909],[3424,2036,38372],[3425,2613,73514],[3426,890,99797],[3427,575,14595],[3428,3176,51865],[3431,3181,29846],[3432,516,14819],[3433,1661,95490],[3434,2479,4972],[3435,700,77306],[3436,2302,58188],[3437,3039,35109],[3438,1389,62043],[3439,2693,36121],[3440,2952,60792],[3441,1006,72116],[3442,2801,17466],[3443,1926,65562],[3444,310,27441],[3445,2587,2756],[3446,609,34200],[3447,1586,79325],[3448,962,31125],[3449,420,39582],[3450,836,64202],[3451,2705,78990],[3452,969,55293],[3454,1466,96623],[3455,1326,81524],[3456,2900,1952],[3457,3334,20818],[3458,1689,56323],[3459,218,20705],[3460,3053,95923],[3461,1854,24855],[3462,2862,18628],[3463,1329,61135],[3464,1667,43002],[3465,1886,52017],[3466,425,58914],[3467,842,11630],[3468,2059,39960],[3469,253,82356],[3470,907,11807],[3471,1453,41374],[3472,1750,96646],[3473,86,45674],[3475,96,80440],[3476,396,32028],[3477,285,53326],[3478,313,83916],[3479,878,89387],[3480,1950,84245],[3481,1570,98142],[3482,203,83760],[3483,1450,30727],[3484,1015,79739],[3485,1061,90217],[3486,2359,1547],[3487,1493,94539],[3489,1413,39733],[3490,2733,89229],[3491,2519,82962],[3492,2568,89919],[3493,161,41101],[3495,689,2388],[3496,1630,12929],[3497,1760,62591],[3499,2071,31301],[3500,346,97849],[3501,1728,89646],[3502,3308,88221],[3503,1461,64053],[3505,2795,14696],[3506,1028,14722],[3507,1735,27753],[3508,1539,99994],[3509,3004,11211],[3510,3045,41732],[3512,152,47657],[3514,2409,88755],[3515,1167,47841],[3516,96,4378],[3517,990,20928],[3518,882,77262],[3519,2752,67334],[3520,2748,91988],[3521,305,3505],[3522,735,14163],[3523,749,80341],[3524,2434,71849],[3525,3118,49254],[3526,2892,76686],[3527,1528,35797],[3528,2011,60716],[3529,666,76699],[3530,922,53458],[3531,2049,75457],[3532,644,37514],[3533,3052,61650],[3534,3133,10714],[3535,662,42608],[3536,3290,999],[3538,1671,14340],[3539,591,31619],[3540,2746,94547],[3541,657,26238],[3542,2512,41534],[3543,2358,16938],[3544,2836,28853],[3545,1627,79591],[3546,371,43672],[3547,49,26568],[3548,631,27499],[3550,2143,35416],[3552,3370,79907],[3553,1820,56181],[3554,581,13365],[3555,1529,96351],[3556,1556,67226],[3557,371,9974],[3559,3318,42182],[3560,94,20988],[3561,1503,1735],[3562,3170,24666],[3564,868,68256],[3565,1720,18565],[3568,990,43599],[3569,3212,69884],[3570,2357,93764],[3571,2727,6985],[3572,3515,1464],[3573,2980,41211],[3574,725,47533],[3575,1121,46300],[3576,1810,72554],[3577,1721,52870],[3578,1388,75108],[3580,836,41583],[3581,1514,95608],[3582,3282,15799],[3583,3279,35417],[3584,1879,82910],[3585,1926,67201],[3587,1189,7064],[3588,1143,51183],[3589,880,62572],[3590,1453,29894],[3591,3314,47586],[3593,1264,20453],[3594,1610,74715],[3595,3047,93557],[3596,2846,61101],[3597,2516,52269],[3598,1403,33592],[3599,2196,1983],[3601,1324,26434],[3602,2710,64814],[3603,270,2485],[3604,2659,56686],[3605,2342,65894],[3606,3523,25760],[3607,1221,35324],[3608,447,53921],[3609,1914,96371],[3610,2913,3169],[3611,780,65565],[3612,475,23248],[3613,829,90868],[3614,2213,70845],[3615,2771,9535],[3616,1808,24471],[3617,1351,31078],[3618,1072,97707],[3619,1429,58390],[3620,2211,67124],[3622,640,48482],[3623,3446,40853],[3624,3163,40710],[3625,2082,11967],[3626,514,86179],[3627,2180,73705],[3628,3349,30939],[3629,1819,10352],[3630,2878,11228],[3631,700,9713],[3633,3298,12169],[3634,506,35798],[3635,2515,33257],[3636,2474,67465],[3637,1920,60172],[3638,1429,11259],[3639,3068,87076],[3640,2106,36092],[3641,345,59082],[3642,2155,12204],[3643,1685,84495],[3644,327,63137],[3645,1445,47601],[3646,475,81389],[3647,1129,44549],[3648,1642,75267],[3649,608,33350],[3650,3353,93343],[3651,1442,30874],[3652,2445,78303],[3653,398,9274],[3654,1233,4518],[3655,3603,97570],[3656,3212,81557],[3657,687,20763],[3658,845,20953],[3659,3622,41873],[3660,3137,645],[3662,64,12841],[3663,1828,87369],[3664,2189,77592],[3665,2639,25201],[3666,141,24112],[3667,1299,10887],[3668,3007,95544],[3669,2565,69989],[3670,2810,34839],[3671,2141,22538],[3672,144,49421],[3673,2866,64737],[3674,3599,71954],[3675,2516,57695],[3676,2427,2911],[3677,1515,10125],[3678,1599,56858],[3679,2262,76511],[3680,1112,98285],[3681,218,27341],[3682,1271,20797],[3683,3413,87948],[3684,2210,24027],[3685,705,88213],[3686,3614,50068],[3687,2202,48147],[3688,910,27202],[3689,1566,32526],[3690,1295,86582],[3691,696,27065],[3692,3167,28903],[3693,1508,99950],[3694,2681,53203],[3695,1902,30928],[3696,2420,41523],[3697,1239,49730],[3698,351,72710],[3699,2167,75430],[3700,1373,65086],[3702,2867,67406],[3703,535,90723],[3704,1558,38269],[3705,3532,43022],[3706,1696,41240],[3707,2900,95390],[3708,3419,58762],[3709,716,36026],[3710,1365,92565],[3711,361,31917],[3712,3693,27630],[3713,3031,25821],[3714,1539,6199],[3715,3392,24181],[3716,2149,14751],[3717,2056,10932],[3718,320,62638],[3719,1119,12849],[3720,1119,13447],[3721,2311,89288],[3722,136,45782],[3723,544,66854],[3724,1383,82359],[3725,2813,73194],[3726,1326,37482],[3727,784,65840],[3728,2862,32972],[3729,2500,68780],[3730,3342,93516],[3731,3356,14558],[3732,257,6309],[3733,2966,99196],[3734,2650,72131],[3735,1359,78534],[3736,2251,88411],[3737,2237,16409],[3738,3637,84288],[3739,3625,23200],[3740,2757,19210],[3741,665,40431],[3742,3541,27198],[3743,2753,71878],[3744,21,32754],[3746,2794,34502],[3747,2905,85352],[3749,2452,65963],[3750,871,62701],[3751,898,48817],[3752,1899,26565],[3753,142,63473],[3754,696,78641],[3755,110,55910],[3756,788,40240],[3757,787,88703],[3759,1300,92839],[3761,2816,51063],[3762,239,28130],[3764,3065,32079],[3766,551,58368],[3767,2918,44300],[3768,2633,56023],[3769,1173,74768],[3770,3297,34093],[3771,3571,55865],[3772,3473,42853],[3773,2609,94216],[3774,524,80240],[3775,1512,61218],[3776,774,114],[3777,2574,29772],[3778,3337,59751],[3779,7,54263],[3780,2410,7680],[3781,2712,24962],[3783,2424,63002],[3784,1748,75858],[3787,2278,59059],[3788,1790,82509],[3789,2487,20840],[3790,474,82318],[3791,2916,39379],[3792,1545,64043],[3793,554,47710],[3794,360,56882],[3795,393,52948],[3796,319,17413],[3797,3430,69387],[3798,3344,64597],[3799,359,51171],[3801,2900,58739],[3803,3375,50687],[3804,1035,19710],[3805,1563,33619],[3806,1043,1031],[3807,3619,19650],[3808,3354,27955],[3809,2623,72158],[3810,1713,1948],[3811,3055,86933],[3812,1072,39565],[3813,1239,45532],[3814,525,34163],[3815,2389,83184],[3816,3760,20841],[3817,826,41128],[3818,772,24196],[3819,2069,40134],[3820,579,57496],[3823,949,30078],[3824,1304,717],[3825,1534,90742],[3826,2989,31303],[3827,3730,49075],[3828,3043,15119],[3829,699,41988],[3830,2053,21362],[3831,593,74588],[3832,300,34513],[3833,3233,374],[3834,1933,43248],[3835,3830,28815],[3836,1611,85313],[3838,75,32775],[3839,3018,35167],[3840,735,92534],[3841,2178,54983],[3842,1540,96942],[3843,908,59838],[3844,2518,27552],[3845,2822,8689],[3846,3723,57227],[3847,533,31867],[3850,2546,36201],[3851,312,76117],[3852,18,71258],[3853,2127,89548],[3854,2843,91546],[3855,1226,39352],[3856,1248,49458],[3858,2388,20377],[3859,2946,54039],[3860,471,75029],[3861,3225,59254],[3862,1344,68785],[3863,1981,26504],[3864,1291,17177],[3865,3121,83935],[3866,1402,70223],[3867,3299,22924],[3869,2491,27971],[3870,2960,33895],[3872,3719,36781],[3873,2608,79133],[3874,2445,6384],[3875,10,59020],[3876,391,30514],[3878,3063,19424],[3879,63,7278],[3880,1110,89746],[3881,193,59876],[3882,3124,27732],[3883,2417,27398],[3884,848,90021],[3886,2403,21505],[3887,346,89082],[3888,2893,82639],[3889,3299,56212],[3890,2169,4415],[3891,1324,49483],[3892,2164,85609],[3893,3471,92981],[3894,2110,25787],[3895,3810,84024],[3897,566,93824],[3898,2615,12776],[3899,2014,64581],[3900,380,30676],[3901,2615,85978],[3902,2490,61083],[3903,3226,78879],[3904,1097,59711],[3905,3865,89809],[3906,1387,76021],[3907,874,45741],[3908,3563,72567],[3910,3066,97191],[3911,2967,18571],[3912,3556,64380],[3913,2312,39457],[3914,3913,36919],[3915,1914,18000],[3916,1181,11882],[3917,1190,75508],[3919,3625,3788],[3920,3109,64471],[3921,2298,99976],[3922,3324,19602],[3923,2666,49424],[3924,261,47288],[3925,741,55572],[3926,553,28661],[3928,538,46241],[3929,904,6963],[3930,447,26209],[3931,727,94064],[3932,2151,78891],[3933,1446,83454],[3934,717,91966],[3935,270,89172],[3936,877,69728],[3937,451,31510],[3938,1506,11533],[3940,881,43805],[3941,3551,6291],[3942,3167,7418],[3943,2614,59385],[3944,2102,60670],[3945,3532,78316],[3946,1255,22836],[3947,106,87236],[3949,75,94036],[3950,96,72612],[3951,3392,93142],[3952,1825,64795],[3953,3044,56774],[3954,2923,24705],[3955,2650,14805],[3956,3612,72674],[3958,892,52212],[3959,2530,56694],[3960,2535,94193],[3961,1613,84258],[3962,2942,85257],[3963,1516,30383],[3964,3481,91700],[3965,3096,63828],[3967,2056,74218],[3968,3733,4179],[3969,3438,78593],[3970,2032,73621],[3971,2945,66094],[3972,2108,88385],[3973,1696,10162],[3974,3104,71140],[3975,2931,20720],[3976,1061,14360],[3977,690,86015],[3978,2851,22537],[3979,1299,2278],[3980,481,34728],[3981,982,81241],[3982,557,65388],[3983,2465,35500],[3984,1963,33812],[3985,1985,2722],[3986,2000,58706],[3987,2448,16751],[3988,2634,95254],[3989,1085,83083],[3990,3542,88404],[3992,783,13924],[3994,2341,87602],[3995,1497,89661],[3996,747,86848],[3997,3479,64499],[3999,3923,36045],[4000,16,29485],[4001,659,306],[4002,718,73872],[4003,1827,51597],[4004,2063,26186],[4005,87,28498],[4006,145,57668],[4008,1995,61378],[4009,165,72312],[4010,558,35815],[4011,3289,8353],[4012,2604,79500],[4014,211,80094],[4015,1966,19010],[4016,543,23261],[4017,1541,91786],[4018,2791,91806],[4019,2265,88329],[4020,613,18832],[4021,2293,67849],[4022,1755,79265],[4023,2762,1719],[4024,2944,40343],[4025,2984,54007],[4026,2081,80786],[4028,1988,75366],[4029,644,16138],[4030,1243,86244],[4031,234,98332],[4032,3818,51150],[4033,229,86004],[4034,272,80257],[4035,686,2904],[4036,340,8840],[4037,3495,30444],[4038,1084,42615],[4039,532,30289],[4040,3726,48824],[4041,2445,99852],[4042,1462,35892],[4043,1577,90744],[4044,2206,13005],[4046,3999,53891],[4047,2772,40110],[4048,250,94129],[4049,3110,81060],[4050,2605,17227],[4051,1646,13376],[4052,1608,9129],[4053,1395,71482],[4054,2546,21830],[4055,2494,9557],[4056,2111,11722],[4057,1920,59533],[4058,3514,95802],[4059,2505,50711],[4063,3073,4670],[4064,418,47303],[4065,1145,46280],[4066,2500,12016],[4067,1012,55986],[4068,1196,71946],[4069,1991,92055],[4070,1424,72881],[4071,2462,40894],[4072,637,26855],[4073,3010,59117],[4074,827,96183],[4075,1879,17630],[4076,3751,42159],[4077,1621,8145],[4078,1414,66958],[4079,3027,18473],[4080,733,92935],[4081,1469,98455],[4082,3929,5149],[4083,1470,85525],[4084,1093,28644],[4085,3884,24248],[4086,247,91417],[4087,1297,91128],[4088,733,85544],[4089,680,90468],[4090,2940,17665],[4091,4045,28725],[4092,308,76866],[4093,1412,56868],[4094,2250,32997],[4095,411,59911],[4096,3997,55692],[4097,1636,53059],[4098,2446,50259],[4099,1054,90347],[4100,1103,55792],[4101,162,74325],[4102,1538,46310],[4103,1498,79094],[4106,1499,41651],[4107,1637,74281],[4108,3228,49223],[4109,43,91715],[4110,1787,55420],[4111,50,73730],[4112,875,56896],[4114,3530,46284],[4115,2659,88521],[4116,442,50351],[4117,653,73798],[4118,2403,72979],[4119,289,31886],[4120,2466,10462],[4122,2204,53914],[4123,2580,36172],[4124,3031,66424],[4125,2416,19125],[4126,4064,75147],[4127,52,69626],[4128,2813,69490],[4129,604,65538],[4130,3000,20816],[4131,340,17382],[4132,3227,91627],[4134,2113,7951],[4135,1280,91731],[4136,2665,234],[4137,3890,30778],[4138,3125,17363],[4139,2957,32354],[4140,3172,28562],[4141,3584,91108],[4142,1170,10884],[4143,2767,72653],[4144,3556,96627],[4145,3756,35818],[4146,901,98429],[4147,1385,63750],[4148,1710,64894],[4149,424,7569],[4150,2118,89731],[4151,1889,39566],[4152,717,87304],[4153,3207,62455],[4154,3119,47468],[4155,3484,7799],[4156,1916,80504],[4157,3921,70371],[4158,3933,11071],[4159,2584,37746],[4160,1043,94835],[4161,1857,80927],[4162,306,63913],[4163,3316,40383],[4164,3915,23962],[4166,2608,70578],[4167,326,77323],[4168,2675,76244],[4169,1879,87354],[4170,3816,28456],[4171,1985,58208],[4172,1881,51251],[4173,3611,82132],[4174,1961,93841],[4175,3875,49326],[4176,3627,39118],[4177,1482,48923],[4178,3757,9396],[4179,1389,79894],[4180,2541,63084],[4181,2260,94731],[4182,2676,6477],[4183,4101,73369],[4184,520,5356],[4186,2740,92133],[4187,3604,41353],[4188,1538,83882],[4189,3829,78354],[4190,805,29224],[4191,2322,708],[4192,262,64725],[4193,3863,68592],[4194,2556,46697],[4195,331,98463],[4196,849,26962],[4197,3976,54483],[4198,834,88233],[4199,1933,73565],[4201,342,8181],[4202,1739,34702],[4203,2193,70660],[4204,2151,8586],[4206,262,35380],[4207,1910,54231],[4208,1805,9399],[4209,874,70099],[4210,2992,50488],[4211,3684,33924],[4213,2691,24917],[4214,1919,81617],[4215,3268,69217],[4216,2772,73350],[4217,416,69141],[4218,2039,23229],[4219,718,63601],[4220,2702,73662],[4221,3970,99696],[4222,3751,9655],[4223,2750,11384],[4224,654,73630],[4225,2305,85096],[4226,3922,14641],[4228,1013,20792],[4229,971,64032],[4230,457,12843],[4231,1152,96364],[4232,526,65961],[4233,2483,52463],[4234,1023,48977],[4235,902,92103],[4236,3450,92134],[4237,1743,43147],[4238,3798,80709],[4240,1539,13879],[4242,1838,84656],[4243,1051,77112],[4244,3366,86421],[4245,1257,11768],[4246,2440,72303],[4247,2377,84806],[4248,2044,16029],[4249,2044,12429],[4250,714,14770],[4251,614,75906],[4252,3247,1610],[4253,3580,65871],[4255,3183,16705],[4256,1229,61933],[4258,1742,43294],[4259,3659,48053],[4260,757,69397],[4261,3538,23148],[4262,2463,45079],[4263,2231,965],[4264,60,15313],[4265,1477,5625],[4266,3295,94178],[4267,4094,42139],[4268,994,13692],[4269,151,5828],[4270,2794,28866],[4271,2262,4444],[4272,2506,42169],[4273,3451,98644],[4274,737,54255],[4276,2340,56078],[4277,2415,27111],[4278,400,76394],[4279,369,76528],[4280,2400,31703],[4281,743,65212],[4282,450,76874],[4283,2907,48893],[4284,4126,29691],[4285,1930,45404],[4286,1678,72814],[4288,1187,28122],[4289,712,95321],[4290,818,30317],[4291,4241,13830],[4292,3023,16875],[4293,716,97696],[4295,2173,36721],[4296,794,1163],[4298,189,12758],[4299,1168,72489],[4300,2894,98625],[4301,3025,57956],[4302,2154,40060],[4303,1116,38444],[4304,1892,86128],[4305,2543,69066],[4307,3668,8750],[4308,1868,60886],[4310,2908,45791],[4311,3013,33909],[4312,167,46457],[4313,2118,29308],[4314,695,44187],[4315,37,10420],[4316,1884,25051],[4317,3748,37996],[4318,4208,71695],[4319,3150,46145],[4320,373,37194],[4321,1503,56806],[4322,704,61158],[4323,3608,81452],[4324,4279,82490],[4325,3939,33970],[4326,628,75564],[4327,1167,37299],[4328,507,23108],[4329,143,25098],[4330,1850,16717],[4331,3895,78807],[4332,1158,462],[4333,1296,7849],[4334,710,42025],[4335,3556,32405],[4336,3360,10547],[4337,3492,67515],[4338,236,59048],[4340,245,36689],[4341,25,45305],[4342,3569,27916],[4343,2566,40436],[4344,2656,95786],[4346,3,30762],[4347,934,68687],[4348,1391,55454],[4349,3967,70198],[4350,3661,41386],[4351,662,85324],[4352,2671,11811],[4353,209,65061],[4354,2746,34715],[4355,476,22683],[4356,3724,99073],[4357,3605,47495],[4358,1506,16395],[4359,1765,98293],[4360,2717,22339],[4361,1449,18170],[4362,2111,44600],[4363,3311,10469],[4364,2648,50151],[4365,2906,60744],[4366,1709,6902],[4367,1954,62076],[4368,4354,74],[4369,3703,9137],[4370,3992,29697],[4371,2328,17369],[4372,3346,33481],[4374,1113,36569],[4375,2439,17384],[4377,1509,19927],[4379,187,4014],[4380,2242,29377],[4381,2292,85635],[4382,2983,75941],[4383,3424,92623],[4384,3146,21370],[4385,3333,65780],[4386,1055,77574],[4387,1097,10963],[4388,2341,48211],[4389,4072,35220],[4390,649,97439],[4391,2565,72473],[4392,592,54171],[4393,1406,53697],[4394,958,11910],[4395,4092,88202],[4396,922,19617],[4397,3021,88737],[4398,1419,68698],[4399,1498,4610],[4400,3494,95688],[4401,815,50937],[4402,3543,22563],[4403,1276,10155],[4404,4187,39475],[4405,1347,79133],[4406,3244,57132],[4408,2489,56226],[4409,2220,25301],[4411,1482,8935],[4412,4124,24797],[4413,3119,13737],[4414,3400,9730],[4415,913,48976],[4416,2250,35040],[4417,1398,42595],[4418,442,38542],[4419,3225,4062],[4420,4201,42624],[4421,2994,42213],[4422,4384,55947],[4423,2843,98129],[4424,1390,45685],[4425,1027,5103],[4426,2655,34488],[4428,852,19412],[4429,4263,11512],[4430,765,59319],[4431,3888,95852],[4432,2395,29391],[4434,3286,42870],[4435,1615,57712],[4436,507,48157],[4437,2034,52642],[4438,3893,32478],[4440,3872,26149],[4441,2693,10991],[4442,79,90521],[4443,2537,10122],[4444,1437,6607],[4445,2089,15752],[4446,3823,6372],[4447,1757,14700],[4448,826,4921],[4449,4341,65553],[4451,3154,97845],[4452,1715,65415],[4453,3490,11734],[4455,2556,39117],[4456,1764,858],[4457,2066,9273],[4458,1245,7333],[4459,1729,41324],[4460,4069,54238],[4461,3239,63038],[4462,2836,91595],[4463,2605,69978],[4464,2043,67282],[4465,2953,1348],[4466,1905,60577],[4467,1694,97941],[4468,4117,95062],[4469,1063,41201],[4470,500,67824],[4471,831,74093],[4472,3568,29757],[4473,1133,38190],[4474,2210,955],[4475,710,74274],[4476,1668,56981],[4477,2780,90085],[4478,3383,72840],[4479,162,53493],[4480,920,40285],[4481,4262,89921],[4482,886,14260],[4485,3151,65582],[4486,4287,37157],[4487,3217,35690],[4488,4334,58308],[4489,411,30585],[4490,955,64378],[4491,389,76702],[4492,2936,86495],[4493,2960,78630],[4494,3336,68705],[4495,3760,50907],[4496,244,21107],[4497,3746,43100],[4498,2302,86858],[4499,331,1146],[4501,492,10622],[4502,3168,78832],[4503,1048,82409],[4504,2451,21823],[4505,147,26654],[4507,2644,19818],[4508,3512,37214],[4509,4148,23850],[4510,2057,29667],[4511,2629,5349],[4513,2338,56643],[4514,287,262],[4515,1863,84310],[4516,1343,50904],[4517,3075,24898],[4519,849,6909],[4520,1218,12657],[4521,1803,73859],[4522,3519,66584],[4523,2888,61987],[4525,3598,52937],[4526,599,53104],[4527,827,24585],[4528,3134,24894],[4529,1429,81999],[4530,2162,53958],[4531,272,69014],[4532,1580,24352],[4533,2524,42459],[4534,137,88046],[4536,1946,58427],[4537,2792,10417],[4538,1773,24880],[4540,2862,74689],[4541,1924,36163],[4542,2624,61649],[4543,1097,2350],[4544,3010,78577],[4545,734,10204],[4546,1137,93033],[4547,1619,56947],[4548,2257,75487],[4549,3708,56245],[4551,1633,4979],[4552,1834,10694],[4553,4279,10740],[4554,1642,63955],[4555,1078,40295],[4556,2727,36377],[4559,221,70913],[4561,4057,80725],[4562,3133,70490],[4564,2586,55600],[4565,2127,2914],[4567,2096,61631],[4568,330,15106],[4569,3505,85057],[4570,625,47390],[4571,42,74390],[4573,339,70910],[4574,531,56403],[4575,2494,62586],[4576,4443,44459],[4577,4069,994],[4578,582,26287],[4579,181,6535],[4580,896,2264],[4581,207,26175],[4582,3930,82741],[4583,554,62338],[4584,2965,43359],[4587,4563,19578],[4588,511,41921],[4589,1317,92930],[4590,3692,32923],[4592,1925,70897],[4593,1341,70465],[4594,1682,62172],[4595,2617,52330],[4596,3620,41755],[4599,3474,74026],[4600,1525,45963],[4601,3301,27392],[4602,2774,80792],[4603,3547,54814],[4604,1733,36031],[4605,2648,23890],[4606,4044,24145],[4607,3935,69],[4608,512,32042],[4609,3843,19351],[4610,219,85944],[4612,236,22376],[4613,3495,55273],[4614,2412,57773],[4615,3029,76077],[4616,4227,92066],[4617,2572,48678],[4618,3961,75607],[4619,4403,53632],[4620,1457,44180],[4621,767,82577],[4622,3610,73591],[4623,3856,96113],[4624,4380,74693],[4625,1741,89687],[4626,3011,72936],[4627,1551,28008],[4629,3863,2790],[4631,3100,88376],[4632,3300,36887],[4633,2078,16920],[4634,880,89834],[4635,3317,10360],[4636,2312,44358],[4637,3941,36694],[4638,3566,50530],[4639,3418,73645],[4640,4076,98068],[4642,2337,76888],[4643,462,7785],[4644,3151,91670],[4645,3556,37850],[4646,1301,54336],[4647,4573,5840],[4648,3498,56437],[4649,351,492],[4650,1941,90555],[4651,712,71979],[4652,1168,59707],[4653,3246,67881],[4654,2803,68071],[4655,1994,19134],[4656,280,36336],[4657,3430,46341],[4658,2464,91836],[4659,767,31303],[4660,3506,99471],[4661,2812,19525],[4662,3614,28252],[4663,1196,20726],[4664,3579,76276],[4665,460,62055],[4666,2523,53987],[4667,2483,51240],[4668,4400,86515],[4669,3875,71449],[4670,275,20068],[4671,2034,70255],[4672,1152,16872],[4673,2066,41395],[4675,4235,48122],[4676,2347,57119],[4677,4639,55483],[4678,348,42413],[4679,1189,54809],[4680,1754,20605],[4681,4153,53813],[4682,2940,67182],[4683,2248,18798],[4684,265,10562],[4685,3445,26879],[4686,3930,40546],[4687,3009,71918],[4689,3468,64514],[4690,3077,7934],[4691,446,54744],[4692,2339,87990],[4694,619,90505],[4695,1027,33571],[4696,1141,17127],[4697,1983,60800],[4698,4251,6593],[4699,2504,8054],[4700,1244,48651],[4701,606,88097],[4702,646,17027],[4703,3056,5260],[4704,1555,81720],[4705,1058,29955],[4706,2266,91377],[4707,1002,420],[4708,4611,92630],[4709,2330,19631],[4710,490,11745],[4711,58,1527],[4712,2375,56187],[4713,697,89398],[4714,2159,51468],[4715,351,84174],[4716,636,85617],[4717,2921,5106],[4718,1912,1617],[4719,1707,87332],[4720,31,92812],[4721,1621,69959],[4722,1227,39705],[4723,3029,28961],[4724,1004,29637],[4725,3228,27106],[4726,304,15665],[4727,4191,5959],[4728,4722,84178],[4729,210,88319],[4730,2663,68302],[4732,1073,98515],[4733,3048,43250],[4734,3137,22302],[4735,3098,65452],[4736,4384,22158],[4737,3939,68858],[4738,1739,39670],[4739,1750,12257],[4740,2660,21235],[4741,189,83097],[4742,2086,960],[4743,4393,9888],[4744,1355,69990],[4745,2385,45075],[4746,408,50863],[4748,4086,63782],[4749,4375,93868],[4750,2895,20883],[4751,4460,89526],[4752,4630,27321],[4753,2033,28438],[4754,984,28540],[4755,3826,66233],[4756,2094,55854],[4757,1039,86123],[4758,2162,20459],[4759,351,72337],[4761,2100,7813],[4762,4573,44134],[4763,2105,89859],[4765,822,38035],[4766,3073,55804],[4767,4599,87743],[4768,4062,38863],[4769,98,54933],[4770,2116,90750],[4771,3201,28480],[4772,315,78300],[4773,4599,26223],[4775,3619,55164],[4776,148,86898],[4777,4556,19967],[4778,146,10762],[4780,1073,49926],[4781,4467,22168],[4782,500,80698],[4783,283,48366],[4784,3464,27727],[4785,1744,56400],[4787,372,71228],[4788,3289,90011],[4789,4480,32684],[4790,2664,41808],[4791,3072,91167],[4792,4016,51614],[4793,3445,9524],[4794,4453,99332],[4795,4559,18133],[4796,3952,51345],[4797,4788,52396],[4799,3346,45613],[4800,2792,77832],[4801,2072,70024],[4802,617,89525],[4803,245,72605],[4804,4086,62971],[4807,2594,12957],[4808,3714,48598],[4809,2616,18693],[4810,74,62882],[4811,2973,85141],[4813,1573,13286],[4814,944,24285],[4815,393,48508],[4816,92,66773],[4817,1613,46200],[4818,1913,85145],[4819,2181,52386],[4820,2187,57368],[4821,2615,35193],[4822,3356,16635],[4823,3936,31775],[4824,345,48639],[4825,3576,51506],[4826,3542,79028],[4827,874,3710],[4829,1028,11124],[4831,4442,75460],[4832,3378,14580],[4833,3632,2249],[4834,3419,47740],[4835,2068,1344],[4836,425,72083],[4837,4825,24877],[4838,909,19974],[4839,2311,25057],[4840,649,22635],[4841,3753,38523],[4842,951,42616],[4843,1665,35938],[4844,3279,86232],[4845,4249,6468],[4846,1085,59835],[4847,2058,71088],[4848,1542,25647],[4849,647,6423],[4850,2809,58914],[4851,3049,59313],[4852,2243,3898],[4853,2961,23283],[4854,3657,66324],[4856,3769,47780],[4857,3439,9107],[4858,4289,30038],[4859,1405,86108],[4860,3061,77134],[4861,3647,9451],[4862,713,88792],[4863,4653,55572],[4864,690,51356],[4865,1796,65553],[4866,1439,98863],[4867,3477,75925],[4868,4573,56861],[4869,2989,1267],[4870,3130,1877],[4872,3504,86658],[4873,3957,9637],[4874,3968,64806],[4875,2497,6132],[4876,1462,22218],[4877,4469,50945],[4878,4148,48824],[4880,61,92026],[4881,4260,12448],[4882,2837,10496],[4883,4176,93769],[4884,1879,78062],[4885,152,64406],[4886,3935,12204],[4887,3086,85979],[4888,86,82447],[4889,545,7204],[4890,2504,34936],[4892,2488,84929],[4893,2558,43869],[4894,4697,16978],[4895,1815,20708],[4896,3725,73424],[4897,2814,74613],[4898,3980,87183],[4900,4264,9058],[4901,741,38291],[4902,2584,63022],[4903,4220,25044],[4904,2247,27226],[4905,2539,15981],[4906,3461,80179],[4908,1385,38442],[4910,1748,2983],[4911,943,95061],[4912,29,77590],[4913,798,81584],[4914,4352,71546],[4915,528,59703],[4916,2635,59531],[4917,515,35616],[4918,2753,96032],[4919,4574,26653],[4920,4195,30293],[4921,2108,86492],[4922,730,73331],[4923,3138,89370],[4924,1032,46090],[4925,1604,2204],[4926,2301,18726],[4927,4593,94106],[4928,2775,60368],[4929,3338,31890],[4930,3697,62844],[4931,1977,68316],[4932,134,71315],[4933,1104,45573],[4935,3020,89794],[4936,2113,77051],[4937,3956,92873],[4938,3814,95395],[4940,525,67922],[4941,1369,76376],[4942,2820,7764],[4944,3583,47405],[4945,678,98918],[4946,3641,74773],[4947,1153,20622],[4948,3818,81255],[4949,4542,95558],[4950,4177,89776],[4951,374,22579],[4953,4468,63616],[4954,3236,32701],[4955,3035,10149],[4956,2197,62741],[4957,3049,13547],[4958,1435,97768],[4959,1322,99894],[4960,2884,83152],[4961,2090,53156],[4962,538,82698],[4963,2588,81769],[4965,2275,21276],[4966,3118,87679],[4967,1177,59162],[4968,4121,65210],[4969,3239,10432],[4970,3068,95365],[4971,2626,90152],[4972,183,60398],[4973,552,32485],[4974,4844,24576],[4975,1795,90547],[4976,678,44605],[4977,2612,78816],[4978,2495,68568],[4979,12,98743],[4980,214,22124],[4981,4755,1098],[4982,174,40306],[4983,4420,79277],[4984,949,1863],[4985,2445,39103],[4986,2451,74467],[4987,4513,49190],[4988,1398,51750],[4989,4347,83126],[4990,754,1916],[4991,1745,44967],[4992,186,15327],[4993,2379,25658],[4994,2315,7855],[4995,2498,68184],[4996,1108,83621],[4997,2430,52934],[4998,4116,61384],[4999,4646,97322],[5000,4815,2714],[5001,3627,58553],[5002,1711,20167],[5003,3474,55766],[5005,2124,13224],[5006,373,24479],[5007,89,94483],[5008,1237,56902],[5009,4012,8034],[5010,833,86052],[5011,4339,87828],[5012,732,44549],[5013,2032,88365],[5014,390,96121],[5015,3054,43712],[5016,1254,11188],[5017,1885,91180],[5018,249,50844],[5019,4340,56613],[5020,396,30292],[5021,289,78828],[5022,3261,51253],[5023,573,2970],[5024,3393,48925],[5026,2952,46418],[5028,2895,83452],[5029,4494,44285],[5030,509,57283],[5031,2342,98032],[5032,511,69230],[5033,4919,48916],[5034,1308,1113],[5035,3827,48818],[5036,2598,41938],[5037,921,5822],[5038,2247,39021],[5039,293,74176],[5040,183,71763],[5041,4333,6059],[5042,4787,11116],[5043,4268,41184],[5044,3453,47232],[5045,885,83205],[5046,2261,55624],[5047,2646,88047],[5048,2027,41878],[5049,3562,33251],[5050,2682,21808],[5052,1810,21656],[5053,1994,6962],[5054,3236,84129],[5055,778,89003],[5058,3076,79013],[5059,3828,64991],[5060,4031,37419],[5061,1087,92433],[5062,191,68136],[5063,2730,46290],[5064,1065,9715],[5065,462,92878],[5066,1766,92559],[5067,2749,95270],[5068,3682,34381],[5069,4977,14521],[5070,4152,9447],[5071,4286,25801],[5072,647,12664],[5074,4526,89566],[5075,2476,16395],[5076,3345,70782],[5077,2681,33611],[5078,3210,93618],[5079,1756,38443],[5080,4425,76266],[5081,3280,13123],[5082,2383,4185],[5083,2799,57486],[5084,4183,28623],[5085,2682,5750],[5086,2453,71868],[5087,1513,18449],[5088,1558,1777],[5089,1970,89319],[5090,696,94673],[5091,4104,59945],[5092,1719,90679],[5093,759,36574],[5094,5016,75070],[5095,1032,79605],[5096,2266,5669],[5097,2407,63608],[5098,4004,61859],[5100,4269,93517],[5101,996,5688],[5102,4371,69525],[5103,1766,45087],[5104,3304,79888],[5105,301,63307],[5106,233,72376],[5107,4104,82064],[5108,3458,56577],[5109,2588,29428],[5110,3446,39224],[5111,5049,88607],[5112,2112,49804],[5113,383,37855],[5114,2169,74609],[5115,2950,20200],[5116,4243,61173],[5117,2460,88193],[5119,183,86090],[5120,3401,49682]]
    

wells = {i:wells_all[i] for i in base}
pipes = [x for x in pipes_all if (x[0] in base) & (x[1] in base)]
    
s = Solution()    
res,well_status,pipe_status = s.minCostToSupplyWater(n, wells, pipes,True)



# print('n:',len(wells))
# print('wells:\n',wells)
# print('pipes:\n',pipes)


well_cost_base = wells
pipe_cost_base = pipes

well_index_mapping = {i:k for i,k in enumerate(well_cost_base)}
well_cost = [well_cost_base[well_index_mapping[i]] for i in range(len(well_index_mapping))]
pipe_cost = [x[2] for x in pipe_cost_base]
target = well_cost+pipe_cost
result = []
for i in range(len(target)):
    for j in range(i+1, len(target)):
        result.append( (i,j,abs(target[i]-target[j])))
        
        
res=set()
for i,j,y in result:
    if y==14144:
        if (i<len(well_cost)) & (j>=len(well_cost)):
#             print(i,j)
            cur_val = well_index_mapping[i]
            tmp = pipe_cost_base[j-len(well_cost)]
            if True and cur_val in tmp:
                res.add(cur_val)

                res.add(tmp[0])
                res.add(tmp[1])
#                 raise

# print(res)
print()
for x in res:
    if x in well_status:
        print(x,True)
    else:
#         print(x,False)
        pass


        


base: 2472
##########################################################################
stack: []
well_status:
 [5, 12, 14, 19, 21, 24, 33, 36, 42, 44, 46, 51, 64, 67, 70, 83, 95, 103, 105, 106, 108, 110, 111, 120, 122, 123, 126, 129, 130, 150, 151, 152, 153, 157, 158, 165, 168, 170, 172, 178, 182, 188, 193, 199, 201, 202, 204, 208, 211, 219, 223, 229, 236, 237, 245, 253, 254, 255, 256, 272, 282, 286, 289, 291, 294, 295, 296, 300, 301, 302, 306, 311, 314, 315, 325, 327, 328, 333, 334, 337, 341, 352, 355, 358, 364, 367, 369, 372, 376, 392, 393, 396, 402, 410, 414, 418, 423, 438, 439, 445, 446, 447, 454, 461, 465, 468, 469, 470, 474, 480, 496, 498, 503, 508, 509, 517, 518, 529, 533, 535, 538, 541, 547, 548, 564, 567, 568, 573, 574, 595, 604, 606, 607, 617, 619, 623, 626, 627, 628, 633, 638, 640, 642, 643, 651, 662, 671, 672, 677, 678, 679, 680, 682, 683, 684, 693, 700, 707, 713, 715, 716, 719, 722, 726, 736, 738, 741, 744, 747, 756, 764, 766, 771, 773, 775, 784, 785, 786, 795, 807, 816, 81

In [6]:
if type(wells)==list:
    wells_new = [int(round(x/1000,0)) for x in wells]
elif type(wells)==dict:
    wells_new ={k:int(round(v/1000,0)) for k,v in wells.items()}
pipes_new = [(x[0],x[1], int(round(x[2]/1000,0))) for x in pipes]



# Init
node_dict = {}
link_dict = {}

if type(wells_new)==list:
    wells_new = {i+1:cost for i,cost in enumerate(wells_new)}


for i,cost in wells_new.items():
    node_dict[i] = {}
    node_dict[i]['id'] = str(i)
    node_dict[i]['info'] = str(i)
    node_dict[i]['label'] = str(cost)
    node_dict[i]['fillColor'] = '#999999'
    node_dict[i]['radius'] = 10
    node_dict[i]['opacity'] = 0.7

for i,j,cost in pipes_new:
    key = tuple(sorted((i,j)))
    link_dict[key] = {}
    link_dict[key]['from'] = str(i)
    link_dict[key]['to'] = str(j)
    link_dict[key]['label'] = str(cost)
    link_dict[key]['fillColor'] = '#999999'
    
    
for i in well_status:
    node_dict[i]['fillColor'] = '#ff6500'
    
for key in pipe_status:
    link_dict[key]['fillColor'] = '#ff6500'

    
for i in res:
    if i in well_status:
        node_dict[i]['fillColor'] = '#800080'
    
    else:
        node_dict[i]['fillColor'] = '#0000ff'
    
    
    
    
    
    
nodes = list(node_dict.values())
links = list(link_dict.values())


output_json = {'version': '1',
     'nodes': nodes,
     'links': links,
     'title': 'test'
    }
dump_json(output_json,'test.json')


In [7]:
wells_key_mapping = {x:i+1 for i,x in enumerate(wells.keys())}
wells_key_mapping_reverse = {i:x for x,i in wells_key_mapping.items()}


wells_list = [wells[wells_key_mapping_reverse[i+1]] for i in range(len(wells_key_mapping))]
pipe_list = [[wells_key_mapping[i],wells_key_mapping[j],x] for i,j,x in pipes]

print(len(wells_list))
print(wells_list)
print(pipe_list)

2472
[95540, 76282, 74797, 32553, 69489, 8659, 90254, 14856, 30762, 3030, 93521, 75424, 95582, 34410, 87433, 3721, 9944, 97187, 27172, 8729, 89006, 73009, 34359, 9910, 57695, 21122, 19429, 14557, 37020, 92358, 54737, 84352, 52697, 52505, 92986, 59256, 12962, 98921, 77535, 10750, 53494, 65547, 39146, 66052, 94339, 31630, 64562, 7069, 8309, 42876, 98921, 38213, 10692, 78632, 15866, 6357, 72160, 8377, 94253, 20515, 14501, 45314, 45221, 52616, 89923, 42337, 25866, 43500, 34049, 64962, 3121, 28277, 96151, 48983, 94788, 97336, 76657, 2166, 15802, 37659, 52970, 20396, 50059, 76487, 67904, 97398, 39806, 92333, 50793, 49047, 10036, 88035, 47257, 62933, 73948, 97709, 90267, 98093, 55202, 50701, 70696, 83146, 99462, 18497, 14443, 78661, 22128, 79519, 3340, 92357, 79102, 66123, 56986, 87552, 24836, 87726, 97919, 52628, 63148, 75631, 87913, 8925, 52684, 10155, 65339, 99664, 19381, 99790, 90039, 68217, 21459, 69677, 63378, 81421, 19141, 90728, 89247, 59497, 65479, 84152, 15928, 33523, 63391, 65897, 

In [8]:
# Get Group Cost

In [58]:


def disconnect(node1,node2):
    # pp(' Disconnect',node1.val,node2.val)
    if (type(node1) in (str,int)):
        node1 = node_dict[node1]
    if (type(node2) in (str,int)):
        node2 = node_dict[node2]
    node1.remove_pipe(node2)
    node2.remove_pipe(node1)

def get_connected_group(node,checked_node_set):
    stack = deque()
    group_set = set()
    stack.append(node)

    while(stack):
        cur = stack.pop()
        group_set.add(cur)
        checked_node_set.add(cur)
        for pipe, cost in list(cur.pipes.items()):
            adj_node = node_dict[pipe]
            if (cost>adj_node.well_cost and cost>node.well_cost):
                disconnect(adj_node,cur)
            else:
                if (not adj_node in group_set):
                        stack.append(adj_node)   
    return group_set


def get_groups(node_list):
    group_list = []
    checked_node_set = set()
    for node in node_list:
        if node in checked_node_set:
            continue
        else:
            group = get_connected_group(node,checked_node_set)
            group_list.append(group)
    return group_list




n = 2472
wells = [40712, 22902, 11267, 98706, 44691, 30785, 89097, 2807, 74834, 44882, 38200, 65030, 22725, 17906, 91949, 98031, 30852, 85656, 53943, 1398, 77320, 81493, 58511, 88049, 11941, 18102, 89282, 67220, 81739, 89253, 34886, 88922, 26551, 11679, 53608, 51266, 57722, 62836, 33376, 99341, 69716, 44508, 47020, 71926, 4970, 92543, 29286, 70234, 77130, 21499, 42814, 9960, 18351, 28979, 40416, 91159, 78138, 59823, 25819, 65029, 35944, 17872, 79905, 71808, 26375, 17276, 25131, 10308, 38148, 99185, 21280, 47968, 95759, 91033, 73978, 84684, 38677, 15292, 81099, 78241, 43650, 72865, 52195, 27022, 11291, 47521, 61757, 50266, 20198, 34755, 8410, 89006, 75102, 86680, 82116, 66532, 76855, 13902, 41026, 23841, 9406, 277, 64458, 10399, 84146, 33661, 83161, 25991, 69578, 15438, 55012, 14395, 95540, 50662, 63248, 45513, 40601, 31882, 99787, 34931, 92410, 49481, 89566, 95936, 29562, 83667, 76998, 79124, 40570, 76340, 70337, 31812, 94026, 39768, 66346, 41237, 12493, 94859, 55147, 75458, 87254, 94647, 14240, 22771, 22430, 84601, 50340, 47131, 61434, 19941, 47215, 84766, 40706, 88971, 47216, 72856, 42401, 50583, 59464, 11235, 14075, 68583, 77660, 31150, 25984, 19666, 43773, 6201, 64481, 52530, 7894, 54636, 81783, 81166, 19161, 47312, 41076, 67013, 28505, 62157, 71734, 25615, 38168, 66862, 4594, 60831, 18034, 43613, 6001, 68877, 46319, 77015, 1750, 9818, 9785, 39213, 65659, 43927, 82036, 83036, 55433, 72458, 17058, 15378, 61206, 35876, 63846, 10002, 94911, 16566, 30490, 4748, 34494, 27446, 29107, 56556, 88561, 55869, 94718, 98330, 4797, 53320, 17673, 81259, 26108, 68218, 15092, 24730, 62171, 51813, 90780, 84949, 87346, 11641, 56197, 36489, 65478, 95540, 76282, 64025, 74797, 32553, 69489, 8659, 91457, 69425, 90254, 98782, 20608, 14856, 72358, 41450, 64339, 55695, 30762, 3030, 9553, 93521, 61207, 75424, 97238, 95582, 81424, 34410, 87433, 3721, 91728, 21955, 41679, 9944, 73049, 97187, 6882, 92234, 27172, 69358, 8729, 82249, 44667, 67526, 73009, 36219, 34359, 87054, 92724, 9910, 51020, 25081, 89519, 57695, 21122, 19429, 43854, 8018, 46685, 14557, 37020, 94748, 1177, 92358, 54737, 51417, 50616, 84352, 52697, 24035, 52505, 99671, 80639, 92986, 73818, 39093, 59256, 72362, 42083, 9524, 65684, 58, 12962, 78500, 98921, 77535, 60926, 17090, 95071, 5195, 38826, 64260, 29977, 39146, 66052, 27084, 31630, 64562, 94339, 8309, 82313, 7069, 98921, 72160, 78632, 8377, 45221, 25866, 20515, 45314, 64962, 3121, 48983, 76657, 2166, 94788, 20396, 97398, 52970, 67904, 49047, 47257, 62933, 98093, 73948, 97709, 57516, 85603, 99462, 18497, 50701, 70696, 79519, 92357, 56986, 24836, 87552, 63148, 15538, 8925, 97919, 10155, 99664, 52684, 20792, 13210, 82734, 63307, 793, 89247, 18524, 37339, 14194, 88599, 98492, 65479, 82518, 32636, 33523, 99719, 53624, 96550, 84152, 65897, 96070, 93971, 42961, 63391, 75927, 22015, 90071, 94324, 97038, 93437, 3667, 78000, 52344, 35605, 85302, 36006, 96446, 84270, 83502, 99343, 80421, 28173, 35611, 35849, 52575, 80954, 39100, 67543, 3805, 37641, 98317, 36326, 4152, 78724, 24681, 39430, 46754, 99970, 76166, 78790, 30927, 64210, 46159, 1426, 93204, 41627, 3539, 29383, 40563, 60377, 82316, 98449, 1383, 24205, 31139, 92270, 84217, 67308, 24119, 75540, 86309, 26315, 54, 61990, 45709, 28084, 10297, 49367, 39804, 4646, 14159, 24397, 90111, 29023, 53494, 71221, 67636, 55676, 39825, 83066, 82011, 36722, 87936, 23003, 44492, 78173, 81928, 83113, 53638, 30388, 8438, 66445, 92082, 74201, 17631, 28501, 37950, 1118, 9973, 11722, 57453, 27952, 87657, 30387, 69458, 44642, 52334, 58705, 27279, 87634, 63675, 37504, 69584, 35679, 69084, 32431, 32362, 71064, 50096, 1347, 24999, 99799, 9314, 38935, 79758, 2035, 61399, 74602, 84902, 14838, 83917, 50023, 8962, 92789, 16163, 12504, 70052, 6379, 50228, 30072, 42981, 65339, 51047, 63378, 19141, 90039, 90728, 69677, 33765, 81421, 96435, 3440, 978, 55927, 56387, 78297, 31749, 34688, 96989, 65029, 61789, 50140, 37905, 38235, 90161, 58607, 79889, 7751, 54661, 33350, 37020, 44260, 58086, 14849, 43710, 46184, 52378, 32704, 66760, 1787, 91898, 27094, 93954, 38755, 81623, 62081, 31545, 62614, 82269, 79952, 40644, 61834, 19287, 57518, 24177, 61926, 75732, 98821, 37259, 33631, 32829, 28044, 28595, 54452, 14811, 51572, 37339, 35096, 74973, 38967, 53729, 94174, 83119, 73311, 856, 69276, 22487, 33301, 85187, 43699, 98773, 59384, 33159, 8159, 30661, 55036, 27791, 87780, 20470, 95229, 34503, 84115, 86690, 17591, 88481, 22643, 77779, 24961, 88724, 1532, 62902, 63291, 35701, 84807, 59296, 27431, 26148, 11969, 61941, 26227, 14462, 91066, 68259, 28572, 47884, 59505, 87971, 36815, 54225, 45356, 50450, 19938, 91481, 81159, 34636, 90696, 85565, 85268, 65125, 4620, 93577, 86753, 7315, 56141, 59549, 74785, 91024, 90781, 15739, 76943, 15950, 2596, 13574, 85610, 98729, 40850, 33746, 59131, 31813, 1478, 7372, 21209, 33646, 54178, 88853, 63859, 16076, 23587, 22946, 58952, 4592, 11521, 20140, 11136, 10240, 87551, 22960, 60684, 54014, 31816, 23953, 78000, 52123, 87986, 67927, 77553, 91248, 50755, 34507, 13442, 13618, 50999, 85198, 65892, 75722, 9908, 34306, 39427, 67481, 8376, 88886, 17027, 69549, 5704, 80787, 94114, 14611, 8692, 13998, 57844, 71866, 46412, 55411, 73825, 16550, 47923, 42192, 25393, 93465, 94352, 35829, 5753, 20259, 89442, 92604, 52408, 27019, 69270, 95524, 88566, 77153, 89877, 37255, 94204, 96751, 24805, 66885, 36114, 60016, 29500, 77226, 49557, 27349, 14864, 7184, 47424, 97663, 19982, 91529, 14965, 69868, 52117, 29235, 46688, 52010, 53707, 46680, 97762, 76670, 95181, 49353, 90483, 31229, 97916, 86153, 48189, 23737, 48324, 71562, 21931, 99922, 86704, 83808, 38909, 88789, 20333, 71809, 38625, 91458, 25599, 14797, 72233, 430, 62557, 774, 81473, 45391, 554, 92055, 14714, 27904, 70729, 2129, 89012, 40285, 33575, 44967, 46905, 69654, 9988, 74765, 69902, 43740, 68788, 31959, 30139, 46554, 10298, 44132, 75580, 87784, 52954, 11622, 54386, 18273, 24828, 60298, 51533, 43476, 7332, 90492, 83198, 40778, 31195, 55851, 4641, 31850, 97848, 38826, 44604, 50244, 48737, 14454, 15430, 56542, 32362, 20660, 5272, 4262, 9553, 90919, 32389, 65877, 87667, 46518, 41753, 15451, 29209, 48611, 4825, 47633, 4005, 29540, 42968, 68661, 50153, 20981, 69822, 98579, 43859, 21454, 52483, 51971, 74183, 49424, 59536, 48421, 77314, 77723, 55879, 92658, 52241, 75458, 41438, 1682, 12363, 66222, 71925, 5910, 13669, 87090, 11056, 17887, 1658, 70196, 50866, 74383, 60402, 82654, 85048, 9901, 33170, 66947, 57160, 91487, 67010, 94244, 78857, 44048, 73565, 21539, 40285, 24519, 33131, 61060, 84186, 24208, 78222, 86944, 95820, 39761, 68019, 88436, 2648, 70823, 45074, 70072, 69463, 91147, 74265, 122, 30498, 31743, 95231, 19103, 81906, 98921, 83777, 66478, 26702, 81944, 12944, 58802, 36798, 71281, 40212, 24387, 76128, 87314, 3679, 42400, 72259, 45613, 35436, 80008, 37106, 529, 87612, 62451, 56911, 18276, 98352, 82128, 66407, 61428, 90216, 61927, 9629, 65609, 43420, 87533, 33281, 94922, 2130, 97250, 68144, 59749, 36830, 95527, 88862, 46343, 76967, 84425, 55092, 53621, 50871, 99525, 45190, 15291, 42738, 70605, 84152, 40746, 89850, 93935, 921, 16621, 34148, 87178, 13938, 5398, 21312, 74387, 23142, 72638, 66761, 83340, 61227, 63868, 63887, 80281, 22089, 17201, 57833, 63819, 35626, 18925, 72992, 11375, 32311, 87302, 71411, 18057, 70844, 1885, 18594, 42434, 30610, 52292, 7753, 20551, 15594, 81253, 32760, 7279, 36600, 9732, 86037, 21724, 60272, 78818, 21543, 94848, 76602, 56808, 7252, 45058, 5949, 53760, 20090, 71683, 87047, 90690, 37126, 21760, 17198, 140, 51568, 14935, 37277, 18477, 84684, 50209, 38500, 29587, 48009, 85144, 13611, 70229, 67561, 77011, 83182, 83699, 68421, 57229, 58897, 97716, 79676, 88468, 73538, 72434, 56735, 27652, 87346, 44389, 47134, 61167, 88011, 93215, 215, 4713, 95421, 53499, 77081, 78506, 94974, 60334, 37356, 75527, 82670, 38617, 86181, 55787, 49424, 99730, 25092, 44092, 98290, 11583, 49499, 13525, 22146, 5449, 41696, 88900, 51856, 62152, 30789, 54031, 50777, 28841, 92693, 64092, 60279, 20657, 44645, 81428, 62298, 56050, 37415, 35692, 80967, 54956, 78232, 69627, 26235, 32838, 25953, 99226, 22827, 73993, 64666, 98763, 80755, 2629, 6658, 87030, 36088, 62940, 91566, 76624, 31437, 35470, 65819, 60734, 63386, 66919, 98327, 81989, 93350, 51011, 45446, 42079, 96684, 23651, 11598, 24666, 16608, 50028, 51941, 91889, 55723, 40880, 66182, 98242, 73631, 59182, 30742, 49379, 69237, 1077, 75573, 5435, 15350, 6553, 88730, 55425, 80424, 26879, 18786, 79195, 30710, 31902, 53447, 63534, 46751, 76838, 99446, 47889, 98216, 50241, 42368, 78240, 50520, 65414, 20384, 75962, 33421, 83767, 15602, 8722, 46000, 687, 63768, 48971, 81328, 23130, 14124, 68346, 77455, 15604, 7077, 86739, 94755, 15025, 18133, 85475, 69255, 49087, 31670, 96087, 12508, 55169, 58037, 97734, 20409, 31215, 24721, 8917, 68532, 99042, 99862, 97478, 9226, 44608, 87163, 82161, 55482, 92752, 44214, 69627, 53772, 36116, 11373, 57425, 31803, 52652, 72430, 83571, 92202, 25451, 23, 38708, 41658, 25678, 76876, 28944, 94860, 76740, 68953, 52470, 59837, 2527, 50705, 26949, 57401, 62238, 24732, 22891, 10804, 22407, 64614, 51416, 13422, 14228, 39724, 30084, 98739, 91103, 20326, 16658, 34322, 80138, 84553, 12836, 35536, 78277, 25252, 22687, 59164, 1727, 85495, 6751, 59169, 92302, 94645, 32160, 95691, 18063, 75726, 80975, 84790, 90064, 38436, 92475, 73473, 74702, 69822, 8899, 98447, 70940, 57847, 59190, 92867, 51211, 5556, 65040, 15898, 5846, 39531, 74169, 5148, 7236, 72514, 99021, 44461, 62058, 65214, 75191, 91044, 20583, 54000, 78327, 66248, 58469, 71837, 4555, 60112, 22749, 50703, 15324, 44766, 56252, 80137, 58428, 9631, 57736, 1193, 13937, 93148, 45593, 58416, 91423, 69858, 66485, 64649, 29503, 4641, 17591, 88302, 88067, 16807, 70778, 15108, 72962, 17678, 4202, 34248, 22933, 288, 52260, 74490, 2565, 68523, 90093, 86616, 38298, 53443, 6615, 4562, 34793, 90425, 82910, 76968, 42735, 25006, 3263, 3623, 9190, 65868, 87268, 33897, 77893, 67461, 70152, 95719, 72771, 90663, 75753, 38083, 8540, 61650, 26666, 91612, 60781, 79414, 50402, 85665, 35399, 95930, 22741, 54391, 91829, 90009, 62969, 77304, 94381, 21799, 90440, 80003, 23874, 48586, 18308, 86685, 63988, 16946, 66010, 40739, 73936, 92452, 10929, 79635, 12792, 1807, 44614, 81274, 19972, 78015, 36336, 41726, 21093, 63597, 45101, 14534, 1574, 63704, 73330, 46150, 3170, 6526, 40782, 29723, 62880, 85538, 56867, 84701, 90467, 33448, 9895, 23512, 88706, 39320, 89394, 58607, 59362, 35386, 39305, 78528, 31713, 43147, 13270, 71241, 1648, 85629, 98801, 79980, 99079, 73186, 38503, 15725, 81195, 53649, 65863, 26006, 49342, 43418, 19105, 14401, 42643, 96315, 80262, 26684, 84442, 60240, 82835, 34614, 62783, 53399, 80425, 63896, 32583, 69325, 75920, 13078, 24545, 42650, 54215, 29118, 27159, 87139, 90297, 65547, 41812, 33395, 61922, 38213, 92481, 6357, 90063, 94253, 52616, 43500, 28277, 79362, 96151, 40680, 37659, 76487, 39806, 31414, 50793, 85396, 10036, 65131, 90267, 83146, 12725, 14443, 95961, 78661, 70908, 66123, 76319, 59056, 19274, 22856, 19381, 57222, 2817, 40276, 48755, 39552, 88423, 37858, 59497, 14057, 15928, 42876, 58897, 72259, 19876, 10692, 6332, 15866, 55420, 5576, 6616, 2381, 95050, 64022, 89923, 86074, 34049, 30648, 84734, 68372, 27765, 97336, 17750, 99561, 15802, 77872, 36675, 50059, 80115, 12108, 36492, 48263, 92333, 78876, 26093, 50987, 8516, 39464, 23696, 88035, 35630, 33460, 28200, 12238, 55671, 90260, 90383, 55202, 97190, 28523, 50330, 56396, 58968, 15412, 22128, 21031, 33821, 79102, 42485, 87726, 56796, 50990, 76496, 52628, 34162, 75631, 70998, 87913, 38304, 35820, 2344, 62213, 68674, 70444, 78422, 92693, 945, 92034, 53193, 10157, 59397, 40456, 98721, 87639, 86826, 3077, 23965, 22750, 58299, 96721, 9188, 45680, 57558, 15201, 34448, 40894, 45456, 68761, 97737, 72131, 38197, 96895, 79019, 87247, 46525, 70111, 99604, 57332, 69473, 82783, 44742, 15685, 19003, 67272, 65309, 16442, 46233, 12507, 48544, 54357, 39895, 53987, 93873, 34658, 81902, 79506, 18364, 79885, 50861, 3856, 40309, 5245, 70315, 90387, 33107, 89444, 89772, 99643, 31030, 28710, 99409, 40562, 87459, 62866, 6811, 4253, 74321, 95938, 56314, 10863, 54205, 60381, 97943, 75193, 8045, 62846, 18073, 80765, 19167, 9201, 76643, 75620, 98522, 2134, 15733, 20970, 52441, 79605, 30010, 21646, 64461, 45526, 86268, 95181, 91699, 2481, 79532, 83196, 16273, 16500, 87286, 12402, 74467, 56288, 99259, 52970, 36175, 36348, 56184, 45498, 37304, 18758, 21394, 65022, 59872, 27488, 23097, 41991, 68217, 52300, 81514, 90096, 66011, 76208, 42706, 26968, 57962, 47844, 93834, 64749, 35261, 43140, 2369, 98339, 13432, 64939, 89034, 80337, 45818, 16143, 50399, 69796, 99013, 34972, 13053, 46756, 58955, 44928, 65582, 21872, 47486, 20177, 23945, 82102, 60222, 15986, 33022, 30067, 38516, 28810, 70666, 39087, 43468, 1315, 91277, 37091, 53608, 89688, 49344, 3277, 6151, 6924, 64854, 51834, 4024, 94425, 99790, 26054, 41342, 21459, 37240, 5697, 39516, 820, 18716, 33199, 65571, 29438, 52004, 49557, 49903, 73718, 70650, 6089, 13090, 9842, 39495, 53489, 1422, 43963, 82471, 66087, 33059, 40637, 38188, 30762, 67693, 26635, 24029, 4310, 55902, 78630, 99070, 13711, 49745, 15189, 64121, 42001, 14764, 76064, 47271, 38983, 60199, 75328, 24699, 72747, 59823, 85938, 46409, 35409, 78426, 57680, 96231, 51161, 48437, 94048, 8730, 69743, 96235, 79483, 13517, 84361, 66673, 65548, 75197, 25325, 47088, 79309, 37641, 21241, 94887, 40025, 70506, 11977, 79021, 76536, 29146, 23765, 96417, 24238, 57108, 94972, 79752, 32493, 3696, 73768, 64115, 93996, 22647, 88189, 77611, 81269, 79403, 12206, 5864, 31319, 2788, 85671, 11092, 6275, 77655, 91767, 61774, 67572, 7373, 91174, 65833, 93411, 17100, 36027, 278, 50241, 61366, 20502, 63746, 86711, 37840, 42480, 49837, 39348, 74682, 42682, 69582, 11165, 90124, 89595, 89085, 34859, 39882, 69754, 54635, 60009, 34452, 97901, 65104, 95021, 17384, 34016, 93094, 45094, 40857, 48132, 95910, 98905, 41865, 38323, 98780, 1276, 37166, 58531, 72750, 31258, 98099, 19807, 90624, 85466, 55655, 49063, 88600, 826, 85963, 77967, 86067, 43234, 67603, 88858, 63372, 98162, 96141, 12848, 48195, 90461, 78580, 45065, 72920, 6596, 67388, 56109, 71460, 46266, 958, 64485, 32320, 91132, 84913, 93082, 27775, 95604, 46317, 58154, 62604, 51633, 25745, 1163, 89843, 77215, 16603, 98929, 77738, 31936, 9359, 52940, 57997, 62495, 97406, 12414, 98598, 82921, 39363, 1264, 603, 57163, 12925, 62529, 40314, 56186, 44394, 66735, 23777, 65341, 702, 80380, 38978, 29204, 77265, 71468, 54602, 18491, 99314, 36569, 98117, 87421, 95972, 943, 36086, 63435, 84833, 61390, 86247, 4581, 56697, 13371, 22197, 61664, 66901, 61311, 61686, 87536, 78146, 75892, 46526, 16438, 96154, 56005, 89325, 22491, 36357, 29131, 94445, 19969, 35152, 91667, 75500, 26311, 51577, 34488, 71395, 67288, 57825, 50095, 83237, 34209, 95007, 26853, 31621, 58666, 42780, 92055, 71525, 30231, 23491, 71769, 51608, 5679, 70456, 32506, 44213, 6369, 83884, 14501, 1181, 29909, 4958, 42337, 63198, 5726, 66244, 21551, 67297, 18915, 35473, 29887, 9810, 57135, 41926, 81496, 84042, 33265, 49806, 3340, 94833, 26189, 25161, 32879, 48969, 42073, 71557, 21134, 71560, 89008, 2250, 31628, 33030, 64432, 89462, 79576, 62928, 28477, 59357, 2841, 10605, 95428, 95909, 42797, 75777, 97963, 54349, 61896, 14516, 76246, 54867, 11646, 35155, 50719, 83856, 41951, 84068, 93703, 80073, 77785, 67725, 4559, 98179, 67319, 94933, 20242, 79944, 63892, 55772, 99094, 97233, 51712, 30451, 56685, 53338, 54910, 56229, 90568, 52094, 56661, 70559, 37426, 99509, 11947, 91704, 98741, 59013, 37972, 28010, 3789, 79132, 28147, 63110, 92828, 37544, 83482, 21449, 32852, 67874, 56253, 59038, 13322, 81532, 9693, 32122, 62730, 20437, 64223, 90921, 19100, 29838, 66231, 3560, 94138, 63795, 44145, 39657, 16469, 84037, 4209, 27956, 29422, 52934, 48873, 14472, 10905, 61335, 56499, 28242, 75748, 75839, 75987, 28644, 31192, 80590, 67303, 26464, 32253, 87318, 37462, 92000, 6603, 24584, 3271, 36874, 59879, 83417, 79809, 86997, 12956, 24564, 4388, 89426, 52079, 63207, 90294, 70248, 68756, 68938, 97569, 83634, 22156, 78601, 54262, 90408, 37616, 91527, 19389, 14541, 64577, 85896, 54069, 6167, 44909, 8466, 83941, 35168, 47906, 22061, 43641, 95235, 33930, 82119, 63457, 79995, 61137, 68183, 39971, 97977, 13572, 35873, 52262, 70646, 30083, 66253, 92112, 55239, 68141, 79481, 14277, 28734, 63869, 86450, 10327, 10241, 88266, 58745, 17247, 17705, 40882, 1851, 51908, 10750, 89919, 38091, 66541, 48673, 42953, 51372, 75528, 7425, 14975, 64666, 307, 30662, 44248, 65762, 5726, 7575, 43645, 47989, 15461, 63575, 36609, 66733, 71138, 45577, 90119, 17145, 61939, 93497, 93626, 17181, 47836, 50667, 80714, 45248, 83783, 43018, 37307, 74373, 98844, 65416, 44134, 8325, 42924, 92407, 69330, 99304, 38357, 35290, 23631, 46721, 90398, 76206, 11773, 52676, 36490, 20168, 69841, 62552, 51811, 35592, 55755, 23078, 31588, 23451, 43399, 90476, 80181, 91728, 6884, 93036, 16750, 68985, 97804, 60362, 71565, 67520, 27969, 29015, 51390, 99093, 6501, 62259, 93394, 23618, 40917, 78859, 16154, 724, 85833, 80988, 33121, 68698, 9972, 71865, 77695, 73574, 34276, 3336, 19831, 22218, 69012, 93290, 67396, 20466, 84536, 59, 86352, 69783, 9081, 39514, 22574, 48044, 42270, 14472, 11178, 75683, 97551, 50814, 60304, 22662, 76122, 23190, 57266]
pipes = [[2325, 2356, 29701], [2328, 2325, 97954], [577, 2328, 14600], [375, 2325, 22110], [337, 2328, 14412], [364, 577, 2560], [363, 577, 30822], [2131, 375, 24812], [2156, 2356, 30583], [2136, 363, 51992], [2140, 2131, 68435], [2139, 577, 73049], [2150, 577, 96887], [2067, 364, 7896], [671, 2067, 87635], [857, 577, 65804], [754, 671, 38626], [755, 2356, 23842], [765, 2156, 13943], [758, 364, 86959], [781, 857, 56993], [785, 671, 65478], [829, 781, 1745], [834, 2150, 87906], [835, 755, 95020], [812, 2356, 32630], [816, 2139, 92967], [824, 816, 82313], [1389, 2356, 99891], [1391, 2139, 34162], [905, 2136, 38087], [907, 671, 57011], [944, 337, 18719], [941, 812, 77119], [939, 944, 21348], [937, 2325, 47063], [935, 907, 49034], [925, 834, 66432], [923, 2328, 84395], [916, 364, 47710], [910, 755, 91789], [2059, 824, 30519], [2065, 781, 43347], [2074, 2139, 67496], [2077, 765, 74982], [2083, 916, 29080], [2084, 944, 88674], [2087, 2074, 11198], [2093, 758, 25532], [2098, 2083, 43847], [2099, 337, 50451], [2101, 910, 65665], [2106, 925, 39159], [2112, 2093, 52212], [2113, 910, 83462], [2123, 2098, 33747], [2125, 905, 89431], [2364, 765, 79585], [2367, 835, 28242], [2370, 907, 62811], [2371, 2140, 48256], [2374, 937, 74903], [2376, 2084, 55040], [2378, 2098, 38146], [2381, 2065, 77445], [692, 1391, 93110], [697, 758, 9519], [698, 829, 34595], [703, 2378, 87776], [705, 835, 37237], [707, 765, 90199], [709, 916, 74313], [713, 2093, 33433], [714, 781, 61179], [715, 2084, 84718], [718, 2106, 13810], [723, 715, 86910], [726, 765, 29483], [730, 2378, 91981], [733, 337, 73538], [735, 2356, 23474], [737, 754, 60080], [739, 2087, 75982], [740, 925, 81311], [741, 715, 74974], [1436, 735, 25868], [1439, 707, 19432], [263, 741, 86544], [268, 816, 22154], [269, 2113, 31854], [271, 2371, 10505], [276, 910, 60703], [282, 2098, 91572], [285, 671, 58132], [287, 2112, 77145], [289, 2093, 48422], [293, 1391, 95248], [294, 2077, 61978], [299, 2084, 76738], [303, 925, 73090], [306, 944, 25304], [308, 941, 79013], [316, 287, 19431], [318, 2370, 38563], [323, 910, 31287], [325, 2084, 76222], [150, 2370, 24477], [153, 364, 76610], [155, 153, 89761], [160, 829, 25753], [165, 160, 67935], [170, 364, 7909], [172, 944, 67848], [181, 2376, 82491], [188, 916, 5625], [190, 2099, 69468], [193, 2376, 9665], [199, 172, 62761], [2086, 2065, 44662], [2092, 160, 40719], [2096, 2325, 35724], [2102, 2087, 35502], [2105, 698, 40461], [2115, 2370, 48251], [2117, 199, 31468], [2120, 816, 28268], [2122, 740, 92398], [2124, 2371, 93630], [2127, 2074, 32983], [2130, 737, 24667], [2134, 316, 83564], [2142, 308, 5146], [2148, 705, 81634], [2153, 263, 49365], [2158, 2325, 2743], [543, 2381, 70265], [549, 318, 93597], [552, 549, 82025], [557, 715, 17511], [561, 293, 43398], [563, 2150, 39531], [565, 735, 98386], [569, 318, 2478], [573, 737, 54275], [575, 2099, 57178], [580, 190, 7814], [588, 172, 77445], [592, 2117, 58732], [595, 2142, 40261], [599, 2381, 36161], [602, 713, 98426], [604, 1391, 5950], [609, 730, 93483], [1680, 2067, 24114], [1681, 2140, 56866], [1684, 2381, 83242], [1696, 785, 70301], [1701, 1391, 12339], [1706, 925, 66095], [1710, 190, 63542], [1715, 916, 80572], [1717, 1710, 49480], [1724, 2130, 28620], [1730, 2130, 15336], [1735, 282, 288], [1737, 2120, 40005], [1748, 2371, 22347], [1754, 709, 30440], [1764, 153, 82039], [1768, 337, 84935], [1774, 263, 26433], [1781, 2153, 1272], [768, 2127, 46568], [776, 2101, 17273], [779, 2112, 16543], [783, 363, 22913], [786, 835, 94387], [793, 2127, 39029], [797, 939, 38436], [800, 563, 55232], [801, 857, 21950], [807, 1774, 29748], [809, 2131, 79111], [815, 602, 1199], [820, 316, 58695], [823, 709, 95981], [828, 2136, 27236], [833, 741, 9606], [838, 153, 91230], [842, 692, 41108], [847, 1735, 57700], [848, 829, 29689], [852, 2374, 85869], [855, 1684, 27571], [861, 2123, 41434], [863, 714, 39683], [867, 857, 81282], [1211, 857, 18774], [1214, 783, 49499], [1217, 834, 35476], [1229, 1724, 74891], [1687, 1229, 47060], [1690, 730, 7734], [1691, 1774, 48920], [1695, 602, 97516], [1703, 1735, 52246], [1707, 268, 43065], [1716, 820, 89573], [1718, 289, 58966], [1728, 2148, 52119], [1733, 939, 62983], [1743, 188, 78837], [1753, 2150, 9200], [1763, 2125, 46561], [1770, 2136, 12250], [1778, 718, 33641], [1212, 1764, 15815], [1218, 740, 24866], [1222, 599, 64697], [1225, 1770, 55239], [1228, 1743, 9797], [1241, 549, 39615], [1243, 1774, 73084], [1246, 2099, 95143], [1249, 323, 43237], [1252, 2124, 60149], [1260, 2153, 87257], [1265, 1214, 91407], [1273, 563, 74371], [2225, 705, 7549], [2234, 2148, 31408], [2236, 2113, 10431], [1443, 2113, 62655], [1446, 2122, 73434], [1458, 316, 90573], [1461, 199, 86501], [1467, 2153, 65834], [1469, 2136, 10629], [1472, 2115, 58288], [1477, 857, 39544], [1479, 1684, 33084], [1483, 1225, 19688], [1487, 588, 58833], [1491, 793, 40730], [1495, 823, 26525], [1498, 363, 91271], [1502, 2140, 92204], [1510, 783, 39725], [1514, 1778, 10054], [1518, 848, 93549], [1521, 268, 79149], [1527, 1724, 49358], [1530, 1225, 68552], [1533, 2234, 91335], [1539, 1467, 16607], [1660, 785, 8117], [1662, 303, 9942], [1667, 2131, 5518], [1671, 754, 81096], [1674, 937, 77858], [1678, 730, 85794], [1686, 1443, 63758], [1688, 2158, 10604], [1699, 714, 32353], [1702, 2136, 49598], [1705, 867, 63931], [1709, 786, 54049], [1712, 707, 30348], [1720, 318, 17723], [1723, 2093, 14597], [1726, 2098, 69424], [1729, 1691, 86389], [1732, 299, 81918], [1742, 2065, 19960], [1745, 2140, 28629], [1749, 599, 93187], [1752, 923, 68123], [1756, 592, 56918], [1759, 1472, 76694], [1760, 294, 63931], [1765, 2074, 442], [1769, 925, 52931], [1776, 1533, 4396], [1779, 2105, 11191], [1782, 293, 43474], [1987, 165, 6269], [1990, 2098, 33667], [1992, 1214, 6513], [1995, 1483, 22053], [1996, 726, 40227], [2000, 1479, 19227], [2001, 2364, 81886], [2002, 1667, 67585], [2006, 1687, 57839], [2011, 577, 30096], [2012, 707, 47372], [2018, 1518, 23678], [2021, 2136, 670], [2026, 1760, 13894], [2029, 1720, 64933], [2031, 833, 93614], [2033, 754, 53580], [2035, 741, 39323], [2037, 1439, 41593], [2040, 2112, 79360], [2046, 2099, 49864], [2047, 1732, 71371], [2049, 2225, 50407], [2241, 1763, 99842], [2244, 1735, 89522], [2248, 713, 47359], [2250, 2120, 10440], [2251, 800, 69916], [2259, 306, 82127], [2262, 718, 16063], [2264, 323, 22127], [2266, 2101, 80682], [2268, 1774, 89040], [2269, 2026, 80592], [2276, 1696, 96631], [2279, 834, 61883], [2281, 2364, 27086], [2287, 1726, 24856], [2288, 2099, 31190], [2289, 2156, 6332], [2293, 1483, 37086], [2296, 820, 66342], [2297, 1477, 63657], [2299, 1781, 66852], [2301, 671, 3203], [2390, 1241, 23276], [2393, 2370, 24924], [2395, 588, 4120], [2403, 2047, 1127], [2404, 852, 86892], [2408, 287, 13695], [2411, 2124, 90507], [2413, 1701, 74287], [2417, 323, 92535], [2422, 740, 27284], [2427, 1756, 48670], [2430, 2376, 90645], [2432, 1781, 85273], [2438, 172, 49536], [2440, 707, 21260], [2444, 1472, 2781], [2451, 1443, 269], [2454, 1776, 60842], [2459, 698, 22681], [2462, 2099, 85785], [2468, 575, 89197], [2471, 2105, 69847], [46, 785, 39904], [144, 697, 19893], [147, 907, 56127], [157, 1709, 48189], [161, 1265, 89934], [166, 698, 66369], [168, 1756, 29590], [176, 2026, 3754], [186, 1765, 9509], [195, 2279, 79715], [198, 2087, 47434], [200, 303, 47336], [202, 809, 69146], [204, 2127, 40688], [211, 1781, 28761], [214, 2099, 40084], [216, 2276, 82437], [218, 916, 42658], [222, 2117, 80995], [224, 2367, 34338], [226, 2124, 40008], [230, 364, 63149], [381, 2096, 31739], [385, 867, 89392], [388, 2122, 63529], [399, 575, 36936], [404, 835, 63126], [411, 293, 31083], [413, 755, 53705], [416, 2156, 56687], [423, 153, 88308], [430, 2451, 22839], [433, 1211, 89481], [439, 168, 6691], [443, 147, 31223], [448, 1510, 23396], [449, 1518, 69334], [453, 448, 91478], [459, 2390, 16589], [460, 707, 8928], [463, 2281, 143], [467, 580, 16663], [471, 939, 81576], [472, 430, 70081], [630, 443, 24489], [631, 2299, 2672], [634, 271, 69484], [636, 2381, 59446], [641, 1469, 52696], [648, 1720, 84846], [652, 2422, 94453], [654, 2106, 46202], [661, 2117, 43707], [664, 754, 80290], [665, 718, 46972], [668, 543, 49175], [669, 1710, 96266], [674, 569, 95130], [676, 797, 35117], [679, 2140, 94893], [682, 1707, 13503], [686, 2462, 90696], [690, 2156, 39992], [874, 2413, 26948], [877, 1229, 27388], [881, 2259, 12535], [883, 2158, 12371], [885, 1265, 39834], [886, 1732, 75407], [888, 1389, 70914], [892, 2378, 54875], [893, 577, 20308], [896, 2408, 85175], [897, 730, 60506], [899, 218, 27167], [900, 1776, 61452], [903, 1688, 35717], [906, 648, 29093], [911, 433, 31228], [913, 195, 19151], [915, 1681, 97610], [919, 459, 21279], [920, 1443, 46233], [924, 848, 69736], [927, 1724, 31508], [931, 834, 44004], [936, 224, 3962], [940, 939, 79612], [943, 289, 4250], [946, 609, 97829], [1058, 1527, 1066], [1059, 2444, 7403], [1060, 543, 37692], [1063, 669, 22687], [1067, 449, 56506], [1070, 282, 52921], [1072, 472, 96586], [1078, 835, 5731], [1083, 1067, 57131], [1085, 848, 96517], [1086, 2269, 99809], [1088, 198, 68589], [1091, 1467, 71272], [1093, 1662, 57985], [1094, 737, 27813], [1096, 1252, 70912], [1100, 2371, 86659], [1106, 2370, 67407], [1107, 2000, 37718], [1112, 1083, 12823], [1115, 2131, 26811], [1117, 2404, 91098], [1119, 155, 52001], [1210, 1748, 21530], [1221, 193, 37370], [1224, 308, 37112], [1235, 1094, 74920], [1239, 881, 91805], [1245, 308, 80731], [1254, 1770, 44481], [1257, 2150, 7873], [1259, 741, 21003], [1270, 867, 47200], [1272, 707, 74453], [1276, 157, 14129], [1287, 935, 58054], [1288, 669, 49276], [1290, 1779, 56280], [1295, 800, 9619], [1298, 665, 78638], [1311, 2262, 10712], [1315, 2454, 66020], [1319, 1720, 80295], [1321, 2297, 28690], [1325, 198, 83992], [1444, 1259, 55730], [1452, 935, 7169], [1456, 1996, 56690], [1460, 202, 85799], [1468, 1479, 3931], [1488, 779, 24810], [1490, 2142, 77452], [1494, 2001, 32911], [1501, 1680, 42964], [1507, 735, 31069], [1516, 1212, 41219], [1517, 1745, 2034], [1519, 676, 40486], [1523, 1456, 62129], [1528, 1321, 7965], [1529, 1239, 91508], [1531, 2364, 47778], [1534, 1754, 10275], [1663, 2413, 94898], [1668, 1732, 60062], [1673, 1688, 93689], [1677, 2124, 13272], [1682, 1100, 78544], [1689, 1100, 47387], [1694, 807, 4860], [1698, 2281, 6421], [1708, 776, 76139], [1719, 1501, 2364], [1721, 1315, 78385], [1725, 1990, 29835], [1736, 2328, 41852], [1741, 2241, 63462], [1751, 781, 93519], [1757, 580, 37296], [1761, 1117, 10557], [1766, 2098, 65851], [1771, 703, 99995], [1772, 472, 12483], [1777, 2096, 37164], [1783, 1495, 14026], [1931, 1702, 29444], [1934, 1228, 7819], [1936, 1218, 82004], [1939, 1458, 47984], [1945, 1469, 29448], [1946, 416, 31385], [1949, 679, 63095], [1951, 905, 98741], [1954, 703, 54784], [1958, 1444, 10447], [1960, 1995, 72751], [1962, 2049, 45488], [1964, 2037, 75688], [1966, 153, 10869], [1968, 1088, 81730], [1970, 1757, 29497], [1971, 1523, 37332], [1975, 931, 42589], [1978, 1273, 53733], [1983, 460, 52573], [2162, 943, 7587], [2166, 682, 19826], [2170, 931, 41327], [2171, 1756, 70389], [2175, 936, 26277], [2181, 765, 39546], [2183, 147, 66509], [2193, 1681, 98920], [2197, 941, 19089], [2202, 2236, 35303], [2208, 881, 4232], [2211, 1741, 54115], [2213, 226, 45857], [2217, 1710, 49621], [2221, 714, 349], [2224, 1443, 63796], [2227, 595, 65490], [2229, 809, 40970], [2237, 144, 52376], [2382, 2289, 34623], [2392, 1954, 98601], [2397, 573, 1562], [2399, 186, 9795], [2414, 1663, 69249], [2416, 943, 46677], [2419, 1260, 78156], [2423, 823, 99773], [2426, 172, 10140], [2436, 453, 15448], [2441, 2000, 40427], [2450, 1468, 43109], [2452, 1677, 19523], [2457, 2404, 10769], [2466, 2374, 37047], [2472, 740, 17012], [56, 741, 60120], [59, 668, 6555], [61, 907, 30469], [63, 2436, 6630], [68, 1452, 76908], [72, 337, 24572], [78, 147, 71619], [79, 1443, 76514], [82, 2356, 32379], [84, 2462, 16104], [90, 1698, 30853], [96, 1721, 31911], [99, 549, 59986], [105, 1246, 91902], [110, 1694, 32858], [115, 282, 49924], [118, 676, 79879], [121, 160, 54271], [126, 877, 74548], [130, 1703, 92122], [131, 2382, 5796], [132, 735, 27394], [327, 2115, 64291], [328, 1067, 92395], [332, 155, 54614], [335, 2142, 8970], [338, 199, 88394], [339, 1249, 77577], [341, 222, 97694], [345, 1975, 84105], [346, 2404, 47890], [352, 781, 8476], [355, 1662, 34665], [356, 160, 93412], [361, 2276, 29571], [362, 575, 81957], [367, 2120, 52023], [368, 82, 16762], [373, 1958, 21122], [377, 692, 63619], [380, 1782, 65787], [542, 364, 31441], [546, 2136, 986], [548, 758, 54773], [550, 1964, 70162], [553, 781, 57184], [556, 72, 90143], [559, 2131, 91283], [564, 367, 13805], [567, 2217, 7617], [572, 2408, 87674], [576, 807, 23108], [578, 703, 98725], [581, 907, 50697], [583, 1753, 48048], [584, 121, 60324], [587, 1719, 93321], [591, 1966, 38489], [596, 204, 5394], [601, 2183, 12787], [603, 2127, 23866], [606, 2120, 90217], [607, 2117, 82553], [610, 581, 7973], [612, 200, 76527], [614, 226, 56597], [2349, 2472, 47951], [92, 1757, 83025], [638, 559, 41159], [616, 1112, 18057], [618, 1684, 30510], [620, 1992, 57167], [621, 460, 7280], [622, 1710, 95340], [625, 153, 42062], [756, 741, 20734], [766, 781, 83850], [770, 1753, 64698], [774, 636, 68193], [789, 299, 75048], [791, 913, 39739], [792, 765, 22937], [796, 2217, 27970], [805, 380, 75522], [808, 801, 43061], [811, 604, 14018], [819, 1741, 96330], [821, 2459, 44728], [830, 1718, 23323], [831, 927, 5042], [836, 709, 54984], [851, 2417, 34668], [858, 1528, 52639], [869, 834, 5296], [871, 1674, 80879], [1007, 1696, 73533], [1008, 1222, 83354], [1011, 1725, 50680], [1017, 2259, 48772], [1020, 1534, 73895], [1021, 195, 24064], [1027, 1778, 14790], [1028, 399, 85873], [1031, 199, 28579], [1035, 1971, 32647], [1043, 1490, 12964], [1050, 2026, 2904], [1054, 2021, 50018], [1120, 1243, 82885], [1124, 306, 99206], [1126, 328, 63921], [1128, 1688, 66754], [1131, 362, 66462], [1134, 294, 96234], [1137, 559, 26666], [1139, 1748, 57581], [1144, 1735, 30112], [1145, 625, 10459], [1146, 2046, 44090], [1150, 2018, 60197], [1159, 1311, 71198], [1161, 299, 54976], [1163, 1252, 55682], [1166, 2416, 59409], [1167, 1146, 89738], [1172, 900, 73013], [1183, 1115, 93100], [1188, 906, 79377], [1190, 811, 69952], [1193, 1771, 23610], [1199, 703, 16640], [1201, 1753, 59724], [1204, 616, 84764], [1387, 1951, 45024], [1393, 2419, 62081], [1398, 584, 79489], [1401, 915, 39178], [1402, 1752, 74206], [1405, 877, 63863], [1410, 559, 66011], [1411, 835, 82499], [1416, 2395, 60177], [1417, 768, 48320], [1420, 335, 58341], [1423, 2125, 56620], [1424, 1199, 82414], [1427, 2065, 10017], [1432, 1126, 23275], [1433, 1996, 34075], [1438, 612, 62803], [1440, 118, 43520], [1442, 285, 52854], [1579, 1741, 51614], [1580, 1433, 89488], [1585, 816, 66757], [1587, 553, 44073], [1589, 1996, 85687], [1595, 2390, 29730], [1597, 2139, 90839], [1600, 847, 41194], [1602, 352, 24041], [1606, 630, 43968], [1607, 1585, 65842], [1610, 2392, 43786], [1613, 614, 18594], [1617, 923, 74456], [1618, 851, 41188], [1622, 2037, 84327], [1628, 1287, 36410], [1630, 147, 35702], [1631, 1060, 34017], [1638, 2153, 85347], [1640, 888, 29166], [1645, 82, 96409], [1647, 2299, 8271], [1657, 1681, 8017], [1845, 467, 23508], [1851, 2124, 32942], [1854, 923, 33343], [1858, 126, 17269], [1859, 899, 976], [1860, 1949, 83076], [1868, 576, 43856], [1870, 1246, 96287], [1874, 634, 2662], [1879, 861, 84453], [1882, 924, 29772], [1886, 587, 33370], [1894, 2293, 75637], [1903, 1131, 13487], [1905, 2411, 48949], [1907, 1210, 47795], [1908, 1433, 93754], [1912, 1490, 28897], [1914, 576, 12717], [1919, 1723, 68809], [1921, 1479, 16171], [1922, 2374, 88933], [1924, 638, 87030], [2054, 2390, 4278], [2055, 1778, 25616], [2057, 1389, 61937], [2061, 1589, 56804], [2063, 147, 80587], [2070, 935, 38880], [2072, 1521, 57168], [2076, 789, 61736], [2079, 144, 84984], [2080, 2224, 52546], [2085, 118, 5940], [2090, 1667, 29613], [2094, 1962, 39580], [2100, 572, 27998], [2103, 1725, 61685], [2108, 230, 74288], [2109, 1756, 20328], [2114, 285, 2969], [2119, 1438, 23636], [2126, 1735, 5032], [2132, 1733, 79476], [2138, 1716, 79966], [2146, 714, 61231], [2155, 1761, 32449], [2306, 1894, 32972], [2309, 1845, 6861], [2310, 463, 90408], [2312, 1707, 29763], [2316, 1494, 30081], [2319, 165, 19984], [2320, 131, 57495], [2322, 621, 63029], [2329, 1290, 10344], [2334, 833, 20088], [2335, 1502, 53171], [2341, 287, 38445], [2344, 2237, 69892], [2348, 1438, 70080], [2351, 170, 88174], [2353, 2221, 27401], [2359, 439, 43755], [2366, 1782, 30755], [2372, 842, 42015], [2377, 823, 49610], [2379, 2356, 69359], [1, 1606, 46559], [2, 1766, 76517], [4, 1736, 16816], [7, 829, 63889], [8, 831, 39489], [9, 690, 17797], [10, 2312, 68888], [12, 631, 52521], [13, 1699, 44975], [17, 411, 9503], [21, 1771, 14523], [22, 885, 77685], [23, 595, 28309], [26, 13, 17487], [28, 193, 60064], [30, 1958, 76168], [36, 2105, 83440], [38, 2001, 82745], [41, 906, 75405], [45, 2102, 82402], [48, 168, 66823], [235, 1606, 48085], [245, 289, 90079], [248, 2250, 5616], [249, 2156, 68623], [251, 2175, 19385], [254, 871, 46757], [261, 82, 55296], [274, 546, 30375], [280, 1483, 91717], [284, 766, 67665], [288, 1607, 52548], [298, 835, 63235], [311, 634, 96433], [315, 1882, 99651], [320, 2124, 76994], [324, 1510, 42643], [477, 1782, 12815], [478, 730, 25495], [480, 2349, 57217], [481, 863, 42236], [484, 1757, 75626], [487, 1257, 26946], [493, 2471, 56242], [496, 1695, 79397], [498, 1628, 53379], [500, 588, 23336], [503, 153, 6950], [505, 230, 696], [510, 1759, 65835], [512, 2211, 61351], [514, 1458, 89541], [515, 2251, 68950], [519, 188, 32748], [520, 2229, 81843], [522, 686, 4490], [525, 587, 72524], [526, 1495, 70036], [528, 214, 52589], [531, 899, 90447], [532, 791, 47774], [533, 1393, 54053], [536, 559, 40525], [538, 2225, 46144], [693, 99, 62853], [694, 559, 54067], [696, 596, 67504], [706, 1868, 80567], [722, 789, 22963], [725, 362, 56242], [727, 607, 6574], [736, 592, 70722], [738, 1647, 45292], [744, 588, 52359], [746, 1613, 39501], [748, 520, 32544], [750, 204, 56884], [947, 315, 41796], [953, 664, 49328], [956, 1970, 11891], [961, 335, 83820], [966, 692, 21744], [967, 1602, 87699], [972, 565, 78040], [976, 830, 71910], [978, 1519, 74307], [981, 1720, 17008], [990, 897, 63190], [993, 573, 41854], [998, 41, 19567], [999, 1393, 96123], [1002, 1060, 36096], [1130, 896, 81426], [1132, 2065, 90421], [1135, 1970, 16556], [1140, 268, 81131], [1141, 1983, 31427], [1143, 22, 89625], [1149, 820, 12035], [1151, 1211, 79609], [1156, 1188, 5052], [1157, 1389, 65741], [1162, 2070, 71008], [1165, 1585, 7224], [1171, 1939, 78199], [1176, 188, 29946], [1177, 30, 8212], [1180, 325, 41003], [1181, 2356, 90961], [1186, 2417, 80313], [1196, 1699, 89332], [1198, 7, 48982], [1200, 1631, 39129], [1203, 1134, 86112], [1207, 2419, 27089], [1328, 2079, 34766], [1331, 2139, 61973], [1335, 1270, 84557], [1336, 1172, 16913], [1340, 828, 59548], [1341, 1134, 32759], [1343, 121, 38234], [1344, 2227, 39254], [1346, 1063, 43711], [1352, 578, 28940], [1355, 796, 33981], [1363, 503, 21487], [1365, 2148, 84244], [1369, 1507, 57088], [1371, 2306, 83288], [1372, 1440, 56168], [1374, 1091, 17644], [1377, 131, 42801], [1380, 828, 57027], [1381, 1945, 96016], [1382, 2112, 26996], [1540, 1446, 75056], [1541, 2079, 13665], [1542, 2367, 88854], [1546, 2269, 23074], [1548, 2250, 2301], [1550, 1934, 12625], [1551, 692, 84195], [1552, 1200, 16484], [1553, 121, 31998], [1555, 2197, 18504], [1557, 254, 99762], [1558, 543, 39921], [1559, 481, 87827], [1561, 855, 27048], [1563, 1745, 94118], [1565, 2115, 87086], [1566, 1962, 72420], [1568, 1945, 44397], [1570, 755, 15100], [1572, 709, 80135], [1577, 248, 58950], [1785, 289, 81901], [1789, 1377, 4709], [1792, 254, 39726], [1793, 2268, 7736], [1794, 1870, 54710], [1797, 288, 62001], [1798, 2181, 29773], [1800, 1141, 65858], [1803, 2417, 95922], [1811, 2452, 2412], [1813, 2348, 68224], [1815, 1021, 83889], [1817, 1501, 79657], [1818, 936, 93207], [1827, 453, 40377], [1833, 883, 58237], [1839, 1241, 29188], [1840, 1617, 25838], [1988, 2468, 56007], [1993, 990, 60140], [1997, 263, 79481], [2004, 783, 27911], [2005, 1199, 3582], [2007, 1420, 49154], [2010, 1050, 87652], [2014, 1017, 73281], [2016, 10, 36274], [2020, 1440, 61660], [2023, 1145, 32181], [2025, 195, 29200], [2030, 2090, 86665], [2032, 2289, 30738], [2034, 335, 70415], [2038, 638, 3994], [2041, 377, 80126], [2043, 214, 71618], [2048, 200, 45588], [2051, 2010, 55626], [2053, 1140, 31588], [2243, 722, 31789], [2247, 1703, 17302], [2249, 1444, 12746], [2252, 1886, 37508], [2253, 2382, 4480], [2255, 1257, 58719], [2256, 2010, 65512], [2257, 1416, 28926], [2260, 953, 45187], [2261, 1860, 78056], [2263, 323, 97799], [2265, 1198, 76020], [2267, 893, 75005], [2270, 885, 1316], [2271, 2243, 81828], [2273, 781, 52498], [2278, 1732, 33525], [2280, 2051, 31623], [2283, 2063, 43117], [2284, 696, 5354], [2285, 413, 7771], [2291, 768, 88131], [2292, 2146, 35519], [2294, 90, 64864], [2298, 805, 25739], [2300, 2005, 13813], [2302, 2260, 42627], [3, 2253, 40231], [6, 45, 87466], [11, 251, 13624], [14, 368, 79724], [16, 1145, 67808], [18, 1145, 52411], [19, 1131, 43981], [24, 311, 68647], [25, 792, 65029], [27, 222, 9544], [29, 90, 38297], [34, 602, 63983], [35, 727, 34481], [40, 1295, 55087], [44, 254, 11747], [47, 1523, 79129], [49, 2127, 32580], [50, 2393, 20877], [51, 328, 71548], [145, 1548, 46879], [156, 953, 5783], [179, 150, 3442], [184, 46, 71950], [187, 2211, 94219], [201, 423, 98335], [203, 1589, 77875], [207, 531, 89787], [208, 1100, 5089], [212, 235, 1950], [215, 2093, 16928], [225, 883, 55387], [228, 2310, 938], [382, 105, 26394], [384, 1874, 60967], [387, 2450, 27009], [391, 72, 82089], [393, 546, 46433], [394, 1107, 24708], [397, 1439, 3945], [401, 2109, 56941], [405, 2349, 64853], [408, 774, 94888], [409, 781, 70744], [417, 1490, 21963], [419, 706, 45436], [422, 905, 64359], [425, 789, 19091], [427, 1566, 87745], [431, 1886, 79986], [432, 1377, 71054], [436, 2085, 1784], [438, 168, 305], [450, 40, 35470], [454, 510, 19927], [456, 10, 34071], [461, 463, 25875], [465, 1382, 33904], [468, 2422, 35240], [469, 900, 96757], [633, 993, 99260], [635, 520, 21105], [643, 1063, 56914], [645, 68, 645], [647, 2083, 96687], [649, 1745, 43050], [651, 9, 99373], [655, 1171, 83226], [657, 2076, 56519], [659, 1124, 47035], [660, 2049, 75986], [663, 1903, 81904], [670, 944, 99363], [678, 2025, 72480], [680, 2113, 45907], [683, 448, 84406], [685, 2012, 70180], [688, 848, 19553], [872, 2072, 8386], [875, 2083, 252], [876, 1162, 97365], [878, 2014, 37848], [884, 2247, 37118], [894, 195, 39041], [901, 375, 99946], [908, 99, 41996], [918, 380, 27106], [921, 2432, 97036], [926, 1070, 77664], [929, 449, 4037], [932, 885, 32401], [934, 510, 85459], [942, 367, 27207], [1057, 1845, 57134], [1062, 2076, 28041], [1066, 2370, 77491], [1069, 1551, 65580], [1071, 920, 87864], [1075, 1988, 15442], [1079, 224, 5407], [1081, 1096, 73219], [1092, 1833, 57141], [1095, 1949, 52713], [1099, 1690, 43624], [1101, 315, 32118], [1103, 838, 47822], [1104, 1514, 29408], [1108, 280, 78049], [1111, 823, 50714], [1223, 815, 86472], [1231, 387, 68476], [1234, 531, 38085], [1236, 1534, 81822], [1244, 1521, 14775], [1250, 1553, 67989], [1258, 683, 52152], [1262, 2366, 92443], [1269, 1410, 37819], [1278, 676, 85861], [1280, 1689, 1976], [1282, 1772, 59940], [1294, 1519, 43078], [1297, 1983, 9528], [1299, 1328, 5966], [1302, 1094, 28826], [1303, 1555, 77664], [1312, 1352, 24149], [1320, 2123, 72484], [1323, 881, 37842], [1447, 1501, 91619], [1450, 34, 67223], [1457, 1741, 78596], [1462, 779, 74695], [1464, 2459, 33923], [1465, 2020, 43447], [1470, 1276, 36399], [1474, 2150, 49664], [1475, 427, 69054], [1481, 1236, 61936], [1492, 1726, 49966], [1496, 235, 42409], [1505, 526, 50699], [1508, 1172, 13183], [1524, 1211, 50778], [1532, 1757, 92462], [1536, 1363, 10492], [1659, 90, 35982], [1661, 2131, 15974], [1666, 828, 67054], [1670, 1141, 86577], [1672, 1249, 7191], [1676, 572, 98932], [1683, 577, 28736], [1692, 2018, 13772], [1697, 792, 7096], [1704, 2007, 54452], [1711, 1958, 19943], [1714, 2053, 35404], [1722, 1717, 96687], [1727, 1302, 26920], [1734, 1108, 458], [1740, 6, 39277], [1744, 216, 57936], [1747, 1140, 3756], [1750, 1229, 52759], [1758, 2471, 38159], [1762, 1677, 69176], [1773, 900, 86148], [1927, 1369, 19519], [1930, 79, 29015], [1932, 1460, 63642], [1935, 2083, 4870], [1937, 655, 43005], [1938, 2134, 57183], [1942, 897, 16655], [1943, 453, 1151], [1944, 2266, 98617], [1948, 303, 89028], [1952, 1678, 79083], [1956, 601, 13599], [1957, 671, 94755], [1959, 480, 1253], [1963, 96, 42950], [1967, 144, 99864], [1969, 2344, 60237], [1972, 789, 44646], [1976, 2122, 27747], [1980, 738, 89232], [1981, 2113, 48528], [1985, 2093, 98831], [2161, 1099, 37847], [2168, 362, 74462], [2172, 1262, 97605], [2173, 2227, 45431], [2186, 993, 38924], [2189, 2441, 73172], [2198, 335, 55698], [2206, 1978, 86420], [2209, 858, 33124], [2215, 1111, 27023], [2216, 1043, 41818], [2219, 575, 22051], [2222, 1647, 91198], [2231, 2126, 55308], [2384, 1320, 8405], [2387, 78, 65093], [2394, 2393, 54963], [2398, 2132, 95585], [2401, 774, 27071], [2406, 1096, 49396], [2410, 1507, 29232], [2421, 160, 90311], [2429, 1761, 14562], [2437, 1745, 77705], [2442, 166, 57376], [2445, 2441, 58445], [2448, 41, 97281], [2449, 649, 21227], [2455, 1756, 25219], [2463, 2040, 26123], [2465, 1433, 19269], [2469, 1365, 36245], [55, 713, 55653], [57, 2278, 72465], [62, 1235, 46922], [66, 2263, 9719], [70, 774, 29885], [76, 961, 47172], [81, 591, 39632], [86, 235, 69567], [88, 1469, 17907], [102, 1103, 21517], [104, 1565, 32961], [108, 269, 50629], [112, 126, 88768], [114, 1680, 61], [119, 630, 3293], [123, 2374, 17187], [127, 1870, 81262], [137, 46, 79260], [138, 449, 29515], [140, 563, 47097], [329, 86, 33223], [330, 2288, 86813], [334, 943, 42502], [336, 51, 75460], [340, 1377, 74267], [343, 1124, 56370], [344, 657, 41896], [347, 1732, 92434], [349, 1502, 8121], [351, 1734, 28006], [353, 268, 19345], [354, 1797, 44573], [359, 299, 34942], [360, 23, 5950], [365, 1962, 2090], [369, 1980, 88550], [371, 1565, 48699], [372, 1436, 4514], [376, 2251, 3184], [378, 349, 77394], [379, 1062, 7405], [545, 2348, 29788], [551, 1432, 36518], [554, 1729, 88711], [560, 894, 37324], [562, 1249, 73937], [568, 756, 19147], [570, 2153, 57215], [586, 2298, 24217], [593, 422, 26681], [600, 328, 37182], [605, 1981, 82393], [613, 2270, 32192], [615, 1221, 39154], [619, 505, 70901], [629, 1257, 77526], [760, 1964, 12651], [763, 1750, 82223], [764, 1017, 79693], [773, 796, 42181], [782, 652, 52425], [784, 981, 66822], [788, 168, 1686], [795, 1442, 65306], [799, 1502, 37170], [804, 1295, 77594], [810, 2219, 89385], [818, 90, 15455], [832, 663, 99751], [841, 1815, 62729], [844, 1754, 92741], [846, 2270, 98670], [856, 831, 73786], [862, 1580, 6316], [866, 999, 52349], [1004, 2419, 97209], [1005, 816, 67989], [1010, 1617, 35613], [1012, 919, 83583], [1013, 2280, 13897], [1016, 1295, 64767], [1019, 1199, 42027], [1022, 2048, 74106], [1023, 768, 8917], [1024, 349, 97126], [1026, 1970, 21924], [1030, 789, 78273], [1032, 1387, 56058], [1034, 770, 44013], [1037, 1259, 38845], [1039, 919, 27218], [1040, 2292, 91012], [1044, 2229, 65079], [1046, 2270, 13174], [1048, 2234, 91914], [1052, 690, 70268], [1055, 138, 43547], [1215, 1770, 95441], [1219, 1922, 18636], [1230, 2090, 90762], [1232, 1728, 58776], [1238, 381, 22743], [1242, 1135, 64933], [1251, 1058, 83847], [1253, 1975, 11468], [1267, 156, 39535], [1268, 2379, 25968], [1274, 2316, 66997], [1279, 2206, 98726], [1284, 2316, 79867], [1291, 2312, 81058], [1296, 613, 12658], [1304, 2108, 71667], [1309, 1483, 94781], [1310, 863, 37981], [1314, 543, 18786], [1316, 344, 88759], [1322, 1963, 78111], [1388, 2406, 13819], [1390, 1563, 43111], [1394, 459, 20474], [1396, 807, 3246], [1399, 1765, 45958], [1400, 176, 53648], [1404, 1143, 33187], [1407, 2215, 83810], [1408, 676, 87228], [1412, 1540, 45312], [1413, 2278, 63390], [1414, 355, 92916], [1415, 12, 47883], [1422, 1461, 53860], [1426, 915, 8672], [1429, 1839, 86785], [1430, 487, 65901], [1431, 481, 51262], [1434, 614, 80207], [1582, 785, 75222], [1583, 2322, 27666], [1586, 2229, 57802], [1590, 1765, 33633], [1596, 459, 50388], [1598, 2414, 23704], [1599, 2371, 74370], [1605, 1936, 77667], [1609, 1234, 5221], [1612, 1166, 43471], [1621, 1492, 61996], [1625, 823, 76535], [1627, 1162, 26831], [1632, 1022, 8445], [1636, 2103, 29489], [1637, 168, 20237], [1639, 453, 44931], [1643, 674, 92900], [1653, 2382, 24770], [1656, 1331, 77359], [1844, 1267, 74314], [1848, 1446, 17531], [1852, 2010, 77686], [1855, 1970, 16228], [1863, 956, 19946], [1869, 1012, 94200], [1871, 1103, 3481], [1876, 1401, 17914], [1883, 505, 1114], [1888, 1631, 61869], [1890, 1321, 98166], [1893, 155, 80127], [1896, 1069, 47055], [1902, 1696, 89446], [1909, 1272, 88789], [1915, 1162, 83878], [1916, 595, 67068], [1923, 1510, 64810], [2060, 1555, 80527], [2064, 38, 88069], [2069, 2423, 52479], [2071, 2376, 50371], [2073, 211, 23137], [2088, 1012, 32526], [2091, 944, 34004], [2095, 1048, 64978], [2104, 1668, 93286], [2110, 1637, 34363], [2116, 705, 37193], [2128, 981, 9181], [2133, 1968, 8918], [2135, 168, 32415], [2145, 2250, 44129], [2147, 2021, 45003], [2149, 805, 6625], [2151, 251, 59018], [2157, 910, 57721], [2303, 528, 85166], [2305, 1010, 65677], [2308, 127, 41424], [2314, 2322, 86664], [2317, 361, 57113], [2318, 863, 85420], [2324, 28, 21810], [2326, 2157, 52942], [2330, 365, 51677], [2336, 2021, 77734], [2339, 153, 61294], [2342, 1531, 56154], [2345, 1815, 29865], [2346, 1800, 907], [2354, 916, 7889], [2360, 1914, 58732], [2363, 693, 79242], [2369, 1246, 75829], [2375, 1742, 67049], [2380, 11, 1592], [54, 1265, 95786], [58, 1278, 24490], [65, 610, 85484], [67, 665, 77635], [69, 2031, 37718], [74, 2234, 4761], [77, 176, 12374], [83, 2451, 11572], [85, 1863, 1816], [87, 1054, 82955], [89, 792, 43306], [94, 1969, 72473], [97, 77, 87446], [98, 193, 27840], [100, 1570, 37426], [107, 1766, 23912], [109, 686, 82520], [113, 727, 96068], [117, 35, 55136], [122, 1687, 86300], [125, 1246, 69768], [128, 765, 28291], [134, 1527, 53655], [135, 573, 32168], [136, 109, 38422], [139, 2257, 96425], [233, 1833, 76632], [236, 202, 96901], [238, 832, 48295], [241, 915, 11112], [243, 1586, 277], [247, 554, 91888], [250, 2339, 2254], [256, 1115, 52498], [260, 906, 50834], [266, 1210, 54532], [270, 791, 84466], [272, 1934, 14627], [281, 651, 14495], [291, 592, 27442], [296, 1770, 7948], [297, 413, 49316], [304, 1267, 79626], [305, 1583, 60102], [310, 1536, 42394], [322, 1424, 27468], [475, 2455, 42013], [483, 1557, 64794], [489, 976, 28827], [490, 1031, 6090], [491, 2335, 66047], [495, 1815, 54962], [497, 1597, 96608], [499, 14, 46048], [501, 1529, 37266], [504, 55, 30408], [506, 671, 26769], [507, 924, 74818], [518, 1692, 68974], [523, 797, 51877], [527, 266, 17931], [530, 405, 74916], [535, 2384, 16680], [537, 1328, 35729], [541, 907, 98928], [699, 1687, 85916], [701, 1432, 38150], [711, 1315, 29668], [717, 2132, 56368], [719, 961, 58075], [721, 916, 69253], [724, 972, 6161], [729, 2411, 51819], [732, 118, 46093], [734, 198, 54299], [742, 110, 14478], [743, 11, 5109], [745, 1112, 71181], [747, 2252, 15817], [749, 841, 80072], [751, 338, 74711], [752, 193, 64622], [753, 2229, 93265], [949, 1632, 60543], [952, 1636, 72127], [954, 2011, 27712], [957, 670, 1410], [959, 260, 16310], [962, 1531, 4071], [963, 872, 3099], [964, 1811, 73169], [969, 327, 78551], [971, 324, 62568], [974, 1229, 21116], [977, 785, 58317], [980, 280, 57855], [982, 829, 65994], [984, 1244, 91291], [985, 568, 47920], [986, 1792, 3364], [987, 2085, 39836], [988, 1740, 24790], [991, 789, 61865], [994, 3, 6234], [996, 2136, 33000], [997, 967, 43408], [1121, 2381, 32464], [1125, 327, 86602], [1129, 855, 61679], [1133, 709, 10910], [1142, 2263, 7689], [1154, 436, 32927], [1155, 2257, 77445], [1158, 562, 64029], [1168, 1719, 7387], [1170, 105, 63811], [1174, 2161, 29992], [1179, 2377, 54374], [1182, 2244, 78783], [1185, 908, 77763], [1189, 1811, 64587], [1191, 682, 36166], [1192, 1548, 41809], [1197, 1676, 20745], [1202, 709, 23467], [1206, 2260, 68460], [1208, 1703, 81702], [1327, 1694, 21373], [1330, 1299, 47377], [1332, 671, 63391], [1333, 939, 70491], [1334, 1674, 85392], [1339, 2085, 23659], [1347, 2273, 70857], [1350, 1502, 12881], [1351, 2322, 13195], [1354, 1320, 73572], [1357, 1214, 4191], [1358, 2206, 88798], [1360, 962, 22906], [1362, 2181, 60074], [1364, 735, 53898], [1368, 1192, 8944], [1370, 184, 18707], [1373, 706, 27129], [1376, 1251, 95229], [1379, 369, 89509], [1383, 2231, 32195], [1384, 746, 27214], [1543, 1871, 81072], [1544, 2138, 73489], [1545, 1844, 25397], [1547, 241, 82341], [1549, 2202, 80009], [1554, 2227, 19999], [1556, 330, 41243], [1560, 2237, 53675], [1562, 495, 84818], [1564, 829, 84877], [1567, 1450, 42945], [1569, 361, 38094], [1571, 1967, 74968], [1573, 280, 76348], [1574, 27, 41404], [1575, 1190, 59921], [1576, 1996, 92885], [1786, 2079, 44622], [1788, 1995, 92351], [1790, 1756, 28089], [1791, 1507, 99775], [1796, 1852, 67022], [1801, 226, 33894], [1805, 88, 83421], [1807, 215, 37277], [1808, 328, 89040], [1809, 838, 56072], [1814, 688, 19675], [1819, 947, 15443], [1821, 1383, 6588], [1825, 1621, 13194], [1829, 779, 37876], [1830, 591, 71611], [1832, 7, 98562], [1834, 233, 25398], [1835, 1368, 54325], [1836, 70, 93158], [1838, 1492, 81026], [1841, 1777, 53355], [1986, 1516, 1611], [1989, 207, 99311], [1991, 2406, 89435], [1994, 123, 69074], [1998, 364, 267], [1999, 2035, 95869], [2003, 2459, 79487], [2008, 1288, 73326], [2009, 1268, 86118], [2013, 1380, 95800], [2015, 1474, 33715], [2017, 2296, 51936], [2019, 1638, 53442], [2022, 1710, 54306], [2024, 1282, 70621], [2027, 1094, 32138], [2028, 2151, 9399], [2036, 1394, 8402], [2039, 1020, 32204], [2042, 2276, 71799], [2044, 467, 26041], [2045, 1203, 94055], [2050, 404, 74525], [2052, 2330, 76596], [2240, 876, 82944], [2242, 144, 32981], [2245, 1839, 6457], [2246, 952, 83851], [2254, 715, 8550], [2258, 1939, 6232], [2272, 2139, 9121], [2274, 1099, 20122], [2275, 1859, 45595], [2277, 2374, 6340], [2282, 2325, 26164], [2286, 1510, 94845], [2290, 2452, 68855], [2295, 1919, 169], [5, 738, 81617], [15, 1155, 29846], [20, 654, 77306], [31, 2023, 65562], [32, 848, 27441], [33, 570, 2756], [37, 1971, 31125], [39, 1295, 64202], [42, 1219, 78990], [43, 1983, 55293], [52, 1989, 20818], [143, 750, 56323], [148, 549, 20705], [149, 504, 95923], [151, 1572, 24855], [159, 1818, 52017], [162, 1533, 58914], [167, 1311, 11630], [171, 212, 39960], [173, 1706, 82356], [175, 1698, 11807], [177, 2076, 41374], [180, 1151, 96646], [182, 2106, 45674], [185, 2367, 80440], [191, 1467, 32028], [196, 861, 83916], [205, 2237, 79739], [209, 1762, 1547], [210, 2316, 94539], [217, 114, 82962], [219, 371, 89919], [220, 323, 41101], [383, 1809, 88221], [386, 2100, 64053], [392, 1590, 14696], [395, 1002, 27753], [400, 13, 99994], [402, 247, 11211], [410, 303, 47657], [415, 2161, 88755], [418, 2367, 4378], [420, 1523, 77262], [424, 1404, 67334], [426, 1394, 91988], [428, 833, 3505], [434, 919, 80341], [437, 2110, 76686], [440, 2, 35797], [442, 25, 60716], [444, 439, 76699], [447, 1751, 53458], [452, 501, 61650], [455, 962, 10714], [457, 433, 42608], [464, 727, 14340], [470, 1388, 94547], [474, 423, 26238], [632, 1758, 16938], [639, 939, 26568], [640, 218, 27499], [673, 874, 47533], [677, 352, 46300], [687, 1295, 41583], [691, 1569, 35417], [873, 2023, 67201], [879, 542, 51183], [880, 1517, 62572], [882, 2076, 29894], [887, 477, 74715], [889, 490, 93557], [890, 1876, 61101], [891, 108, 52269], [895, 942, 1983], [898, 1402, 26434], [902, 1764, 2485], [904, 1704, 65894], [909, 434, 25760], [912, 1705, 53921], [914, 2007, 96371], [917, 2305, 3169], [922, 1995, 23248], [928, 1372, 24471], [930, 1432, 31078], [933, 82, 97707], [938, 381, 48482], [945, 405, 11967], [1061, 2015, 30939], [1064, 2073, 11228], [1065, 654, 9713], [1068, 1796, 12169], [1073, 2442, 67465], [1074, 2014, 60172], [1076, 456, 36092], [1077, 1753, 59082], [1080, 746, 84495], [1082, 1691, 63137], [1084, 2061, 47601], [1087, 1995, 81389], [1089, 361, 44549], [1090, 528, 75267], [1097, 2054, 30874], [1098, 2231, 78303], [1102, 1472, 9274], [1105, 902, 97570], [1109, 1319, 20953], [1110, 938, 41873], [1113, 971, 645], [1114, 910, 12841], [1116, 932, 77592], [1118, 799, 25201], [1329, 1625, 34839], [1337, 895, 71954], [1338, 108, 57695], [1342, 2359, 10125], [1345, 339, 98285], [1348, 549, 27341], [1349, 1050, 20797], [1353, 1075, 24027], [1356, 661, 88213], [1359, 1708, 27202], [1361, 1166, 86582], [1366, 1129, 28903], [1367, 2348, 99950], [1375, 1602, 65086], [1378, 2269, 90723], [1385, 682, 36026], [1448, 1367, 27630], [1454, 13, 6199], [1459, 680, 14751], [1463, 203, 10932], [1478, 1628, 82359], [1480, 1632, 73194], [1482, 1085, 65840], [1486, 2008, 93516], [1493, 2027, 14558], [1497, 1724, 6309], [1500, 841, 72131], [1504, 1579, 78534], [1511, 1074, 84288], [1513, 945, 23200], [1522, 474, 27198], [1526, 2067, 32754], [1537, 1494, 62701], [1658, 276, 63473], [1664, 705, 55910], [1665, 1093, 40240], [1669, 1091, 88703], [1675, 1636, 51063], [1679, 602, 28130], [1685, 523, 32079], [1693, 2296, 58368], [1700, 612, 74768], [1713, 182, 42853], [1731, 378, 29772], [1738, 1998, 59751], [1739, 337, 54263], [1746, 1242, 24962], [1755, 1149, 75858], [1767, 2465, 20840], [1775, 1992, 82318], [1780, 21, 64043], [1784, 2301, 47710], [1925, 1228, 56882], [1926, 1461, 52948], [1928, 1217, 17413], [1929, 2009, 64597], [1933, 1225, 51171], [1940, 2240, 50687], [1941, 2426, 19710], [1947, 2022, 27955], [1950, 764, 72158], [1953, 976, 1948], [1955, 82, 39565], [1961, 1270, 41128], [1965, 2440, 57496], [1973, 1951, 30078], [1974, 1183, 717], [1977, 8, 90742], [1979, 1486, 49075], [1982, 483, 15119], [1984, 815, 34513], [2160, 478, 85313], [2167, 908, 54983], [2174, 112, 27552], [2179, 2266, 31867], [2184, 855, 76117], [2191, 1869, 91546], [2192, 851, 39352], [2195, 1020, 49458], [2207, 1358, 59254], [2212, 753, 83935], [2223, 67, 33895], [2230, 615, 79133], [2233, 2231, 6384], [2235, 363, 59020], [2386, 2102, 59876], [2389, 952, 27732], [2396, 1325, 90021], [2412, 1402, 49483], [2418, 177, 92981], [2424, 465, 25787], [2425, 1953, 84024], [2431, 27, 64581], [2434, 2225, 30676], [2439, 2469, 61083], [2446, 2212, 89809], [2458, 83, 18571], [2464, 2007, 18000], [462, 945, 3788], [142, 742, 64471], [146, 1481, 99976], [152, 1737, 47288], [154, 900, 55572], [158, 2299, 28661], [163, 2276, 46241], [164, 1689, 6963], [169, 1705, 26209], [174, 685, 78891], [178, 2063, 83454], [183, 1764, 89172], [189, 1723, 31510], [192, 2344, 11533], [194, 1519, 43805], [197, 1129, 7418], [206, 698, 87236], [213, 2367, 72612], [221, 841, 14805], [223, 922, 72674], [227, 137, 56694], [229, 480, 84258], [331, 721, 63828], [333, 203, 74218], [342, 734, 71140], [348, 1888, 22537], [350, 2171, 81241], [357, 2298, 2722], [358, 788, 95254], [366, 1083, 13924], [370, 913, 86848], [374, 2140, 29485], [544, 2112, 28498], [547, 282, 57668], [555, 150, 72312], [558, 1576, 8353], [566, 2148, 80094], [571, 2267, 19010], [574, 2281, 23261], [579, 1586, 91806], [582, 1282, 88329], [585, 1470, 67849], [589, 1162, 79265], [590, 1413, 1719], [594, 2363, 40343], [597, 122, 54007], [598, 401, 80786], [608, 580, 86004], [611, 1768, 80257], [617, 110, 42615], [623, 2231, 99852], [626, 2103, 35892], [627, 249, 90744], [628, 1069, 13005], [761, 1696, 94129], [767, 324, 9129], [771, 57, 9557], [775, 468, 11722], [777, 2014, 59533], [778, 415, 95802], [790, 1518, 47303], [794, 546, 46280], [798, 2229, 55986], [802, 766, 71946], [806, 3, 92055], [814, 2410, 40894], [825, 260, 59117], [826, 1272, 96183], [839, 496, 8145], [840, 1868, 66958], [845, 304, 18473], [850, 2126, 98455], [853, 164, 5149], [859, 130, 28644], [864, 2396, 24248], [868, 1684, 91417], [1003, 472, 90468], [1006, 842, 76866], [1009, 1244, 32997], [1014, 328, 55792], [1015, 325, 74325], [1018, 2329, 79094], [1025, 522, 74281], [1029, 1391, 91715], [1033, 447, 46284], [1036, 1688, 50351], [1038, 413, 73798], [1041, 783, 31886], [1042, 2421, 10462], [1045, 551, 36172], [1047, 2173, 19125], [1049, 790, 75147], [1051, 935, 69626], [1053, 1632, 69490], [1056, 161, 65538], [1209, 241, 20816], [1216, 1360, 91627], [1226, 1137, 91731], [1248, 606, 10884], [1255, 1665, 35818], [1256, 1677, 98429], [1261, 1631, 63750], [1264, 972, 64894], [1277, 1327, 62455], [1281, 751, 47468], [1283, 205, 7799], [1285, 2010, 80504], [1289, 146, 70371], [1292, 178, 11071], [1293, 562, 37746], [1300, 838, 63913], [1305, 1821, 40383], [1308, 2464, 23962], [1313, 615, 70578], [1317, 1690, 77323], [1318, 1019, 76244], [1326, 2298, 58208], [1386, 1811, 51251], [1392, 2235, 49326], [1395, 2155, 48923], [1397, 1669, 9396], [1403, 336, 63084], [1406, 1015, 73369], [1409, 2248, 5356], [1418, 1707, 98463], [1419, 1444, 26962], [1421, 1288, 88233], [1425, 1130, 34702], [1428, 685, 8586], [1435, 136, 50488], [1437, 1353, 33924], [1441, 1040, 24917], [1584, 1514, 69141], [1591, 1399, 11384], [1593, 416, 73630], [1604, 1742, 12843], [1615, 2397, 48977], [1616, 1682, 92103], [1620, 39, 92134], [1623, 1140, 43147], [1624, 1929, 80709], [1629, 13, 13879], [1633, 1555, 84656], [1635, 2044, 86421], [1642, 1943, 84806], [1648, 676, 14770], [1649, 186, 75906], [1651, 1384, 1610], [1655, 687, 65871], [1847, 1110, 48053], [1853, 464, 23148], [1857, 1111, 965], [1862, 1791, 94178], [1864, 1009, 42139], [1865, 2193, 13692], [1866, 299, 5828], [1873, 42, 98644], [1877, 893, 54255], [1881, 1697, 56078], [1884, 2172, 27111], [1887, 1252, 76528], [1889, 1981, 31703], [1892, 903, 65212], [1895, 1720, 76874], [1897, 2147, 48893], [1899, 1049, 29691], [1900, 2030, 45404], [1911, 1245, 30317], [1917, 682, 97696], [2056, 603, 72489], [2058, 2116, 98625], [2062, 296, 57956], [2066, 345, 38444], [2068, 1827, 86128], [2075, 1793, 60886], [2078, 2149, 45791], [2081, 266, 33909], [2082, 153, 46457], [2089, 812, 10420], [2097, 1815, 25051], [2107, 988, 46145], [2111, 1265, 37194], [2118, 912, 81452], [2121, 1887, 82490], [2129, 216, 75564], [2137, 2425, 78807], [2141, 583, 462], [2143, 1167, 7849], [2144, 668, 42025], [2152, 219, 67515], [2154, 595, 59048], [2304, 1680, 36689], [2307, 754, 45305], [2311, 2328, 30762], [2313, 1640, 55454], [2315, 333, 70198], [2321, 433, 85324], [2323, 1013, 11811], [2327, 1388, 34715], [2331, 1996, 22683], [2332, 1478, 99073], [2333, 904, 47495], [2337, 2344, 16395], [2338, 1181, 98293], [2340, 2072, 18170], [2343, 468, 44600], [2347, 832, 50151], [2350, 2145, 60744], [2352, 2249, 62076], [2355, 2327, 74], [2357, 1378, 9137], [2358, 366, 29697], [2361, 1666, 17369], [2362, 2013, 33481], [2365, 341, 36569], [2368, 2219, 17384], [2373, 2351, 19927], [231, 117, 75941], [240, 2466, 77574], [246, 404, 97439], [252, 1854, 53697], [253, 1964, 11910], [257, 1006, 88202], [259, 1751, 19617], [267, 2329, 4610], [273, 632, 22563], [278, 1126, 10155], [279, 1427, 79133], [295, 2155, 8935], [302, 751, 13737], [309, 1721, 48976], [312, 1244, 35040], [314, 1688, 38542], [317, 1358, 4062], [326, 1869, 98129], [476, 1638, 45685], [479, 856, 34488], [482, 1452, 19412], [485, 1857, 11512], [486, 1058, 59319], [488, 1972, 29391], [492, 1574, 42870], [494, 484, 57712], [502, 2418, 32478], [508, 2083, 90521], [509, 330, 10122], [511, 419, 15752], [513, 1973, 6372], [516, 1165, 14700], [517, 1270, 4921], [521, 2307, 65553], [524, 981, 65415], [529, 1180, 858], [534, 806, 54238], [539, 179, 67282], [540, 2380, 1348], [757, 947, 97941], [759, 1038, 95062], [762, 61, 41201], [769, 1075, 955], [772, 668, 74274], [780, 325, 53493], [787, 1531, 14260], [803, 2144, 58308], [813, 1960, 64378], [817, 2346, 86495], [822, 67, 78630], [827, 1994, 68705], [837, 609, 21107], [843, 1707, 1146], [849, 2021, 10622], [854, 1133, 78832], [860, 2384, 21823], [865, 287, 26654], [870, 818, 19818], [1122, 410, 37214], [1123, 1264, 23850], [1127, 207, 29667], [1136, 1692, 56643], [1138, 779, 262], [1147, 1444, 6909], [1148, 830, 12657], [1152, 424, 66584], [1153, 2095, 61987], [1160, 1272, 24585], [1164, 963, 24894], [1169, 1768, 69014], [1173, 254, 24352], [1175, 127, 42459], [1178, 263, 88046], [1184, 1196, 24880], [1187, 2020, 36163], [1194, 260, 78577], [1195, 888, 10204], [1205, 1262, 75487], [1445, 515, 4979], [1449, 1551, 10694], [1451, 1887, 10740], [1453, 528, 63955], [1455, 96, 40295], [1466, 557, 70913], [1471, 777, 80725], [1473, 962, 70490], [1476, 568, 55600], [1484, 1703, 15106], [1485, 392, 85057], [1489, 1389, 74390], [1499, 2264, 56403], [1503, 57, 62586], [1506, 509, 44459], [1509, 806, 994], [1512, 190, 6535], [1515, 1668, 2264], [1520, 169, 82741], [1525, 2301, 62338], [1535, 1387, 92930], [1538, 1366, 32923], [1787, 629, 52330], [1795, 2379, 45963], [1799, 639, 54814], [1802, 832, 23890], [1804, 628, 24145], [1806, 183, 69], [1810, 552, 85944], [1812, 595, 22376], [1816, 310, 76077], [1820, 229, 75607], [1822, 278, 53632], [1823, 2085, 44180], [1824, 1060, 82577], [1826, 917, 73591], [1828, 2195, 96113], [1831, 1135, 89687], [1837, 729, 88376], [1842, 393, 16920], [2159, 1517, 89834], [2177, 1756, 7785], [2190, 603, 59707], [2194, 1383, 67881], [2199, 6, 19134], [2205, 1060, 31303], [2214, 766, 20726], [2220, 123, 53987], [2226, 2235, 71449], [2239, 1616, 48122], [2383, 1727, 57119], [2385, 1763, 42413], [2388, 1157, 20605], [2391, 1277, 53813], [2400, 1236, 18798], [2402, 1748, 10562], [2405, 33, 26879], [2407, 169, 40546], [2409, 256, 71918], [2415, 171, 64514], [2420, 1702, 54744], [2428, 198, 90505], [2433, 2292, 60800], [2435, 1649, 6593], [2443, 166, 88097], [2447, 507, 5260], [2453, 2472, 29955], [2456, 2208, 420], [2460, 2018, 11745], [2461, 923, 1527], [2467, 876, 51468], [2470, 226, 85617], [624, 2317, 5106], [232, 2004, 1617], [234, 966, 87332], [237, 758, 92812], [239, 496, 69959], [242, 310, 28961], [244, 2213, 29637], [255, 1005, 68302], [258, 84, 98515], [262, 491, 43250], [264, 971, 22302], [265, 724, 65452], [275, 1130, 39670], [277, 1151, 12257], [283, 252, 9888], [286, 1438, 69990], [290, 1957, 45075], [292, 1491, 50863], [300, 868, 63782], [301, 2368, 93868], [307, 534, 89526], [313, 2175, 28540], [319, 431, 55854], [321, 2436, 86123], [637, 454, 89859], [642, 2371, 54933], [644, 1197, 28480], [646, 867, 78300], [650, 289, 86898], [653, 285, 10762], [656, 84, 49926], [658, 757, 22168], [662, 1141, 56400], [666, 1260, 71228], [667, 1576, 90011], [672, 574, 51614], [675, 33, 9524], [681, 1466, 18133], [684, 667, 52396], [689, 2013, 45613], [948, 195, 89525], [950, 1680, 72605], [951, 868, 62971], [955, 1454, 48598], [958, 2074, 62882], [960, 97, 85141], [965, 1461, 48508], [968, 2125, 66773], [970, 480, 46200], [973, 2005, 85145], [975, 926, 57368], [979, 2027, 16635], [983, 1753, 48639], [989, 508, 75460], [992, 2242, 14580], [995, 1533, 72083], [1000, 404, 22635], [1001, 1658, 38523], [1213, 1569, 86232], [1220, 208, 71088], [1227, 399, 6423], [1233, 495, 59313], [1237, 1223, 3898], [1240, 69, 23283], [1247, 1700, 47780], [1263, 1089, 9451], [1266, 674, 88792], [1271, 2194, 55572], [1275, 1921, 98863], [1286, 959, 1877], [1301, 2103, 22218], [1306, 762, 50945], [1307, 1264, 48824], [1324, 303, 64406], [1578, 183, 12204], [1581, 2106, 82447], [1588, 2433, 16978], [1592, 1381, 20708], [1594, 1480, 73424], [1601, 900, 38291], [1603, 562, 63022], [1608, 1234, 27226], [1611, 334, 15981], [1614, 151, 80179], [1619, 1631, 38442], [1626, 1149, 2983], [1634, 2323, 71546], [1641, 1499, 26653], [1644, 1418, 30293], [1646, 883, 73331], [1650, 974, 89370], [1652, 2419, 46090], [1654, 315, 2204], [1843, 1426, 60368], [1846, 1999, 31890], [1849, 2284, 68316], [1850, 1436, 71315], [1856, 281, 89794], [1861, 223, 92873], [1867, 1595, 76376], [1872, 691, 47405], [1875, 467, 98918], [1878, 1077, 74773], [1880, 567, 20622], [1885, 1395, 89776], [1891, 759, 63616], [1898, 322, 10149], [1901, 495, 13547], [1904, 1914, 97768], [1906, 1398, 99894], [1910, 422, 53156], [1913, 2276, 82698], [1918, 1312, 21276], [1920, 92, 59162], [2163, 2297, 32485], [2164, 1213, 24576], [2165, 1355, 90547], [2169, 467, 44605], [2176, 2156, 98743], [2178, 2158, 22124], [2180, 172, 40306], [2182, 1951, 1863], [2185, 2231, 39103], [2187, 2384, 74467], [2188, 1136, 49190], [2196, 927, 1916], [2200, 1143, 44967], [2201, 199, 15327], [2203, 1524, 7855], [2204, 66, 68184], [2210, 1036, 61384], [2218, 965, 2714], [2228, 1265, 24479], [2232, 1008, 56902], [2238, 1287, 86052], [53, 886, 44549], [60, 506, 43712], [64, 1817, 91180], [71, 2304, 56613], [73, 1467, 30292], [75, 783, 78828], [80, 2427, 2970], [91, 827, 44285], [93, 2047, 57283], [95, 1704, 98032], [101, 1641, 48916], [103, 1190, 1113], [106, 1979, 48818], [111, 1234, 39021], [116, 2143, 6059], [120, 666, 11116], [124, 1865, 41184], [129, 1529, 83205], [133, 49, 41878], [141, 6, 6962], [389, 537, 79013], [390, 1982, 64991], [396, 115, 92433], [398, 2096, 68136], [403, 1291, 46290], [406, 68, 9715], [407, 1756, 92878], [412, 1396, 95270], [414, 1349, 34381], [421, 399, 12664], [429, 2445, 16395], [435, 1333, 93618], [441, 1571, 13123], [445, 1598, 57486], [446, 1406, 28623], [451, 2387, 71868], [458, 2273, 89319], [466, 936, 36574], [473, 2419, 79605], [695, 1985, 63608], [700, 1866, 93517], [702, 2197, 5688], [704, 2361, 69525], [708, 1805, 79888], [710, 820, 63307], [712, 588, 72376], [716, 143, 56577], [720, 469, 49804], [728, 2375, 20200], [731, 2406, 88193]]



node_dict = packupNode(n, wells, pipes)
node_list = list(node_dict.values())

group_list = get_groups(node_list)

In [59]:
len(group_list)

138

In [60]:
# Round1 Edge to Fork (or edge)
def get_pipe_key(node1,node2):
    if (type(node1)==str) or (type(node1)==int):
        key = tuple(sorted([node1, node2]))
    else:
        key = tuple(sorted([node1.val, node2.val]))
    return key

def nodeGroup2dict(group,output_format='dict'):
    well_dict = {}
    pipe_dict = {}
    node_dict = {node.val:node for node in group}
    for node in group:
        well_dict[node.val]=node.well_cost
        for adj,cost in node.pipes.items():
            key = get_pipe_key(node.val,adj)
            pipe_dict[key]=cost
    
    if output_format=='dict':
        return node_dict, pipe_dict
    elif output_format=='list':
        wells_key_mapping = {x:i+1 for i,x in enumerate(well_dict.keys())}
        wells_key_mapping_reverse = {i:x for x,i in wells_key_mapping.items()}
        wells_list = [well_dict[wells_key_mapping_reverse[i+1]] for i in range(len(wells_key_mapping))]
        
        pipes = [[k[0],k[1],v] for k,v in pipe_dict.items()]
        pipe_list = [[wells_key_mapping[i],wells_key_mapping[j],x] for i,j,x in pipes]
    
        return wells_list,pipe_list
    
    else:
        return None,None
    

In [38]:
tmp = group_list
base = tmp[3]

wells_list,pipe_list = nodeGroup2dict(base,output_format='list')
n = len(wells_list)


print(n)
print(wells_list)
print(pipe_list)

2125
[84270, 83502, 99343, 80421, 28173, 35611, 35849, 52575, 39100, 67543, 3805, 37641, 98317, 36326, 4152, 24681, 39430, 46754, 99970, 76166, 64210, 1426, 93204, 41627, 3539, 29383, 40563, 60377, 53943, 98449, 1383, 31139, 78138, 17276, 47968, 52530, 79905, 28979, 99185, 77130, 70234, 18351, 71926, 25819, 95071, 42814, 25131, 38148, 59823, 64260, 20515, 25866, 40416, 45314, 95759, 64481, 21280, 76657, 2166, 71808, 48983, 3121, 64962, 8377, 78632, 72160, 29286, 91033, 98921, 7069, 82313, 8309, 94339, 64562, 91159, 4970, 31630, 27084, 66052, 39146, 13517, 84361, 66673, 65548, 75197, 25325, 79309, 21241, 94887, 70506, 11977, 79021, 76536, 29146, 23765, 96417, 24238, 57108, 94972, 79752, 32493, 3696, 73768, 64115, 93996, 22647, 88189, 77611, 81269, 79403, 5864, 31319, 2788, 85671, 11092, 6275, 77655, 91767, 43963, 66087, 73631, 40637, 30742, 38188, 49379, 69237, 1077, 67693, 75573, 26635, 5435, 15350, 4310, 55902, 6553, 78630, 55425, 99070, 13711, 80424, 26879, 49745, 15189, 18786, 79195

In [39]:
s = Solution()    
res,well_status,pipe_status = s.minCostToSupplyWater(n, wells_list, pipe_list,True)
res

##########################################################################
stack: []
well_status:
 [7, 9, 11, 14, 15, 16, 18, 22, 24, 25, 27, 31, 34, 38, 42, 44, 46, 47, 48, 49, 51, 52, 54, 57, 58, 59, 60, 62, 63, 64, 66, 67, 70, 72, 76, 77, 78, 81, 82, 83, 86, 88, 89, 91, 94, 95, 97, 100, 101, 102, 103, 106, 111, 112, 114, 115, 116, 121, 122, 123, 127, 128, 131, 132, 133, 135, 137, 139, 140, 141, 142, 143, 144, 147, 148, 149, 152, 155, 158, 160, 161, 165, 166, 169, 173, 174, 178, 180, 181, 182, 183, 184, 185, 189, 191, 193, 194, 203, 206, 207, 208, 209, 211, 213, 214, 222, 225, 226, 228, 232, 233, 234, 235, 236, 240, 241, 249, 252, 254, 257, 261, 263, 265, 267, 268, 269, 273, 274, 275, 276, 278, 280, 281, 284, 287, 290, 292, 294, 295, 297, 300, 301, 304, 305, 306, 308, 310, 312, 313, 314, 315, 318, 321, 322, 324, 325, 326, 331, 333, 334, 335, 336, 340, 342, 344, 346, 347, 349, 351, 356, 358, 359, 360, 361, 362, 363, 364, 366, 368, 369, 370, 371, 374, 382, 384, 385, 386, 389, 394, 395,

66344852

In [23]:



# print('n:',len(wells))
# print('wells:\n',wells)
# print('pipes:\n',pipes)


well_cost_base = wells
pipe_cost_base = pipes

well_index_mapping = {i:k for i,k in enumerate(well_cost_base)}
well_cost = [well_cost_base[well_index_mapping[i]] for i in range(len(well_index_mapping))]
pipe_cost = [x[2] for x in pipe_cost_base]
target = well_cost+pipe_cost
result = []
for i in range(len(target)):
    for j in range(i+1, len(target)):
        result.append( (i,j,abs(target[i]-target[j])))
        
        
res=set()
for i,j,y in result:
    if y==14144:
        if (i<len(well_cost)) & (j>=len(well_cost)):
#             print(i,j)
            cur_val = well_index_mapping[i]
            tmp = pipe_cost_base[j-len(well_cost)]
            if True and cur_val in tmp:
                res.add(cur_val)

                res.add(tmp[0])
                res.add(tmp[1])
#                 raise

# print(res)
print()
for x in res:
    if x in well_status:
        print(x,True)
    else:
#         print(x,False)
        pass


        


base: 2125


IndexError: list index out of range

In [ ]:
# Good
2125
[84270, 83502, 99343, 80421, 28173, 35611, 35849, 52575, 39100, 67543, 3805, 37641, 98317, 36326, 4152, 24681, 39430, 46754, 99970, 76166, 64210, 1426, 93204, 41627, 3539, 29383, 40563, 60377, 53943, 98449, 1383, 31139, 78138, 17276, 47968, 52530, 79905, 28979, 99185, 77130, 70234, 18351, 71926, 25819, 95071, 42814, 25131, 38148, 59823, 64260, 20515, 25866, 40416, 45314, 95759, 64481, 21280, 76657, 2166, 71808, 48983, 3121, 64962, 8377, 78632, 72160, 29286, 91033, 98921, 7069, 82313, 8309, 94339, 64562, 91159, 4970, 31630, 27084, 66052, 39146, 13517, 84361, 66673, 65548, 75197, 25325, 79309, 21241, 94887, 70506, 11977, 79021, 76536, 29146, 23765, 96417, 24238, 57108, 94972, 79752, 32493, 3696, 73768, 64115, 93996, 22647, 88189, 77611, 81269, 79403, 5864, 31319, 2788, 85671, 11092, 6275, 77655, 91767, 43963, 66087, 73631, 40637, 30742, 38188, 49379, 69237, 1077, 67693, 75573, 26635, 5435, 15350, 4310, 55902, 6553, 78630, 55425, 99070, 13711, 80424, 26879, 49745, 15189, 18786, 79195, 64121, 42001, 30710, 14764, 53447, 76064, 63534, 47271, 38983, 46751, 76838, 60199, 75328, 99446, 47889, 24699, 72747, 98216, 50241, 59823, 42368, 85938, 78240, 35409, 50520, 78426, 65414, 57680, 20384, 75962, 96231, 33421, 51161, 48437, 15602, 8730, 8722, 69743, 46000, 687, 96235, 63768, 65479, 20583, 82518, 54000, 78327, 32636, 33523, 66248, 99719, 58469, 71837, 53624, 96550, 84152, 60112, 22749, 96070, 93971, 15324, 44766, 42961, 56252, 80137, 58428, 57736, 1193, 13937, 93148, 45593, 58416, 91423, 69858, 66485, 64649, 4641, 88302, 88067, 90071, 16807, 70778, 94324, 97038, 93437, 72962, 17678, 3667, 78000, 4202, 34248, 52344, 22933, 35605, 85302, 288, 75927, 52260, 36006, 63391, 74490, 96446, 35873, 6201, 52262, 70646, 11235, 84766, 30083, 42401, 66253, 47216, 55239, 68141, 79481, 14277, 28734, 63869, 86450, 10327, 88971, 25984, 10241, 14075, 88266, 58745, 59464, 19666, 17247, 17705, 40882, 51908, 10750, 89919, 47215, 38091, 66541, 48673, 61434, 42953, 68583, 31150, 51372, 75528, 7425, 40706, 14975, 64666, 307, 22430, 30662, 44248, 50583, 43773, 5726, 7575, 43645, 47989, 15461, 39804, 33022, 49367, 30067, 81328, 10297, 38516, 23130, 28084, 28810, 14124, 45709, 70666, 68346, 61990, 77455, 39087, 54, 43468, 26315, 1315, 7077, 86309, 91277, 86739, 75540, 37091, 94755, 53608, 15025, 67308, 18133, 89688, 84217, 85475, 49344, 92270, 3277, 69255, 6151, 49087, 31670, 6924, 51834, 12508, 55169, 4024, 94425, 58037, 97734, 99790, 20409, 41342, 31215, 21459, 24721, 8917, 37240, 68532, 5697, 99042, 39516, 99862, 820, 18716, 9226, 33199, 65571, 44608, 29438, 52004, 55482, 49903, 92752, 44214, 70650, 69627, 6089, 53772, 13090, 36116, 11373, 39495, 53489, 31803, 95604, 46317, 62604, 51633, 25745, 1163, 89843, 16603, 98929, 77738, 31936, 9359, 52940, 57997, 62495, 97406, 12414, 98598, 39363, 1264, 57163, 12925, 62529, 40314, 56186, 66735, 23777, 65341, 702, 80380, 77265, 71468, 54602, 18491, 20437, 64223, 19100, 29838, 66231, 3560, 94138, 63795, 44145, 39657, 16469, 84037, 4209, 27956, 29422, 52934, 14472, 10905, 61335, 56499, 28242, 75748, 75839, 75987, 28644, 31192, 80590, 67303, 26464, 32253, 87318, 92000, 6603, 24584, 3271, 36874, 59879, 79809, 17090, 60926, 77535, 98921, 12962, 78500, 65684, 9524, 59256, 72362, 45513, 59872, 4646, 14159, 27488, 23097, 24397, 90111, 29023, 29562, 68217, 40601, 44882, 53494, 52300, 71221, 92410, 81514, 67636, 90096, 38200, 55012, 55676, 66011, 76208, 83066, 83161, 69578, 82011, 26968, 74834, 57962, 47844, 23003, 93834, 44492, 64749, 89566, 34931, 35261, 50662, 81928, 43140, 83113, 2369, 53638, 98339, 95540, 30388, 13432, 8438, 25991, 33661, 66445, 89034, 89097, 92082, 80337, 15438, 45818, 16143, 63248, 76998, 50399, 31882, 69796, 79124, 75458, 99013, 55147, 94859, 13053, 46756, 41237, 58955, 94647, 44928, 65582, 21872, 94026, 47486, 87254, 20177, 98706, 31812, 44691, 82102, 70337, 17631, 30785, 60222, 76340, 74201, 40570, 15986, 26148, 61941, 26227, 91066, 28572, 47884, 59505, 87971, 45356, 50450, 19938, 91481, 34636, 90696, 85565, 85268, 65125, 86753, 7315, 59549, 74785, 91024, 90781, 15739, 76943, 15950, 2596, 13574, 85610, 98729, 40850, 33746, 59131, 37020, 97943, 63846, 86997, 33350, 14811, 75193, 51572, 12956, 10002, 37339, 35876, 24564, 7751, 4388, 35096, 62846, 79889, 61206, 15378, 18073, 74973, 58607, 89426, 52079, 17058, 90161, 53729, 63207, 72458, 38235, 19167, 55433, 9201, 90294, 37905, 94174, 83036, 76643, 83119, 70248, 50140, 82036, 75620, 73311, 61789, 68756, 856, 98522, 43927, 65029, 68938, 65659, 69276, 97569, 96989, 22487, 83634, 15733, 39213, 34688, 20970, 33301, 9785, 31749, 22156, 78601, 9818, 52441, 85187, 1750, 43699, 79605, 54262, 77015, 30010, 98773, 59384, 21646, 46319, 91527, 68877, 64461, 45526, 6001, 19389, 14541, 30661, 43613, 64577, 55036, 95181, 60831, 91699, 27791, 54069, 87780, 2481, 4594, 6167, 20470, 79532, 83196, 38168, 44909, 8466, 34503, 25615, 16273, 71734, 84115, 83941, 16500, 62157, 87286, 12402, 17591, 47906, 74467, 88481, 22061, 56288, 41076, 22643, 43641, 47312, 33930, 24961, 52970, 19161, 88724, 82119, 36175, 81166, 36348, 81783, 1532, 63457, 54636, 79995, 56184, 62902, 63291, 45498, 61137, 35701, 37304, 68183, 39971, 18758, 21394, 97977, 27431, 13572, 62969, 31813, 77304, 1478, 94381, 21209, 90440, 80003, 54178, 23874, 63859, 18308, 16076, 86685, 23587, 63988, 58952, 16946, 4592, 40739, 11521, 20140, 73936, 11136, 92452, 10929, 10240, 87551, 79635, 12792, 22960, 1807, 60684, 44614, 54014, 46685, 81274, 31816, 19429, 19972, 23953, 14557, 78000, 78015, 37020, 36336, 52123, 94748, 1177, 87986, 41726, 21093, 67927, 39093, 77553, 73818, 91248, 92986, 45101, 14534, 50755, 34507, 1574, 80639, 63704, 13442, 99671, 73330, 13618, 50999, 24035, 54737, 85198, 3170, 6526, 65892, 52697, 75722, 40782, 92358, 29723, 9908, 50616, 34306, 62880, 39427, 85538, 8018, 51417, 56867, 67481, 84701, 43854, 8376, 27349, 14864, 7184, 47424, 97663, 19982, 28501, 91529, 14965, 37950, 69868, 52117, 9973, 11722, 29235, 57453, 27952, 87657, 53707, 30387, 46680, 97762, 69458, 76670, 52334, 95181, 49353, 58705, 27279, 90483, 31229, 63675, 97916, 48189, 23737, 35679, 69084, 48324, 32431, 71562, 32362, 21931, 99922, 71064, 86704, 50096, 83808, 1347, 38909, 99799, 88789, 9314, 71809, 38625, 79758, 2035, 91458, 25599, 61399, 74602, 14797, 84902, 72233, 14838, 430, 83917, 62557, 83237, 2565, 774, 34209, 81473, 68523, 95007, 45391, 86616, 26853, 554, 38298, 31621, 53443, 58666, 14714, 27904, 42780, 6615, 70729, 4562, 92055, 71525, 2129, 34793, 90425, 89012, 30231, 82910, 23491, 76968, 33575, 42735, 14240, 44967, 25006, 46905, 69654, 3263, 70456, 32506, 3623, 9988, 9190, 44213, 6369, 65868, 87268, 83884, 43740, 33897, 68788, 14501, 77893, 1181, 31959, 30139, 67461, 29909, 4958, 46554, 70152, 10298, 42337, 95719, 72771, 63198, 44132, 75580, 5726, 90663, 66244, 87784, 21551, 38083, 52954, 8540, 67297, 11622, 18915, 61650, 54386, 35473, 26666, 24828, 91612, 29887, 60781, 9810, 60298, 51533, 57135, 79414, 50402, 43476, 41926, 85665, 7332, 84042, 90492, 35399, 22771, 95930, 83198, 33265, 22741, 49806, 40778, 31195, 3340, 55851, 94833, 91829, 90009, 26189, 65030, 22725, 88886, 17027, 17906, 69549, 91949, 5704, 98031, 80787, 94114, 8692, 13998, 57844, 71866, 30852, 46412, 85656, 55411, 73825, 16550, 47923, 42192, 25393, 93465, 94352, 35829, 5753, 20259, 92604, 52408, 69270, 95524, 88566, 94204, 96751, 24805, 66885, 36114, 60016, 29500, 77226, 49557, 31850, 97848, 38826, 44604, 50244, 48737, 14454, 15430, 56542, 32362, 20660, 5272, 4262, 9553, 90919, 32389, 65877, 87667, 46518, 29209, 48611, 4825, 47633, 4005, 29540, 42968, 68661, 50153, 20981, 98579, 43859, 21454, 52483, 51971, 74183, 49424, 59536, 48421, 77314, 49047, 38304, 77723, 44742, 67904, 89247, 35820, 55879, 15685, 47257, 19003, 2344, 62933, 18524, 67272, 52241, 62213, 98093, 65309, 75458, 68674, 16442, 41438, 97709, 1682, 46233, 78422, 57516, 92693, 12507, 85603, 48544, 945, 99462, 54357, 92034, 18497, 53193, 39895, 50701, 10157, 13669, 53987, 70696, 87090, 59397, 93873, 34658, 40456, 11056, 79519, 81902, 98721, 17887, 92357, 1658, 87639, 79506, 18364, 70196, 24836, 79885, 50866, 3077, 14194, 74383, 50861, 23965, 87552, 60402, 3856, 22750, 52970, 40309, 58299, 97398, 5245, 85048, 9901, 70315, 9188, 33170, 45680, 66947, 57558, 33107, 89444, 57160, 15201, 91487, 89772, 34448, 793, 67010, 40894, 45456, 31030, 63307, 78857, 68761, 82734, 28710, 13210, 97737, 44048, 73565, 72131, 40562, 94788, 87459, 21539, 20792, 40285, 96895, 52684, 6811, 79019, 24519, 99664, 4253, 87247, 10155, 46525, 74321, 97919, 70111, 95938, 84186, 8925, 56314, 99604, 24208, 98492, 15538, 10863, 57332, 78222, 37339, 63148, 86944, 54205, 69473, 20396, 95820, 60381, 99314, 52652, 18102, 39761, 52195, 11941, 36569, 72430, 68019, 43650, 89282, 88436, 98117, 27022, 2648, 87421, 89253, 92202, 88049, 11291, 25451, 70823, 67220, 95972, 72865, 45074, 943, 23, 81739, 38708, 36086, 26551, 64458, 88922, 69463, 63435, 34886, 41658, 91147, 84833, 25678, 277, 61390, 74265, 86247, 122, 62836, 33376, 94860, 4581, 30498, 9406, 56697, 51266, 23841, 95231, 68953, 19103, 52470, 11679, 53608, 13902, 22197, 81906, 61664, 57722, 99341, 76855, 66901, 2527, 66532, 50705, 61311, 83777, 81493, 82116, 26949, 61686, 86680, 44508, 57401, 87536, 26702, 75102, 47020, 81944, 78146, 62238, 89006, 24732, 75892, 12944, 46526, 58802, 22891, 8410, 10804, 16438, 36798, 58511, 34755, 96154, 71281, 22407, 84146, 1398, 40212, 56005, 20198, 24387, 51416, 89325, 50266, 22491, 76128, 13422, 61757, 36357, 14228, 29131, 3679, 94445, 42400, 30084, 19969, 73978, 72259, 98739, 35152, 91103, 45613, 20326, 91667, 84684, 75500, 16658, 80008, 26311, 37106, 38677, 80138, 51577, 87612, 84553, 34488, 81099, 71395, 12836, 35536, 56911, 78241, 78277, 98352, 25252, 50095, 82128, 61774, 67572, 66407, 61428, 91174, 90216, 61927, 65833, 9629, 93411, 65609, 17100, 43420, 87533, 50241, 33281, 61366, 20502, 63746, 97250, 68144, 86711, 59749, 37840, 42480, 95527, 49837, 39348, 88862, 46343, 76967, 42682, 69582, 84425, 11165, 55092, 53621, 90124, 89595, 50871, 45190, 34859, 39882, 15291, 42738, 69754, 70605, 60009, 34452, 40746, 97901, 89850, 93935, 65104, 921, 16621, 17384, 34148, 34016, 87178, 45094, 13938, 40857, 5398, 48132, 21312, 74387, 98905, 23142, 44260, 36675, 50059, 14849, 43710, 12108, 36492, 46184, 48263, 92333, 78876, 26093, 50987, 8516, 39464, 23696, 88035, 28200, 55671, 90260, 90383, 55202, 97190, 28523, 50330, 56396, 58968, 22128, 21031, 79102, 42485, 87726, 56796, 50990, 76496, 34162, 75631, 70998, 72638, 66761, 83340, 61227, 63868, 63887, 80281, 17201, 57833, 63819, 35626, 18925, 72992, 32311, 87302, 71411, 18057, 70844, 1885, 18594, 42434, 30610, 52292, 7753, 15594, 81253, 32760, 7279, 36600, 9732, 60272, 78818, 21543, 94848, 76602, 56808, 87346, 29107, 27446, 88561, 55869, 94911, 16566, 84949, 30490, 90780, 81424, 95582, 62171, 64025, 51813, 76282, 74797, 68218, 32553, 24730, 69489, 8659, 91457, 69425, 34494, 90254, 98782, 53320, 4797, 20608, 14856, 98330, 26108, 72358, 41450, 64339, 55695, 30762, 3030, 9553, 93521, 61207, 81259, 17673, 97238, 41865, 38323, 66123, 76319, 98780, 59056, 1276, 19274, 37166, 58531, 22856, 72750, 19381, 31258, 57222, 98099, 19807, 40276, 48755, 90624, 85466, 55655, 88423, 37858, 49063, 52378, 59497, 88600, 14057, 15928, 66760, 85963, 77967, 1787, 42876, 86067, 58897, 72259, 27094, 43234, 67603, 93954, 19876, 38755, 10692, 88858, 63372, 81623, 6332, 98162, 15866, 62081, 96141, 31545, 62614, 12848, 6616, 48195, 79952, 90461, 2381, 78580, 40644, 95050, 61834, 45065, 64022, 72920, 19287, 89923, 6596, 86074, 67388, 24177, 34049, 30648, 61926, 56109, 71460, 84734, 98821, 46266, 37259, 27765, 33631, 97336, 64485, 17750, 99561, 28044, 91132, 84913, 28595, 15802, 54452, 77872, 93082, 63575, 35692, 45058, 25161, 80967, 32879, 5949, 53760, 48969, 66733, 54956, 78232, 71138, 20090, 42073, 69627, 71557, 45577, 71683, 90119, 87047, 71560, 17145, 90690, 37126, 61939, 89008, 25953, 21760, 99226, 93497, 31628, 17198, 93626, 22827, 33030, 73993, 140, 17181, 64432, 51568, 47836, 89462, 50667, 98763, 14935, 80755, 79576, 37277, 80714, 2629, 18477, 62928, 84684, 83783, 6658, 87030, 50209, 43018, 59357, 37307, 36088, 38500, 62940, 10605, 29587, 91566, 95428, 85144, 65416, 76624, 95909, 42797, 13611, 31437, 8325, 35470, 75777, 67561, 65819, 42924, 60734, 77011, 61896, 63386, 83182, 99304, 66919, 83699, 14516, 98327, 38357, 81989, 54867, 57229, 93350, 23631, 11646, 58897, 35155, 46721, 51011, 50719, 90398, 79676, 45446, 76206, 88468, 42079, 96684, 73538, 11773, 41951, 52676, 93703, 36490, 11598, 27652, 20168, 24666, 80073, 87346, 69841, 62552, 44389, 50028, 67725, 51941, 51811, 4559, 61167, 98179, 91889, 35592, 55723, 67319, 88011, 55755, 34410, 23078, 31588, 23451, 43399, 80181, 91728, 6884, 93036, 16750, 68985, 97804, 71565, 67520, 27969, 29015, 51390, 99093, 6501, 62259, 93394, 23618, 40917, 78859, 16154, 724, 85833, 80988, 33121, 68698, 9972, 71865, 73574, 34276, 3336, 22218, 69012, 93290, 67396, 80425, 20466, 63896, 84536, 59, 32583, 69325, 86352, 69783, 75920, 73049, 13078, 9081, 39514, 44667, 22574, 42650, 41679, 48044, 9944, 54215, 42270, 29118, 27159, 14472, 87139, 11178, 90297, 75683, 97551, 65547, 41812, 60304, 33395, 34359, 22662, 91728, 76122, 38213, 21122, 87054, 23190, 92481, 97187, 57266, 6357, 92234, 90063, 94253, 52616, 43500, 21955, 28277, 27172, 87433, 79362, 89519, 9910, 96151, 36219, 40680, 8729, 3721, 76487, 51020, 39806, 36489, 25081, 65478, 31414, 50793, 11641, 6882, 85396, 10036, 82249, 65131, 83146, 69358, 12725, 14443, 67526, 78661, 90467, 94933, 50023, 8962, 59164, 33448, 215, 20242, 1727, 79944, 4713, 95421, 16163, 23512, 63892, 85495, 53499, 6751, 55772, 88706, 12504, 99094, 59169, 39320, 70052, 77081, 97233, 92302, 78506, 6379, 51712, 94645, 94974, 58607, 32160, 30451, 59362, 60334, 30072, 95691, 56685, 37356, 65339, 75527, 18063, 82670, 51047, 78528, 75726, 54910, 38617, 80975, 63378, 56229, 31713, 90568, 43147, 86181, 19141, 90064, 90039, 52094, 13270, 55787, 90728, 38436, 71241, 56661, 70559, 1648, 92475, 69677, 37426, 25092, 73473, 99509, 74702, 44092, 98801, 81421, 79980, 96435, 98290, 11947, 69822, 8899, 91704, 11583, 99079, 98741, 978, 98447, 49499, 73186, 13525, 55927, 59013, 70940, 57847, 22146, 15725, 37972, 81195, 59190, 5449, 28010, 53649, 3789, 92867, 41696, 88900, 65863, 79132, 51211, 28147, 5556, 51856, 26006, 49342, 65040, 62152, 63110, 92828, 43418, 30789, 37544, 19105, 54031, 5846, 83482, 14401, 50777, 39531, 42643, 21449, 74169, 28841, 96315, 92693, 32852, 7236, 80262, 67874, 64092, 26684, 60279, 72514, 56253, 99021, 20657, 59038, 84442, 44645, 44461, 60240, 62058, 81428, 81532, 65214, 62298, 34614, 78297, 56050, 75191, 53399, 37415, 62730, 91044]
[[1, 303, 4378], [2, 781, 45436], [2, 879, 15752], [3, 2109, 77262], [201, 4, 12664], [5, 1136, 64359], [5, 1669, 26681], [5, 84, 53156], [291, 6, 88308], [632, 6, 98335], [6, 480, 26238], [7, 920, 67334], [7, 1702, 66584], [8, 871, 19091], [9, 1952, 87745], [9, 816, 69054], [10, 964, 3505], [11, 1868, 16395], [12, 1873, 22839], [12, 478, 70081], [13, 151, 79986], [466, 13, 55854], [14, 212, 71054], [15, 145, 89481], [15, 1156, 31228], [15, 338, 42608], [15, 739, 85324], [16, 2017, 93618], [17, 1301, 1784], [17, 1709, 32927], [18, 1394, 76686], [249, 19, 305], [249, 20, 6691], [20, 293, 43755], [20, 22, 76699], [1262, 21, 60716], [23, 1661, 57486], [24, 923, 28623], [25, 658, 53458], [25, 1527, 46284], [26, 2058, 23396], [26, 31, 91478], [26, 594, 84406], [27, 2088, 69334], [27, 1700, 56506], [27, 1216, 4037], [542, 27, 29515], [1323, 28, 35470], [29, 2052, 43981], [30, 863, 61650], [31, 1860, 15448], [31, 317, 40377], [31, 113, 1151], [31, 1508, 44931], [32, 1348, 10714], [33, 456, 72465], [33, 1071, 9557], [33, 2027, 62586], [34, 442, 9719], [34, 1972, 68184], [35, 71, 24572], [35, 2119, 90143], [35, 1244, 82089], [36, 1142, 7909], [36, 281, 88174], [37, 1860, 6630], [38, 373, 95786], [39, 849, 29885], [39, 342, 93158], [40, 944, 32580], [40, 551, 41878], [41, 249, 66823], [42, 1100, 44549], [43, 1592, 11747], [44, 582, 6555], [45, 1299, 76222], [45, 1793, 41003], [45, 1465, 74325], [45, 856, 53493], [46, 50, 71548], [46, 72, 75460], [47, 579, 77635], [47, 2054, 33895], [47, 934, 78630], [48, 1692, 37718], [48, 320, 23283], [49, 1264, 24490], [50, 1700, 92395], [50, 2031, 63921], [50, 1685, 37182], [50, 537, 89040], [50, 1464, 55792], [51, 1455, 84105], [51, 417, 38444], [52, 574, 41896], [52, 1405, 88759], [53, 800, 55653], [53, 867, 30408], [54, 1813, 47890], [55, 793, 30292], [56, 1143, 26209], [56, 2095, 82741], [56, 1821, 40546], [57, 685, 56613], [58, 1740, 81241], [59, 1238, 28006], [60, 552, 91180], [61, 2024, 8121], [61, 1233, 77394], [61, 1481, 97126], [62, 154, 22537], [63, 1231, 92434], [64, 1037, 71140], [1583, 65, 97694], [65, 301, 36569], [66, 212, 74267], [67, 2109, 79129], [68, 2097, 4761], [69, 345, 77577], [69, 2071, 98285], [642, 70, 88394], [70, 1056, 74711], [71, 250, 14412], [71, 1271, 18719], [71, 1358, 50451], [71, 1035, 73538], [71, 715, 84935], [71, 601, 54263], [72, 916, 63084], [73, 984, 8970], [73, 961, 58341], [73, 1342, 83820], [73, 390, 70415], [73, 1827, 55698], [74, 1268, 42502], [74, 1694, 15981], [75, 1044, 60120], [76, 1370, 82402], [561, 76, 87466], [625, 77, 74218], [77, 720, 70198], [257, 78, 54614], [79, 822, 63828], [80, 614, 86813], [80, 1935, 41243], [80, 876, 10122], [144, 81, 47795], [998, 82, 93754], [383, 83, 88789], [334, 85, 30317], [1971, 86, 28897], [1489, 87, 12717], [87, 294, 58732], [183, 87, 97768], [1674, 88, 67068], [593, 89, 97696], [1202, 90, 68809], [90, 651, 169], [1345, 91, 59162], [827, 92, 16171], [93, 1717, 88933], [160, 93, 18636], [2058, 94, 64810], [727, 95, 87030], [177, 96, 56882], [779, 97, 52948], [203, 98, 19519], [156, 99, 17413], [100, 1636, 64597], [1496, 100, 80709], [1402, 101, 29015], [1133, 102, 29444], [778, 103, 63642], [172, 104, 51171], [177, 105, 7819], [1923, 105, 12625], [1918, 105, 14627], [106, 1296, 4870], [159, 107, 82004], [1680, 107, 77667], [572, 108, 43005], [109, 960, 57183], [773, 110, 47984], [1768, 110, 78199], [110, 438, 6232], [111, 1850, 19710], [1110, 112, 16655], [1510, 113, 84806], [114, 445, 98617], [799, 115, 29448], [216, 115, 96016], [1955, 115, 44397], [244, 116, 31385], [117, 1671, 27955], [836, 118, 89028], [802, 119, 5828], [770, 119, 93517], [1489, 120, 43856], [781, 120, 80567], [983, 120, 66958], [121, 1122, 89332], [1807, 121, 24880], [336, 122, 96287], [504, 122, 54710], [534, 122, 81262], [527, 123, 48982], [123, 444, 76020], [1815, 124, 3481], [1906, 124, 81072], [775, 125, 16640], [125, 972, 82414], [125, 1628, 3582], [1472, 125, 42027], [126, 1500, 39129], [126, 1925, 16484], [127, 668, 59724], [1336, 128, 98644], [788, 129, 23467], [716, 130, 2662], [1209, 130, 60967], [2067, 131, 86112], [131, 399, 94055], [653, 132, 84764], [243, 133, 17914], [1103, 133, 61101], [1106, 134, 54255], [135, 367, 75487], [1735, 136, 74773], [137, 1844, 27089], [1077, 138, 84453], [617, 139, 20622], [140, 1137, 81702], [1572, 141, 20816], [1115, 142, 56078], [1199, 143, 29772], [144, 643, 21530], [1937, 144, 54532], [1073, 145, 18774], [1699, 145, 79609], [145, 2112, 50778], [870, 146, 1114], [147, 1745, 27111], [148, 702, 15815], [148, 2081, 41219], [231, 149, 89776], [862, 150, 49499], [150, 373, 91407], [150, 1602, 6513], [150, 2113, 4191], [1654, 151, 33370], [151, 432, 37508], [152, 721, 95441], [349, 153, 76528], [153, 927, 82490], [763, 153, 10740], [1500, 154, 61869], [155, 2125, 91627], [966, 156, 35476], [157, 1466, 31703], [1966, 158, 98166], [1043, 159, 24866], [1336, 160, 78990], [1062, 161, 63616], [1131, 162, 65212], [609, 163, 71088], [670, 164, 37370], [647, 164, 39154], [257, 165, 80127], [1683, 166, 64697], [1453, 166, 83354], [167, 640, 75637], [167, 689, 32972], [917, 168, 86472], [168, 312, 3898], [1705, 169, 47055], [820, 170, 37112], [171, 997, 48893], [172, 721, 55239], [172, 838, 19688], [172, 1877, 68552], [464, 173, 10149], [2078, 174, 91731], [201, 175, 6423], [1542, 176, 29691], [177, 620, 9797], [178, 1691, 45404], [560, 179, 13547], [180, 1321, 90762], [2052, 181, 13487], [578, 181, 81904], [1125, 182, 68476], [184, 1219, 58776], [560, 185, 59313], [186, 1837, 48949], [913, 187, 38085], [187, 1688, 5221], [187, 1686, 27226], [493, 187, 39021], [188, 1638, 14696], [188, 843, 85057], [1753, 189, 86582], [190, 2018, 46433], [190, 352, 16920], [191, 1775, 60074], [865, 192, 21487], [192, 1891, 10492], [193, 1828, 24708], [194, 1443, 27753], [1039, 195, 53898], [533, 196, 92433], [197, 1001, 84244], [197, 1910, 36245], [2045, 198, 28903], [198, 1897, 32923], [199, 1015, 3945], [200, 1347, 68136], [201, 1486, 36936], [201, 1523, 85873], [1832, 202, 8944], [202, 340, 54325], [203, 2046, 57088], [204, 1391, 56941], [204, 1681, 80786], [1581, 205, 11211], [206, 689, 83288], [207, 1017, 56168], [1210, 207, 24471], [208, 1315, 46290], [781, 209, 27129], [1780, 210, 17644], [211, 1672, 65086], [559, 212, 42801], [212, 495, 4709], [213, 448, 90723], [213, 290, 9137], [1162, 214, 89509], [951, 215, 57027], [215, 1647, 95800], [216, 1645, 20708], [217, 1401, 26996], [316, 217, 33904], [218, 2083, 32195], [218, 562, 6588], [218, 1812, 67881], [1051, 219, 27214], [219, 1517, 1610], [593, 220, 36026], [221, 543, 51251], [222, 1820, 13819], [474, 222, 94547], [222, 248, 34715], [223, 1949, 43111], [224, 974, 34162], [759, 224, 93110], [846, 224, 95248], [1695, 224, 5950], [224, 1130, 12339], [1524, 224, 91715], [225, 678, 92878], [226, 2100, 49326], [227, 1844, 62081], [1961, 227, 54053], [1438, 227, 96123], [228, 849, 94888], [229, 858, 70744], [836, 230, 47657], [230, 2015, 37214], [231, 1021, 48923], [900, 232, 3246], [234, 232, 95270], [846, 233, 31083], [1045, 233, 9503], [235, 1164, 9396], [1644, 236, 79489], [237, 1059, 53705], [237, 603, 7771], [798, 237, 49316], [237, 1532, 73798], [238, 708, 45958], [238, 1643, 11384], [239, 2086, 34381], [240, 1706, 88755], [240, 853, 95802], [725, 241, 53648], [242, 278, 64853], [242, 911, 74916], [242, 1278, 11967], [1171, 243, 39178], [244, 1024, 56687], [244, 1649, 73630], [245, 970, 63126], [245, 404, 74525], [1579, 245, 97439], [245, 1441, 22635], [246, 661, 74206], [1111, 246, 26434], [246, 1838, 49483], [247, 1971, 21963], [248, 288, 74], [249, 678, 29590], [249, 869, 1686], [249, 1506, 20237], [249, 963, 32415], [751, 250, 97954], [1626, 250, 14600], [1195, 250, 84395], [1246, 250, 41852], [705, 250, 30762], [1313, 251, 10344], [1470, 251, 79094], [1911, 251, 4610], [252, 952, 25753], [252, 267, 67935], [252, 1329, 40719], [488, 252, 54271], [252, 1116, 93412], [252, 1846, 90311], [253, 1252, 47288], [1145, 254, 51677], [406, 254, 76596], [255, 1158, 48189], [255, 389, 14129], [1610, 256, 22683], [291, 257, 89761], [257, 2001, 52001], [1134, 258, 47495], [964, 259, 20088], [2024, 260, 53171], [522, 260, 66047], [1668, 261, 77734], [262, 274, 16395], [1799, 263, 98293], [291, 264, 61294], [1586, 264, 2254], [421, 265, 18170], [266, 1119, 55572], [267, 1475, 6269], [267, 731, 19984], [1939, 268, 38445], [269, 373, 89934], [269, 1548, 65538], [1880, 270, 56154], [307, 271, 44600], [272, 554, 52017], [273, 768, 66369], [273, 1866, 57376], [273, 1867, 88097], [2108, 274, 69892], [1445, 274, 60237], [674, 274, 11533], [549, 275, 29865], [518, 276, 907], [922, 276, 86495], [1011, 277, 70080], [546, 277, 68224], [2016, 277, 29788], [278, 1919, 47951], [497, 278, 57217], [992, 279, 60744], [280, 1603, 24855], [280, 1484, 80179], [281, 1715, 19927], [429, 282, 62076], [2044, 283, 27401], [284, 867, 95923], [1174, 285, 7889], [286, 1884, 58914], [287, 1228, 6963], [287, 1022, 5149], [751, 289, 29701], [1024, 289, 30583], [1059, 289, 23842], [910, 289, 32630], [1039, 289, 23474], [1281, 289, 32379], [289, 1736, 69359], [1799, 289, 90961], [291, 1142, 76610], [291, 702, 82039], [291, 977, 91230], [291, 1440, 10869], [291, 687, 42062], [291, 865, 6950], [291, 1292, 46457], [1148, 292, 29697], [2056, 293, 10125], [295, 1920, 46879], [1152, 296, 17369], [777, 296, 69525], [1647, 297, 33481], [298, 666, 28661], [299, 1392, 11630], [1065, 300, 79585], [1620, 300, 81886], [459, 300, 27086], [1880, 300, 47778], [476, 302, 30755], [367, 302, 92443], [970, 303, 28242], [1598, 303, 34338], [1905, 303, 88854], [698, 303, 80440], [1580, 303, 72612], [2033, 304, 17384], [825, 304, 93868], [305, 1119, 96757], [305, 819, 49804], [306, 1301, 44180], [307, 1847, 35240], [307, 850, 11722], [1550, 308, 82577], [309, 1886, 81822], [309, 832, 61936], [309, 1801, 18798], [310, 1634, 16663], [310, 357, 23508], [310, 398, 26041], [310, 1733, 44605], [1493, 311, 13194], [313, 1239, 36574], [1177, 314, 73591], [1223, 315, 22743], [316, 1848, 25787], [317, 419, 86128], [1095, 318, 91805], [318, 1875, 91508], [319, 837, 14340], [319, 372, 23148], [321, 1818, 96113], [322, 459, 143], [322, 704, 90408], [327, 322, 25875], [855, 323, 37876], [324, 1278, 3788], [1663, 325, 71611], [2068, 326, 64933], [326, 633, 24962], [2068, 328, 89687], [329, 734, 73084], [2003, 329, 82885], [330, 784, 8928], [330, 1469, 52573], [330, 676, 7280], [527, 331, 98562], [332, 2099, 14775], [1415, 332, 91291], [1456, 332, 32997], [807, 332, 35040], [1097, 333, 58237], [1783, 333, 57141], [1556, 333, 76632], [820, 334, 80731], [335, 452, 89319], [336, 1358, 95143], [1361, 336, 91902], [336, 1698, 75829], [481, 336, 69768], [1556, 337, 25398], [339, 1126, 47780], [484, 341, 34071], [341, 1730, 36092], [607, 343, 10884], [844, 344, 88376], [463, 345, 43237], [345, 1175, 7191], [641, 345, 73937], [346, 1927, 67989], [1981, 347, 81026], [1488, 348, 25838], [349, 936, 60149], [1796, 349, 70912], [1742, 349, 55682], [350, 1455, 11468], [746, 351, 53355], [353, 721, 44481], [354, 1149, 35818], [979, 355, 60368], [356, 1192, 98429], [357, 699, 6861], [1549, 357, 57134], [358, 1006, 7873], [513, 358, 26946], [358, 435, 58719], [700, 358, 77526], [359, 1616, 31890], [594, 360, 52152], [1044, 361, 21003], [361, 1028, 55730], [1531, 361, 38845], [1964, 362, 48053], [363, 1013, 87257], [363, 1844, 78156], [580, 363, 71228], [756, 364, 17531], [365, 1500, 63750], [366, 461, 68316], [367, 1745, 97605], [1007, 368, 71315], [369, 936, 32942], [1383, 370, 64894], [2021, 370, 23850], [370, 1380, 48824], [371, 1640, 77686], [508, 371, 67022], [1099, 373, 39834], [373, 1398, 37194], [373, 2072, 24479], [1195, 374, 33343], [1590, 374, 53697], [375, 1447, 16228], [376, 1736, 25968], [376, 1636, 86118], [1968, 377, 965], [507, 377, 11512], [378, 935, 37819], [1083, 379, 47200], [379, 2028, 84557], [379, 1431, 41128], [888, 379, 4921], [1114, 380, 976], [380, 454, 45595], [381, 1812, 55572], [382, 1413, 83076], [382, 440, 78056], [784, 383, 74453], [946, 383, 96183], [1732, 383, 24585], [1599, 384, 92873], [635, 385, 74371], [385, 1460, 53733], [386, 724, 66997], [1320, 387, 19946], [1276, 387, 1816], [1456, 388, 42139], [389, 804, 36399], [1044, 391, 39323], [1616, 391, 95869], [1015, 392, 41593], [1437, 392, 75688], [1494, 392, 84327], [727, 393, 3994], [1474, 394, 32204], [395, 1401, 79360], [395, 1899, 26123], [1236, 396, 80126], [1558, 397, 71618], [1504, 398, 86421], [400, 1358, 49864], [2110, 400, 44090], [1231, 401, 71371], [401, 1811, 1127], [1340, 401, 57283], [637, 402, 45588], [1478, 402, 74106], [403, 2063, 50407], [1434, 403, 45488], [575, 403, 75986], [1640, 405, 55626], [405, 458, 31623], [2089, 407, 31588], [1173, 407, 35404], [749, 408, 25616], [1693, 409, 72489], [410, 1411, 98625], [941, 411, 30519], [1933, 412, 80527], [1637, 413, 56804], [1760, 413, 47601], [795, 414, 57956], [1303, 415, 88069], [858, 416, 43347], [416, 1741, 77445], [416, 1306, 44662], [616, 416, 19960], [980, 416, 10017], [2057, 416, 90421], [1142, 418, 7896], [1200, 418, 24114], [2118, 418, 32754], [420, 1723, 50371], [2099, 421, 57168], [1088, 421, 8386], [1564, 422, 23137], [1554, 422, 11228], [423, 974, 67496], [423, 1309, 11198], [423, 944, 32983], [708, 423, 442], [1330, 423, 62882], [823, 424, 31789], [424, 450, 81828], [1241, 425, 89522], [1803, 425, 78783], [1307, 426, 83851], [1137, 427, 17302], [1098, 427, 37118], [800, 428, 47359], [933, 428, 5356], [1028, 429, 12746], [924, 430, 10440], [1582, 430, 5616], [1920, 430, 2301], [992, 430, 44129], [889, 431, 69916], [884, 431, 68950], [1240, 431, 3184], [1052, 432, 15817], [433, 1747, 4480], [806, 434, 8550], [1640, 436, 65512], [950, 437, 28926], [541, 437, 96425], [1713, 437, 77445], [824, 439, 82127], [1095, 439, 12535], [1468, 439, 48772], [814, 441, 16063], [1392, 441, 10712], [463, 442, 97799], [2096, 442, 7689], [463, 443, 22127], [2012, 443, 56403], [1368, 445, 80682], [1769, 445, 31867], [1106, 446, 75005], [600, 446, 19010], [734, 447, 89040], [503, 447, 7736], [1682, 448, 80592], [1913, 448, 23074], [1099, 449, 1316], [639, 449, 32192], [999, 449, 98670], [1539, 449, 13174], [974, 451, 9121], [858, 452, 52498], [2077, 452, 70857], [1805, 453, 20122], [455, 1717, 6340], [1231, 456, 33525], [943, 456, 63390], [966, 457, 61883], [663, 457, 79715], [1462, 458, 13897], [1120, 459, 6421], [1485, 459, 23261], [460, 751, 26164], [764, 461, 5354], [462, 2058, 42643], [462, 1379, 62568], [462, 1067, 9129], [463, 1153, 31287], [463, 1842, 92535], [1587, 463, 41101], [464, 972, 27468], [465, 1860, 86123], [467, 936, 76994], [468, 2116, 4062], [1939, 469, 19431], [469, 960, 83564], [469, 930, 58695], [469, 773, 90573], [470, 1757, 28540], [471, 1225, 38542], [472, 986, 6059], [473, 1013, 1272], [473, 666, 66852], [473, 1856, 85273], [1564, 473, 28761], [475, 1251, 81576], [846, 476, 43474], [1226, 476, 65787], [490, 476, 12815], [1994, 477, 14026], [478, 1718, 96586], [478, 728, 12483], [478, 1446, 90468], [479, 1844, 79605], [480, 2102, 27198], [1931, 482, 81901], [1317, 483, 55136], [483, 1565, 75941], [484, 711, 68888], [484, 1656, 36274], [485, 1879, 42013], [486, 1280, 44622], [487, 1507, 45685], [488, 1644, 60324], [488, 2061, 38234], [488, 1927, 31998], [700, 489, 52330], [490, 1101, 74715], [491, 1609, 92351], [492, 1589, 13624], [492, 1739, 1592], [492, 1048, 5109], [494, 847, 25495], [494, 1703, 85313], [678, 496, 28089], [497, 1429, 1253], [1568, 497, 84258], [497, 1377, 46200], [498, 712, 23912], [499, 597, 82520], [499, 545, 38422], [500, 1079, 42236], [500, 1940, 87827], [500, 991, 51262], [1592, 501, 39726], [1418, 501, 3364], [502, 757, 17797], [503, 1257, 60886], [505, 682, 75626], [505, 564, 57712], [506, 1736, 45963], [1704, 508, 12169], [509, 1717, 17187], [509, 1607, 69074], [509, 2041, 53987], [510, 580, 11116], [1942, 511, 62001], [1187, 511, 44573], [512, 1200, 61], [512, 1559, 82962], [513, 989, 65901], [514, 1775, 29773], [515, 1451, 29391], [730, 516, 54814], [517, 1395, 28827], [2094, 518, 65858], [519, 837, 96068], [520, 1526, 6090], [520, 1102, 93557], [1573, 521, 33894], [1567, 522, 43250], [523, 1892, 50629], [523, 1104, 52269], [523, 2042, 57695], [524, 1461, 48818], [525, 1606, 42870], [526, 1842, 95922], [527, 952, 63889], [528, 1916, 56242], [696, 529, 24145], [530, 1247, 32858], [530, 1046, 14478], [530, 656, 42615], [1369, 531, 83421], [786, 531, 79888], [703, 532, 69], [533, 1934, 49924], [534, 695, 41424], [534, 1779, 42459], [1557, 535, 37277], [536, 587, 79879], [536, 1017, 43520], [536, 1301, 5940], [536, 1033, 46093], [538, 1065, 28291], [539, 635, 47097], [977, 540, 56072], [1212, 540, 88221], [542, 1547, 43547], [543, 1874, 2412], [1355, 543, 73169], [1823, 543, 64587], [1674, 544, 22376], [545, 1002, 50488], [547, 1046, 64471], [599, 548, 19675], [1476, 549, 83889], [987, 549, 62729], [560, 549, 54962], [549, 1349, 25051], [811, 550, 76077], [2020, 552, 79657], [561, 553, 6962], [1239, 554, 93207], [555, 1246, 16816], [556, 1039, 27394], [557, 1041, 81617], [1568, 558, 75607], [559, 1747, 5796], [559, 737, 57495], [560, 1948, 84818], [561, 606, 39277], [561, 1829, 19134], [1375, 562, 40383], [563, 1137, 92122], [563, 1075, 28644], [565, 1875, 83205], [1950, 566, 53632], [567, 1296, 96687], [568, 631, 43050], [568, 1871, 21227], [1931, 569, 86898], [570, 1847, 94453], [570, 859, 52425], [571, 1381, 46202], [1362, 571, 77306], [571, 1555, 9713], [572, 1768, 83226], [1270, 573, 49926], [574, 1263, 56519], [576, 915, 43707], [576, 2111, 88213], [577, 2094, 56400], [579, 814, 46972], [579, 1346, 78638], [581, 1611, 90011], [581, 595, 52396], [2004, 582, 49175], [582, 988, 42025], [582, 1072, 74274], [583, 1159, 96266], [583, 1553, 22687], [583, 1305, 49276], [1485, 584, 51614], [585, 1090, 47533], [1288, 586, 9524], [587, 882, 35117], [587, 2091, 40486], [587, 1264, 85861], [587, 928, 87228], [587, 1515, 14770], [1220, 588, 46300], [589, 1679, 72480], [590, 978, 94893], [590, 1413, 63095], [591, 1403, 45907], [591, 776, 14751], [592, 790, 18133], [593, 1150, 13503], [593, 1724, 19826], [593, 1830, 36166], [596, 1646, 70180], [733, 596, 78891], [596, 985, 8586], [597, 1896, 90696], [897, 597, 4490], [598, 1333, 41583], [598, 1113, 65871], [599, 1005, 19553], [602, 913, 89787], [602, 2035, 29667], [604, 1013, 57215], [1288, 604, 2756], [605, 1466, 82393], [1421, 606, 24790], [607, 924, 90217], [2058, 608, 94845], [609, 1808, 5089], [610, 915, 82553], [610, 837, 6574], [611, 768, 87236], [1213, 612, 24856], [613, 1060, 19147], [613, 1416, 47920], [613, 818, 55600], [1358, 614, 31190], [1634, 615, 86004], [616, 1720, 67049], [1676, 616, 12843], [617, 2026, 7617], [618, 2108, 79739], [618, 1286, 7799], [619, 944, 40688], [619, 1677, 5394], [619, 1055, 56884], [688, 620, 78837], [621, 847, 93483], [621, 1282, 97829], [621, 975, 21107], [622, 1001, 80094], [1024, 623, 6332], [623, 1747, 34623], [1697, 623, 30738], [624, 1874, 68855], [625, 1637, 77875], [625, 783, 10932], [626, 1039, 98386], [626, 1383, 78040], [627, 715, 80257], [1068, 628, 88131], [629, 904, 69146], [629, 778, 85799], [629, 1941, 96901], [1151, 630, 13805], [631, 978, 28629], [2084, 631, 2034], [1949, 631, 94118], [631, 1861, 77705], [994, 634, 35519], [1533, 634, 91012], [634, 1857, 60800], [635, 1006, 39531], [635, 889, 55232], [637, 636, 76527], [636, 1011, 62803], [636, 1126, 74768], [637, 836, 47336], [2089, 638, 3756], [639, 1337, 12658], [838, 640, 37086], [641, 1725, 64029], [641, 1326, 37746], [641, 1675, 63022], [738, 642, 62761], [642, 915, 31468], [642, 779, 86501], [642, 1526, 28579], [642, 1959, 15327], [643, 1707, 22347], [2085, 643, 57581], [643, 1809, 10562], [1573, 644, 56597], [644, 1483, 18594], [644, 1000, 80207], [846, 645, 43398], [1357, 646, 64864], [647, 2080, 79133], [647, 1397, 70578], [1683, 648, 93187], [649, 1309, 47434], [649, 1771, 68589], [649, 1985, 83992], [649, 1037, 54299], [649, 1852, 90505], [650, 1107, 37324], [652, 2045, 7418], [653, 1969, 18057], [930, 654, 66342], [1658, 654, 51936], [1242, 654, 58368], [655, 954, 91283], [655, 727, 41159], [655, 2078, 26666], [655, 935, 66011], [1978, 655, 40525], [655, 762, 54067], [821, 657, 63657], [1966, 657, 28690], [1714, 657, 32485], [858, 658, 93519], [1597, 658, 19617], [659, 1077, 83916], [660, 1611, 8353], [1195, 661, 68123], [662, 1211, 30510], [663, 1165, 19151], [663, 1476, 24064], [663, 1679, 29200], [663, 1107, 39041], [663, 1291, 89525], [664, 806, 17511], [664, 790, 70913], [898, 665, 25739], [1652, 665, 24217], [1112, 665, 2722], [1989, 665, 58208], [706, 666, 2672], [1514, 666, 8271], [667, 2091, 43805], [668, 1006, 9200], [1642, 668, 48048], [1070, 668, 64698], [1735, 668, 59082], [1412, 668, 48639], [870, 669, 70901], [670, 1723, 9665], [1279, 670, 60064], [1318, 670, 27840], [670, 1057, 64622], [671, 1602, 57167], [788, 672, 30440], [1886, 672, 10275], [996, 672, 92741], [1628, 673, 13813], [2120, 675, 75858], [676, 744, 63029], [677, 1159, 95340], [1665, 678, 56918], [678, 1851, 48670], [678, 1740, 70389], [678, 1391, 20328], [678, 1879, 25219], [678, 1762, 7785], [679, 793, 32028], [907, 680, 85166], [681, 1358, 69468], [681, 1634, 7814], [681, 1159, 63542], [681, 2064, 6535], [1634, 682, 37296], [682, 1447, 29497], [1345, 682, 83025], [1881, 682, 92462], [683, 1916, 38159], [710, 683, 16938], [684, 1202, 31510], [1200, 685, 36689], [1457, 686, 65677], [1177, 686, 3169], [687, 2107, 10459], [688, 1174, 5625], [688, 893, 32748], [688, 1782, 29946], [690, 1374, 35892], [841, 691, 63931], [691, 1682, 13894], [692, 708, 9509], [692, 1798, 9795], [692, 1516, 75906], [1995, 693, 10557], [693, 1021, 32449], [693, 1853, 14562], [1585, 694, 90744], [696, 1705, 13005], [1192, 697, 69176], [1561, 697, 1547], [701, 937, 46561], [701, 1756, 42413], [1127, 702, 2485], [703, 702, 89172], [703, 1615, 12204], [1575, 704, 938], [1030, 706, 52521], [707, 1381, 45674], [707, 1170, 42853], [1638, 708, 33633], [709, 1723, 82491], [1947, 710, 22563], [1150, 711, 29763], [1315, 711, 81058], [712, 1354, 65851], [713, 1699, 96646], [714, 1903, 20840], [1761, 715, 69014], [1885, 716, 69484], [809, 716, 96433], [717, 744, 86664], [1201, 718, 52931], [895, 719, 21105], [721, 967, 12250], [795, 721, 7948], [722, 1263, 41374], [722, 1843, 92981], [723, 1741, 59446], [723, 849, 68193], [1991, 724, 30081], [1294, 724, 79867], [1562, 724, 94539], [725, 1682, 3754], [1396, 725, 12374], [1079, 726, 85420], [1284, 728, 59940], [729, 1120, 11807], [730, 1251, 26568], [1119, 732, 86148], [1566, 734, 26433], [900, 734, 29748], [1234, 734, 48920], [735, 1146, 82356], [1560, 736, 27499], [738, 1271, 67848], [738, 1655, 77445], [738, 1862, 49536], [738, 1850, 10140], [738, 1770, 40306], [740, 1602, 82318], [741, 799, 52696], [742, 1707, 54933], [1884, 743, 4396], [743, 1878, 60842], [1119, 743, 61452], [1623, 744, 27666], [2093, 744, 13195], [745, 1553, 56914], [746, 1347, 37164], [1462, 747, 11811], [1503, 747, 71546], [1279, 748, 21810], [814, 749, 33641], [2075, 749, 10054], [1522, 749, 14790], [750, 1378, 11191], [1313, 750, 56280], [1245, 751, 22110], [1243, 751, 47063], [1347, 751, 35724], [1029, 751, 2743], [752, 1083, 78300], [1026, 753, 52942], [884, 754, 4979], [755, 1647, 45613], [756, 929, 73434], [756, 1900, 75056], [757, 1024, 39992], [757, 1544, 70268], [758, 2020, 91619], [759, 990, 41108], [1236, 759, 63619], [759, 1363, 21744], [759, 1924, 84195], [760, 1924, 10694], [1316, 761, 67223], [761, 1954, 42945], [1677, 764, 67504], [907, 765, 63955], [766, 1061, 9519], [1016, 766, 19893], [1031, 767, 6199], [1314, 767, 48598], [768, 952, 34595], [768, 1378, 40461], [768, 1888, 22681], [1327, 769, 40295], [771, 1610, 56690], [771, 2109, 62129], [772, 995, 38150], [883, 773, 89541], [774, 1824, 5688], [775, 1731, 87776], [775, 1422, 54784], [965, 779, 53860], [1359, 779, 48508], [780, 970, 37237], [780, 1001, 81634], [780, 2063, 7549], [780, 1411, 37193], [780, 1147, 55910], [855, 782, 74695], [784, 1065, 90199], [784, 1015, 19432], [784, 1169, 30348], [784, 1646, 47372], [784, 1864, 21260], [785, 1888, 33923], [787, 1666, 43447], [788, 1174, 74313], [788, 938, 95981], [788, 971, 54984], [788, 1603, 80135], [788, 2062, 10910], [789, 1021, 8935], [791, 930, 63307], [792, 1977, 50863], [793, 1013, 65834], [793, 1898, 16607], [1780, 793, 71272], [794, 1404, 29668], [1655, 796, 72376], [797, 827, 3931], [797, 1872, 43109], [799, 967, 10629], [1369, 799, 17907], [800, 1334, 33433], [1690, 800, 98426], [801, 970, 63235], [802, 1299, 76738], [802, 1231, 81918], [802, 871, 75048], [802, 1734, 54976], [802, 1124, 34942], [803, 858, 61179], [803, 1079, 39683], [803, 1122, 32353], [803, 2044, 349], [803, 994, 61231], [1648, 804, 67849], [852, 805, 80725], [806, 1299, 84718], [806, 826, 86910], [806, 1044, 74974], [948, 808, 56577], [1275, 809, 68647], [810, 955, 56368], [811, 1891, 42394], [1574, 811, 28961], [1348, 812, 70490], [813, 1006, 49664], [813, 1653, 33715], [814, 1381, 13810], [815, 1342, 58075], [817, 1197, 48976], [820, 1260, 79013], [820, 984, 5146], [1073, 821, 39544], [822, 1174, 69253], [823, 871, 22963], [824, 1271, 25304], [827, 1211, 33084], [827, 1617, 19227], [828, 1501, 73194], [828, 1651, 73424], [829, 1383, 6161], [1929, 829, 65452], [830, 1623, 60102], [1135, 831, 56242], [833, 1084, 63782], [1763, 834, 65840], [835, 1065, 29483], [835, 1610, 40227], [836, 1201, 73090], [836, 1140, 9942], [836, 1980, 64406], [1317, 837, 34481], [838, 1609, 22053], [1956, 838, 91717], [1386, 838, 94781], [839, 1720, 20200], [840, 1137, 15106], [841, 1269, 61978], [841, 2067, 96234], [842, 1056, 13737], [844, 1837, 51819], [845, 1633, 93516], [845, 1461, 49075], [847, 1731, 91981], [847, 1232, 7734], [847, 1194, 85794], [847, 1110, 60506], [848, 881, 42181], [849, 1804, 27071], [851, 1368, 17273], [851, 1154, 76139], [852, 1650, 59533], [854, 1253, 79397], [854, 982, 8145], [1571, 854, 69959], [855, 1401, 16543], [855, 1963, 24810], [855, 2082, 262], [857, 1657, 96608], [858, 1073, 56993], [858, 952, 1745], [1220, 858, 8476], [2048, 858, 57184], [1066, 858, 83850], [1034, 860, 46048], [861, 1655, 23336], [1139, 862, 22913], [862, 2058, 39725], [862, 1625, 27911], [862, 1534, 31886], [1382, 862, 78828], [863, 1875, 37266], [864, 1843, 32478], [866, 970, 94387], [866, 1158, 54049], [868, 1880, 14260], [1121, 869, 95254], [1570, 870, 696], [871, 1263, 61736], [871, 1451, 44646], [871, 1525, 78273], [871, 1426, 61865], [872, 1199, 74818], [872, 1870, 5260], [873, 2088, 47303], [873, 1542, 75147], [874, 1165, 39739], [1960, 874, 47774], [1894, 874, 84466], [875, 1296, 90521], [875, 1423, 75460], [876, 2038, 44459], [1065, 877, 22937], [1262, 877, 65029], [877, 1115, 7096], [1365, 877, 43306], [878, 944, 39029], [878, 1977, 40730], [2018, 880, 46280], [881, 2026, 27970], [881, 2106, 33981], [882, 1251, 38436], [899, 882, 51877], [885, 2079, 55986], [886, 1748, 14700], [887, 2024, 37170], [887, 1999, 25201], [889, 1333, 9619], [890, 1073, 21950], [890, 901, 43061], [891, 1237, 68974], [1066, 892, 71946], [894, 988, 58308], [895, 2079, 81843], [895, 1053, 32544], [896, 1333, 77594], [897, 1520, 74281], [1226, 898, 75522], [898, 1003, 6625], [899, 1215, 32079], [900, 1247, 4860], [1489, 900, 23108], [902, 1654, 72524], [903, 1994, 70036], [903, 2036, 50699], [904, 954, 79111], [904, 2079, 40970], [905, 2033, 89385], [1937, 906, 17931], [1558, 907, 52589], [907, 1776, 75267], [1695, 908, 14018], [908, 1825, 69952], [909, 1793, 858], [910, 1260, 77119], [910, 1319, 10420], [912, 1430, 64378], [913, 1114, 90447], [914, 1835, 40894], [1665, 915, 58732], [1583, 915, 80995], [1690, 917, 1199], [917, 1471, 34513], [1161, 918, 81452], [919, 974, 92967], [919, 941, 82313], [1926, 919, 22154], [919, 924, 28268], [919, 1627, 66757], [919, 1449, 67989], [2101, 920, 33187], [1011, 921, 23636], [1465, 923, 73369], [1252, 924, 40005], [1151, 924, 52023], [1357, 925, 15455], [925, 1086, 19818], [926, 2019, 83810], [1043, 929, 92398], [1249, 929, 63529], [1458, 929, 27747], [930, 1182, 89573], [930, 2120, 12035], [931, 1888, 44728], [1354, 932, 33747], [1077, 932, 41434], [1962, 932, 72484], [936, 1707, 93630], [936, 1837, 90507], [1573, 936, 40008], [1192, 936, 13272], [1136, 937, 89431], [968, 937, 56620], [1371, 937, 66773], [938, 1994, 26525], [938, 1728, 49610], [938, 1968, 50714], [938, 1497, 76535], [970, 939, 82499], [940, 1900, 45312], [1241, 942, 5032], [942, 2083, 55308], [1012, 942, 98455], [1659, 943, 1719], [1068, 944, 46568], [1693, 944, 23866], [1184, 945, 92916], [1030, 947, 47883], [948, 1055, 56323], [949, 1607, 68705], [1352, 949, 44285], [950, 1784, 60177], [951, 967, 27236], [951, 2049, 59548], [951, 1152, 67054], [952, 1005, 29689], [952, 1409, 65994], [952, 1951, 84877], [1068, 953, 48320], [1245, 954, 24812], [954, 978, 68435], [1157, 954, 5518], [1986, 954, 26811], [1138, 954, 15974], [1235, 955, 79476], [955, 1794, 95585], [956, 1150, 98463], [956, 1511, 30293], [957, 1188, 23323], [957, 1395, 71910], [957, 2117, 12657], [958, 1028, 26962], [1444, 959, 8918], [961, 1632, 49154], [1305, 962, 88233], [1044, 964, 9606], [964, 1692, 93614], [966, 1006, 87906], [966, 1201, 66432], [966, 1222, 44004], [966, 1085, 5296], [1139, 967, 51992], [1136, 967, 38087], [1133, 967, 49598], [1668, 967, 670], [2018, 967, 986], [1433, 967, 33000], [969, 1849, 78807], [1059, 970, 95020], [970, 1738, 5731], [1182, 973, 79966], [1908, 973, 73489], [1626, 974, 73049], [1657, 974, 90839], [2009, 974, 61973], [2050, 976, 34702], [977, 1815, 47822], [977, 1353, 63913], [978, 1707, 48256], [1203, 978, 56866], [2024, 978, 92204], [1250, 978, 29485], [1171, 979, 8672], [1889, 980, 79133], [1642, 981, 462], [1674, 984, 40261], [1971, 984, 77452], [1754, 986, 7849], [1054, 987, 80072], [987, 2014, 72131], [1584, 987, 14805], [990, 1710, 42015], [990, 1452, 76866], [993, 1150, 1146], [2031, 995, 23275], [2039, 995, 36518], [1217, 995, 31078], [1668, 997, 45003], [998, 1610, 34075], [998, 1618, 89488], [998, 1903, 19269], [1219, 1001, 52119], [1001, 2097, 31408], [1272, 1003, 45791], [1004, 1241, 57700], [1004, 1667, 41194], [1005, 2088, 93549], [1005, 1199, 69736], [1005, 1763, 96517], [1290, 1005, 27441], [1626, 1006, 96887], [1039, 1007, 25868], [1172, 1007, 4514], [2098, 1008, 33924], [1009, 1668, 10622], [1589, 1010, 59018], [1687, 1010, 9399], [1932, 1011, 69990], [1566, 1013, 49365], [1507, 1013, 85347], [1014, 1842, 34668], [1014, 1490, 41188], [1014, 1810, 39352], [1016, 2108, 52376], [1016, 1280, 84984], [1016, 2124, 32981], [1017, 1666, 61660], [1018, 1717, 85869], [1018, 1813, 86892], [1674, 1019, 59048], [1533, 1020, 24917], [1023, 2062, 78832], [1065, 1024, 13943], [1585, 1024, 68623], [1025, 1211, 27571], [1025, 1945, 27048], [1025, 2045, 61679], [1025, 1781, 76117], [1153, 1026, 57721], [1027, 1403, 62655], [1027, 1218, 63758], [1027, 1873, 269], [1189, 1027, 46233], [1027, 2059, 63796], [1402, 1027, 76514], [1028, 1427, 10447], [2114, 1028, 6909], [1225, 1029, 10604], [1097, 1029, 12371], [1029, 1765, 22124], [1031, 1122, 44975], [1031, 1259, 17487], [1031, 1499, 13879], [1032, 1820, 88193], [1034, 1155, 79724], [1036, 1713, 29846], [1038, 2107, 67808], [1039, 2046, 31069], [1665, 1040, 70722], [1041, 1514, 45292], [1041, 1463, 89232], [1042, 1309, 75982], [1043, 1201, 81311], [1043, 1847, 27284], [1043, 1919, 17012], [1566, 1044, 86544], [1044, 1060, 20734], [1047, 2107, 52411], [1655, 1049, 52359], [1050, 1969, 71181], [1051, 1483, 39501], [1051, 1746, 84495], [1056, 1277, 47468], [1058, 2079, 93265], [1058, 2007, 83935], [1059, 1153, 91789], [1059, 1957, 15100], [1142, 1061, 86959], [1061, 1334, 25532], [2029, 1061, 54773], [1943, 1061, 92812], [1062, 1532, 95062], [1063, 1437, 12651], [1064, 1468, 79693], [1064, 1414, 72158], [1065, 1269, 74982], [1065, 1775, 39546], [1915, 1066, 67665], [1066, 2013, 20726], [1068, 1479, 8917], [1069, 1726, 955], [1070, 1528, 44013], [1626, 1073, 65804], [1073, 1083, 81282], [1074, 2122, 52639], [1074, 1993, 33124], [1076, 1752, 21823], [1078, 1618, 6316], [1079, 1388, 37981], [1080, 1789, 24248], [1939, 1081, 26654], [1082, 1438, 52349], [1083, 1143, 63931], [1204, 1083, 89392], [1084, 1211, 91417], [1084, 1302, 62971], [1087, 1183, 80879], [1592, 1087, 46757], [1088, 1350, 3099], [1089, 1673, 67201], [1090, 1839, 26948], [1091, 1296, 252], [1092, 1650, 37848], [2000, 1093, 51183], [1094, 2084, 62572], [1095, 1988, 4232], [1095, 1975, 37842], [1096, 1263, 29894], [1588, 1097, 55387], [1097, 1513, 73331], [1331, 1099, 77685], [1099, 1224, 32401], [1100, 1231, 75407], [1105, 1731, 54875], [1626, 1106, 20308], [1108, 1265, 1983], [1108, 2034, 71954], [1109, 1826, 85175], [1109, 2050, 81426], [1110, 1425, 63190], [1560, 1114, 27167], [1117, 1364, 64053], [2009, 1118, 77359], [1119, 1772, 73013], [1119, 1670, 38291], [1357, 1120, 30853], [1123, 1203, 8017], [1125, 1872, 27009], [1127, 1822, 97570], [1953, 1128, 63473], [1442, 1128, 38523], [1356, 1129, 5950], [1130, 1839, 74287], [1131, 1225, 35717], [1357, 1132, 35982], [1133, 1845, 54744], [1134, 1141, 65894], [1135, 1486, 81957], [1135, 2052, 66462], [1135, 1729, 74462], [1137, 1241, 52246], [1139, 1626, 30822], [1139, 2005, 91271], [1139, 2100, 59020], [1786, 1140, 57985], [1184, 1140, 34665], [1141, 1632, 54452], [1332, 1141, 98032], [1142, 1626, 2560], [1142, 1174, 47710], [1570, 1142, 63149], [1142, 2000, 31441], [1142, 1614, 267], [1161, 1143, 53921], [1144, 1839, 94898], [1144, 1840, 69249], [1145, 1434, 2090], [1201, 1146, 66095], [1148, 1755, 13924], [1786, 1149, 40240], [1926, 1150, 43065], [1151, 1265, 27207], [1153, 1368, 65665], [1153, 1403, 83462], [1953, 1153, 60703], [1153, 1983, 12841], [2121, 1154, 27202], [1281, 1155, 16762], [1157, 1621, 67585], [1157, 1321, 29613], [1159, 1185, 49480], [1159, 2026, 49621], [1159, 1671, 54306], [1160, 1231, 60062], [1160, 1376, 93286], [2076, 1160, 2264], [1162, 1463, 88550], [1163, 1427, 19943], [1780, 1164, 88703], [1166, 1165, 86848], [1167, 1632, 96371], [2094, 1168, 86577], [1171, 1203, 97610], [1572, 1171, 11112], [1174, 1296, 29080], [1174, 1178, 80572], [1560, 1174, 42658], [1176, 1982, 79013], [1179, 1225, 93689], [1180, 1427, 21122], [1226, 1181, 27106], [1243, 1183, 77858], [2023, 1183, 85392], [1185, 1198, 96687], [1505, 1186, 51063], [1931, 1188, 58966], [1189, 1716, 87864], [1190, 1856, 97036], [2020, 1191, 2364], [1654, 1191, 93321], [1759, 1191, 7387], [1192, 1874, 19523], [1193, 1609, 23248], [1599, 1193, 72674], [1194, 1417, 79083], [1195, 1488, 74456], [1195, 1893, 1527], [1690, 1196, 28130], [1404, 1197, 78385], [1327, 1197, 31911], [2020, 1200, 42964], [1300, 1200, 72605], [1201, 1381, 39159], [1202, 1334, 14597], [1205, 1711, 77664], [1205, 1393, 57368], [1808, 1206, 78544], [1487, 1206, 92103], [1626, 1207, 28736], [1208, 1480, 29835], [1459, 1208, 50680], [1208, 1374, 61685], [1211, 1741, 83242], [1213, 1354, 69424], [1981, 1213, 49966], [1361, 1214, 26394], [1220, 1672, 24041], [1234, 1221, 86389], [2053, 1221, 88711], [1222, 1455, 42589], [1222, 1737, 41327], [1223, 1347, 31739], [1223, 1248, 48482], [1224, 1990, 77592], [2040, 1225, 66754], [1530, 1225, 50351], [1233, 1227, 29772], [1808, 1228, 47387], [1274, 1228, 1976], [1281, 1229, 97707], [1230, 1552, 7405], [1805, 1232, 43624], [1408, 1232, 77323], [1751, 1234, 63137], [1251, 1235, 62983], [1237, 1660, 13772], [2074, 1237, 56643], [1834, 1238, 458], [1598, 1239, 3962], [1239, 1757, 26277], [1934, 1241, 288], [2103, 1241, 30112], [1243, 1717, 74903], [1992, 1247, 21373], [1248, 1964, 41873], [1251, 1271, 21348], [1251, 1255, 79612], [1251, 2017, 70491], [1690, 1253, 97516], [1926, 1254, 19345], [1256, 1614, 59751], [1258, 1992, 62455], [1258, 1773, 53813], [1260, 1824, 19089], [1261, 1873, 11572], [1261, 1887, 18571], [1552, 1263, 28041], [1266, 1663, 39632], [1267, 1583, 9544], [1267, 1606, 41404], [1267, 1855, 64581], [1931, 1268, 4250], [1268, 1712, 7587], [1268, 1841, 46677], [1270, 1896, 16104], [1270, 1596, 98515], [1271, 1299, 88674], [1271, 1325, 34004], [1273, 1427, 76168], [1273, 1785, 8212], [1278, 2069, 23200], [1997, 1280, 34766], [1902, 1280, 13665], [1281, 1512, 96409], [1281, 1600, 55296], [1281, 1424, 39565], [1283, 2059, 52546], [1284, 1678, 70621], [1639, 1284, 88329], [1285, 1357, 38297], [1937, 1287, 33909], [1288, 1816, 26879], [1289, 1825, 1113], [1293, 1673, 65562], [1295, 1501, 60543], [1296, 1354, 43847], [1297, 1640, 80504], [1298, 1815, 21517], [1299, 1723, 55040], [1419, 1301, 39836], [2043, 1301, 23659], [1303, 1620, 82745], [1304, 1333, 64202], [1304, 1492, 92134], [1305, 1633, 73326], [1307, 1505, 72127], [1307, 1767, 27732], [1308, 1509, 48916], [1309, 1370, 35502], [1310, 1378, 83440], [1311, 1957, 37426], [1312, 1641, 27712], [1316, 1690, 63983], [1320, 1447, 11891], [1691, 1321, 86665], [1322, 1450, 31125], [1323, 1333, 55087], [1396, 1324, 87446], [1324, 1339, 85141], [1327, 1436, 42950], [1328, 2091, 43078], [1331, 2101, 89625], [1467, 1333, 64767], [1334, 1401, 52212], [1931, 1334, 48422], [1557, 1334, 16928], [1335, 1445, 72473], [1434, 1338, 39580], [1341, 1469, 55293], [1390, 1342, 47172], [1541, 1343, 64978], [1708, 1343, 61987], [1344, 1469, 9528], [1348, 1880, 4071], [1348, 2125, 22906], [1350, 1744, 24894], [1351, 1997, 5966], [1351, 2006, 47377], [1354, 1731, 38146], [1934, 1354, 91572], [1480, 1354, 33667], [1356, 1674, 28309], [1486, 1358, 57178], [1358, 1896, 85785], [1558, 1358, 40084], [1359, 2030, 2714], [1360, 1374, 22218], [1361, 1764, 63811], [1946, 1363, 87332], [1482, 1364, 27998], [1366, 1672, 87699], [1366, 1435, 43408], [1367, 1933, 77664], [1370, 1758, 59876], [1372, 1389, 71667], [1373, 1546, 82955], [1505, 1374, 29489], [1378, 1916, 69847], [1379, 1974, 645], [1836, 1379, 22302], [1619, 1381, 82447], [1384, 1901, 23962], [1421, 1385, 46145], [1387, 1628, 85145], [1570, 1389, 74288], [1727, 1392, 71198], [1506, 1394, 34363], [1395, 1420, 1948], [1399, 2091, 74307], [2004, 1400, 18786], [1939, 1401, 77145], [2010, 1401, 28498], [1892, 1403, 31854], [1403, 2105, 10431], [1466, 1403, 48528], [1404, 1878, 66020], [1956, 1406, 57855], [1407, 1851, 2970], [1472, 1410, 76244], [1792, 1413, 52713], [1420, 1849, 84024], [1422, 1778, 98601], [1425, 1605, 60140], [1428, 2124, 14580], [1430, 1609, 72751], [1432, 1884, 72083], [1952, 1434, 72420], [1973, 1436, 78111], [2037, 1437, 70162], [1591, 1437, 11910], [1439, 1864, 57496], [1663, 1440, 38489], [1443, 1550, 36096], [1771, 1444, 81730], [2068, 1447, 16556], [1521, 1447, 21924], [1448, 1844, 97209], [1593, 1449, 68302], [2109, 1450, 37332], [1529, 1450, 32647], [1595, 1452, 88202], [1453, 2087, 56902], [1806, 1454, 717], [1457, 1488, 35613], [1460, 1979, 86420], [1468, 1650, 73281], [2094, 1469, 31427], [2081, 1473, 1611], [1474, 1886, 73895], [1474, 1818, 49458], [1477, 1907, 56007], [1726, 1477, 15442], [1478, 1501, 8445], [1482, 1826, 87674], [1486, 1907, 89197], [1486, 2033, 22051], [1487, 2115, 48122], [1491, 1500, 38442], [1981, 1493, 61996], [2089, 1495, 43147], [2002, 1497, 34839], [2120, 1498, 2983], [1550, 1500, 34017], [1545, 1501, 69490], [1933, 1502, 84656], [1507, 1662, 53442], [2012, 1509, 26653], [1514, 2047, 91198], [1516, 1859, 6593], [1518, 1844, 46090], [1519, 1747, 24770], [1530, 1998, 61384], [1535, 1846, 10462], [1536, 1971, 12964], [1536, 2025, 41818], [1537, 2079, 65079], [2039, 1538, 36172], [1540, 1750, 19125], [1541, 2097, 91914], [1543, 1682, 2904], [1543, 1640, 87652], [1543, 2086, 20797], [1546, 1668, 50018], [2004, 1550, 37692], [1550, 1976, 31303], [1551, 1653, 30939], [1553, 2073, 43711], [1563, 1625, 1617], [1566, 1612, 79481], [1566, 1788, 88046], [1569, 1904, 77574], [1572, 1917, 82341], [1573, 2011, 45857], [1573, 1912, 85617], [1576, 1629, 277], [1577, 2011, 29637], [1578, 1931, 90079], [1581, 2053, 91888], [1582, 1613, 58950], [1589, 1757, 19385], [1590, 1909, 9888], [1592, 1774, 24352], [1594, 1986, 52498], [1594, 1831, 71918], [1598, 1743, 5407], [1601, 1820, 89435], [1956, 1604, 76348], [1825, 1608, 59921], [1766, 1609, 81389], [1637, 1610, 85687], [1611, 1610, 92885], [1828, 1617, 37718], [1617, 1865, 40427], [1991, 1620, 32911], [1622, 1888, 79487], [2075, 1624, 69141], [1626, 1641, 30096], [1627, 1684, 65842], [1748, 1627, 7224], [1629, 2079, 57802], [1631, 1629, 91806], [2048, 1630, 44073], [1632, 1901, 18000], [1635, 1857, 16978], [1722, 1650, 60172], [1655, 1958, 58833], [1655, 1784, 4120], [2088, 1660, 23678], [2123, 1660, 60197], [1660, 1890, 11745], [1661, 1840, 23704], [1664, 1707, 74370], [1914, 1665, 27442], [1817, 1666, 36163], [2107, 1673, 32181], [1674, 2070, 65490], [1683, 1741, 36161], [1942, 1684, 52548], [1689, 1778, 43786], [1693, 1800, 59707], [1753, 1696, 43471], [1936, 1699, 12257], [1700, 1755, 57131], [2084, 1701, 89834], [1705, 1924, 65580], [1805, 1706, 37847], [1777, 1706, 29992], [1885, 1707, 10505], [1808, 1707, 86659], [1934, 1711, 52921], [1717, 1904, 37047], [2106, 1719, 90547], [1721, 1866, 67465], [1722, 2060, 84288], [1723, 1854, 90645], [1726, 2098, 24027], [1790, 1728, 54374], [1996, 1739, 1348], [2004, 1741, 70265], [2008, 1741, 32464], [1749, 1796, 73219], [1750, 2070, 45431], [1962, 1752, 8405], [1970, 1752, 16680], [1791, 1752, 74467], [1753, 1841, 59409], [2110, 1754, 89738], [1755, 1969, 12823], [1772, 2032, 16913], [1772, 2051, 13183], [1787, 2083, 39103], [1985, 1789, 90021], [2074, 1795, 49190], [1796, 1820, 49396], [1797, 1865, 73172], [1802, 2083, 78303], [1986, 1806, 93100], [1995, 1813, 91098], [1813, 1883, 10769], [1814, 1842, 80313], [1819, 2075, 29408], [1933, 1824, 18504], [1939, 1826, 13695], [1832, 1920, 41809], [2101, 1833, 44967], [1956, 1834, 78049], [2046, 1835, 29232], [2063, 1858, 30676], [1863, 1910, 61083], [1865, 1868, 58445], [2007, 1869, 89809], [1876, 1919, 29955], [1988, 1882, 420], [1884, 2097, 91335], [1991, 1895, 62701], [1921, 2018, 30375], [1922, 1965, 80009], [1926, 2099, 79149], [1926, 2089, 81131], [1928, 2050, 39670], [1930, 2070, 19999], [1934, 2022, 57668], [2004, 1938, 39921], [1944, 2108, 53675], [1950, 2031, 10155], [1962, 2104, 73572], [1965, 2105, 35303], [1966, 2122, 7965], [2112, 1967, 7855], [1968, 2019, 27023], [2116, 1979, 88798], [1982, 1997, 35729], [2116, 1984, 59254], [1987, 2063, 46144], [2090, 2024, 12881], [2067, 2055, 32759], [2065, 2100, 71449], [2066, 2070, 39254], [2083, 2092, 6384]]

In [40]:
[len(x) for x in group_list]

[35,
 2,
 2,
 2125,
 12,
 4,
 2,
 12,
 16,
 2,
 2,
 1,
 3,
 3,
 2,
 2,
 1,
 2,
 2,
 3,
 4,
 1,
 16,
 2,
 18,
 2,
 8,
 11,
 2,
 1,
 2,
 4,
 14,
 2,
 2,
 2,
 5,
 1,
 3,
 1,
 4,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 1,
 4,
 3,
 2,
 3,
 1,
 1,
 1,
 3,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 5,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [44]:
group_list_II = [x for x in group_list if len(x)<2000]
new_group = []
for x in group_list_II:
    new_group+=x

In [45]:
base = new_group

wells_list,pipe_list = nodeGroup2dict(base,output_format='list')
n = len(wells_list)


print(n)
print(wells_list)
print(pipe_list)

print('\n'*5)
s = Solution()    
res,well_status,pipe_status = s.minCostToSupplyWater(n, wells_list, pipe_list,True)
print(res)


347
[86153, 85896, 40712, 50340, 89442, 99787, 93577, 46159, 95229, 90476, 38967, 68259, 4748, 48586, 66222, 92724, 89877, 36722, 84790, 65029, 35944, 56197, 68372, 61060, 7894, 29204, 44642, 90408, 86826, 82835, 12238, 81159, 67288, 54391, 91898, 78790, 22902, 2130, 11267, 2807, 39305, 77785, 74765, 78173, 99643, 96087, 86037, 35436, 94244, 62783, 83767, 77320, 40880, 65022, 99259, 36830, 69716, 12493, 70444, 94048, 39552, 82783, 15092, 52010, 66862, 99525, 92543, 50703, 529, 56986, 92407, 63597, 86268, 36609, 21499, 77695, 87634, 21724, 56735, 57825, 81496, 24205, 94718, 69330, 90267, 9960, 95910, 17872, 48971, 26375, 22015, 10308, 59296, 74373, 82316, 15292, 47521, 29977, 41026, 71925, 10399, 28477, 14395, 49481, 75732, 95936, 84068, 1422, 83667, 73718, 90093, 15451, 39768, 94922, 76246, 68421, 14611, 62451, 58086, 89394, 35290, 62866, 66346, 80115, 5679, 45248, 99409, 64614, 958, 76740, 84601, 24119, 44134, 82921, 64854, 15604, 33765, 15108, 47131, 90921, 82471, 15898, 84152, 55420

In [52]:
group_list

[{<__main__.Node at 0x23900ac1550>,
  <__main__.Node at 0x239e60609d0>},
 {<__main__.Node at 0x2397b83f850>, <__main__.Node at 0x239e60608b0>},
 {<__main__.Node at 0x2397b83fbe0>, <__main__.Node at 0x23980034490>},
 {<__main__.Node at 0x239e6060070>,
  ...},
 {<__main__.Node at 0x2397b7e3df0>,
  <__main__.Node at 0x239e602a820>},
 {<__main__.Node at 0x2397b6f36a0>,
  <__main__.Node at 0x2397b80cfd0>},
 {<__main__.Node at 0x2397b6f37f0>, <__main__.Node at 0x23980034610>},
 {<__main__.Node at 0x23900ac13a0>,
  <__main__.Node at 0x239801988e0>},
 {<__main__.Node at 0x23900ac1ee0>,
  <__main__.Node at 0x239e6060fa0>},
 {<__main__.Node at 0x23900ac1610>, <__main__.Node at 0x239e5f34f70>},
 {<__main__.Node at 0x23900ac1e50>, <__main__.Node at 0x2397b806070>},
 {<__main__.Node at 0x23900ac1a90>},
 {<__main__.Node at 0x23900ac1b50>,
  <__main__.Node at 0x23980624bb0>},
 {<__main__.Node at 0x23980193df0>,
  <__main__.Node at 0x239e6060cd0>},
 {<__main__.Node at 0x2397f141910>, <__main__.Node at

In [61]:
tmp_group = group_list


new_group = []
for x in tmp_group:
    new_group+=x


base = new_group

wells_list,pipe_list = nodeGroup2dict(base,output_format='list')
n = len(wells_list)


print(n)
print(wells_list)
print(pipe_list)

print('\n'*5)
s = Solution()    
res,well_status,pipe_status = s.minCostToSupplyWater(n, wells_list, pipe_list,True)
print(res)


2472
[92724, 84790, 56197, 61060, 29204, 90408, 93577, 89442, 67288, 54391, 91898, 65029, 86153, 35944, 4748, 85896, 12238, 81159, 82835, 36722, 46159, 86826, 50340, 90476, 95229, 38967, 68372, 7894, 68259, 44642, 99787, 40712, 48586, 66222, 89877, 78790, 22902, 2130, 11267, 83917, 94755, 50023, 86739, 8962, 7077, 16163, 77455, 12504, 68346, 70052, 14124, 6379, 23130, 81328, 30072, 63768, 65339, 687, 51047, 46000, 63378, 8722, 19141, 15602, 90039, 90728, 33421, 69677, 75962, 20384, 81421, 65414, 96435, 50520, 78240, 978, 42368, 55927, 50241, 98216, 78297, 47889, 31749, 99446, 34688, 76838, 96989, 46751, 65029, 63534, 61789, 53447, 50140, 37905, 30710, 38235, 79195, 90161, 18786, 58607, 26879, 79889, 80424, 7751, 55425, 33350, 6553, 37020, 15350, 44260, 5435, 140, 51568, 81166, 81783, 14935, 37277, 19161, 18477, 59464, 84684, 42401, 50209, 11235, 50583, 38500, 29587, 41076, 64481, 85144, 13611, 52530, 67561, 47216, 77011, 83182, 31150, 83699, 43773, 19666, 25984, 57229, 47312, 58897, 54

In [56]:
78007608 - 77658595

17975

In [62]:
n = 5120
wells = [21128,75528,97977,52262,52378,15538,38502,82313,47321,85603,57516,70456,3340,10685,14501,58724,42337,74534,6019,4958,57135,702,93577,99406,31850,89442,92604,98339,96751,92818,99355,69270,69868,52010,69654,68788,31959,72233,81473,89012,98204,77272,17591,88067,1682,66222,2648,39761,53101,86944,93661,24208,61060,27169,4096,61950,91487,5713,66947,92700,69000,22407,50866,30970,13669,67723,25650,12925,86768,18176,23777,47704,6001,28669,18491,41794,81633,98117,86153,84833,61390,56697,61686,16438,96154,89325,19969,34488,71395,80075,9290,42780,71525,98954,98949,5726,47989,36609,66733,90119,61939,93626,50667,21209,30272,16076,23587,31502,20140,67856,10240,22960,54014,52123,87986,67927,37322,50755,41996,82535,85198,9227,9908,8376,93639,65630,69549,98730,80787,14611,13998,57844,71866,29050,79414,35399,32945,81424,21955,41679,73049,11354,69358,18489,38909,36219,92724,51020,89519,43854,8018,1177,50616,24035,99671,51378,69801,9524,58,94173,21476,60926,95071,97791,80030,19941,13664,40706,47216,14832,11235,1618,25984,52530,54636,98407,18467,39794,71734,32048,43613,68877,1750,36231,85772,10250,82036,61698,4581,8832,61311,75892,50035,22491,94445,97822,57825,83237,26853,58666,92055,23491,5679,34791,6369,37423,20900,97724,5726,10614,38469,29887,78640,84042,26189,58524,51047,68102,33765,3440,20862,31749,61789,37905,90161,54661,58086,43710,74402,1787,8606,1694,84348,62614,69986,61834,24177,37259,66432,28044,54452,43908,57008,24151,74973,15201,34448,68761,76275,34812,82783,25593,65309,54357,34658,50861,40309,99643,49225,62866,56314,54205,7414,6946,42236,75620,19893,79605,292,63612,83196,30451,12402,36348,60488,35269,59872,86581,75990,85045,31249,82809,36114,78399,32493,64972,47424,91529,68309,29235,7094,53707,6607,64663,31229,23737,21931,99922,71809,91458,774,14714,2129,46905,36074,43740,10298,27174,52954,60298,51533,83198,55851,50244,14454,20660,16223,79195,53447,76838,34390,47889,83767,86683,23837,38197,87247,46525,69473,90769,46233,39895,90331,3856,5245,6039,27673,40562,74321,55080,71112,18073,10865,95454,9593,52441,74657,76750,79532,39354,56288,50675,18758,99925,37918,30710,51440,99446,42368,65414,33421,19571,17110,42778,15604,86739,22061,18133,49087,12508,8185,5696,68532,44608,10581,55129,36116,77450,8836,87767,28010,31303,52670,11378,32852,56253,74493,91829,77304,28107,5736,53220,40739,10929,12977,71900,19972,36336,63597,61179,38395,73330,3170,26604,25879,62880,81557,90467,88706,59362,78528,16957,71241,49440,15725,1915,26006,35380,14401,80262,37337,62783,92025,63896,30995,75920,74312,69685,27159,32062,70444,92693,52586,13090,59397,33566,86826,58154,22750,45680,24536,72131,96895,16672,63780,19003,16442,48544,93873,79506,90387,89772,28710,87459,4253,22432,5508,62846,19167,98522,15840,20970,21646,86268,95181,16273,74467,26720,45498,21394,27488,41400,40857,98905,38323,37166,58531,19807,90624,85466,826,66915,89977,67603,88858,54190,56391,53628,29255,44732,90461,87316,72920,10294,94325,46266,32320,84913,27775,46317,25259,62604,98683,1163,50831,20996,67535,28022,31936,9359,57997,53293,16166,90968,5610,98677,87204,32122,64223,49981,66231,94138,63795,34347,44108,16931,12803,52934,90050,10905,38807,56499,69434,75748,75987,28644,8009,9228,37462,36282,74376,24584,18419,36874,84941,10567,28562,24564,4388,89426,70248,97569,83634,78601,90408,24587,876,11564,24023,44134,92407,99304,59507,52676,36490,51811,98936,23078,23451,6912,91728,97804,45705,29015,6501,50648,93394,31486,85833,33121,77695,79898,97513,79456,93290,84536,31622,32336,39514,82426,42270,20375,45038,64750,60304,23190,92543,22771,50340,91463,87263,59310,30115,42401,14075,1839,19666,6201,91372,92347,47312,90591,45247,41791,60831,25649,2183,9785,95802,43927,83036,16446,72458,15378,24760,11584,30490,27446,56556,3947,90231,55869,96391,53320,81259,99427,68218,73419,51813,73002,20792,32167,793,37339,42973,29669,38921,84152,28165,63391,35823,3667,42668,52344,36006,66088,22539,35611,25678,377,10698,37641,4152,25553,19953,79344,76166,46159,94346,81931,29383,40563,1383,33101,24119,67774,75540,54,10297,14159,24397,95229,34503,17591,22643,53823,96961,99632,62902,37168,33152,76881,50996,11969,42557,28398,44837,30287,91066,90878,28572,50129,52533,87683,68314,50450,81159,34636,16141,97481,85268,65125,54871,56141,74785,15739,2596,30257,15052,40850,43972,46468,1478,12625,90174,87667,15451,6492,74535,47633,29540,68661,50153,19973,69822,22596,52483,51971,59536,48421,77723,55879,60123,75458,12363,49026,87090,17887,70196,82654,85048,29062,57160,36882,94244,94262,21539,6242,79710,84186,71852,95820,75439,88436,45074,86037,21724,60272,82634,94848,71507,45058,93257,20090,62973,87047,94777,74805,25555,33799,46180,51568,38778,50209,29587,48009,13611,77011,83699,68421,58897,30217,73538,51615,16195,47134,61167,16458,14992,95421,71095,48104,78506,91875,60334,4114,75527,18786,82351,35532,50241,50520,17270,90251,48971,88583,68346,3848,15025,70858,97811,58037,68107,31215,8917,75557,44214,53772,11833,19310,31803,82175,64860,28944,94860,68953,26949,44917,24732,60193,86879,64017,16658,29788,12836,22687,88036,1727,92302,90009,51181,58144,48586,39924,16946,92452,33424,77306,60740,78015,53771,95852,38535,88149,17595,63739,33295,17344,81835,33448,23512,58607,9968,13270,15219,66043,99079,98340,8320,43418,19105,42643,84442,75356,53399,80425,32583,13078,68747,65624,83734,893,54635,945,79073,87847,40456,74898,90253,45601,23965,9188,40894,3754,79019,99794,57332,15685,91816,83086,53987,43455,70315,33107,31030,3535,24937,10863,8045,84647,82713,5071,15733,42459,14557,64461,91699,16500,99259,52970,37304,4569,23097,13179,29011,88814,16161,3696,93996,93160,88189,69671,79403,62556,33777,11092,13584,6275,61774,7373,93411,71739,58735,50241,73185,20502,25261,86711,42480,39348,42682,11165,90124,39882,11329,60009,82621,81196,22652,17384,71993,8830,86961,23286,56008,21564,42073,97841,81333,36967,89008,64432,89462,31948,59357,50876,92984,31854,75777,54349,75612,2645,83708,73798,84068,18782,67725,51197,20242,78221,15405,88975,51712,53338,56229,70559,93094,42660,91704,31559,37972,79132,63110,24211,87803,59038,15146,80714,72374,73476,17009,52153,42924,35290,46721,63103,9738,13920,69124,11726,43399,80181,93036,60362,94235,27969,45172,23065,6605,16154,26321,68698,56945,10420,90505,69012,67396,89141,47554,69783,82993,4910,66633,78642,60171,97551,24823,8750,57266,74022,91159,57012,25819,35944,79905,10308,46203,47968,95061,15292,81099,74545,72865,27022,98151,11036,34755,58121,66532,41026,3114,54703,84146,15231,15438,9441,74611,63248,31882,92410,83667,89292,64519,76340,70337,31812,11008,76279,20963,31667,75575,38826,59449,64260,24512,27084,33892,94339,65486,7069,17944,87392,98921,78632,58804,4891,20515,45314,61484,68694,49231,94788,35669,52970,67904,36921,6102,58638,41658,73948,97709,50701,70696,58743,94303,62964,87389,73320,87552,16086,97919,92033,52684,65339,37318,90039,69677,81421,978,78297,96989,38235,53587,79889,44260,46184,32704,91898,93954,38755,31545,15054,40644,34988,61926,80204,32829,81642,28595,76613,51572,37339,38967,94174,73311,80507,10750,89006,24961,87684,69276,23445,33301,43699,98773,59384,30661,56264,90151,5850,18180,52408,54646,22614,67348,22613,24805,45947,29500,43287,14864,62135,84902,60446,18211,92923,76670,89875,49353,90483,48189,38758,88789,38625,14797,30270,92055,27904,9988,74765,30139,22782,16514,11091,10572,75398,90492,40731,97848,97439,4262,90919,63479,11307,69550,67871,99525,45190,99847,43972,84558,40801,70605,11570,15760,46120,69357,16621,13938,5398,23311,5720,83340,61227,93356,99224,80281,63292,72418,96202,63819,15608,6783,71776,85763,18057,91241,19993,63631,93025,83683,7753,32760,24773,82670,49424,25092,98290,13525,41696,62152,54031,36463,60279,20657,44645,60436,37415,23642,94797,85267,18304,99226,73993,98763,6658,87030,68355,31437,43022,29304,23320,75433,61626,42079,67876,58293,16608,51941,41930,40880,86964,49379,1077,15350,72960,13884,29503,39425,86562,70778,59841,34248,52260,74490,90093,53189,99288,80278,83298,99575,72594,4562,34793,95116,31373,51355,25006,17448,3263,44201,212,65868,34255,48071,77893,67461,72771,99698,77863,70132,61650,65378,26666,84315,85665,95930,54391,31646,2817,40276,6652,59497,25345,15928,46708,51400,58897,97320,55420,18055,6616,64022,86074,68372,33385,27765,99561,36675,36492,48263,50987,56466,33460,12238,55671,45851,96091,48186,41076,56396,15412,36161,42485,56796,56296,88870,46899,8233,2344,2885,41342,31776,35569,18716,95261,29438,73718,70650,6089,1471,68298,17162,68760,66087,37528,40637,26635,99070,31718,15189,55957,76064,22738,60905,42989,85938,97466,50427,27183,8730,96235,13517,84361,25325,79309,15929,77996,70506,79021,76536,23765,82921,39363,603,40314,44394,53286,29204,71468,57635,34317,63272,36569,95972,943,32101,86247,30776,61664,87536,56005,36357,91667,75500,67288,79466,95007,48684,30231,18924,70970,32506,76005,10590,29909,64396,23674,89669,18915,18373,14115,49806,71403,75369,64577,75182,6167,44909,93849,4498,83941,33918,43641,82119,63457,61137,26533,70646,73593,7797,68141,79481,22767,9325,10241,17247,19839,51908,38091,7748,48673,84165,20139,60560,64666,90829,57871,43645,98423,27581,71138,47389,59847,93497,17181,40712,3395,22902,78394,78270,98706,52705,89097,2807,74834,44882,65030,77593,22725,62296,6737,35532,30852,9400,77320,86042,81493,58511,29323,18102,85251,67220,89253,31261,52417,72485,51266,64426,62836,69716,29557,4970,75907,70234,11188,43237,25894,38253,56197,13576,9403,40668,47581,9439,91457,98782,20608,72358,4031,55695,43430,67698,97238,73269,36596,41483,51941,92234,55751,53951,67526,91253,87054,22002,25081,83501,26084,94748,83638,94522,1796,73818,70702,42083,92016,96918,78500,17090,53413,67636,55676,96824,83066,82011,23003,70910,81928,64516,25695,92082,28501,1118,11722,80628,87657,69458,55520,7348,87634,19571,37504,35679,69084,50096,1347,89816,99799,37729,79758,2035,23594,74602,83917,50023,8962,3168,12504,7853,6379,81057,86219,33646,54178,63859,65010,34130,70307,94265,87551,79211,34376,12890,54603,34827,67616,34647,89431,36513,50999,75722,34306,56047,55858,7534,89107,8928,94114,88974,8692,93547,39919,7773,81787,73825,47923,35681,36029,25393,94352,48007,47078,68736,29625,70072,71398,28919,31743,9530,94858,19626,81906,78556,76955,94531,81944,72022,40212,24387,29111,72259,10573,97955,37106,87612,18276,64241,31172,88174,9686,31806,792,67672,43420,94922,43945,24676,79613,99315,32058,36830,95527,71541,93854,87934,76967,94523,48348,11352,49499,22146,88900,32017,50777,28841,64092,77829,25565,56050,39276,35692,32393,14183,26235,32838,64666,3338,2629,27823,76624,87858,63386,66919,57822,93350,51011,1742,9730,91361,11598,67320,46985,78953,73631,30742,69237,5435,55425,82558,95691,77875,80975,23059,92475,73473,98447,70940,32906,59190,92867,68348,5556,57533,5148,93146,99021,46642,8157,59752,24720,97493,78327,58469,4701,22749,61142,61444,15324,44766,80137,74351,75926,6247,57736,93148,45593,46209,58416,55245,56892,18536,87139,90297,65547,11931,19977,54649,71025,10933,38213,6357,90482,94253,35511,52616,43500,28277,37740,96151,2685,37659,76487,39806,50793,10036,71092,39660,90267,17428,83146,20686,14443,78661,68681,66123,18875,82484,70883,5379,17918,66891,19381,68217,54588,53006,88269,66011,26968,57962,47844,35261,24554,43140,4818,98339,45157,89034,12249,47427,14749,36123,13053,58955,18342,65582,47486,20177,41972,66933,65616,83409,7520,70666,41162,14711,50553,53608,27560,87432,44220,19385,64854,85848,51834,37139,54625,48132,98780,4655,72750,45522,23574,49063,88600,85963,28537,43234,20117,12524,86640,98162,87622,12848,51964,45065,2120,67388,71460,63350,86999,91132,93082,95604,49405,51633,8271,85487,89843,16603,42176,59270,41587,52940,79603,97406,98598,39718,4251,63950,64811,73550,20437,44042,29838,3560,44145,39657,84037,4209,27956,48873,97320,14472,41480,61335,28242,75839,27906,31192,80590,26464,19645,62303,43108,6603,3271,83417,79809,79697,86997,87711,14463,63207,90294,68756,22156,54262,37616,74081,46293,52772,11267,58551,74985,30785,4630,68565,90050,91024,38200,25704,52810,17906,98031,54381,85656,53943,92632,87993,48477,88049,11941,43800,11045,89282,81739,67607,11679,53608,75578,63129,40714,99341,107,71926,31410,29286,77130,21499,42814,26904,22430,5573,72087,64072,72856,12668,64005,67179,15146,37712,21322,82204,28505,39434,60242,66862,18034,57636,33432,85578,73059,34883,27960,55433,82561,17058,63846,10002,4748,61897,29107,92117,19716,32815,82725,36591,26108,95863,86442,24730,51740,46974,13210,51092,63307,18524,98492,82518,32636,53624,96070,75927,94324,97038,37728,51123,33693,96446,83502,28173,43153,52575,39100,98317,36326,70750,39430,44577,99970,24960,90983,34050,61476,60377,24205,92270,89175,21395,86309,45709,49367,39804,25348,20130,86690,90056,88481,68610,7588,95258,15043,22077,35701,32950,59296,26148,96232,61941,14533,49683,14462,34807,47884,4539,87971,54225,91864,45356,91481,19425,77604,47517,71776,4620,30934,45565,90781,37093,76943,13574,98729,77965,59131,89289,90560,32389,9672,46518,41753,29209,2495,47792,42968,91282,49067,37914,59085,74008,84292,67233,74183,23725,56254,92658,99788,34515,66217,71925,40893,51299,81484,60402,9901,69941,62615,79540,67010,44048,40285,61933,33131,61030,26847,38267,38427,68019,83502,9732,52662,70270,21543,37753,7252,33018,5749,53760,71683,13714,49260,21760,51785,66894,52450,14935,18477,91117,30,76977,19710,66006,83182,92762,57229,88468,72434,27652,87346,22963,88011,5623,4713,79992,50880,60056,3975,68946,61503,75702,45244,4232,8881,49090,78240,16620,71194,8722,63768,81328,67869,94755,58265,37032,31670,18900,18995,24721,99862,44229,92752,34348,70048,53240,72430,92202,23,39098,2296,54344,50705,62238,22891,64614,51416,17044,34322,46537,46347,69272,59164,39471,6751,14592,94381,80003,36032,68365,66010,79635,1807,44614,11912,64073,41726,24672,14534,1574,25125,40782,45502,67397,83307,21575,39320,35386,91056,77997,98801,39821,73186,54941,11349,62029,74196,19504,60240,51942,47773,34788,69325,53380,42650,52871,82481,52491,68674,78422,36432,10157,94419,41688,87639,3077,96721,59910,45456,95875,81095,70111,69307,44742,72594,12507,81902,79885,77407,89444,99409,57156,95938,33157,75193,80765,76643,2134,92626,96024,91929,45526,2481,48015,73769,36175,34060,41579,40862,30987,57966,57108,31040,32493,73768,22647,77611,81269,3727,31319,2788,85671,1550,91767,2060,91174,88800,36027,278,63106,61366,30825,372,58844,37840,74682,69582,8124,89595,25539,69754,58881,97901,65104,37638,42371,80762,78709,67005,2639,93094,7934,48969,24945,17455,54564,94063,31628,79576,62928,12150,44378,5761,83162,58070,83952,76246,89737,44145,35155,31442,98690,4559,78946,37369,18335,99094,70935,30451,91512,52094,56661,99509,32490,72150,98741,60800,61749,37544,836,46036,98983,35019,70565,83783,50794,74373,76206,69330,30868,23631,76206,25542,69841,8844,55755,29633,16328,65545,68985,14548,37910,99093,32966,74018,1496,724,9972,97043,73574,19831,22218,29644,59,64508,63119,7295,90883,902,14472,75683,33424,61851,22662,10033,40416,46165,78138,10410,17872,79971,17276,99185,92762,45786,84684,43650,53479,72079,5388,47521,50266,35462,46298,46748,75288,277,10399,41142,25991,7659,14395,50662,53096,99787,85113,89566,76998,63888,52547,70722,35917,47566,12493,94859,29722,75458,87735,25570,29977,39146,82507,64562,67560,8309,17455,64868,1028,72160,84062,47017,45221,25866,64962,6166,48983,2166,20396,97398,25129,20483,68305,7220,62933,92988,98093,97713,99462,8880,81503,79519,56986,31881,24836,71154,90968,8925,10155,99664,17661,47209,19141,84285,96435,55927,45206,65029,50140,98098,7751,33350,38010,45222,75128,51682,98162,62081,67166,42538,19287,92037,44474,33631,57479,17408,14811,89646,11304,9015,83119,9634,856,50989,85187,72807,50417,13607,40718,90950,33187,20470,42955,93058,88566,9267,37255,94204,11603,62841,1341,45421,27349,60869,56422,52117,46688,56547,97762,82184,80347,86153,14670,71562,38909,19882,25599,51008,554,92186,60684,75790,69902,5906,87784,54386,24828,68036,32485,4641,21748,48737,32362,4673,64196,53621,55092,53621,41687,79798,35481,70316,8771,42738,84152,40746,85060,921,15094,87178,59400,21312,74387,23142,66761,77467,29193,63887,22089,79143,57833,36849,18925,37113,11375,32311,67069,48409,76371,70844,18594,24975,30610,73192,15594,7279,40164,63534,83854,47889,23665,67277,15602,46000,31993,14124,7077,1295,52849,96087,55169,14966,1937,6924,82161,55482,62459,77542,11373,83571,68837,82138,41658,48654,52470,57401,60478,38162,81217,13422,91103,20326,84553,35536,13638,53079,61523,85495,91430,20907,4641,88302,15108,17678,22933,288,68523,46200,56806,38298,53443,70613,62562,32996,87451,90425,82910,76968,42735,45889,5564,79522,53787,52951,52868,61298,24467,33897,19426,95719,75753,24456,38083,8540,73329,85622,91612,68821,95612,63791,85748,23294,39552,88423,35241,14057,3803,22588,45023,72259,95016,5355,5576,2381,95050,30646,84734,25421,17750,77872,48873,5430,26093,27439,23696,35630,73749,90260,52953,4989,28523,50330,58968,33821,2873,52426,60092,74590,66956,70998,35820,31882,26054,26563,27709,5697,94421,33199,52004,45997,7390,78934,39495,95019,67882,70302,31972,33059,38188,92355,4310,82834,64121,99908,55510,38983,75328,59823,38474,35409,69678,94048,89858,23507,8962,66673,61917,37641,21241,64437,67867,57187,29146,83277,14917,80060,71933,62529,42437,66735,38978,77265,54602,15810,27158,13003,47161,5905,97220,75586,13371,66901,78146,85225,49487,29131,26311,44404,50095,75831,17716,80676,71769,62170,3060,44213,83884,2271,28941,83628,67297,35473,9810,41926,91269,94833,91527,14541,3056,54069,22129,23060,18354,70078,47906,95235,33930,62060,39971,81442,13572,30083,37232,52203,14277,68192,86450,96615,88266,17705,40882,8317,17799,42953,78266,68182,91105,307,65762,86308,4501,49946,63575,79013,17145,35705,66913,47836,28979,53816,59823,90208,40034,26375,25131,38148,20604,91033,38677,9427,88544,52195,11291,71449,61757,20198,86680,76855,13902,23841,85046,83161,69578,55935,35367,95540,37388,40601,49481,29562,71865,79124,3467,1374,39768,66346,41237,96376,34719,55147,12156,87346,36489,95540,74797,37542,69489,56788,90254,14856,24395,91439,25953,3030,75424,38736,87710,3721,9944,97187,63589,86659,84201,37363,73009,11055,1279,75794,21122,14557,37020,84352,52697,92986,55206,88807,68447,81550,51892,77535,37285,84591,53494,76655,12243,77960,41655,87936,14095,38370,53638,30388,8438,17631,37950,9973,57453,30387,44642,29163,52334,10004,72158,29199,41541,60221,71064,54534,74471,9314,52249,49077,20818,61399,14838,53537,53617,26242,16163,68726,70052,30511,14824,42981,49817,86533,58579,22946,4592,5424,82076,38727,22929,60457,23953,8640,71032,91248,72338,34507,13618,98256,65892,40759,67481,17027,41414,56238,5704,18903,30758,39670,65266,46412,22397,55411,16550,42192,15645,50637,96232,93465,33712,35829,5753,20259,91147,67798,30498,24047,95231,49812,33776,98921,66478,77036,61962,12944,58802,36798,87314,42400,35436,59065,74320,529,94954,56911,98352,66407,71811,61428,90216,61927,9629,78414,87533,2130,13844,2739,68144,59749,68782,82357,22263,59333,72011,84676,38617,99730,98271,11583,5449,24466,89532,71396,70,92693,47326,89119,31097,7516,10951,78232,31229,69627,25953,27923,39917,93665,36088,91566,65819,35045,81989,15525,42815,45446,57392,23651,76214,20471,50028,91889,55723,59182,87938,75573,88730,80424,80905,32160,75726,84790,90064,38436,65043,8899,13568,20759,40620,55390,42919,65040,39531,74169,72514,62058,7239,65214,9870,91044,89263,54000,66248,60112,50703,60109,27628,24071,4413,56252,9631,93167,39080,13937,21509,61919,52693,95432,91423,69858,90216,1751,41848,40512,16004,41812,33395,6984,61922,92481,6523,90063,85178,78092,84525,54555,20106,79362,41937,40680,58008,28539,31414,85396,46811,14339,19476,65131,11410,12725,69293,95961,70908,69982,29716,76319,1280,41564,59056,19274,34312,22856,17685,52300,39328,90096,76208,42706,79285,31443,64749,46330,70489,28147,13432,61560,45818,50399,85065,69796,99013,4211,50930,6575,44928,19382,73888,23945,60222,55596,38833,38516,35713,43468,1315,37091,52416,89688,27658,49344,3277,6924,24083,4024,45094,46511,95910,41865,1276,31258,98099,54524,55655,80741,77967,87767,86067,11357,63372,48613,32980,96141,48195,10632,78580,14005,6596,56109,958,64485,92986,97413,5373,58154,63043,79631,25745,77215,98929,77738,20535,54438,78645,9919,62495,12414,83591,9693,69570,87240,62730,12750,90921,19100,15691,12141,65027,16469,16275,29422,29579,91147,16486,44133,24889,15823,73537,59632,38869,67303,32253,87318,7449,59751,92000,57284,59879,57547,85419,12956,17892,72398,52079,7696,68938,72632,46570,37455,4847,90911,15298,69960,74862,80849,44691,86966,51584,7027,39784,61434,32931,72636,12106,91949,25868,16181,15711,1398,36397,32480,63799,95284,11559,24211,60099,34886,88922,26551,46385,39425,57722,50215,33376,44508,47020,88448,78103,5111,271,9960,14240,47131,61434,47215,55257,6417,30439,59464,68583,43773,7894,81783,19161,41076,62157,25615,19706,4594,46319,37257,39213,86849,94477,36802,40846,71856,61206,82575,94911,16566,34953,49861,80548,88561,94718,98330,66056,23865,48949,84409,67127,51217,34208,77806,82734,89247,18289,65479,60489,33523,65897,93971,48013,88725,93437,35070,85302,75788,84270,75712,99343,35849,80954,67543,64979,78724,94960,62146,46754,78790,64210,1426,3539,39852,6782,98449,31139,84217,24245,26652,61990,89290,4646,29023,61764,84115,39211,6641,65885,88724,1532,32465,2076,31387,63334,98997,3737,82569,58311,56461,8666,2160,88393,23915,63479,84364,45463,64733,66762,65776,69803,82684,67648,69721,82981,88118,7315,91024,22481,10811,46392,4335,33746,70292,34233,7372,72673,65877,63502,92207,48611,4825,4005,60406,28520,15672,20981,98579,43859,21454,46138,49424,5096,77314,21855,52241,85245,41438,5910,17767,11056,1658,74383,95020,33170,61827,24351,28630,78857,73565,24519,39613,39850,49098,78222,62004,12384,70823,50736,91128,78818,82819,76602,56808,45062,5949,16140,83728,90690,37126,17748,25034,17198,140,64986,37277,84684,38500,85144,70229,67561,85312,22664,97716,79676,56735,71555,44389,40859,99313,93215,215,53499,4981,77081,85505,94974,37356,50899,14165,52473,20842,18063,24077,42600,31831,74702,69822,47774,57847,75284,4221,51211,15898,5846,6761,7236,44461,57589,35286,75191,65596,20583,19437,71837,4555,7100,63527,32833,44053,57011,40634,58428,29431,30463,1193,55281,39153,86994,30185,22081,66485,34032,22519,21799,74942,86685,72624,73936,12792,29561,47571,65188,42324,12950,45723,46150,6526,73437,29723,27386,62676,84701,89394,39305,78134,43147,85629,53964,78979,81195,65863,60695,61927,26684,58651,34614,14697,58233,20356,50110,88109,54215,97414,88645,62213,92478,92034,53193,98721,16421,37804,98510,58299,57558,13455,97737,14057,99604,65994,86153,67272,55621,82583,18364,36714,6979,48761,69118,6811,60381,97943,67346,9201,15301,47501,30010,79740,30878,40696,57019,87286,56184,76333,65022,41991,96417,24238,94972,43582,79752,64115,79161,16824,10262,12206,5864,78301,94350,60832,77655,67572,65833,99506,17100,87807,37696,81933,20085,63746,49165,1962,49837,32257,36210,89085,34859,54635,22291,34452,95021,55980,60352,99429,34016,12184,24039,65951,32879,39819,44390,15488,60109,2250,65366,96790,28477,3101,65864,95909,84900,43296,19649,61896,23238,21883,83856,41951,80073,70217,55637,27776,84955,97233,31659,79663,62480,54910,42329,40308,5110,53744,59013,15541,28405,92828,21449,67874,15321,33049,39504,37615,50391,37307,65416,16304,38357,86734,79546,93282,67193,90754,86931,31588,13000,6884,71565,67520,21148,71447,19868,62259,40917,26731,80988,69143,24415,34276,91080,43835,83296,21658,36551,9081,21128,11569,8783,11178,35165,41674,69369,26315,94647,84601,87098,40285,84766,88971,50583,52459,77660,31150,64481,43667,81166,67013,25659,38168,3906,6001,77015,46570,9818,99108,65659,91776,22567,16943,40380,35876,97739,45779,34494,1175,33688,97673,82939,4797,17673,68318,63741,15092,67403,62171,93647,37611,38823,66052,16070,31630,97142,34754,45263,43574,75819,11586,8377,55469,60822,42990,3121,17208,87130,76657,11331,10329,39243,49047,92234,20450,47257,6783,13608,33385,18497,96710,88915,27737,92357,65528,49701,8700,63148,85518,54204,52750,66585,63378,90728,54041,62584,56387,77629,34688,28587,55506,58607,37020,14849,35642,66760,27094,85536,81623,82269,79952,57518,75732,98821,11282,21708,50659,69105,82248,72216,35096,53729,40435,57711,34978,22487,88482,99431,33159,55036,27791,87780,58914,34225,59246,77153,20312,67532,16414,66885,31514,20352,77226,7184,97663,19982,86141,69841,43923,12305,344,95181,97916,631,48324,86704,20333,68493,62557,6229,40285,33575,60122,35958,44132,75580,18273,32511,7332,40778,61307,38826,15430,5272,58748,2765,84425,37602,71905,50871,19840,15291,20505,56193,75538,86994,96945,89850,93935,47791,34148,83936,56757,50698,72638,51296,63868,71512,64289,16861,54018,17201,47812,35626,72992,11308,87302,71411,27793,77758,1885,98490,42434,52292,20551,81253,36600,26879,71292,46751,98476,47684,20384,60749,93371,55041,31423,48033,15030,69255,78059,15085,97734,20409,99042,9226,39539,65063,15928,61686,52652,25451,38708,25678,76740,59837,2527,99947,49495,10804,14228,98739,1935,80138,78277,25252,65263,22754,94645,64649,2988,589,16807,87639,72962,4202,85100,2565,52295,51005,86616,6615,63571,73961,15299,61909,39942,33179,79553,42456,90042,52345,3623,9190,6751,87268,80056,17307,65339,70152,30028,90663,27947,94233,36203,20312,22578,60781,50402,64770,22741,58316,24762,40033,37858,65451,55924,37534,99646,35116,19876,6332,73889,48003,21625,81732,9129,30648,13203,20011,17916,80115,12108,78876,8516,39464,41078,28200,38436,90383,25873,97190,18158,57244,21031,26680,3992,50990,76496,34162,38304,5096,55735,26267,80526,74898,37240,32162,65571,69621,49903,83551,85054,9842,53489,1422,43963,77659,26865,54481,67693,55902,42595,49745,42001,7877,47271,60199,72747,46409,78426,96231,51161,22179,43747,86724,69346,75197,15754,78972,94887,25979,74197,22549,44259,6630,1264,57163,56186,27236,65341,80380,83245,68321,87749,99314,67032,87421,36086,63435,59477,71354,22197,46526,22336,84842,35152,51577,19450,21407,34209,31621,1970,51608,8186,39645,94610,80118,1181,63198,66244,21551,67514,98781,81496,33265,12263,19389,85896,40260,56074,39478,60499,8466,15322,35168,22061,49372,79995,68183,30470,35873,66253,92112,55239,28734,63869,74422,10327,58745,99513,1851,89919,96368,66541,51372,7425,14975,30662,44248,79217,7575,15461,62990,45577,76424,40555,41462,20780,90780,46334,81230,12844,64025,62602,7801,69675,69425,6809,99807,41450,64339,9553,61207,22305,62112,91728,70450,62487,6882,82249,82352,44667,80189,67688,21590,16017,46361,46685,12810,51417,8725,80639,39093,83757,72362,65684,78483,19829,72481,5195,71221,94648,39825,52115,36722,44492,78173,5636,83113,46509,66445,74201,13467,51882,27952,19941,2290,58966,58705,27279,6701,63675,69584,32431,32362,24999,64996,5941,38935,46882,69950,93729,84902,93202,79468,96861,92789,83527,14532,36853,50228,30072,1406,27019,95524,89877,9354,60251,60112,83493,60016,49557,78042,35210,60794,14965,56891,78142,46680,65392,12355,25844,53129,28127,83808,62718,430,72997,45391,70729,44967,93516,46554,34551,63939,11622,35100,43476,31195,88034,44604,56542,28924,9553,86181,55787,44092,75674,8530,51856,30789,63315,67796,10190,54537,81428,62298,2826,80967,54956,13040,74375,37974,22827,80755,6767,24337,62940,35470,60734,98327,550,82739,66199,96684,35874,14648,24666,71733,73243,66182,98242,16020,33092,6553,64541,41616,62969,90440,23874,18308,63988,61921,71731,96330,81274,24542,21093,45101,97560,63704,71507,2632,44914,85538,56867,61886,9895,59770,91508,31713,1648,79980,38503,54297,53649,49342,57272,96315,82835,21519,34409,70242,83629,2125,24545,29118,50842,91458,48664,65802,81514,55461,79547,76147,93929,93834,87581,81279,2369,73991,64939,80337,16143,93021,52943,34972,64429,46756,69125,91955,21872,57040,8277,82102,15986,33022,30067,28810,39087,20150,91277,26679,91667,77151,97365,71976,6151,25083,94425,25161,6358,19464,36560,60205,46734,80530,90413,71582,10605,4573,50793,75696,4142,94,47430,97037,12452,50719,93703,92317,98179,56545,13635,54749,55772,41390,84185,65341,90568,8566,40283,11947,4181,22598,3789,57401,7874,68754,64245,13834,81532,45248,44716,43018,7677,98844,8325,66173,90398,11773,20168,62552,35592,17722,90476,80442,16750,60557,78518,51390,80931,64500,23618,78859,67890,24760,71865,26988,3336,28811,20466,1458,86352,11701,73112,22574,48044,10514,11157,50814,12299,76122,8159,84949,11641,65478,76282,4254,32553,8659,40415,40736,70092,6880,57519,30762,23437,93521,95582,34410,87433,40356,36619,27172,8729,53434,94080,91305,34359,9910,57695,19429,69541,92358,54737,107,52505,20220,41853,59256,15869,12962,98921,53892,92398,8873,89592,40109,77779,24441,24548,38231,68151,84184,63291,18312,84807,27431,43834,29498,96210,26227,43342,68259,2223,59505,36815,87077,42863,35869,19938,29866,90696,85565,4391,42027,1767,86753,59549,49910,15950,74330,85610,71181,31813,34641,43897,69463,74265,122,50100,87541,2636,19103,10499,83777,26702,6900,50459,69182,71281,76128,3679,45613,91349,80008,26930,62451,96895,82128,26558,84845,11932,60692,49608,98772,65609,33281,19180,73051,68029,97250,33612,29706,66441,88862,46343,35617,68696,31902,49127,46744,98216,66352,75962,98126,687,23130,77455,60698,15602,85475,41110,34790,20172,72425,97478,87163,69627,22512,54505,57425,2628,3435,77765,76876,82620,46507,1768,61103,11723,22407,39724,30084,52989,41098,8072,91150,40577,12416,59169,57222,71173,48755,85597,9720,32363,83050,22514,42876,10692,15866,22845,73653,5473,81825,89923,34049,89427,97336,15802,50059,74560,92333,21117,88035,23304,99272,72767,55202,18104,834,4967,43767,22128,79102,24115,87726,52628,75631,87913,31640,50689,99790,21459,26636,39516,820,60504,57633,49557,5486,13090,56078,15882,1801,82471,29914,79397,30762,24029,78630,13711,52041,99675,14764,54417,80684,24699,45287,57680,14773,48437,69743,79483,81854,65548,47088,56030,18712,40025,12189,11977,36866,9402,54647,57579,5240,71557,21134,71560,33030,69395,51564,2841,95428,41459,42797,2729,97963,14516,54867,11646,28343,75107,77785,67319,94933,48069,79944,63892,31982,20253,56685,98010,37426,22251,81046,27472,34048,42116,28147,70139,83482,60935,13322,44296,18351,66464,1989,65029,18875,71808,6522,21280,95759,73978,99212,78241,97990,85089,94966,12160,54424,8410,75102,82116,86704,9406,64458,33661,77597,67972,55012,9965,62058,45513,34931,95936,64923,40570,59206,26266,94026,97975,96992,83193,27368,87254,83971,23368,25015,7945,14194,88599,95306,99719,96550,42961,22015,90071,90435,78000,35605,44032,48207,18629,80421,31648,17061,3805,15114,65016,24681,92855,86274,30927,46413,93204,41627,90116,82316,79212,4821,67308,96416,24863,46486,28084,91464,90111,51754,88853,93820,27547,58952,11521,11136,63803,60684,31816,78000,71484,77553,32201,21700,88097,13442,84322,85018,39427,73720,88886,64031,40618]
pipes = [[2,1,29701],[3,2,97954],[4,3,14600],[5,2,22110],[7,3,14412],[8,6,35742],[9,4,2560],[10,4,30822],[11,5,24812],[12,1,30583],[13,6,70159],[14,10,51992],[15,8,1765],[16,11,68435],[17,13,26992],[18,15,46846],[19,4,73049],[20,4,96887],[21,9,7896],[22,21,87635],[23,15,29266],[24,4,65804],[25,22,38626],[26,1,23842],[28,12,13943],[29,8,95626],[30,23,90303],[31,9,86959],[32,24,56993],[33,22,65478],[34,32,1745],[35,20,87906],[36,26,95020],[37,1,32630],[38,19,92967],[39,38,82313],[40,27,73805],[41,15,48963],[42,1,99891],[43,19,34162],[44,14,38087],[45,22,57011],[46,7,18719],[47,37,77119],[48,8,19496],[49,46,21348],[51,2,47063],[52,45,49034],[53,23,75239],[54,48,41897],[56,35,66432],[58,3,84395],[59,29,4102],[60,27,35845],[62,9,47710],[63,61,11283],[64,26,91789],[65,59,43149],[66,65,68682],[67,39,30519],[69,61,5680],[70,32,43347],[71,66,84590],[72,59,52431],[73,48,19756],[74,19,67496],[75,57,87276],[76,68,40757],[77,28,74982],[78,71,91211],[79,62,29080],[80,46,88674],[81,74,11198],[82,31,25532],[83,79,43847],[84,7,50451],[85,64,65665],[86,56,39159],[87,82,52212],[88,64,83462],[89,29,97270],[91,83,33747],[92,44,89431],[93,68,84122],[95,28,79585],[96,36,28242],[97,45,62811],[98,16,48256],[99,51,74903],[100,80,55040],[101,83,38146],[102,70,77445],[103,43,93110],[105,31,9519],[106,34,34595],[108,101,87776],[110,36,37237],[111,28,90199],[112,62,74313],[113,82,33433],[114,32,61179],[115,80,84718],[117,86,13810],[118,54,28086],[119,59,85376],[120,115,86910],[121,53,51790],[122,28,29483],[123,101,91981],[125,124,9644],[126,7,73538],[128,1,23474],[129,25,60080],[130,81,75982],[131,56,81311],[132,115,74974],[133,109,69225],[134,128,25868],[135,111,19432],[136,118,92403],[137,132,86544],[138,38,22154],[139,88,31854],[140,98,10505],[142,64,60703],[143,66,80450],[144,17,71173],[145,83,91572],[146,22,58132],[147,87,77145],[148,82,48422],[149,43,95248],[150,77,61978],[151,80,76738],[152,56,73090],[153,46,25304],[154,47,79013],[156,66,77969],[157,147,19431],[158,97,38563],[160,60,12524],[161,64,31287],[162,80,76222],[163,143,32137],[164,66,43935],[165,97,24477],[166,159,94280],[167,9,76610],[168,167,89761],[169,27,67260],[170,34,25753],[171,72,92307],[172,170,67935],[173,9,7909],[174,46,67848],[175,93,78956],[176,72,75403],[177,89,21917],[178,100,82491],[179,40,67362],[180,62,5625],[181,84,69468],[182,100,9665],[185,109,95626],[186,174,62761],[188,70,44662],[190,170,40719],[191,2,35724],[193,81,35502],[194,106,40461],[195,76,55697],[196,97,48251],[197,186,31468],[198,38,28268],[199,131,92398],[200,98,93630],[201,74,32983],[202,129,24667],[203,160,98338],[204,157,83564],[205,76,34448],[206,144,18206],[207,54,21036],[208,154,5146],[209,17,88955],[211,110,81634],[212,169,49900],[213,137,49365],[214,2,2743],[216,102,70265],[217,6,7473],[218,158,93597],[219,218,82025],[220,57,93583],[221,115,17511],[222,149,43398],[223,20,39531],[224,128,98386],[225,158,2478],[226,129,54275],[227,84,57178],[228,76,67722],[229,181,7814],[230,144,31563],[231,176,80503],[232,121,7717],[233,174,77445],[235,197,58732],[236,208,40261],[237,102,36161],[239,113,98426],[240,43,5950],[241,234,44069],[242,215,20773],[244,123,93483],[245,21,24114],[246,16,56866],[247,102,83242],[248,207,42104],[249,228,33878],[250,33,70301],[252,43,12339],[253,56,66095],[254,181,63542],[255,62,80572],[256,254,49480],[257,202,28620],[258,175,21567],[259,202,15336],[260,145,288],[261,198,40005],[262,189,68227],[263,232,50808],[264,72,26817],[265,98,22347],[267,112,30440],[268,18,40357],[269,116,96319],[270,167,82039],[271,207,37300],[272,7,84935],[273,137,26433],[274,195,20485],[275,155,3191],[276,213,1272],[277,207,59565],[280,144,99450],[281,59,91363],[282,201,46568],[283,263,40687],[284,274,43775],[285,159,71086],[286,85,17273],[287,87,16543],[289,10,22913],[290,6,75939],[291,36,94387],[292,30,65390],[293,6,90943],[294,201,39029],[295,49,38436],[296,223,55232],[297,24,21950],[298,273,29748],[299,11,79111],[300,239,1199],[301,157,58695],[302,112,95981],[303,14,27236],[304,40,33067],[305,132,9606],[306,167,91230],[307,176,73139],[308,103,41108],[309,260,57700],[310,34,29689],[311,99,85869],[312,247,27571],[313,91,41434],[314,114,39683],[315,24,81282],[316,205,46011],[317,24,18774],[318,289,49499],[319,35,35476],[320,283,37597],[322,257,74891],[325,322,47060],[326,123,7734],[327,273,48920],[328,239,97516],[329,249,15805],[330,260,52246],[331,138,43065],[332,234,95662],[333,301,89573],[334,148,58966],[335,175,22325],[336,288,90484],[337,211,52119],[338,49,62983],[340,59,75987],[341,180,78837],[342,13,24516],[343,60,20307],[344,263,58061],[345,20,9200],[346,205,71813],[347,258,77860],[348,92,46561],[349,155,95175],[350,14,12250],[351,280,92298],[352,117,33641],[353,329,68156],[354,263,36526],[355,270,15815],[356,23,42824],[357,131,24866],[358,237,64697],[359,350,55239],[360,341,9797],[361,136,89544],[362,73,43099],[363,207,24181],[364,218,39615],[365,273,73084],[366,6,864],[367,84,95143],[368,161,43237],[369,200,60149],[370,207,10091],[371,248,66612],[372,213,87257],[373,318,91407],[374,207,19219],[375,155,27372],[376,223,74371],[378,18,10894],[379,118,87212],[380,110,7549],[381,18,27123],[382,71,61836],[384,211,31408],[385,88,10431],[386,329,86168],[387,88,62655],[388,199,73434],[389,321,31213],[390,285,46509],[392,157,90573],[393,186,86501],[394,251,52085],[396,213,65834],[397,14,10629],[398,196,58288],[399,189,401],[400,29,42458],[401,24,39544],[402,247,33084],[403,293,90173],[404,391,70184],[405,359,19688],[406,230,49814],[407,233,58833],[408,294,40730],[409,302,26525],[410,10,91271],[412,16,92204],[413,89,19524],[414,289,39725],[415,66,20872],[416,352,10054],[417,277,31609],[418,310,93549],[419,138,79149],[420,48,84584],[421,257,49358],[422,391,46325],[423,359,68552],[424,232,17270],[425,384,91335],[426,144,83940],[427,90,98409],[428,396,16607],[429,370,78462],[430,33,8117],[431,152,9942],[432,424,69632],[433,30,85543],[434,11,5518],[436,25,81096],[437,292,82378],[438,51,77858],[439,123,85794],[440,248,42626],[441,387,63758],[442,214,10604],[443,215,41259],[444,116,54035],[445,114,32353],[446,14,49598],[447,315,63931],[448,291,54049],[449,111,30348],[450,158,17723],[451,82,14597],[452,83,69424],[453,327,86389],[454,151,81918],[456,189,96242],[457,70,19960],[458,16,28629],[459,237,93187],[460,159,73685],[461,58,68123],[462,235,56918],[463,398,76694],[464,150,63931],[465,74,442],[466,56,52931],[468,425,4396],[469,194,11191],[470,149,43474],[471,363,54711],[472,172,6269],[473,83,33667],[474,318,6513],[475,405,22053],[476,122,40227],[477,402,19227],[478,95,81886],[479,434,67585],[480,325,57839],[482,471,37584],[483,4,30096],[484,111,47372],[485,203,38591],[487,366,6141],[488,377,54203],[489,59,94963],[490,418,23678],[491,307,17408],[492,14,670],[493,437,77484],[494,280,82940],[495,464,13894],[496,450,64933],[497,305,93614],[498,25,53580],[499,132,39323],[500,371,98644],[501,135,41593],[502,231,47066],[503,87,79360],[504,159,9643],[505,275,38103],[507,248,54503],[508,84,49864],[509,454,71371],[510,380,50407],[512,205,67971],[513,467,97712],[514,374,33787],[515,15,28496],[516,60,76410],[517,348,99842],[518,260,89522],[520,113,47359],[521,198,10440],[522,296,69916],[523,63,83891],[524,60,89749],[525,391,54598],[526,175,5846],[527,153,82127],[528,383,38116],[529,117,16063],[530,399,79407],[531,161,22127],[532,230,76530],[533,85,80682],[534,273,89040],[535,495,80592],[536,109,39842],[537,57,15843],[538,250,96631],[539,493,83378],[540,263,69270],[541,35,61883],[542,159,57255],[543,95,27086],[544,215,6833],[545,73,35151],[546,243,9761],[547,452,24856],[548,84,31190],[549,12,6332],[550,405,37086],[551,301,66342],[552,401,63657],[553,276,66852],[554,22,3203],[555,228,37338],[556,176,22010],[557,381,66493],[558,75,66208],[559,364,23276],[560,97,24924],[561,233,4120],[562,107,67561],[563,509,1127],[564,311,86892],[565,147,13695],[566,440,92139],[567,200,90507],[568,252,74287],[570,161,92535],[571,131,27284],[572,155,30566],[573,462,48670],[574,100,90645],[576,276,85273],[578,174,49536],[579,111,21260],[580,398,2781],[581,525,26866],[582,544,32893],[583,444,84344],[584,387,269],[585,468,60842],[586,6,37878],[587,575,473],[588,106,22681],[589,185,27433],[590,84,85785],[594,227,89197],[595,194,69847],[596,33,39904],[597,105,19893],[598,45,56127],[599,435,44844],[600,415,50742],[601,556,36073],[602,558,1862],[603,448,48189],[604,373,89934],[605,530,56263],[606,106,66369],[607,462,29590],[609,343,81848],[610,495,3754],[611,403,68366],[612,356,4017],[613,400,25750],[614,465,9509],[615,435,72301],[617,541,79715],[618,424,63813],[619,81,47434],[620,152,47336],[621,251,98166],[622,299,69146],[623,201,40688],[624,542,15112],[625,504,14571],[626,276,28761],[627,84,40084],[628,538,82437],[630,104,83542],[631,62,42658],[632,195,89255],[633,197,80995],[634,96,34338],[635,511,49439],[636,200,40008],[637,321,82504],[638,9,63149],[640,191,31739],[641,205,86562],[642,315,89392],[643,199,63529],[644,275,27480],[645,616,68136],[647,227,36936],[648,268,35143],[649,36,63126],[650,370,84467],[651,149,31083],[653,26,53705],[654,12,56687],[656,163,51549],[657,167,88308],[658,78,60839],[659,324,13943],[661,584,22839],[662,317,89481],[663,163,94885],[664,288,85440],[666,607,6691],[667,598,31223],[669,217,86682],[670,414,23396],[671,418,69334],[672,670,91478],[673,53,4716],[674,559,16589],[675,207,79062],[676,111,8928],[677,543,143],[678,229,16663],[679,49,81576],[680,661,70081],[681,667,24489],[682,553,2672],[683,140,69484],[684,102,59446],[685,363,25059],[686,6,62764],[687,283,1382],[688,397,52696],[689,230,58658],[690,176,90750],[691,608,1352],[692,159,36270],[693,450,84846],[694,344,81318],[695,116,90234],[696,404,9678],[697,519,29049],[698,571,94453],[699,644,96083],[700,86,46202],[701,613,99800],[703,277,38585],[704,73,86680],[705,197,43707],[706,25,80290],[707,117,46972],[708,600,85249],[709,90,76060],[710,216,49175],[711,254,96266],[712,266,90950],[713,225,95130],[714,295,35117],[715,16,94893],[716,331,13503],[717,659,11049],[719,590,90696],[720,491,72175],[721,316,86679],[722,12,39992],[724,177,11080],[725,568,26948],[726,322,27388],[727,76,27145],[728,266,75008],[729,527,12535],[730,214,12371],[731,373,39834],[732,454,75407],[734,42,70914],[736,101,54875],[737,4,20308],[738,565,85175],[739,123,60506],[740,631,27167],[741,468,61452],[743,442,35717],[744,693,29093],[746,662,31228],[747,617,19151],[748,246,97610],[749,674,21279],[750,387,46233],[751,546,39260],[752,310,69736],[753,566,9630],[754,257,31508],[755,668,84160],[756,35,44004],[757,658,83541],[759,634,3962],[760,347,75632],[761,49,79612],[762,192,24690],[763,148,4250],[764,244,97829],[765,421,1066],[766,580,7403],[767,216,37692],[769,711,22687],[770,404,88094],[771,671,56506],[772,755,58255],[773,145,52921],[774,609,83960],[775,680,96586],[776,703,99713],[777,599,52375],[779,144,44736],[780,632,89757],[781,36,5731],[782,351,30366],[783,771,57131],[784,310,96517],[785,535,99809],[786,619,68589],[787,396,71272],[788,431,57985],[789,129,27813],[790,369,70912],[791,281,92982],[792,98,86659],[793,546,48369],[795,97,67407],[796,477,37718],[797,66,87249],[799,783,12823],[801,437,3536],[802,11,26811],[804,564,91098],[805,482,27517],[806,168,52001],[807,265,21530],[808,53,76266],[809,361,80924],[810,182,37370],[811,154,37112],[812,556,41290],[814,789,74920],[815,800,56751],[816,729,91805],[817,107,94232],[818,154,80731],[819,268,14737],[820,375,4887],[821,350,44481],[822,72,89776],[823,20,7873],[824,132,21003],[825,528,54949],[826,315,47200],[827,111,74453],[828,512,94245],[829,577,59313],[830,603,14129],[831,690,26452],[832,413,54620],[833,52,58054],[834,711,49276],[835,469,56280],[836,296,9619],[837,371,27141],[838,707,78638],[839,762,2177],[840,136,98498],[841,175,16939],[842,529,10712],[843,320,905],[844,585,66020],[845,450,80295],[846,381,90691],[847,552,28690],[848,619,83992],[849,824,55730],[850,363,43803],[852,52,7169],[853,444,56336],[854,476,56690],[855,622,85799],[857,774,96678],[858,424,49449],[859,402,3931],[861,59,68039],[862,283,69225],[863,668,97126],[865,837,35760],[866,562,10999],[867,185,55837],[868,404,30814],[869,287,24810],[870,208,77452],[871,478,32911],[872,249,33444],[873,245,42964],[874,426,98778],[875,697,70505],[876,128,31069],[877,23,62263],[878,589,55131],[879,355,41219],[880,458,2034],[881,714,40486],[882,854,62129],[883,558,53708],[884,847,7965],[885,816,91508],[886,95,47778],[887,267,10275],[889,206,26758],[890,593,30540],[891,346,26922],[893,568,94898],[894,695,17579],[895,274,37331],[896,454,60062],[897,258,78827],[898,203,1875],[899,665,47366],[900,442,93689],[901,200,13272],[902,792,78544],[903,266,40584],[904,792,47387],[905,659,3977],[906,298,4860],[907,543,6421],[909,40,98118],[910,286,76139],[911,471,29719],[912,873,2364],[913,844,78385],[914,473,29835],[915,89,15642],[916,589,32405],[917,3,41852],[918,517,63462],[920,377,28193],[921,6,27665],[922,32,93519],[923,422,14364],[925,229,37296],[926,804,10557],[927,83,65851],[928,108,99995],[929,680,12483],[930,191,37164],[931,279,80905],[932,409,14026],[933,61,48110],[934,540,10928],[937,446,29444],[938,360,7819],[939,471,40358],[940,357,82004],[941,819,38581],[942,392,47984],[943,652,30315],[944,605,38241],[945,397,29448],[946,915,19271],[947,654,31385],[948,715,63095],[949,44,98741],[950,108,54784],[951,179,54911],[953,849,10447],[954,339,60110],[955,475,72751],[956,511,27716],[957,510,45488],[958,501,75688],[959,167,10869],[960,786,81730],[961,925,29497],[962,882,37332],[963,756,42589],[964,695,69758],[965,376,53733],[967,277,40479],[968,637,55910],[969,676,52573],[970,500,35584],[971,866,76667],[972,335,78272],[973,864,7167],[975,656,95465],[976,763,7587],[977,946,9897],[978,572,15324],[979,791,68963],[980,716,19826],[981,756,41327],[982,462,70389],[983,694,19291],[984,759,26277],[985,269,96491],[986,171,72450],[987,335,33769],[988,28,39546],[989,598,66509],[990,815,58010],[994,246,98920],[995,952,92564],[996,47,19089],[997,815,52660],[998,385,35303],[999,136,34329],[1000,648,73110],[1001,243,253],[1002,729,4232],[1003,918,54115],[1004,636,45857],[1005,254,49621],[1006,915,61092],[1007,664,36241],[1008,114,349],[1009,712,73965],[1010,387,63796],[1011,236,65490],[1012,299,40970],[1014,1007,61676],[1015,597,52376],[1017,549,34623],[1019,1016,25540],[1020,136,52380],[1021,791,24927],[1022,950,98601],[1023,226,1562],[1024,614,9795],[1025,335,30621],[1026,593,56947],[1027,241,25080],[1028,809,15470],[1029,650,35961],[1030,893,69249],[1031,763,46677],[1032,372,78156],[1033,302,99773],[1034,883,85578],[1035,174,10140],[1036,243,41563],[1039,672,15448],[1041,477,40427],[1042,440,94716],[1044,583,53646],[1045,859,43109],[1046,901,19523],[1047,60,71221],[1048,187,43065],[1049,564,10769],[1050,956,54814],[1051,65,81441],[1052,206,18660],[1053,862,27312],[1054,515,6942],[1055,99,37047],[1056,616,41243],[1057,284,33037],[1058,131,17012],[1060,132,60120],[1061,424,65010],[1062,710,6555],[1063,45,30469],[1064,1039,6630],[1065,852,76908],[1067,7,24572],[1068,558,80311],[1069,598,71619],[1070,387,76514],[1072,1,32379],[1073,590,16104],[1074,107,75199],[1075,712,19319],[1076,907,30853],[1078,913,31911],[1079,218,59986],[1080,1051,55941],[1081,54,61929],[1082,367,91902],[1083,924,43746],[1084,906,32858],[1085,183,34173],[1086,375,76226],[1087,145,49924],[1088,714,79879],[1089,170,54271],[1090,726,74548],[1091,798,77665],[1092,60,20506],[1093,330,92122],[1094,1017,5796],[1095,128,27394],[1098,878,52275],[1100,1048,47647],[1101,196,64291],[1102,29,57258],[1103,771,92395],[1104,292,70900],[1105,168,54614],[1106,340,13278],[1107,208,8970],[1108,727,15460],[1109,186,88394],[1110,872,17516],[1112,368,77577],[1113,633,97694],[1114,794,38711],[1115,524,69183],[1116,963,84105],[1117,564,47890],[1118,635,33266],[1119,164,77356],[1120,228,13586],[1121,32,8476],[1122,23,97217],[1123,431,34665],[1124,170,93412],[1125,703,87334],[1126,694,81732],[1127,1068,44270],[1129,538,29571],[1130,227,81957],[1131,198,52023],[1132,1072,16762],[1134,354,99400],[1135,699,46641],[1136,860,68542],[1137,903,34620],[1138,953,21122],[1139,339,99965],[1140,103,63619],[1141,1044,12550],[1142,470,65787],[1143,9,31441],[1144,897,43870],[1145,14,986],[1146,31,54773],[1147,958,70162],[1148,32,57184],[1149,1067,90143],[1150,11,91283],[1151,1131,13805],[1153,1005,7617],[1154,565,87674],[1155,298,23108],[1156,108,98725],[1157,45,50697],[1158,345,48048],[1159,1089,60324],[1160,912,93321],[1161,374,64541],[1162,959,38489],[1163,1007,41410],[1164,623,5394],[1166,989,12787],[1167,723,63135],[1168,201,23866],[1169,883,18205],[1170,198,90217],[1171,197,82553],[1172,1157,7973],[1173,620,76527],[1174,636,56597],[1175,971,93165],[1176,1058,47951],[1177,925,83025],[1178,1150,41159],[1179,41,1078],[1180,799,18057],[1181,600,23302],[1182,247,30510],[1183,474,57167],[1184,676,7280],[1185,254,95340],[1186,167,42062],[1187,422,44527],[1188,780,53033],[1189,353,53881],[1190,155,96905],[1191,132,20734],[1192,192,35140],[1194,184,61905],[1195,973,58618],[1196,32,83850],[1197,30,50460],[1198,345,64698],[1199,18,94838],[1200,684,68193],[1201,390,8651],[1202,258,16873],[1203,1139,70156],[1204,349,36248],[1205,290,66944],[1206,151,75048],[1207,1165,76526],[1208,747,39739],[1209,28,22937],[1210,1005,27970],[1212,1142,75522],[1213,297,43061],[1214,240,14018],[1215,602,61861],[1216,918,96330],[1217,588,44728],[1218,334,23323],[1219,754,5042],[1220,112,54984],[1221,903,90527],[1224,718,9033],[1225,536,51629],[1226,570,34668],[1227,956,39257],[1228,884,52639],[1229,782,89207],[1230,35,5296],[1231,438,80879],[1233,440,68052],[1234,883,63213],[1235,179,96141],[1236,250,73533],[1237,358,83354],[1238,370,68408],[1239,878,58233],[1240,911,75054],[1241,812,64691],[1242,914,50680],[1243,770,50604],[1244,382,93458],[1245,575,81450],[1246,266,68534],[1247,527,48772],[1248,887,73895],[1249,617,24064],[1251,1245,66677],[1252,352,14790],[1253,647,85873],[1254,572,48089],[1256,186,28579],[1257,23,28457],[1259,612,11103],[1260,962,32647],[1262,195,36388],[1263,829,6800],[1264,891,63034],[1265,870,12964],[1266,1047,76341],[1267,587,90963],[1268,1175,58228],[1269,185,32283],[1270,687,10575],[1271,495,2904],[1272,492,50018],[1274,365,82885],[1275,153,99206],[1276,1103,63921],[1277,442,66754],[1278,1130,66462],[1279,150,96234],[1280,1150,26666],[1281,265,57581],[1282,1027,19629],[1283,260,30112],[1284,1186,10459],[1285,508,44090],[1286,831,16935],[1287,490,60197],[1288,187,34489],[1289,1211,49901],[1290,562,21963],[1292,842,71198],[1293,151,54976],[1294,369,55682],[1295,1031,59409],[1296,1285,89738],[1297,175,84503],[1298,741,73013],[1299,709,71213],[1300,163,69017],[1301,377,97307],[1302,644,47722],[1303,48,28534],[1304,802,93100],[1305,183,2428],[1306,143,81543],[1307,744,79377],[1308,1214,69952],[1309,526,98676],[1310,928,23610],[1311,485,32011],[1312,108,16640],[1313,345,59724],[1314,1180,84764],[1315,429,91811],[1316,687,93029],[1317,949,45024],[1318,1021,27980],[1319,555,66426],[1320,1032,62081],[1321,1229,10487],[1322,1159,79489],[1323,748,39178],[1324,461,74206],[1325,726,63863],[1326,909,35424],[1328,635,7373],[1330,583,42792],[1331,856,56747],[1332,1150,66011],[1333,36,82499],[1334,346,69398],[1336,858,65762],[1337,561,60177],[1338,59,45990],[1339,282,48320],[1340,599,97713],[1341,618,88764],[1342,1107,58341],[1343,262,29797],[1345,92,56620],[1346,1312,82414],[1347,70,10017],[1349,1290,64396],[1351,1276,23275],[1352,48,33954],[1353,476,34075],[1354,1125,78417],[1355,1173,62803],[1356,1088,43520],[1357,146,52854],[1358,107,34794],[1359,918,51614],[1360,1353,89488],[1361,908,38986],[1362,38,66757],[1363,268,68867],[1364,1148,44073],[1365,532,72633],[1366,1136,89196],[1367,476,85687],[1368,602,46477],[1369,559,29730],[1370,916,76393],[1371,19,90839],[1372,309,41194],[1373,1121,24041],[1374,681,43968],[1375,613,97413],[1376,1362,65842],[1377,1022,43786],[1378,1174,18594],[1379,58,74456],[1380,1226,41188],[1381,501,84327],[1382,1097,4589],[1383,833,36410],[1384,598,35702],[1385,767,34017],[1386,444,89645],[1388,542,80738],[1389,90,2826],[1390,213,85347],[1391,734,29166],[1393,1072,96409],[1394,553,8271],[1395,179,2230],[1396,1261,73418],[1399,246,8017],[1400,1092,65801],[1401,678,23508],[1402,629,82835],[1404,200,32942],[1405,40,8439],[1406,58,33343],[1407,1090,17269],[1408,740,976],[1409,948,83076],[1410,1327,81532],[1412,75,25421],[1414,1155,43856],[1415,899,40062],[1416,367,96287],[1417,683,2662],[1418,313,84453],[1419,389,97720],[1420,752,29772],[1421,832,74634],[1422,1160,33370],[1425,903,2804],[1426,550,75637],[1427,1040,87343],[1428,1366,14820],[1429,415,38785],[1430,1278,13487],[1431,567,48949],[1432,807,47795],[1433,1353,93754],[1434,870,28897],[1435,1155,12717],[1436,1054,93012],[1437,1258,70247],[1438,451,68809],[1439,402,16171],[1440,99,88933],[1441,1178,87030],[1442,559,4278],[1443,352,25616],[1444,42,61937],[1445,1367,56804],[1446,598,80587],[1448,52,38880],[1449,419,57168],[1450,1262,70998],[1451,800,54836],[1453,1206,61736],[1454,597,84984],[1455,1010,52546],[1456,1290,21483],[1457,1088,5940],[1458,169,99165],[1459,434,29613],[1460,957,39580],[1461,1154,27998],[1462,914,61685],[1463,638,74288],[1464,462,20328],[1465,146,2969],[1466,55,24516],[1467,1355,23636],[1469,260,5032],[1472,338,79476],[1474,794,31960],[1475,333,79966],[1476,941,34278],[1478,704,43050],[1479,114,61231],[1480,801,98296],[1481,718,2047],[1482,926,32449],[1483,443,67181],[1484,1413,91672],[1485,1426,32972],[1486,278,15385],[1487,1401,6861],[1488,677,90408],[1489,1096,21566],[1490,1452,13504],[1491,331,29763],[1492,6,76860],[1493,871,30081],[1494,172,19984],[1495,1094,57495],[1496,1184,63029],[1497,1354,12928],[1498,835,10344],[1499,895,81466],[1500,1349,70236],[1501,305,20088],[1502,412,53171],[1503,659,78764],[1504,655,25927],[1505,147,38445],[1506,1015,69892],[1507,1128,65963],[1508,1355,70080],[1509,173,88174],[1510,758,51179],[1511,1008,27401],[1513,878,94695],[1514,406,31807],[1515,666,43755],[1516,939,81974],[1517,1016,8833],[1518,470,30755],[1519,50,41369],[1520,275,37836],[1521,308,42015],[1522,1412,15351],[1523,285,57167],[1524,302,49610],[1525,1,69359],[1526,1374,46559],[1527,1251,61083],[1528,927,76517],[1529,931,1797],[1530,863,2776],[1531,917,16816],[1532,335,78842],[1533,34,63889],[1534,1219,39489],[1535,722,17797],[1536,1491,68888],[1537,682,52521],[1539,445,44975],[1541,278,57900],[1542,599,18322],[1543,651,9503],[1544,258,62891],[1545,928,14523],[1546,94,71006],[1547,731,77685],[1548,236,28309],[1550,1539,17487],[1551,868,17325],[1552,182,60064],[1553,953,76168],[1554,482,26176],[1555,1554,41161],[1556,656,22451],[1557,194,83440],[1558,48,79620],[1559,478,82745],[1560,744,75405],[1561,1542,32021],[1562,193,82402],[1563,1415,10703],[1564,607,66823],[1565,865,15584],[1566,500,22983],[1567,637,41866],[1568,1133,14499],[1569,1374,48085],[1570,1099,29295],[1571,1061,58459],[1572,1427,50462],[1575,148,90079],[1576,521,5616],[1577,12,68623],[1578,984,19385],[1579,875,28175],[1580,1231,46757],[1582,701,82062],[1583,1072,55296],[1584,76,43894],[1585,1254,11593],[1586,978,36258],[1587,1549,33871],[1588,1145,30375],[1590,1458,81330],[1591,405,91717],[1593,1196,67665],[1594,1516,73678],[1595,1376,52548],[1596,511,45447],[1597,648,93984],[1598,36,63235],[1599,1141,70106],[1600,1538,93658],[1601,176,14318],[1602,683,96433],[1603,6,3566],[1604,1420,99651],[1605,356,1541],[1607,200,76994],[1608,414,42643],[1609,209,31702],[1610,470,12815],[1611,123,25495],[1612,1477,25216],[1613,1176,57217],[1614,314,42236],[1615,925,75626],[1616,356,36801],[1617,823,26946],[1619,1341,29413],[1620,595,56242],[1621,328,79397],[1622,1383,53379],[1623,233,23336],[1624,1541,33776],[1625,167,6950],[1626,638,696],[1627,164,48174],[1628,875,59196],[1629,463,65835],[1631,1003,61351],[1632,392,89541],[1633,522,68950],[1634,180,32748],[1635,1012,81843],[1637,719,4490],[1639,1160,72524],[1640,409,70036],[1642,627,52589],[1643,740,90447],[1644,1208,47774],[1645,1320,54053],[1646,1106,69528],[1647,1150,40525],[1648,1597,18],[1649,380,46144],[1650,90,68408],[1651,1529,47294],[1652,1079,62853],[1653,1150,54067],[1654,1164,67504],[1655,351,20105],[1656,1452,78587],[1657,1396,90214],[1658,278,59150],[1659,1414,80567],[1660,1021,32666],[1661,696,96834],[1662,413,38056],[1664,644,1247],[1665,432,9235],[1666,169,43365],[1668,776,98200],[1669,1206,22963],[1670,1130,56242],[1671,1171,6574],[1674,1363,10204],[1675,1195,84699],[1676,234,93506],[1677,235,70722],[1679,1394,45292],[1680,1529,51633],[1681,1135,47195],[1682,542,35535],[1683,324,33859],[1684,233,52359],[1685,1378,39501],[1686,159,61649],[1687,422,17006],[1688,1635,32544],[1689,623,56884],[1691,803,83],[1692,758,71204],[1693,878,76748],[1694,1604,41796],[1695,1471,73553],[1696,1344,29484],[1697,706,49328],[1698,1061,77002],[1700,701,38657],[1701,961,11891],[1702,1403,76762],[1703,1473,18829],[1704,858,34472],[1705,1107,83820],[1707,103,21744],[1708,1373,87699],[1709,1254,53381],[1710,224,78040],[1711,1026,91283],[1712,1510,81904],[1713,1218,71910],[1714,881,74307],[1715,450,17008],[1716,524,20758],[1717,1077,18922],[1718,426,6142],[1719,1673,24708],[1720,1672,22673],[1721,1061,51947],[1722,1387,85400],[1723,739,63190],[1724,226,41854],[1726,210,64512],[1727,347,24203],[1728,1040,94220],[1729,689,9995],[1730,1560,19567],[1731,1320,96123],[1732,703,82424],[1733,755,78220],[1734,1648,26714],[1735,767,36096],[1737,1009,25871],[1738,1572,5145],[1739,738,81426],[1740,70,90421],[1741,961,16556],[1742,1421,3976],[1743,138,81131],[1744,969,31427],[1745,1547,89625],[1746,1666,38553],[1747,757,37813],[1748,301,12035],[1750,317,79609],[1751,978,3946],[1752,378,88630],[1753,1307,5052],[1754,42,65741],[1755,1448,71008],[1756,1674,20456],[1757,1362,7224],[1758,782,85199],[1759,942,78199],[1761,180,29946],[1762,1553,8212],[1763,868,24273],[1764,162,41003],[1765,1,90961],[1766,1405,55774],[1767,762,95592],[1768,1726,10598],[1769,570,80313],[1771,324,36741],[1772,362,25140],[1773,445,89332],[1774,1533,48982],[1775,1385,39129],[1776,1279,86112],[1777,1032,27089],[1779,1454,34766],[1780,724,87661],[1781,19,61973],[1782,444,99469],[1783,826,84557],[1784,1298,16913],[1785,303,59548],[1786,1279,32759],[1787,591,47664],[1788,1089,38234],[1789,1011,39254],[1791,769,43711],[1792,1778,10440],[1793,1156,28940],[1794,974,75378],[1795,1210,33981],[1796,72,52449],[1798,1718,20993],[1799,663,17307],[1800,1565,50282],[1801,1625,21487],[1802,211,84244],[1804,876,57088],[1805,1474,64278],[1806,872,52006],[1807,1485,83288],[1808,1356,56168],[1809,787,17644],[1810,1674,4892],[1811,1424,98678],[1812,1181,6610],[1813,1094,42801],[1814,303,57027],[1815,945,96016],[1816,1540,90279],[1817,87,26996],[1818,566,89527],[1820,1268,61447],[1821,388,75056],[1822,1454,13665],[1823,96,88854],[1824,93,26858],[1826,1674,7684],[1827,641,26221],[1828,179,97599],[1829,535,23074],[1830,521,2301],[1832,938,12625],[1834,103,84195],[1835,1775,16484],[1836,1089,31998],[1837,488,43211],[1838,996,18504],[1839,1261,92238],[1840,1580,99762],[1841,216,39921],[1842,1614,87827],[1843,312,27048],[1844,458,94118],[1845,238,32505],[1846,417,67817],[1847,196,87086],[1848,481,87862],[1849,957,72420],[1850,924,21972],[1851,945,44397],[1852,26,15100],[1854,112,80135],[1856,877,98938],[1857,1826,12135],[1858,762,78788],[1859,1071,14583],[1861,1576,58950],[1862,148,81901],[1863,920,34735],[1864,500,14383],[1865,486,53463],[1866,1813,4709],[1867,1580,39726],[1868,534,7736],[1869,1416,54710],[1870,1595,62001],[1871,171,52530],[1872,988,29773],[1873,1099,46458],[1874,1744,65858],[1875,569,55721],[1876,570,95922],[1877,990,98942],[1878,1846,22826],[1879,983,74666],[1880,1413,17988],[1881,1046,2412],[1882,1508,68224],[1883,971,36791],[1884,1249,83889],[1885,873,79657],[1886,759,93207],[1888,63,4417],[1889,75,90303],[1892,672,40377],[1893,1043,80449],[1894,699,84171],[1896,730,58237],[1898,780,17909],[1900,1691,47016],[1901,364,29188],[1902,1134,1388],[1903,1379,25838],[1904,1746,44877],[1905,794,7078],[1906,594,56007],[1907,1723,60140],[1908,1650,47677],[1909,137,79481],[1912,289,27911],[1913,1312,3582],[1914,1342,49154],[1915,1139,25802],[1916,1271,87652],[1918,1361,20539],[1920,1247,73281],[1921,562,27815],[1922,1536,36274],[1923,1510,3428],[1924,1356,61660],[1925,1638,26168],[1926,1284,32181],[1927,617,29200],[1928,163,73573],[1929,1563,4238],[1930,1459,86665],[1931,549,30738],[1932,1107,70415],[1933,1211,48121],[1934,1178,3994],[1936,1716,55675],[1937,1140,80126],[1938,627,71618],[1939,1458,93084],[1940,1344,82402],[1941,481,1261],[1942,620,45588],[1943,936,85286],[1944,1916,55626],[1945,1743,31588],[1946,866,88852],[1947,1935,95125],[1948,897,32841],[1949,1340,50949],[1950,934,95611],[1951,1669,31789],[1952,288,71059],[1953,330,17302],[1954,849,12746],[1955,1422,37508],[1956,1017,4480],[1957,823,58719],[1958,1916,65512],[1959,1337,28926],[1960,1697,45187],[1961,645,79442],[1962,1409,78056],[1963,1590,96112],[1964,161,97799],[1965,1774,76020],[1966,737,75005],[1967,1516,68113],[1968,731,1316],[1969,1951,81828],[1970,32,52498],[1971,829,6600],[1972,587,3639],[1973,1211,34628],[1974,454,33525],[1975,1944,31623],[1976,1446,43117],[1977,1654,5354],[1978,1139,28086],[1979,653,7771],[1980,1309,71940],[1982,282,88131],[1983,1479,35519],[1984,1076,64864],[1985,1212,25739],[1986,1913,13813],[1987,1960,42627],[1989,1165,7472],[1990,863,33477],[1991,1956,40231],[1992,889,20056],[1993,18,29013],[1994,1562,87466],[1995,583,55010],[1997,856,77515],[1998,382,87168],[1999,1578,13624],[2001,1108,48441],[2002,1132,79724],[2003,1284,67808],[2004,641,8894],[2005,1284,52411],[2006,1278,43981],[2008,1752,29244],[2009,65,81523],[2010,1602,68647],[2011,1209,65029],[2012,1452,71790],[2013,1628,37263],[2014,633,9544],[2015,1076,38297],[2016,813,49450],[2017,239,63983],[2018,1671,34481],[2019,443,11346],[2021,1282,18896],[2022,836,55087],[2023,1627,78152],[2024,1580,11747],[2025,1797,87033],[2026,882,79129],[2027,201,32580],[2028,560,20877],[2029,1103,71548],[2030,1863,77127],[2031,1830,46879],[2032,751,62894],[2033,777,51242],[2034,1858,73690],[2035,1697,5783],[2036,1812,13180],[2037,846,21033],[2039,575,39011],[2041,1899,84186],[2042,1706,75012],[2043,165,3442],[2045,1961,10443],[2046,596,71950],[2047,1003,94219],[2048,601,31053],[2049,63,89948],[2050,944,7438],[2051,639,56694],[2052,685,20260],[2053,1856,68744],[2054,657,98335],[2055,2009,84753],[2056,1367,77875],[2057,1643,89787],[2058,792,5089],[2059,1569,1950],[2060,169,56565],[2061,82,16928],[2062,1573,470],[2063,542,6916],[2064,1943,66462],[2067,730,55387],[2069,1250,64247],[2070,1488,938],[2071,94,90514],[2072,605,29025],[2073,1082,26394],[2074,1202,94264],[2075,1417,60967],[2076,1045,27009],[2077,1067,82089],[2078,1145,46433],[2079,796,24708],[2080,135,3945],[2081,1464,56941],[2082,1176,64853],[2083,1200,94888],[2084,32,70744],[2086,343,65223],[2087,27,47010],[2088,870,21963],[2089,1659,45436],[2090,44,64359],[2091,2052,32531],[2092,1206,19091],[2093,1849,87745],[2094,1422,79986],[2095,1813,71054],[2096,1447,78772],[2097,1457,1784],[2099,607,305],[2101,1590,47210],[2103,1111,38168],[2104,2022,35470],[2105,1629,19927],[2106,1536,34071],[2108,566,92823],[2109,677,25875],[2110,1817,33904],[2111,571,35240],[2112,741,96757],[2113,1486,78613],[2114,2053,38392],[2115,1724,99260],[2116,1845,66895],[2117,1635,21105],[2118,891,34050],[2119,708,24250],[2120,1331,82794],[2121,956,84026],[2122,1486,21995],[2123,769,56914],[2124,694,80412],[2125,1065,645],[2126,79,96687],[2127,1504,59408],[2128,458,43050],[2130,1594,13307],[2131,1535,99373],[2132,1321,97868],[2133,1759,83226],[2134,1561,37970],[2135,1453,56519],[2136,1275,47035],[2137,1480,44677],[2138,510,75986],[2139,1430,81904],[2141,395,98604],[2143,1812,64733],[2144,46,99363],[2146,1207,17310],[2147,1927,72480],[2148,675,20605],[2149,88,45907],[2150,670,84406],[2151,484,70180],[2152,675,36255],[2153,310,19553],[2154,1068,31005],[2155,703,10580],[2156,1449,8386],[2157,1349,20511],[2158,79,252],[2159,1755,97365],[2160,1920,37848],[2161,601,31738],[2163,1953,37118],[2165,850,54036],[2166,1327,61832],[2167,919,87251],[2168,505,73976],[2169,1618,17476],[2170,544,72201],[2171,617,39041],[2172,1507,56703],[2173,1425,29231],[2174,5,99946],[2175,1234,15449],[2176,1746,84835],[2177,215,12768],[2178,1079,41996],[2179,1483,99892],[2181,1888,42817],[2182,1142,27106],[2183,576,97036],[2184,1318,25675],[2185,935,65725],[2186,1111,52746],[2187,773,77664],[2188,671,4037],[2189,731,32401],[2190,343,28439],[2191,1629,85459],[2192,1663,22616],[2194,723,64190],[2196,1131,27207],[2198,1401,57134],[2199,1992,51555],[2200,2055,54755],[2201,1453,28041],[2203,97,77491],[2204,692,4818],[2205,1767,91743],[2206,1834,65580],[2207,750,87864],[2208,1599,96553],[2209,1338,41079],[2210,1906,15442],[2211,1036,91071],[2212,2179,31891],[2213,116,38651],[2214,634,5407],[2215,790,73219],[2216,1627,46632],[2217,1233,92692],[2218,1389,11892],[2219,1091,310],[2220,2042,4138],[2221,1896,57141],[2222,1943,48854],[2223,948,52713],[2224,326,43624],[2225,1604,32118],[2226,306,47822],[2227,416,29408],[2229,1591,78049],[2230,1331,54044],[2231,302,50714],[2232,394,27780],[2233,2167,35092],[2234,1760,18359],[2235,593,16597],[2236,1405,21589],[2239,777,17972],[2240,2213,50321],[2241,471,33198],[2242,1286,88861],[2243,300,86472],[2245,2132,35193],[2246,2076,68476],[2247,1643,38085],[2248,887,81822],[2250,419,14775],[2251,1034,13565],[2252,791,20932],[2253,1836,67989],[2254,1009,51834],[2255,1331,83849],[2256,2150,52152],[2257,1518,92443],[2258,993,59409],[2259,1332,37819],[2260,133,95590],[2261,1134,7218],[2262,215,2803],[2263,714,85861],[2264,904,1976],[2265,929,59940],[2267,724,19013],[2268,2127,58898],[2269,881,43078],[2270,969,9528],[2271,1779,5966],[2272,789,28826],[2273,1838,77664],[2274,1887,20241],[2275,1793,24149],[2277,444,39736],[2279,91,72484],[2280,908,25158],[2281,729,37842],[2282,55,79670],[2283,873,91619],[2284,2017,67223],[2285,931,27948],[2286,805,44287],[2287,918,78596],[2288,287,74695],[2289,588,33923],[2290,1924,43447],[2291,1871,19670],[2292,1742,23092],[2293,830,36399],[2294,1021,90140],[2295,20,49664],[2296,2093,69054],[2297,1551,34537],[2298,2248,61936],[2300,709,31920],[2301,485,32202],[2302,1828,83649],[2303,452,49966],[2304,1569,42409],[2305,898,66026],[2306,2053,65774],[2307,1640,50699],[2308,72,18222],[2309,1298,13183],[2310,874,52600],[2311,611,78138],[2312,1080,47769],[2313,1554,32481],[2314,1796,45496],[2315,317,50778],[2316,415,72322],[2317,406,83473],[2319,925,92462],[2320,557,59251],[2321,1801,10492],[2322,143,36709],[2324,263,87646],[2325,1076,35982],[2326,11,15974],[2327,1799,27301],[2328,303,67054],[2329,1933,42549],[2330,1737,31216],[2331,1744,86577],[2332,368,7191],[2333,1154,98932],[2335,4,28736],[2336,1782,50871],[2337,1319,84767],[2338,490,13772],[2339,220,15341],[2340,1209,7096],[2341,1234,9301],[2342,1914,54452],[2343,953,19943],[2344,1945,35404],[2345,692,93408],[2346,256,96687],[2347,2272,26920],[2348,819,4325],[2349,2229,458],[2350,2294,71856],[2351,1994,39277],[2352,628,57936],[2353,1743,3756],[2354,322,52759],[2355,1995,1101],[2357,1125,81963],[2358,595,38159],[2359,901,69176],[2360,426,80204],[2361,1110,3807],[2362,741,86148],[2363,175,33573],[2364,909,40522],[2365,1771,914],[2366,2297,55208],[2367,1890,56235],[2368,1804,19519],[2369,850,68637],[2370,1070,29015],[2371,855,63642],[2372,79,4870],[2373,2133,43005],[2374,204,57183],[2376,739,16655],[2377,672,1151],[2378,533,98617],[2379,911,68820],[2380,152,89028],[2381,1556,64362],[2382,439,79083],[2383,2140,49268],[2384,1166,13599],[2385,22,94755],[2386,1692,34636],[2387,1613,1253],[2388,133,64777],[2389,2217,48738],[2390,185,19364],[2391,1078,42950],[2392,597,99864],[2393,1506,60237],[2394,1021,71844],[2395,1206,44646],[2396,379,60834],[2397,199,27747],[2398,1478,36858],[2399,1679,89232],[2400,88,48528],[2401,2050,46105],[2403,1600,69274],[2404,971,21419],[2405,1000,97446],[2406,1624,26158],[2407,82,98831],[2408,1512,25186],[2409,2224,37847],[2411,381,14252],[2412,1061,9908],[2413,41,63348],[2414,1130,74462],[2415,2257,97605],[2416,1011,45431],[2417,526,61727],[2419,630,2855],[2420,460,18714],[2422,2276,5996],[2423,1724,38924],[2425,1447,90106],[2426,1041,73172],[2427,1695,73883],[2428,1812,60478],[2429,1107,55698],[2430,1340,69150],[2431,1725,1998],[2432,1452,44897],[2433,965,86420],[2434,1594,49807],[2435,1228,33124],[2436,189,36504],[2437,2231,27023],[2438,1265,41818],[2439,227,22051],[2440,491,25706],[2441,1458,24269],[2442,1394,91198],[2443,515,60214],[2444,2285,21829],[2445,1469,55308],[2446,2118,47692],[2447,332,91416],[2448,2420,3001],[2450,1655,12678],[2451,2279,8405],[2452,779,41602],[2453,1069,65093],[2454,403,78828],[2455,560,54963],[2457,1472,95585],[2458,1200,27071],[2460,790,49396],[2461,205,26025],[2462,876,29232],[2463,1000,2969],[2466,170,90311],[2467,819,25840],[2468,175,20096],[2469,926,14562],[2470,659,52428],[2471,1436,41523],[2472,2316,41998],[2473,458,77705],[2474,606,57376],[2476,1041,58445],[2477,1560,97281],[2478,2128,21227],[2479,1203,77451],[2480,462,25219],[2481,646,8349],[2482,2274,44492],[2483,375,60246],[2484,406,26080],[2485,1205,67669],[2486,503,26123],[2487,1353,19269],[2488,556,14243],[2489,1425,38485],[2490,1802,36245],[2491,2464,73756],[2492,113,55653],[2493,1303,82183],[2494,1974,72465],[2495,288,70579],[2496,814,46922],[2497,2277,97279],[2498,1964,9719],[2499,1200,29885],[2500,600,36273],[2501,1824,76022],[2502,1705,47172],[2503,1162,39632],[2504,1963,12157],[2505,800,34134],[2506,2459,41601],[2507,1569,69567],[2508,397,17907],[2509,1873,96135],[2510,1772,18118],[2511,1686,39480],[2512,55,56888],[2513,2226,21517],[2514,1847,32961],[2515,2200,63196],[2516,139,50629],[2517,592,73465],[2518,1090,88768],[2519,245,61],[2520,889,46999],[2521,681,3293],[2522,266,40421],[2523,99,17187],[2524,1416,81262],[2525,2339,47652],[2526,2411,59280],[2527,2176,45584],[2528,768,90991],[2529,277,92706],[2530,596,79260],[2531,671,29515],[2532,1126,96099],[2533,223,47097],[2534,525,95300],[2535,1306,70047],[2536,2507,33223],[2537,548,86813],[2538,673,86717],[2539,763,42502],[2541,2029,75460],[2543,777,32627],[2544,1106,68201],[2545,1813,74267],[2546,2044,7021],[2547,2113,36265],[2548,1275,56370],[2549,2135,41896],[2550,454,92434],[2551,378,37484],[2552,412,8121],[2553,2349,28006],[2554,138,19345],[2555,1870,44573],[2556,892,89085],[2557,1848,33033],[2558,2055,75789],[2559,1961,69333],[2560,151,34942],[2561,539,12572],[2562,1548,5950],[2563,1181,48440],[2564,957,2090],[2565,2103,99594],[2566,630,23679],[2567,2399,88550],[2568,1847,48699],[2569,951,38980],[2570,134,4514],[2571,2454,59093],[2572,2285,48149],[2573,522,3184],[2574,2552,77394],[2575,2201,7405],[2576,2472,57539],[2577,2294,19400],[2578,1508,29788],[2579,621,98430],[2580,1351,36518],[2581,453,88711],[2582,2240,12986],[2583,2171,37324],[2584,368,73937],[2585,2184,30738],[2586,1191,19147],[2587,213,57215],[2588,1581,58394],[2589,2411,46165],[2590,1175,12592],[2591,2301,46319],[2593,1985,24217],[2594,946,9783],[2595,335,90881],[2596,2090,26681],[2597,2322,41002],[2598,2334,49024],[2599,1103,37182],[2600,2345,55152],[2601,2334,97269],[2602,2400,82393],[2603,343,99315],[2604,2472,73789],[2605,2055,39070],[2606,1968,32192],[2608,810,39154],[2609,185,54290],[2610,1626,70901],[2611,189,75295],[2613,625,64436],[2614,1660,9165],[2616,2595,39171],[2617,823,77526],[2619,1738,8080],[2620,958,12651],[2622,2354,82223],[2623,1247,79693],[2625,2266,27173],[2626,1111,96604],[2628,1210,42181],[2629,542,12964],[2630,1540,2210],[2631,698,52425],[2632,1715,66822],[2634,607,1686],[2635,2411,36748],[2636,544,19209],[2637,1357,65306],[2638,2317,80823],[2639,412,37170],[2640,836,77594],[2641,1993,80238],[2642,2439,89385],[2643,1824,35166],[2644,1076,15455],[2645,717,99577],[2646,591,73160],[2647,1290,82080],[2648,2139,99751],[2649,489,79278],[2650,1884,62729],[2651,267,92741],[2652,1968,98670],[2655,1219,73786],[2656,1700,63321],[2657,1360,6316],[2658,1731,52349],[2659,2411,44954],[2661,545,93459],[2662,1032,97209],[2663,38,67989],[2664,1567,51330],[2666,125,16130],[2667,2155,95280],[2668,1523,14665],[2669,1379,35613],[2670,749,83583],[2671,1975,13897],[2673,836,64767],[2674,539,77392],[2675,1312,42027],[2676,2262,97176],[2677,1942,74106],[2678,282,8917],[2679,2552,97126],[2680,961,21924],[2681,363,47092],[2682,2154,90139],[2683,1206,78273],[2684,1317,56058],[2685,1221,51641],[2686,1198,44013],[2688,824,38845],[2690,749,27218],[2691,1983,91012],[2693,2241,16990],[2695,1012,65079],[2696,1968,13174],[2697,562,85512],[2698,384,91914],[2699,2310,76270],[2700,722,70268],[2701,2531,43547],[2702,1706,49714],[2703,350,95441],[2704,2629,54206],[2705,1440,18636],[2706,2638,90986],[2707,413,73248],[2708,1459,90762],[2709,337,58776],[2710,2646,3684],[2711,640,22743],[2712,1741,64933],[2714,592,96687],[2715,765,83847],[2716,963,11468],[2718,1681,48558],[2719,2528,1117],[2720,2035,39535],[2721,1525,25968],[2722,813,2185],[2723,93,68941],[2724,1493,66997],[2725,2433,98726],[2726,2689,63219],[2727,1266,68093],[2728,1493,79867],[2729,1102,31808],[2730,1491,81058],[2731,2606,12658],[2733,1343,85065],[2734,1043,77699],[2735,1463,71667],[2736,405,94781],[2737,314,37981],[2738,216,18786],[2739,2549,88759],[2740,1736,86297],[2741,413,87614],[2742,2403,47522],[2743,2391,78111],[2744,1686,52105],[2746,2460,13819],[2747,1844,43111],[2748,674,20474],[2749,298,3246],[2750,465,45958],[2751,610,53648],[2752,1745,33187],[2753,755,91346],[2754,1234,4138],[2755,2437,83810],[2756,714,87228],[2757,1770,45270],[2758,1083,85423],[2759,829,29033],[2760,909,87960],[2761,1821,45312],[2762,1974,63390],[2763,1123,92916],[2764,1537,47883],[2765,542,94799],[2766,1119,20726],[2767,2636,1863],[2768,2244,49905],[2770,332,9915],[2771,420,20266],[2772,1846,16998],[2773,393,53860],[2774,1473,20372],[2775,748,8672],[2776,1901,86785],[2777,1880,60247],[2778,1617,65901],[2779,1614,51262],[2780,1890,1496],[2781,656,37311],[2782,1174,80207],[2784,2714,52303],[2786,1663,17983],[2787,558,94367],[2788,33,75222],[2789,1496,27666],[2790,2175,62718],[2791,1012,57802],[2792,283,39345],[2793,2394,66716],[2794,727,9993],[2795,465,33633],[2797,768,75734],[2798,674,50388],[2799,1030,23704],[2800,98,74370],[2801,776,12792],[2802,940,77667],[2803,572,38708],[2804,2247,5221],[2805,1295,43471],[2806,2758,51458],[2807,1027,90532],[2808,2303,61996],[2809,432,33438],[2810,302,76535],[2811,1755,26831],[2813,2677,8445],[2814,109,74368],[2816,1462,29489],[2817,607,20237],[2818,672,44931],[2819,713,92900],[2820,2175,51526],[2822,545,76339],[2823,2218,13347],[2824,1887,57825],[2825,1017,24770],[2826,1781,77359],[2827,210,94704],[2828,2720,74314],[2829,1941,23895],[2830,659,37003],[2831,388,17531],[2833,1916,77686],[2834,961,16228],[2835,2601,15387],[2837,2614,94323],[2838,1701,19946],[2839,1529,76664],[2840,516,87873],[2841,2699,30643],[2842,2286,33140],[2843,2670,94200],[2844,2226,3481],[2845,2661,69372],[2846,1323,17914],[2847,1366,58503],[2848,1626,1114],[2849,2468,23375],[2850,1266,86616],[2851,1385,61869],[2852,847,98166],[2853,168,80127],[2854,2402,7142],[2855,2206,47055],[2856,285,19093],[2857,250,89446],[2858,760,40701],[2859,332,68552],[2860,1995,86362],[2861,827,88789],[2862,2823,99570],[2863,1755,83878],[2864,236,67068],[2865,562,44904],[2866,2118,18041],[2867,1187,14428],[2868,414,64810],[2871,27,69285],[2872,1797,95509],[2873,1838,80527],[2874,379,10532],[2875,1559,88069],[2876,1033,52479],[2877,100,50371],[2878,626,23137],[2879,1047,39045],[2881,1904,62987],[2882,1343,51764],[2883,591,42800],[2884,2532,86560],[2885,1770,87623],[2886,2670,32526],[2887,46,34004],[2888,2698,64978],[2890,837,15182],[2891,896,93286],[2892,2817,34363],[2894,110,37193],[2895,611,2370],[2896,460,43976],[2897,1555,49623],[2898,1715,9181],[2900,2369,15110],[2901,960,8918],[2902,607,32415],[2903,774,72117],[2904,1628,40657],[2906,521,44129],[2907,492,45003],[2908,1212,6625],[2909,1578,59018],[2910,1091,48434],[2911,64,57721],[2912,1642,85166],[2913,2669,65677],[2914,2167,55693],[2915,2524,41424],[2916,889,80133],[2917,324,1216],[2919,353,86028],[2920,1496,86664],[2921,1129,57113],[2922,314,85420],[2923,2334,60876],[2924,1552,21810],[2925,68,42398],[2926,2911,52942],[2927,2564,51677],[2928,488,56831],[2930,492,77734],[2931,2511,15570],[2932,167,61294],[2933,2692,20385],[2934,886,56154],[2935,1884,29865],[2936,1874,907],[2937,1227,50054],[2938,2361,17921],[2939,62,7889],[2940,1119,34604],[2942,2589,26756],[2943,1435,58732],[2944,1652,79242],[2945,329,32984],[2946,581,49522],[2947,212,66951],[2948,367,75829],[2949,2774,44360],[2950,457,67049],[2951,1995,43559],[2952,1660,21471],[2953,1999,1592],[2954,373,95786],[2955,17,85899],[2956,2263,24490],[2957,2074,55499],[2958,2193,19217],[2959,1172,85484],[2960,707,77635],[2961,497,37718],[2962,1025,89242],[2963,384,4761],[2964,610,12374],[2965,1589,1662],[2966,1989,14142],[2967,584,11572],[2968,2838,1816],[2969,1300,53073],[2970,1272,82955],[2971,1209,43306],[2972,2393,72473],[2973,2964,87446],[2974,182,27840],[2975,1852,37426],[2976,696,20566],[2977,927,23912],[2978,719,82520],[2979,2484,6463],[2981,1671,96068],[2982,817,67210],[2983,2018,55136],[2984,325,86300],[2985,367,69768],[2986,1794,8343],[2987,28,28291],[2989,2893,78264],[2990,421,53655],[2991,226,32168],[2992,2978,38422],[2993,1811,55839],[2994,1411,25383],[2995,1959,96425],[2997,1896,76632],[2998,622,96901],[2999,2648,48295],[3000,748,11112],[3001,1500,68925],[3002,2791,277],[3003,2065,66688],[3004,2581,91888],[3005,2932,2254],[3006,2053,11673],[3007,290,32993],[3008,1897,98847],[3009,802,52498],[3010,744,50834],[3011,2255,91561],[3012,2208,42429],[3013,807,54532],[3014,1208,84466],[3015,938,14627],[3016,1126,92723],[3017,2157,61461],[3018,1437,80824],[3019,851,1174],[3020,2131,14495],[3021,2440,56194],[3022,231,49020],[3023,2510,1812],[3024,235,27442],[3025,350,7948],[3026,653,49316],[3027,2720,79626],[3028,2789,60102],[3029,2321,42394],[3030,2654,22147],[3031,262,68260],[3033,2742,14922],[3035,1346,27468],[3036,1696,44688],[3037,993,34914],[3038,2480,42013],[3040,1798,6908],[3043,1840,64794],[3044,2707,33629],[3046,1713,28827],[3047,1256,6090],[3048,1502,66047],[3049,1884,54962],[3050,1371,96608],[3051,2002,46048],[3052,885,37266],[3053,2492,30408],[3054,22,26769],[3055,897,97225],[3056,752,74818],[3058,1503,31249],[3059,2540,76375],[3060,2145,92508],[3061,2835,48064],[3062,2338,68974],[3063,1097,34123],[3064,648,63193],[3065,295,51877],[3067,1990,20799],[3068,1350,96691],[3069,3013,17931],[3070,2082,74916],[3071,68,623],[3072,686,19849],[3073,1386,2408],[3074,2451,16680],[3076,1779,35729],[3077,983,24351],[3078,3023,13177],[3079,45,98928],[3080,2525,92410],[3082,2396,21590],[3083,325,85916],[3084,1351,38150],[3086,1510,11356],[3087,2258,39831],[3088,2360,82080],[3089,1513,96803],[3090,844,29668],[3091,2636,51743],[3092,73,22754],[3093,1472,56368],[3094,1152,19168],[3095,1705,58075],[3096,62,69253],[3097,694,34465],[3098,1710,6161],[3099,487,47053],[3100,567,51819],[3101,1088,46093],[3102,2479,5185],[3103,1139,5158],[3104,619,54299],[3105,2122,15826],[3107,2193,96891],[3108,1998,38971],[3109,1084,14478],[3110,3011,99360],[3111,1999,5109],[3112,799,71181],[3113,1955,15817],[3114,1057,93834],[3116,2561,98403],[3117,2650,80072],[3118,1037,93759],[3119,1109,74711],[3120,182,64622],[3121,1012,93265],[3122,2813,60543],[3124,2816,72127],[3125,897,80640],[3126,483,27712],[3127,2143,42858],[3128,1092,78056],[3129,2144,1410],[3130,3010,16310],[3131,894,17500],[3133,886,4071],[3134,2156,3099],[3135,1881,73169],[3136,1101,78551],[3137,1608,62568],[3138,322,21116],[3139,2589,34685],[3140,1014,35459],[3141,33,58317],[3142,2341,57668],[3143,1591,57855],[3144,34,65994],[3145,2250,91291],[3146,2233,77397],[3147,2586,47920],[3148,1867,3364],[3149,1457,39836],[3150,2351,24790],[3151,2501,39785],[3152,1206,61865],[3153,1991,6234],[3154,2772,8447],[3155,1599,36331],[3156,14,33000],[3157,1708,43408],[3158,2986,88977],[3159,1223,25324],[3160,723,33845],[3161,2240,58316],[3162,555,47559],[3164,102,32464],[3165,1101,86602],[3166,690,47446],[3167,312,61679],[3168,112,10910],[3169,3099,41671],[3170,2905,74939],[3171,2129,48434],[3172,2544,84068],[3173,1964,7689],[3174,2260,16939],[3175,3128,21086],[3177,2793,50695],[3178,1709,90979],[3179,2097,32927],[3180,2179,80388],[3181,1959,77445],[3182,2584,64029],[3183,1941,39436],[3184,1468,16634],[3186,912,7387],[3187,1082,63811],[3188,2409,29992],[3189,1329,97068],[3190,1524,54374],[3191,1215,11757],[3192,1693,52017],[3193,518,78783],[3195,2178,77763],[3196,2722,19141],[3198,1881,64587],[3199,716,36166],[3200,1830,41809],[3201,2333,20745],[3202,696,45232],[3203,112,23467],[3204,1960,68460],[3205,330,81702],[3207,906,21373],[3208,2271,47377],[3209,22,63391],[3210,49,70491],[3211,438,85392],[3212,2074,61377],[3213,1457,23659],[3214,1108,86807],[3217,2713,76033],[3218,1503,16533],[3219,1970,70857],[3220,412,12881],[3221,1496,13195],[3222,2279,73572],[3223,318,4191],[3224,241,16034],[3225,2433,88798],[3226,1111,45760],[3227,3133,22906],[3228,1538,94500],[3229,988,60074],[3230,128,53898],[3231,3200,8944],[3232,2046,18707],[3233,1554,72060],[3235,1691,18011],[3236,1929,33635],[3237,1659,27129],[3238,2715,95229],[3239,2850,71640],[3240,2122,85451],[3241,2567,89509],[3243,263,17190],[3244,1077,98533],[3246,2445,32195],[3247,1685,27214],[3248,2889,40874],[3249,2847,82418],[3250,379,72502],[3253,2844,81072],[3254,1475,73489],[3255,2517,62456],[3256,2828,25397],[3257,3000,82341],[3258,2865,44401],[3259,998,80009],[3260,1656,87029],[3261,288,22462],[3262,278,16984],[3263,2515,3584],[3264,57,7056],[3265,1011,19999],[3266,2365,75802],[3267,2537,41243],[3268,1831,98833],[3269,979,48232],[3270,1015,53675],[3271,3049,84818],[3272,703,49001],[3273,1570,27072],[3274,997,67429],[3275,34,84877],[3276,210,98405],[3277,2284,42945],[3278,1541,65035],[3279,1129,38094],[3280,2392,74968],[3281,2066,36828],[3282,2434,74952],[3283,1591,76348],[3284,467,20332],[3285,40,89247],[3286,2014,41404],[3287,1308,59921],[3288,378,85141],[3289,476,92885],[3290,217,36789],[3291,1454,44622],[3292,160,55387],[3293,475,92351],[3294,462,28089],[3295,876,99775],[3296,248,69274],[3297,277,56546],[3298,2833,67022],[3299,2100,9270],[3300,3068,86791],[3301,2506,95256],[3302,636,33894],[3304,2508,83421],[3305,2061,37277],[3306,2396,78245],[3307,1103,89040],[3308,306,56072],[3309,1480,89322],[3310,2647,83904],[3312,2153,19675],[3314,2865,8579],[3315,1694,15443],[3316,3246,6588],[3317,1321,82767],[3318,2849,44655],[3319,2808,13194],[3320,2406,45072],[3321,287,37876],[3322,1162,71611],[3323,1533,98562],[3324,1805,11089],[3325,2997,25398],[3326,17,73766],[3327,3231,54325],[3328,2499,93158],[3329,2303,81026],[3330,853,72611],[3331,930,53355],[3332,879,1611],[3333,2038,65225],[3334,2057,99311],[3335,2460,89435],[3336,2523,69074],[3337,9,267],[3338,499,95869],[3339,1702,40196],[3340,588,79487],[3341,583,5735],[3342,834,73326],[3343,877,12359],[3344,2721,86118],[3346,1814,95800],[3347,1270,11129],[3348,258,25518],[3349,2295,33715],[3350,551,51936],[3351,1481,13969],[3352,1390,53442],[3353,2982,97667],[3354,254,54306],[3355,2265,70621],[3356,789,32138],[3357,2909,9399],[3358,519,40628],[3359,2228,88214],[3360,1737,88378],[3361,2748,8402],[3362,1706,52937],[3363,3174,62587],[3364,1248,32204],[3365,538,71799],[3366,678,26041],[3367,1776,94055],[3368,279,62427],[3370,3071,94713],[3371,2641,44716],[3372,649,74525],[3373,2927,76596],[3374,1648,8584],[3375,2159,82944],[3376,2526,52645],[3377,1989,38689],[3378,597,32981],[3379,3023,74310],[3380,1901,6457],[3381,3124,83851],[3382,3314,7860],[3383,1203,26163],[3384,1995,49543],[3385,115,8550],[3386,828,84986],[3387,942,6232],[3388,2004,47858],[3389,3170,5380],[3390,629,18989],[3391,159,14677],[3392,865,41940],[3393,936,92207],[3394,1800,52392],[3395,2327,7842],[3396,2030,55326],[3397,19,9121],[3398,2224,20122],[3399,1408,45595],[3400,668,72503],[3401,1484,78181],[3402,99,6340],[3403,124,90328],[3404,2,26164],[3405,3345,49029],[3406,116,47524],[3407,414,94845],[3408,1655,92235],[3409,2341,58404],[3410,1046,68855],[3411,1766,52738],[3412,1438,169],[3413,1837,71120],[3414,1163,98348],[3415,2772,12479],[3416,177,66832],[3417,2515,72020],[3418,2102,75559],[3419,983,93372],[3420,2403,45295],[3421,274,43962],[3422,1679,81617],[3423,1716,82909],[3424,2036,38372],[3425,2613,73514],[3426,890,99797],[3427,575,14595],[3428,3176,51865],[3431,3181,29846],[3432,516,14819],[3433,1661,95490],[3434,2479,4972],[3435,700,77306],[3436,2302,58188],[3437,3039,35109],[3438,1389,62043],[3439,2693,36121],[3440,2952,60792],[3441,1006,72116],[3442,2801,17466],[3443,1926,65562],[3444,310,27441],[3445,2587,2756],[3446,609,34200],[3447,1586,79325],[3448,962,31125],[3449,420,39582],[3450,836,64202],[3451,2705,78990],[3452,969,55293],[3454,1466,96623],[3455,1326,81524],[3456,2900,1952],[3457,3334,20818],[3458,1689,56323],[3459,218,20705],[3460,3053,95923],[3461,1854,24855],[3462,2862,18628],[3463,1329,61135],[3464,1667,43002],[3465,1886,52017],[3466,425,58914],[3467,842,11630],[3468,2059,39960],[3469,253,82356],[3470,907,11807],[3471,1453,41374],[3472,1750,96646],[3473,86,45674],[3475,96,80440],[3476,396,32028],[3477,285,53326],[3478,313,83916],[3479,878,89387],[3480,1950,84245],[3481,1570,98142],[3482,203,83760],[3483,1450,30727],[3484,1015,79739],[3485,1061,90217],[3486,2359,1547],[3487,1493,94539],[3489,1413,39733],[3490,2733,89229],[3491,2519,82962],[3492,2568,89919],[3493,161,41101],[3495,689,2388],[3496,1630,12929],[3497,1760,62591],[3499,2071,31301],[3500,346,97849],[3501,1728,89646],[3502,3308,88221],[3503,1461,64053],[3505,2795,14696],[3506,1028,14722],[3507,1735,27753],[3508,1539,99994],[3509,3004,11211],[3510,3045,41732],[3512,152,47657],[3514,2409,88755],[3515,1167,47841],[3516,96,4378],[3517,990,20928],[3518,882,77262],[3519,2752,67334],[3520,2748,91988],[3521,305,3505],[3522,735,14163],[3523,749,80341],[3524,2434,71849],[3525,3118,49254],[3526,2892,76686],[3527,1528,35797],[3528,2011,60716],[3529,666,76699],[3530,922,53458],[3531,2049,75457],[3532,644,37514],[3533,3052,61650],[3534,3133,10714],[3535,662,42608],[3536,3290,999],[3538,1671,14340],[3539,591,31619],[3540,2746,94547],[3541,657,26238],[3542,2512,41534],[3543,2358,16938],[3544,2836,28853],[3545,1627,79591],[3546,371,43672],[3547,49,26568],[3548,631,27499],[3550,2143,35416],[3552,3370,79907],[3553,1820,56181],[3554,581,13365],[3555,1529,96351],[3556,1556,67226],[3557,371,9974],[3559,3318,42182],[3560,94,20988],[3561,1503,1735],[3562,3170,24666],[3564,868,68256],[3565,1720,18565],[3568,990,43599],[3569,3212,69884],[3570,2357,93764],[3571,2727,6985],[3572,3515,1464],[3573,2980,41211],[3574,725,47533],[3575,1121,46300],[3576,1810,72554],[3577,1721,52870],[3578,1388,75108],[3580,836,41583],[3581,1514,95608],[3582,3282,15799],[3583,3279,35417],[3584,1879,82910],[3585,1926,67201],[3587,1189,7064],[3588,1143,51183],[3589,880,62572],[3590,1453,29894],[3591,3314,47586],[3593,1264,20453],[3594,1610,74715],[3595,3047,93557],[3596,2846,61101],[3597,2516,52269],[3598,1403,33592],[3599,2196,1983],[3601,1324,26434],[3602,2710,64814],[3603,270,2485],[3604,2659,56686],[3605,2342,65894],[3606,3523,25760],[3607,1221,35324],[3608,447,53921],[3609,1914,96371],[3610,2913,3169],[3611,780,65565],[3612,475,23248],[3613,829,90868],[3614,2213,70845],[3615,2771,9535],[3616,1808,24471],[3617,1351,31078],[3618,1072,97707],[3619,1429,58390],[3620,2211,67124],[3622,640,48482],[3623,3446,40853],[3624,3163,40710],[3625,2082,11967],[3626,514,86179],[3627,2180,73705],[3628,3349,30939],[3629,1819,10352],[3630,2878,11228],[3631,700,9713],[3633,3298,12169],[3634,506,35798],[3635,2515,33257],[3636,2474,67465],[3637,1920,60172],[3638,1429,11259],[3639,3068,87076],[3640,2106,36092],[3641,345,59082],[3642,2155,12204],[3643,1685,84495],[3644,327,63137],[3645,1445,47601],[3646,475,81389],[3647,1129,44549],[3648,1642,75267],[3649,608,33350],[3650,3353,93343],[3651,1442,30874],[3652,2445,78303],[3653,398,9274],[3654,1233,4518],[3655,3603,97570],[3656,3212,81557],[3657,687,20763],[3658,845,20953],[3659,3622,41873],[3660,3137,645],[3662,64,12841],[3663,1828,87369],[3664,2189,77592],[3665,2639,25201],[3666,141,24112],[3667,1299,10887],[3668,3007,95544],[3669,2565,69989],[3670,2810,34839],[3671,2141,22538],[3672,144,49421],[3673,2866,64737],[3674,3599,71954],[3675,2516,57695],[3676,2427,2911],[3677,1515,10125],[3678,1599,56858],[3679,2262,76511],[3680,1112,98285],[3681,218,27341],[3682,1271,20797],[3683,3413,87948],[3684,2210,24027],[3685,705,88213],[3686,3614,50068],[3687,2202,48147],[3688,910,27202],[3689,1566,32526],[3690,1295,86582],[3691,696,27065],[3692,3167,28903],[3693,1508,99950],[3694,2681,53203],[3695,1902,30928],[3696,2420,41523],[3697,1239,49730],[3698,351,72710],[3699,2167,75430],[3700,1373,65086],[3702,2867,67406],[3703,535,90723],[3704,1558,38269],[3705,3532,43022],[3706,1696,41240],[3707,2900,95390],[3708,3419,58762],[3709,716,36026],[3710,1365,92565],[3711,361,31917],[3712,3693,27630],[3713,3031,25821],[3714,1539,6199],[3715,3392,24181],[3716,2149,14751],[3717,2056,10932],[3718,320,62638],[3719,1119,12849],[3720,1119,13447],[3721,2311,89288],[3722,136,45782],[3723,544,66854],[3724,1383,82359],[3725,2813,73194],[3726,1326,37482],[3727,784,65840],[3728,2862,32972],[3729,2500,68780],[3730,3342,93516],[3731,3356,14558],[3732,257,6309],[3733,2966,99196],[3734,2650,72131],[3735,1359,78534],[3736,2251,88411],[3737,2237,16409],[3738,3637,84288],[3739,3625,23200],[3740,2757,19210],[3741,665,40431],[3742,3541,27198],[3743,2753,71878],[3744,21,32754],[3746,2794,34502],[3747,2905,85352],[3749,2452,65963],[3750,871,62701],[3751,898,48817],[3752,1899,26565],[3753,142,63473],[3754,696,78641],[3755,110,55910],[3756,788,40240],[3757,787,88703],[3759,1300,92839],[3761,2816,51063],[3762,239,28130],[3764,3065,32079],[3766,551,58368],[3767,2918,44300],[3768,2633,56023],[3769,1173,74768],[3770,3297,34093],[3771,3571,55865],[3772,3473,42853],[3773,2609,94216],[3774,524,80240],[3775,1512,61218],[3776,774,114],[3777,2574,29772],[3778,3337,59751],[3779,7,54263],[3780,2410,7680],[3781,2712,24962],[3783,2424,63002],[3784,1748,75858],[3787,2278,59059],[3788,1790,82509],[3789,2487,20840],[3790,474,82318],[3791,2916,39379],[3792,1545,64043],[3793,554,47710],[3794,360,56882],[3795,393,52948],[3796,319,17413],[3797,3430,69387],[3798,3344,64597],[3799,359,51171],[3801,2900,58739],[3803,3375,50687],[3804,1035,19710],[3805,1563,33619],[3806,1043,1031],[3807,3619,19650],[3808,3354,27955],[3809,2623,72158],[3810,1713,1948],[3811,3055,86933],[3812,1072,39565],[3813,1239,45532],[3814,525,34163],[3815,2389,83184],[3816,3760,20841],[3817,826,41128],[3818,772,24196],[3819,2069,40134],[3820,579,57496],[3823,949,30078],[3824,1304,717],[3825,1534,90742],[3826,2989,31303],[3827,3730,49075],[3828,3043,15119],[3829,699,41988],[3830,2053,21362],[3831,593,74588],[3832,300,34513],[3833,3233,374],[3834,1933,43248],[3835,3830,28815],[3836,1611,85313],[3838,75,32775],[3839,3018,35167],[3840,735,92534],[3841,2178,54983],[3842,1540,96942],[3843,908,59838],[3844,2518,27552],[3845,2822,8689],[3846,3723,57227],[3847,533,31867],[3850,2546,36201],[3851,312,76117],[3852,18,71258],[3853,2127,89548],[3854,2843,91546],[3855,1226,39352],[3856,1248,49458],[3858,2388,20377],[3859,2946,54039],[3860,471,75029],[3861,3225,59254],[3862,1344,68785],[3863,1981,26504],[3864,1291,17177],[3865,3121,83935],[3866,1402,70223],[3867,3299,22924],[3869,2491,27971],[3870,2960,33895],[3872,3719,36781],[3873,2608,79133],[3874,2445,6384],[3875,10,59020],[3876,391,30514],[3878,3063,19424],[3879,63,7278],[3880,1110,89746],[3881,193,59876],[3882,3124,27732],[3883,2417,27398],[3884,848,90021],[3886,2403,21505],[3887,346,89082],[3888,2893,82639],[3889,3299,56212],[3890,2169,4415],[3891,1324,49483],[3892,2164,85609],[3893,3471,92981],[3894,2110,25787],[3895,3810,84024],[3897,566,93824],[3898,2615,12776],[3899,2014,64581],[3900,380,30676],[3901,2615,85978],[3902,2490,61083],[3903,3226,78879],[3904,1097,59711],[3905,3865,89809],[3906,1387,76021],[3907,874,45741],[3908,3563,72567],[3910,3066,97191],[3911,2967,18571],[3912,3556,64380],[3913,2312,39457],[3914,3913,36919],[3915,1914,18000],[3916,1181,11882],[3917,1190,75508],[3919,3625,3788],[3920,3109,64471],[3921,2298,99976],[3922,3324,19602],[3923,2666,49424],[3924,261,47288],[3925,741,55572],[3926,553,28661],[3928,538,46241],[3929,904,6963],[3930,447,26209],[3931,727,94064],[3932,2151,78891],[3933,1446,83454],[3934,717,91966],[3935,270,89172],[3936,877,69728],[3937,451,31510],[3938,1506,11533],[3940,881,43805],[3941,3551,6291],[3942,3167,7418],[3943,2614,59385],[3944,2102,60670],[3945,3532,78316],[3946,1255,22836],[3947,106,87236],[3949,75,94036],[3950,96,72612],[3951,3392,93142],[3952,1825,64795],[3953,3044,56774],[3954,2923,24705],[3955,2650,14805],[3956,3612,72674],[3958,892,52212],[3959,2530,56694],[3960,2535,94193],[3961,1613,84258],[3962,2942,85257],[3963,1516,30383],[3964,3481,91700],[3965,3096,63828],[3967,2056,74218],[3968,3733,4179],[3969,3438,78593],[3970,2032,73621],[3971,2945,66094],[3972,2108,88385],[3973,1696,10162],[3974,3104,71140],[3975,2931,20720],[3976,1061,14360],[3977,690,86015],[3978,2851,22537],[3979,1299,2278],[3980,481,34728],[3981,982,81241],[3982,557,65388],[3983,2465,35500],[3984,1963,33812],[3985,1985,2722],[3986,2000,58706],[3987,2448,16751],[3988,2634,95254],[3989,1085,83083],[3990,3542,88404],[3992,783,13924],[3994,2341,87602],[3995,1497,89661],[3996,747,86848],[3997,3479,64499],[3999,3923,36045],[4000,16,29485],[4001,659,306],[4002,718,73872],[4003,1827,51597],[4004,2063,26186],[4005,87,28498],[4006,145,57668],[4008,1995,61378],[4009,165,72312],[4010,558,35815],[4011,3289,8353],[4012,2604,79500],[4014,211,80094],[4015,1966,19010],[4016,543,23261],[4017,1541,91786],[4018,2791,91806],[4019,2265,88329],[4020,613,18832],[4021,2293,67849],[4022,1755,79265],[4023,2762,1719],[4024,2944,40343],[4025,2984,54007],[4026,2081,80786],[4028,1988,75366],[4029,644,16138],[4030,1243,86244],[4031,234,98332],[4032,3818,51150],[4033,229,86004],[4034,272,80257],[4035,686,2904],[4036,340,8840],[4037,3495,30444],[4038,1084,42615],[4039,532,30289],[4040,3726,48824],[4041,2445,99852],[4042,1462,35892],[4043,1577,90744],[4044,2206,13005],[4046,3999,53891],[4047,2772,40110],[4048,250,94129],[4049,3110,81060],[4050,2605,17227],[4051,1646,13376],[4052,1608,9129],[4053,1395,71482],[4054,2546,21830],[4055,2494,9557],[4056,2111,11722],[4057,1920,59533],[4058,3514,95802],[4059,2505,50711],[4063,3073,4670],[4064,418,47303],[4065,1145,46280],[4066,2500,12016],[4067,1012,55986],[4068,1196,71946],[4069,1991,92055],[4070,1424,72881],[4071,2462,40894],[4072,637,26855],[4073,3010,59117],[4074,827,96183],[4075,1879,17630],[4076,3751,42159],[4077,1621,8145],[4078,1414,66958],[4079,3027,18473],[4080,733,92935],[4081,1469,98455],[4082,3929,5149],[4083,1470,85525],[4084,1093,28644],[4085,3884,24248],[4086,247,91417],[4087,1297,91128],[4088,733,85544],[4089,680,90468],[4090,2940,17665],[4091,4045,28725],[4092,308,76866],[4093,1412,56868],[4094,2250,32997],[4095,411,59911],[4096,3997,55692],[4097,1636,53059],[4098,2446,50259],[4099,1054,90347],[4100,1103,55792],[4101,162,74325],[4102,1538,46310],[4103,1498,79094],[4106,1499,41651],[4107,1637,74281],[4108,3228,49223],[4109,43,91715],[4110,1787,55420],[4111,50,73730],[4112,875,56896],[4114,3530,46284],[4115,2659,88521],[4116,442,50351],[4117,653,73798],[4118,2403,72979],[4119,289,31886],[4120,2466,10462],[4122,2204,53914],[4123,2580,36172],[4124,3031,66424],[4125,2416,19125],[4126,4064,75147],[4127,52,69626],[4128,2813,69490],[4129,604,65538],[4130,3000,20816],[4131,340,17382],[4132,3227,91627],[4134,2113,7951],[4135,1280,91731],[4136,2665,234],[4137,3890,30778],[4138,3125,17363],[4139,2957,32354],[4140,3172,28562],[4141,3584,91108],[4142,1170,10884],[4143,2767,72653],[4144,3556,96627],[4145,3756,35818],[4146,901,98429],[4147,1385,63750],[4148,1710,64894],[4149,424,7569],[4150,2118,89731],[4151,1889,39566],[4152,717,87304],[4153,3207,62455],[4154,3119,47468],[4155,3484,7799],[4156,1916,80504],[4157,3921,70371],[4158,3933,11071],[4159,2584,37746],[4160,1043,94835],[4161,1857,80927],[4162,306,63913],[4163,3316,40383],[4164,3915,23962],[4166,2608,70578],[4167,326,77323],[4168,2675,76244],[4169,1879,87354],[4170,3816,28456],[4171,1985,58208],[4172,1881,51251],[4173,3611,82132],[4174,1961,93841],[4175,3875,49326],[4176,3627,39118],[4177,1482,48923],[4178,3757,9396],[4179,1389,79894],[4180,2541,63084],[4181,2260,94731],[4182,2676,6477],[4183,4101,73369],[4184,520,5356],[4186,2740,92133],[4187,3604,41353],[4188,1538,83882],[4189,3829,78354],[4190,805,29224],[4191,2322,708],[4192,262,64725],[4193,3863,68592],[4194,2556,46697],[4195,331,98463],[4196,849,26962],[4197,3976,54483],[4198,834,88233],[4199,1933,73565],[4201,342,8181],[4202,1739,34702],[4203,2193,70660],[4204,2151,8586],[4206,262,35380],[4207,1910,54231],[4208,1805,9399],[4209,874,70099],[4210,2992,50488],[4211,3684,33924],[4213,2691,24917],[4214,1919,81617],[4215,3268,69217],[4216,2772,73350],[4217,416,69141],[4218,2039,23229],[4219,718,63601],[4220,2702,73662],[4221,3970,99696],[4222,3751,9655],[4223,2750,11384],[4224,654,73630],[4225,2305,85096],[4226,3922,14641],[4228,1013,20792],[4229,971,64032],[4230,457,12843],[4231,1152,96364],[4232,526,65961],[4233,2483,52463],[4234,1023,48977],[4235,902,92103],[4236,3450,92134],[4237,1743,43147],[4238,3798,80709],[4240,1539,13879],[4242,1838,84656],[4243,1051,77112],[4244,3366,86421],[4245,1257,11768],[4246,2440,72303],[4247,2377,84806],[4248,2044,16029],[4249,2044,12429],[4250,714,14770],[4251,614,75906],[4252,3247,1610],[4253,3580,65871],[4255,3183,16705],[4256,1229,61933],[4258,1742,43294],[4259,3659,48053],[4260,757,69397],[4261,3538,23148],[4262,2463,45079],[4263,2231,965],[4264,60,15313],[4265,1477,5625],[4266,3295,94178],[4267,4094,42139],[4268,994,13692],[4269,151,5828],[4270,2794,28866],[4271,2262,4444],[4272,2506,42169],[4273,3451,98644],[4274,737,54255],[4276,2340,56078],[4277,2415,27111],[4278,400,76394],[4279,369,76528],[4280,2400,31703],[4281,743,65212],[4282,450,76874],[4283,2907,48893],[4284,4126,29691],[4285,1930,45404],[4286,1678,72814],[4288,1187,28122],[4289,712,95321],[4290,818,30317],[4291,4241,13830],[4292,3023,16875],[4293,716,97696],[4295,2173,36721],[4296,794,1163],[4298,189,12758],[4299,1168,72489],[4300,2894,98625],[4301,3025,57956],[4302,2154,40060],[4303,1116,38444],[4304,1892,86128],[4305,2543,69066],[4307,3668,8750],[4308,1868,60886],[4310,2908,45791],[4311,3013,33909],[4312,167,46457],[4313,2118,29308],[4314,695,44187],[4315,37,10420],[4316,1884,25051],[4317,3748,37996],[4318,4208,71695],[4319,3150,46145],[4320,373,37194],[4321,1503,56806],[4322,704,61158],[4323,3608,81452],[4324,4279,82490],[4325,3939,33970],[4326,628,75564],[4327,1167,37299],[4328,507,23108],[4329,143,25098],[4330,1850,16717],[4331,3895,78807],[4332,1158,462],[4333,1296,7849],[4334,710,42025],[4335,3556,32405],[4336,3360,10547],[4337,3492,67515],[4338,236,59048],[4340,245,36689],[4341,25,45305],[4342,3569,27916],[4343,2566,40436],[4344,2656,95786],[4346,3,30762],[4347,934,68687],[4348,1391,55454],[4349,3967,70198],[4350,3661,41386],[4351,662,85324],[4352,2671,11811],[4353,209,65061],[4354,2746,34715],[4355,476,22683],[4356,3724,99073],[4357,3605,47495],[4358,1506,16395],[4359,1765,98293],[4360,2717,22339],[4361,1449,18170],[4362,2111,44600],[4363,3311,10469],[4364,2648,50151],[4365,2906,60744],[4366,1709,6902],[4367,1954,62076],[4368,4354,74],[4369,3703,9137],[4370,3992,29697],[4371,2328,17369],[4372,3346,33481],[4374,1113,36569],[4375,2439,17384],[4377,1509,19927],[4379,187,4014],[4380,2242,29377],[4381,2292,85635],[4382,2983,75941],[4383,3424,92623],[4384,3146,21370],[4385,3333,65780],[4386,1055,77574],[4387,1097,10963],[4388,2341,48211],[4389,4072,35220],[4390,649,97439],[4391,2565,72473],[4392,592,54171],[4393,1406,53697],[4394,958,11910],[4395,4092,88202],[4396,922,19617],[4397,3021,88737],[4398,1419,68698],[4399,1498,4610],[4400,3494,95688],[4401,815,50937],[4402,3543,22563],[4403,1276,10155],[4404,4187,39475],[4405,1347,79133],[4406,3244,57132],[4408,2489,56226],[4409,2220,25301],[4411,1482,8935],[4412,4124,24797],[4413,3119,13737],[4414,3400,9730],[4415,913,48976],[4416,2250,35040],[4417,1398,42595],[4418,442,38542],[4419,3225,4062],[4420,4201,42624],[4421,2994,42213],[4422,4384,55947],[4423,2843,98129],[4424,1390,45685],[4425,1027,5103],[4426,2655,34488],[4428,852,19412],[4429,4263,11512],[4430,765,59319],[4431,3888,95852],[4432,2395,29391],[4434,3286,42870],[4435,1615,57712],[4436,507,48157],[4437,2034,52642],[4438,3893,32478],[4440,3872,26149],[4441,2693,10991],[4442,79,90521],[4443,2537,10122],[4444,1437,6607],[4445,2089,15752],[4446,3823,6372],[4447,1757,14700],[4448,826,4921],[4449,4341,65553],[4451,3154,97845],[4452,1715,65415],[4453,3490,11734],[4455,2556,39117],[4456,1764,858],[4457,2066,9273],[4458,1245,7333],[4459,1729,41324],[4460,4069,54238],[4461,3239,63038],[4462,2836,91595],[4463,2605,69978],[4464,2043,67282],[4465,2953,1348],[4466,1905,60577],[4467,1694,97941],[4468,4117,95062],[4469,1063,41201],[4470,500,67824],[4471,831,74093],[4472,3568,29757],[4473,1133,38190],[4474,2210,955],[4475,710,74274],[4476,1668,56981],[4477,2780,90085],[4478,3383,72840],[4479,162,53493],[4480,920,40285],[4481,4262,89921],[4482,886,14260],[4485,3151,65582],[4486,4287,37157],[4487,3217,35690],[4488,4334,58308],[4489,411,30585],[4490,955,64378],[4491,389,76702],[4492,2936,86495],[4493,2960,78630],[4494,3336,68705],[4495,3760,50907],[4496,244,21107],[4497,3746,43100],[4498,2302,86858],[4499,331,1146],[4501,492,10622],[4502,3168,78832],[4503,1048,82409],[4504,2451,21823],[4505,147,26654],[4507,2644,19818],[4508,3512,37214],[4509,4148,23850],[4510,2057,29667],[4511,2629,5349],[4513,2338,56643],[4514,287,262],[4515,1863,84310],[4516,1343,50904],[4517,3075,24898],[4519,849,6909],[4520,1218,12657],[4521,1803,73859],[4522,3519,66584],[4523,2888,61987],[4525,3598,52937],[4526,599,53104],[4527,827,24585],[4528,3134,24894],[4529,1429,81999],[4530,2162,53958],[4531,272,69014],[4532,1580,24352],[4533,2524,42459],[4534,137,88046],[4536,1946,58427],[4537,2792,10417],[4538,1773,24880],[4540,2862,74689],[4541,1924,36163],[4542,2624,61649],[4543,1097,2350],[4544,3010,78577],[4545,734,10204],[4546,1137,93033],[4547,1619,56947],[4548,2257,75487],[4549,3708,56245],[4551,1633,4979],[4552,1834,10694],[4553,4279,10740],[4554,1642,63955],[4555,1078,40295],[4556,2727,36377],[4559,221,70913],[4561,4057,80725],[4562,3133,70490],[4564,2586,55600],[4565,2127,2914],[4567,2096,61631],[4568,330,15106],[4569,3505,85057],[4570,625,47390],[4571,42,74390],[4573,339,70910],[4574,531,56403],[4575,2494,62586],[4576,4443,44459],[4577,4069,994],[4578,582,26287],[4579,181,6535],[4580,896,2264],[4581,207,26175],[4582,3930,82741],[4583,554,62338],[4584,2965,43359],[4587,4563,19578],[4588,511,41921],[4589,1317,92930],[4590,3692,32923],[4592,1925,70897],[4593,1341,70465],[4594,1682,62172],[4595,2617,52330],[4596,3620,41755],[4599,3474,74026],[4600,1525,45963],[4601,3301,27392],[4602,2774,80792],[4603,3547,54814],[4604,1733,36031],[4605,2648,23890],[4606,4044,24145],[4607,3935,69],[4608,512,32042],[4609,3843,19351],[4610,219,85944],[4612,236,22376],[4613,3495,55273],[4614,2412,57773],[4615,3029,76077],[4616,4227,92066],[4617,2572,48678],[4618,3961,75607],[4619,4403,53632],[4620,1457,44180],[4621,767,82577],[4622,3610,73591],[4623,3856,96113],[4624,4380,74693],[4625,1741,89687],[4626,3011,72936],[4627,1551,28008],[4629,3863,2790],[4631,3100,88376],[4632,3300,36887],[4633,2078,16920],[4634,880,89834],[4635,3317,10360],[4636,2312,44358],[4637,3941,36694],[4638,3566,50530],[4639,3418,73645],[4640,4076,98068],[4642,2337,76888],[4643,462,7785],[4644,3151,91670],[4645,3556,37850],[4646,1301,54336],[4647,4573,5840],[4648,3498,56437],[4649,351,492],[4650,1941,90555],[4651,712,71979],[4652,1168,59707],[4653,3246,67881],[4654,2803,68071],[4655,1994,19134],[4656,280,36336],[4657,3430,46341],[4658,2464,91836],[4659,767,31303],[4660,3506,99471],[4661,2812,19525],[4662,3614,28252],[4663,1196,20726],[4664,3579,76276],[4665,460,62055],[4666,2523,53987],[4667,2483,51240],[4668,4400,86515],[4669,3875,71449],[4670,275,20068],[4671,2034,70255],[4672,1152,16872],[4673,2066,41395],[4675,4235,48122],[4676,2347,57119],[4677,4639,55483],[4678,348,42413],[4679,1189,54809],[4680,1754,20605],[4681,4153,53813],[4682,2940,67182],[4683,2248,18798],[4684,265,10562],[4685,3445,26879],[4686,3930,40546],[4687,3009,71918],[4689,3468,64514],[4690,3077,7934],[4691,446,54744],[4692,2339,87990],[4694,619,90505],[4695,1027,33571],[4696,1141,17127],[4697,1983,60800],[4698,4251,6593],[4699,2504,8054],[4700,1244,48651],[4701,606,88097],[4702,646,17027],[4703,3056,5260],[4704,1555,81720],[4705,1058,29955],[4706,2266,91377],[4707,1002,420],[4708,4611,92630],[4709,2330,19631],[4710,490,11745],[4711,58,1527],[4712,2375,56187],[4713,697,89398],[4714,2159,51468],[4715,351,84174],[4716,636,85617],[4717,2921,5106],[4718,1912,1617],[4719,1707,87332],[4720,31,92812],[4721,1621,69959],[4722,1227,39705],[4723,3029,28961],[4724,1004,29637],[4725,3228,27106],[4726,304,15665],[4727,4191,5959],[4728,4722,84178],[4729,210,88319],[4730,2663,68302],[4732,1073,98515],[4733,3048,43250],[4734,3137,22302],[4735,3098,65452],[4736,4384,22158],[4737,3939,68858],[4738,1739,39670],[4739,1750,12257],[4740,2660,21235],[4741,189,83097],[4742,2086,960],[4743,4393,9888],[4744,1355,69990],[4745,2385,45075],[4746,408,50863],[4748,4086,63782],[4749,4375,93868],[4750,2895,20883],[4751,4460,89526],[4752,4630,27321],[4753,2033,28438],[4754,984,28540],[4755,3826,66233],[4756,2094,55854],[4757,1039,86123],[4758,2162,20459],[4759,351,72337],[4761,2100,7813],[4762,4573,44134],[4763,2105,89859],[4765,822,38035],[4766,3073,55804],[4767,4599,87743],[4768,4062,38863],[4769,98,54933],[4770,2116,90750],[4771,3201,28480],[4772,315,78300],[4773,4599,26223],[4775,3619,55164],[4776,148,86898],[4777,4556,19967],[4778,146,10762],[4780,1073,49926],[4781,4467,22168],[4782,500,80698],[4783,283,48366],[4784,3464,27727],[4785,1744,56400],[4787,372,71228],[4788,3289,90011],[4789,4480,32684],[4790,2664,41808],[4791,3072,91167],[4792,4016,51614],[4793,3445,9524],[4794,4453,99332],[4795,4559,18133],[4796,3952,51345],[4797,4788,52396],[4799,3346,45613],[4800,2792,77832],[4801,2072,70024],[4802,617,89525],[4803,245,72605],[4804,4086,62971],[4807,2594,12957],[4808,3714,48598],[4809,2616,18693],[4810,74,62882],[4811,2973,85141],[4813,1573,13286],[4814,944,24285],[4815,393,48508],[4816,92,66773],[4817,1613,46200],[4818,1913,85145],[4819,2181,52386],[4820,2187,57368],[4821,2615,35193],[4822,3356,16635],[4823,3936,31775],[4824,345,48639],[4825,3576,51506],[4826,3542,79028],[4827,874,3710],[4829,1028,11124],[4831,4442,75460],[4832,3378,14580],[4833,3632,2249],[4834,3419,47740],[4835,2068,1344],[4836,425,72083],[4837,4825,24877],[4838,909,19974],[4839,2311,25057],[4840,649,22635],[4841,3753,38523],[4842,951,42616],[4843,1665,35938],[4844,3279,86232],[4845,4249,6468],[4846,1085,59835],[4847,2058,71088],[4848,1542,25647],[4849,647,6423],[4850,2809,58914],[4851,3049,59313],[4852,2243,3898],[4853,2961,23283],[4854,3657,66324],[4856,3769,47780],[4857,3439,9107],[4858,4289,30038],[4859,1405,86108],[4860,3061,77134],[4861,3647,9451],[4862,713,88792],[4863,4653,55572],[4864,690,51356],[4865,1796,65553],[4866,1439,98863],[4867,3477,75925],[4868,4573,56861],[4869,2989,1267],[4870,3130,1877],[4872,3504,86658],[4873,3957,9637],[4874,3968,64806],[4875,2497,6132],[4876,1462,22218],[4877,4469,50945],[4878,4148,48824],[4880,61,92026],[4881,4260,12448],[4882,2837,10496],[4883,4176,93769],[4884,1879,78062],[4885,152,64406],[4886,3935,12204],[4887,3086,85979],[4888,86,82447],[4889,545,7204],[4890,2504,34936],[4892,2488,84929],[4893,2558,43869],[4894,4697,16978],[4895,1815,20708],[4896,3725,73424],[4897,2814,74613],[4898,3980,87183],[4900,4264,9058],[4901,741,38291],[4902,2584,63022],[4903,4220,25044],[4904,2247,27226],[4905,2539,15981],[4906,3461,80179],[4908,1385,38442],[4910,1748,2983],[4911,943,95061],[4912,29,77590],[4913,798,81584],[4914,4352,71546],[4915,528,59703],[4916,2635,59531],[4917,515,35616],[4918,2753,96032],[4919,4574,26653],[4920,4195,30293],[4921,2108,86492],[4922,730,73331],[4923,3138,89370],[4924,1032,46090],[4925,1604,2204],[4926,2301,18726],[4927,4593,94106],[4928,2775,60368],[4929,3338,31890],[4930,3697,62844],[4931,1977,68316],[4932,134,71315],[4933,1104,45573],[4935,3020,89794],[4936,2113,77051],[4937,3956,92873],[4938,3814,95395],[4940,525,67922],[4941,1369,76376],[4942,2820,7764],[4944,3583,47405],[4945,678,98918],[4946,3641,74773],[4947,1153,20622],[4948,3818,81255],[4949,4542,95558],[4950,4177,89776],[4951,374,22579],[4953,4468,63616],[4954,3236,32701],[4955,3035,10149],[4956,2197,62741],[4957,3049,13547],[4958,1435,97768],[4959,1322,99894],[4960,2884,83152],[4961,2090,53156],[4962,538,82698],[4963,2588,81769],[4965,2275,21276],[4966,3118,87679],[4967,1177,59162],[4968,4121,65210],[4969,3239,10432],[4970,3068,95365],[4971,2626,90152],[4972,183,60398],[4973,552,32485],[4974,4844,24576],[4975,1795,90547],[4976,678,44605],[4977,2612,78816],[4978,2495,68568],[4979,12,98743],[4980,214,22124],[4981,4755,1098],[4982,174,40306],[4983,4420,79277],[4984,949,1863],[4985,2445,39103],[4986,2451,74467],[4987,4513,49190],[4988,1398,51750],[4989,4347,83126],[4990,754,1916],[4991,1745,44967],[4992,186,15327],[4993,2379,25658],[4994,2315,7855],[4995,2498,68184],[4996,1108,83621],[4997,2430,52934],[4998,4116,61384],[4999,4646,97322],[5000,4815,2714],[5001,3627,58553],[5002,1711,20167],[5003,3474,55766],[5005,2124,13224],[5006,373,24479],[5007,89,94483],[5008,1237,56902],[5009,4012,8034],[5010,833,86052],[5011,4339,87828],[5012,732,44549],[5013,2032,88365],[5014,390,96121],[5015,3054,43712],[5016,1254,11188],[5017,1885,91180],[5018,249,50844],[5019,4340,56613],[5020,396,30292],[5021,289,78828],[5022,3261,51253],[5023,573,2970],[5024,3393,48925],[5026,2952,46418],[5028,2895,83452],[5029,4494,44285],[5030,509,57283],[5031,2342,98032],[5032,511,69230],[5033,4919,48916],[5034,1308,1113],[5035,3827,48818],[5036,2598,41938],[5037,921,5822],[5038,2247,39021],[5039,293,74176],[5040,183,71763],[5041,4333,6059],[5042,4787,11116],[5043,4268,41184],[5044,3453,47232],[5045,885,83205],[5046,2261,55624],[5047,2646,88047],[5048,2027,41878],[5049,3562,33251],[5050,2682,21808],[5052,1810,21656],[5053,1994,6962],[5054,3236,84129],[5055,778,89003],[5058,3076,79013],[5059,3828,64991],[5060,4031,37419],[5061,1087,92433],[5062,191,68136],[5063,2730,46290],[5064,1065,9715],[5065,462,92878],[5066,1766,92559],[5067,2749,95270],[5068,3682,34381],[5069,4977,14521],[5070,4152,9447],[5071,4286,25801],[5072,647,12664],[5074,4526,89566],[5075,2476,16395],[5076,3345,70782],[5077,2681,33611],[5078,3210,93618],[5079,1756,38443],[5080,4425,76266],[5081,3280,13123],[5082,2383,4185],[5083,2799,57486],[5084,4183,28623],[5085,2682,5750],[5086,2453,71868],[5087,1513,18449],[5088,1558,1777],[5089,1970,89319],[5090,696,94673],[5091,4104,59945],[5092,1719,90679],[5093,759,36574],[5094,5016,75070],[5095,1032,79605],[5096,2266,5669],[5097,2407,63608],[5098,4004,61859],[5100,4269,93517],[5101,996,5688],[5102,4371,69525],[5103,1766,45087],[5104,3304,79888],[5105,301,63307],[5106,233,72376],[5107,4104,82064],[5108,3458,56577],[5109,2588,29428],[5110,3446,39224],[5111,5049,88607],[5112,2112,49804],[5113,383,37855],[5114,2169,74609],[5115,2950,20200],[5116,4243,61173],[5117,2460,88193],[5119,183,86090],[5120,3401,49682]]
    



node_dict = packupNode(n, wells, pipes)
node_list = list(node_dict.values())

group_list = get_groups(node_list)

In [63]:
tmp_group = group_list


new_group = []
for x in tmp_group:
    new_group+=x


base = new_group

wells_list,pipe_list = nodeGroup2dict(base,output_format='list')
n = len(wells_list)


print(n)
print(wells_list)
print(pipe_list)

print('\n'*5)
s = Solution()    
res,well_status,pipe_status = s.minCostToSupplyWater(n, wells_list, pipe_list,True)
print(res)


5120
[93497, 49440, 87286, 61311, 16223, 47633, 49981, 31881, 79195, 74898, 87240, 98730, 215, 58651, 65885, 1276, 71865, 93215, 16414, 99719, 99406, 98782, 2807, 52483, 68309, 29050, 60381, 14395, 87767, 94324, 86616, 21128, 67698, 60123, 49231, 45818, 78142, 87657, 51020, 96210, 30776, 90387, 98339, 67693, 97336, 82161, 25984, 53101, 82834, 99506, 13090, 66222, 61757, 6575, 69307, 75528, 88049, 80590, 74973, 89442, 50701, 86999, 91132, 59910, 78790, 34886, 72131, 96231, 31993, 57284, 4388, 61897, 14443, 50616, 82507, 98449, 52010, 87803, 75439, 46468, 83757, 73419, 68674, 60298, 52684, 73002, 77996, 7696, 93661, 78500, 98905, 90039, 73192, 57135, 81421, 49098, 90624, 46203, 84790, 98949, 60499, 89977, 74545, 93929, 95759, 61962, 81279, 70456, 57516, 85603, 47321, 84042, 64022, 87314, 69270, 95582, 96615, 38502, 81633, 72514, 49353, 97977, 99355, 66043, 52262, 52378, 30272, 54000, 54014, 81931, 93350, 65609, 70337, 80425, 48104, 52970, 92207, 92543, 56697, 21564, 77965, 78221, 74534, 

In [64]:
178954501-178939525

14976